In [1]:
import hashlib
import functools
import string
import binascii

# Exercise 1: PGPv9.2

In [2]:
# Main goal of the exercise is to factorize N
def PGP(listX, listMod, pk, rsa, pwd):
    e, N = pk
    #From hint 1 knowing the entropy is low we can deduce the one of the factor of N from the given list of moduli.
    r = 1
    for elem in listMod:
        tmp_r = gcd(elem, N)
        if tmp_r != 1:
            r = tmp_r
    N_bis = N/r
    #From hint 2, from the x'!= x, we can recover one factor of N because modulo N the result is false but mod p or q we have the good result.
    x_diff, dec_diff = 0,0
    for x, dec in listX:
        if x - dec != 0:
            x_diff = x
            dec_diff = dec
    diff = abs(x_diff - dec_diff)
    q = gcd(diff, N_bis)
    p = N_bis/q
    
    #Compute phi of N.
    phi_N = (p-1)*(q-1)*(r-1)
    d = inverse_mod(e, phi_N)
    
    #Decyphering rsa (x^e mod N) and computing the key.
    x = rsa.powermod(d, N)
    key = hashlib.sha512(str(x).encode('ascii')).digest()
    ct_pwd = bytes(a ^^ b for (a, b) in zip(key, pwd))
    return ct_pwd.decode('ascii')
    

In [3]:
#Test params
e = 3
N = 654032078828158004479561125076917751960218115958449966396073894966965597590508952033405786698739540724708083662315074065830344628106475494551249714696105362529966978945647150542063883753999989797718111113517466533085449033495449408003774358585561413790437244826450323335154941395301964762596917256318739893022733981253943989882389117953312620841491439098404231131575863126087020934329150434477218974136881108394562225854972691844695580415002478480748654665076457
listX = [(115963774772220810043182850365803488571526308949834730535835020963661240896197507058552248717332607598077260241174995885866261373798446888352200387120359233987312952065050486761657736547759439324511724800503349295617703635143296294734938288818491181321640907537083732467731710981116829054851094401980987877179076712307974340060014498464309477937637686708417156063342741451053147159580224134263101723607135197531306625855612563069882726496307889178391665417857230, 115963774772220810043182850365803488571526308949834730535835020963661240896197507058552248717332607598077260241174995885866261373798446888352200387120359233987312952065050486761657736547759439324511724800503349295617703635143296294734938288818491181321640907537083732467731710981116829054851094401980987877179076712307974340060014498464309477937637686708417156063342741451053147159580224134263101723607135197531306625855612563069882726496307889178391665417857230), (593089147881448099088538093048144061370031487519498315113735355305247161096028429113082390275722204025531787397031608162733945130611700939173546032247817161422274830331388239925978188316696731557471072381258695327827459679884187493268080799598447530439733366018958966255824242019911427036355323481879170847077573574153793333985805990463154859886821728955059268906733894169628898126475191207605711785974664067782550109682595606571001822541570254178645888760245182, 593089147881448099088538093048144061370031487519498315113735355305247161096028429113082390275722204025531787397031608162733945130611700939173546032247817161422274830331388239925978188316696731557471072381258695327827459679884187493268080799598447530439733366018958966255824242019911427036355323481879170847077573574153793333985805990463154859886821728955059268906733894169628898126475191207605711785974664067782550109682595606571001822541570254178645888760245182), (522608397034874198856960074519318465784313595632425984872947732658036502254136059711940496471703497104063178838388708951083486683519567676971656890489905394181769402669153115866276266443911805521135792371133592186499012985166279884935536200698049243886099553052991931609312520591569929240046247266614607098678254712606052531322350221435391236524557492979880133575569544412477453351790145437531843315696294229652819665343149095826849446668253745788455211269065043, 522608397034874198856960074519318465784313595632425984872947732658036502254136059711940496471703497104063178838388708951083486683519567676971656890489905394181769402669153115866276266443911805521135792371133592186499012985166279884935536200698049243886099553052991931609312520591569929240046247266614607098678254712606052531322350221435391236524557492979880133575569544412477453351790145437531843315696294229652819665343149095826849446668253745788455211269065043), (94065509575279323001870981880174196625758174365966104277549384695927635101490532336312649701041310863886239454263764529720753963645105352459847457956003380942267173251650027361367400662150011974698986392452178929041121516098491635529899658619608148101732905484091983430713374473114699877898102828072592911872213863515302296865789810950913527353714170674093346856493768422167123739119564123425009965006079285496734655724530266427725350886169701273634916612559070, 579779774031696687089363461878937841277675912639289860017545468289840572787141261599317817076306231699659895418670846940262401094372873338185752498499668188751286631269776846988243194594214564837376408780979108494302453024953816277375657128144785776749144230410550900614651577021961686555552030786934057847801143054865474706973428397488074801321028610444139193042732565035227260097797356623701456620404857454859391230685495284994678252805800160051831765930135604), (91211839659019289158008153581476582963926434466379970396397153906443336082137044183226748436654906897503519757167613730406179626247186656900140651545402682516497348108265257273700006131221622524479642752514635496154113319310794173880875306674596268248420275326367035335276879296239911068224732545651409061317454172310265140265369367787297890238484140370648637771823359188799193767099918713158000298304595218041668723587293662290294649139237061046841010045384994, 91211839659019289158008153581476582963926434466379970396397153906443336082137044183226748436654906897503519757167613730406179626247186656900140651545402682516497348108265257273700006131221622524479642752514635496154113319310794173880875306674596268248420275326367035335276879296239911068224732545651409061317454172310265140265369367787297890238484140370648637771823359188799193767099918713158000298304595218041668723587293662290294649139237061046841010045384994), (483886482423109377099591120687146459262094403483616844006739591268833343905761504822424873716390583585608030612730401642751805046014811744563964789987289513375579708483987583170916321510748394645977409865914575036054724742991845184605065691940199403917922497096984072102349677154211366920347821489876322722865767755296684414294762172534917178698076275329076839278564024818528832208528869786721141110023949752360102420805580705436108502281212841845540729396903162, 483886482423109377099591120687146459262094403483616844006739591268833343905761504822424873716390583585608030612730401642751805046014811744563964789987289513375579708483987583170916321510748394645977409865914575036054724742991845184605065691940199403917922497096984072102349677154211366920347821489876322722865767755296684414294762172534917178698076275329076839278564024818528832208528869786721141110023949752360102420805580705436108502281212841845540729396903162), (154035460055209138974920765471504218814522930758386669259619056482142819058817424080912538838630848667688518600730217343921141464731908112644002874055920141385011551681435013170080155211623556084964281924262103709881853800944780531832568260531089558737367443964725652775340714053040873635347390288434285035887822541676042856856810187119970461281861944489400410697012426497830094678540377558545663355926521827750276050028738611644007227428730363890478514047635490, 154035460055209138974920765471504218814522930758386669259619056482142819058817424080912538838630848667688518600730217343921141464731908112644002874055920141385011551681435013170080155211623556084964281924262103709881853800944780531832568260531089558737367443964725652775340714053040873635347390288434285035887822541676042856856810187119970461281861944489400410697012426497830094678540377558545663355926521827750276050028738611644007227428730363890478514047635490), (607070063127371922145703769643802648288099481439362598400443861050417681605802477552810247778419335954631710547514637419308075972178510469065040997618522282432170231519485808568436207572534498586166108577836443077133969800406803748955683433563926570701374254888683694402491244741675108714332976863442107167948505056405522864954326482839213190904149823237437021317222415342110621194299682278442738777269717868097942365548570502654251415051086083823792315132696762, 607070063127371922145703769643802648288099481439362598400443861050417681605802477552810247778419335954631710547514637419308075972178510469065040997618522282432170231519485808568436207572534498586166108577836443077133969800406803748955683433563926570701374254888683694402491244741675108714332976863442107167948505056405522864954326482839213190904149823237437021317222415342110621194299682278442738777269717868097942365548570502654251415051086083823792315132696762), (246890220787420370835704466384516785996311437909681762670770827070048142868229723786311139954730625602084784479543631164739192098290148674375730791029276291276014132984896018566083871789350154071464779929637833986310860770178274582224277694069245639236952718364817764163414842233052089624722819236893856355377304664716210320089826804943399284239862605527793253578595222107759008146356095032380354376008726951729003030830414780479892552695217847905687488514894724, 246890220787420370835704466384516785996311437909681762670770827070048142868229723786311139954730625602084784479543631164739192098290148674375730791029276291276014132984896018566083871789350154071464779929637833986310860770178274582224277694069245639236952718364817764163414842233052089624722819236893856355377304664716210320089826804943399284239862605527793253578595222107759008146356095032380354376008726951729003030830414780479892552695217847905687488514894724), (255611033680370499609906602994339393499035119542914979128754946090886090924432408933678628608052105774158527240190451943584437650730754108585688453506484055996759002662581957163380741836526754615823837470262870541270343617149084016065019100290036171046430753894336284913704458790270543979621910639591866652382377544843322144842448887138927456227859342201332616854781580855553053524147459705287921939567004267455415148069898241804693213041996405518771791491623192, 255611033680370499609906602994339393499035119542914979128754946090886090924432408933678628608052105774158527240190451943584437650730754108585688453506484055996759002662581957163380741836526754615823837470262870541270343617149084016065019100290036171046430753894336284913704458790270543979621910639591866652382377544843322144842448887138927456227859342201332616854781580855553053524147459705287921939567004267455415148069898241804693213041996405518771791491623192), (126091571090770103808983663682276380729502389921243916242777412219317551636252286620989548393841189964074807581808012282217261297800132411634863049491190029690865781840398037386670304300447435634723901053721699620255853199085961726756814558744162816202952090130377340929034260517793174136167066151613277937409847977208174916601526829455420902549474855927352772474091775818057595870429142355136811483292420113176578795472101767829680713373577502359533061999582177, 126091571090770103808983663682276380729502389921243916242777412219317551636252286620989548393841189964074807581808012282217261297800132411634863049491190029690865781840398037386670304300447435634723901053721699620255853199085961726756814558744162816202952090130377340929034260517793174136167066151613277937409847977208174916601526829455420902549474855927352772474091775818057595870429142355136811483292420113176578795472101767829680713373577502359533061999582177), (367685766936887352314148044842951547696404540664325752594640910958497464456176435816161150531559106705426023889730937238656222279537370383886844276164324385481783740965878958841566872154237550440527555829254811834086555438274880440941640798066434082891031840470336083271033211975750242569043650917196308853353770824482612254439497437661197544596067370318533347450826599768065913820058556205724002153609594565110991111354863835863465268474508221195714794993738765, 367685766936887352314148044842951547696404540664325752594640910958497464456176435816161150531559106705426023889730937238656222279537370383886844276164324385481783740965878958841566872154237550440527555829254811834086555438274880440941640798066434082891031840470336083271033211975750242569043650917196308853353770824482612254439497437661197544596067370318533347450826599768065913820058556205724002153609594565110991111354863835863465268474508221195714794993738765), (23597093494755215227955834741574240329676624515687471313612392886547685297825565177730232457182031996217930997019613525815256901067893860074884994227892136243193415250393674377417222756647790985529107684175029633715264731351512542509550134932748926987247006617927680347525120326212866823173117084626983805009820668042413155617078860491197008360486388180296642328557138587140953145878859079860573123093762212405326049525475711569083994472763612943535011487836688, 23597093494755215227955834741574240329676624515687471313612392886547685297825565177730232457182031996217930997019613525815256901067893860074884994227892136243193415250393674377417222756647790985529107684175029633715264731351512542509550134932748926987247006617927680347525120326212866823173117084626983805009820668042413155617078860491197008360486388180296642328557138587140953145878859079860573123093762212405326049525475711569083994472763612943535011487836688), (93553755189848638012338659673375584674936537808684571609487412905706716483228418613013789748317669757894115018213499797793404634381651389489512661174720561406135293458733472042843060169705930999639376378777580575929744098536160415433556990316455595795400313647575972960226729647966534381897815815991001198933496310713509802061302905296200382604747872965936009754110919367195433355177945358117562522612456717294004972422895241128418791012978090879140241849417526, 93553755189848638012338659673375584674936537808684571609487412905706716483228418613013789748317669757894115018213499797793404634381651389489512661174720561406135293458733472042843060169705930999639376378777580575929744098536160415433556990316455595795400313647575972960226729647966534381897815815991001198933496310713509802061302905296200382604747872965936009754110919367195433355177945358117562522612456717294004972422895241128418791012978090879140241849417526), (184510633038099406858956417635943396922590857237175523642001327509891353416255293084578166686973789957077190675799631401432300825454687488017918044145877996013035524689844354906045733347607151440542688057333682640295940109569923497246207594212902136998336940375363222476373069218386302755276499759241827028623144323095168818810208674972774122692603875611617383128222690321022271261186126543228038354424565029840074447480526443467222763383247598908662994685106512, 184510633038099406858956417635943396922590857237175523642001327509891353416255293084578166686973789957077190675799631401432300825454687488017918044145877996013035524689844354906045733347607151440542688057333682640295940109569923497246207594212902136998336940375363222476373069218386302755276499759241827028623144323095168818810208674972774122692603875611617383128222690321022271261186126543228038354424565029840074447480526443467222763383247598908662994685106512), (313883761387729108558174484702252649459258651206438896466515457482468530320785916446291015373146172791315362138351005614454784436667178728533999727387760716081845947501254901551155035834083915744717444113533667885439078608445190007840835606881796124027430178877129784262818087155125275528639154591625279482643060100086478467884856351359437787098097639648634877701310797078341980502597006950898536067898450186030900315532650792285412195341790996491089294018240094, 313883761387729108558174484702252649459258651206438896466515457482468530320785916446291015373146172791315362138351005614454784436667178728533999727387760716081845947501254901551155035834083915744717444113533667885439078608445190007840835606881796124027430178877129784262818087155125275528639154591625279482643060100086478467884856351359437787098097639648634877701310797078341980502597006950898536067898450186030900315532650792285412195341790996491089294018240094), (432289976018011261169940914026286643724217241728113443854410655957833312784169019104703382315574541755613817445952545698570002622394837049878738307214629674390935447174219790649543692804798912194307486896996502004641900073129517192946146623163333951920679961268919005354577918770046216960570977454042401738554843406917629658200330083540748963332777292104807234044328378010653973986824475292571339654343836244655304163043082001668629418200524116272411148381021801, 432289976018011261169940914026286643724217241728113443854410655957833312784169019104703382315574541755613817445952545698570002622394837049878738307214629674390935447174219790649543692804798912194307486896996502004641900073129517192946146623163333951920679961268919005354577918770046216960570977454042401738554843406917629658200330083540748963332777292104807234044328378010653973986824475292571339654343836244655304163043082001668629418200524116272411148381021801), (618158432649215355948633931201562976809079772575047647914588322612699747936374934270034296249155890769734707271491530013347410450723728713367116552007135893769147086347955333917141754054987632400673784451780034601560128239866091696239418459339456103095519885250496913376311643354859718458060771621797859575366922600264108412250346108990174575340815346972796716590401890465600084704639308315157642480366216712413381404821021537458652708146078981055368558548732434, 618158432649215355948633931201562976809079772575047647914588322612699747936374934270034296249155890769734707271491530013347410450723728713367116552007135893769147086347955333917141754054987632400673784451780034601560128239866091696239418459339456103095519885250496913376311643354859718458060771621797859575366922600264108412250346108990174575340815346972796716590401890465600084704639308315157642480366216712413381404821021537458652708146078981055368558548732434), (566966315699108376067894219336421868529938074262835240641117393019836090984613770075894960107500630648588180066544474084271734354595637412926613720007333886581872374093907682034226230075113500073402344327160659368147743109130627909804364370989437199913093839732226744550825434759465245438329303417762531567688594131676469015525816833608495549904196618172883384257929085501664893381406185596501446024154130484665576034715298505774816415529495383500195107667134530, 566966315699108376067894219336421868529938074262835240641117393019836090984613770075894960107500630648588180066544474084271734354595637412926613720007333886581872374093907682034226230075113500073402344327160659368147743109130627909804364370989437199913093839732226744550825434759465245438329303417762531567688594131676469015525816833608495549904196618172883384257929085501664893381406185596501446024154130484665576034715298505774816415529495383500195107667134530), (613082542048157489987244252914908235828611899179532954548493134616684026723413802329657817977358011499997896390184618670935416183611348776862993234015367962619119639260535567414347893856449773636274084302317950882798612449245494584318202033567743178528433345156763522028407296985262610447841539573236979642192580362712929410858170246984941735636702565075326603954622586185465679401329717090522503811518290862733106525279850341497585045043973452779854131961506915, 613082542048157489987244252914908235828611899179532954548493134616684026723413802329657817977358011499997896390184618670935416183611348776862993234015367962619119639260535567414347893856449773636274084302317950882798612449245494584318202033567743178528433345156763522028407296985262610447841539573236979642192580362712929410858170246984941735636702565075326603954622586185465679401329717090522503811518290862733106525279850341497585045043973452779854131961506915), (525841027784276028809507848310553779525532108024075555182069209257741149801295348490272732998385577844271805651387662889546182522249137217058832036420315503070879148469788567920962604903510714321404295114318395332591815304902927498957255323000880123016444082148341474030897094552347170969223159983259774154918796421755476513807043693990065692044259469642127533172845636845785917559875210495962156905370087447393965018725320626384818921449679421747544241165264892, 525841027784276028809507848310553779525532108024075555182069209257741149801295348490272732998385577844271805651387662889546182522249137217058832036420315503070879148469788567920962604903510714321404295114318395332591815304902927498957255323000880123016444082148341474030897094552347170969223159983259774154918796421755476513807043693990065692044259469642127533172845636845785917559875210495962156905370087447393965018725320626384818921449679421747544241165264892)]
listMod = [1324402492843207866561425832097409112743122617938013099297275876188193530249596156061072959979811126050357866352331129505194587096872891224777496523262295761681761406312202563123737465753323798244443518331534601462866069765093900021668683066079373844146037391824358516735469380300397702121329477901465888001362781455324022425752503895988398134877432762866650425271137315743997066508844296088669069734652110450832743547876580696199239532070617397068880921609564827, 866633907947364670100754027743232325438049218410965998522474394342437880625593365404619028807423160365120462863563178979105797285451363516595260151370407771007563291845554786908111095807136792427558918891631430790171425935360914995634593335402679898758833654636625417457062312732770827471401019070437208900237548290477374960264493373343770691486920042143457776685634123916748457546008894562465042952047130368335733370692718068427526324522419483045627121801841797, 877770228212254034064730821063244518021437151922451124142458101632057269201893009821817373884544345442037040205134578716213478004870127849932737387514795009026675660082067694870091281237690090911394763896773497182618050259636105736126125612464697466121831973128040965035860918139939305423347821304097892160591251133470571275671437558270579841970371075402799893546189313620263468302080159931686634769676649668154075834995860083191681473566665181570653634650262463, 806671414905661003805740511486115776294259411236973992431897439163454303132713723495196511102826935764118673859513360578065977783766408878776641656861636607136088400921165199158155872667259280766455157612136853933127827651605373637447511088692958871011221254880786259987368604034959259876930387659926832965905873610394263504497326189594365641711284992816994302251546142315850058338479502826033457723677733697085581010374510981735145537499036813580742033501958767, 803127886388603694110706145219936598026327803928723268030509854580505609573968785475707997810804897088412471280786273674895553928692190377055866038066119713287082434307885661429369782483573372797794405603427760689717044605676626821858944838867150997904270334501759987500217244780708906991995535347179566195472934757995051048376881028374954961086301116685867491543255211328061839425759046128427103651249759624337721523436021941650679597538224361268930492435320439, 973663257537876233821660944266348973437010252140686420329805815628665335522543082604961527100986891656068740778686235707542468040566994179334798957472277013680946376595550035470784798352116049223501188102654963969358780951287830171195074029148702488301816225485573557058593043055361008554498773583512319663800856041517832260743030148831293334778123393172462624852284201076630984420508572742656150323421211684037685460530351499962218818459749114258915606636554999]
pk = (e, N)
rsa = 141204424870829329443866715848746153690342235983374570289144738712150822811932537323312196146311281709159340410068376829689640624192557133341110516093265660510567583864690731696694441416721797208638892282230141231243410799511582688393681499526865659432688862856921957682891893182158446717796136818980346882647341519631848124941704393180498469857216332672958690372866037882810947165118890291044356610275866673164866597002284998532914756933556395468060602388153888
pwd = b'^\xdf\xc1UWr\xdc\xc9'

In [4]:
PGP(listX, listMod, pk, rsa, pwd)

'trashy38'

In [5]:
#Submission params
N = 1079125639777997072304940783481668580166893346462343679540015172306296500356269749293855964098133740864114610558027592759590605645667111623913500733144655953569269560718126915457853300293871817017653260217436660012117190556122017459475128245954393688736456182601291807956229581723292120604200600082515717199150845042948029961651890683956361923828698614198763114531056685366303030861977129534480108028625480400866455926459669854305819626492231239648194160270748653
e =  3
pk = (e, N)
listMod = [862100532584019625871560248636287566345996172471800454688765650513697739674590790317889689411971710153089736606608191349938799644916218254264057060081031441825318080240956918190315731980350703885717498360925909355225988256930237778613638661175604875897417407940559756447766905863662659297327047359411274244219354775603873785185521856973228857761686100085593913542597494086124699684564917552921604543723156737672361253143173557405957975185861063575826169280946539, 827284894288536865895776099291714839716958039245022932351276202842812753491400381441537589988662434340414987889132855039255388644380124107212356657712599999491647616781818288579188169074453869588132502759571003064114234076091069104403312764343410693045519811993651378942631251238605647174396771435646061442187065402867306203816035423593595974517324382163151566436134135304569689824345172366728000440713675085414386822523846695101611398679314601853077258159680529, 2173183074281411402864063455283647942167764961388881418093096495988926061576642310061694229314698920977596240000556727537753308563797733464257411714614092905426764428539252101818522102854266766883959991406441933829026743682859525629851682267930949043088305310182411791914740334897037685193791185710646212895199638953069290071566323547560004536347628889649929855037725023528341974683726546020505330397807557513698828882516832970296103973152417836927559796322245497, 867729829989562339110157940741333729999197130725685597017391951865122376460803036830799843749839146022855598658512249821392176669532473030266524322318357656451600503534433879352378661358881414669149212616779338361873070614855743562700226930957711684902291613815127489765974184740884370892167724076109556215282983444820720876901154050471565236351850491796217044307571945999779825824831510139661894401877986979227662383593010987655507028130941714269191289196827027, 1260433499704474228531792084151155492821321725377884397599224586212206239093568179677807309663894901769473364408395557339899953652497903561804856328850330643555815634798447643870634161219608936835310343190767408573238938641828791631685112329088866487263149349771094296000306715232708164733342789468136880673603753474596289433825917712917560939905348900942328997169932219906434364222875443002971779138065696817317399101521849961772882000851012099833485057864496807, 376893432335857004669560332124637538477652394318727587302109691527824882103749638725656475573923568743945916693049396781173823566446303427889502982284911699849743434065893048749969065071482201110227145125775485945209575494294797800117128428355673619667471264116646077383441431893211242140206259071551308168348331473308901244796296182908997754506998523295741788372985884294008288152065861184759706696969865547160797271796825611750374800867204108303156546654065941]
listX =  [(1044299057871373315669713603780807916564871120912747782284738359548793297508314237069338778783979652274617547067279325605206365661617927421953853217335328108531191038056398959605140668677627798758329877908028452585381498217043613701603369488053363373256759382055361484619059165917563497575972957953055697012839214841367931076681614266177726411662004629687125563839019983144715365893587682696919677438895000609232692263574919618983385468398714273970895365759739714, 1044299057871373315669713603780807916564871120912747782284738359548793297508314237069338778783979652274617547067279325605206365661617927421953853217335328108531191038056398959605140668677627798758329877908028452585381498217043613701603369488053363373256759382055361484619059165917563497575972957953055697012839214841367931076681614266177726411662004629687125563839019983144715365893587682696919677438895000609232692263574919618983385468398714273970895365759739714), (367446925612879142174100384532087106410407671200829266667549440622340631070330823176723939698259638629638600439412676136075938701200277536445692461698218982805846685868138776196982321961815042010926578796002705092469644104795269672946128035688781151158095758897878074276675998263950085421424693269310020221317614866783632419443287636683852767176202772795053654197083495013439806179542883892012212888413720765726451136254981427321580792853120517406718532274252967, 367446925612879142174100384532087106410407671200829266667549440622340631070330823176723939698259638629638600439412676136075938701200277536445692461698218982805846685868138776196982321961815042010926578796002705092469644104795269672946128035688781151158095758897878074276675998263950085421424693269310020221317614866783632419443287636683852767176202772795053654197083495013439806179542883892012212888413720765726451136254981427321580792853120517406718532274252967), (179479257313326940246951187224016707798715876333988500128299546594155461266793196811105424311984391715366116314820910430793093632436590921113830521236795217665900070512137054987084722112412910885491948502729658871780842412535625346058689543392463676817609083713079936066951835925102564248908155028290361615959541098862010729240752679720111191921789140693410786621813034318580938321979319232463513308825411559368094754721624987474619989572422239029171092514490126, 179479257313326940246951187224016707798715876333988500128299546594155461266793196811105424311984391715366116314820910430793093632436590921113830521236795217665900070512137054987084722112412910885491948502729658871780842412535625346058689543392463676817609083713079936066951835925102564248908155028290361615959541098862010729240752679720111191921789140693410786621813034318580938321979319232463513308825411559368094754721624987474619989572422239029171092514490126), (350830443451863416874487057565441337455157197482233145278140619699334037691389426297377678547428530263582891678349176195848142304450455187243980686876998334571853477302117082864877874313425336411067530053126855926304380063107818742044313894973552210808489282344083395123491829998721601825957230245212019172442328883015072080238633887854148783275018145839849554188373496779507926873811906441715791022780924170269924686961067813757373421376366679697931204042625771, 350830443451863416874487057565441337455157197482233145278140619699334037691389426297377678547428530263582891678349176195848142304450455187243980686876998334571853477302117082864877874313425336411067530053126855926304380063107818742044313894973552210808489282344083395123491829998721601825957230245212019172442328883015072080238633887854148783275018145839849554188373496779507926873811906441715791022780924170269924686961067813757373421376366679697931204042625771), (1075426929103101777923412489492622465437471171136735625809458994202765547053240252715464736572994763823994034957431929746574669918032956135199851732140644424383638689773575801005688305769584326102167182867025734038523175140865571025091294828206148727401014871146341661368354071376742058905363464768864287214750442784575423693847743616460008680599521881988235274286396336011142375706580758405963653351238473449469406829466284204885413489631102492732442659899620023, 1075426929103101777923412489492622465437471171136735625809458994202765547053240252715464736572994763823994034957431929746574669918032956135199851732140644424383638689773575801005688305769584326102167182867025734038523175140865571025091294828206148727401014871146341661368354071376742058905363464768864287214750442784575423693847743616460008680599521881988235274286396336011142375706580758405963653351238473449469406829466284204885413489631102492732442659899620023), (324477279667536469875498597034832144599004616610370303305387277989412813478690233847063974714717773484382941935780189610242456439009311984433566665263301265898744480893039179564080416469527022933040594807807698525480159610883194233889532041416905781350673313812450989580501484164282269348071323655918433023146130264444920876203024920293873586236637769900045735379509363070412544424089281234277473904577082354881509979648784698056449722392888482084233315318441245, 970308419197815825530277661917367302869051037154841028293853609200285588851940841442964616169099260107440659125265544324075033649820309325794312700950126248530830966394906166998460231006221414937533232574466533332325162323517244554601863714709538659662889705087416150068396900906851327832562539710756453022179125356162688117705069272575464879179346998755034607619623279959106981106449754923374025209966307537440503659339160741082488609310346057150687720405824649), (700610055113954102278388575360988186822202793736961756893015980277763092072585466984203924542073575005197067608816753478650145073023152951070959900457701469365618105835235488558526731744693179161725337719424582605530451444233703682748893927643924503891688392374798110260594997867746491128470722670729144612749400677743893912237711122320207916337264180158984993050377086166050523218916333337932584867172062536936659580062496376725031761947853797616144722951150366, 700610055113954102278388575360988186822202793736961756893015980277763092072585466984203924542073575005197067608816753478650145073023152951070959900457701469365618105835235488558526731744693179161725337719424582605530451444233703682748893927643924503891688392374798110260594997867746491128470722670729144612749400677743893912237711122320207916337264180158984993050377086166050523218916333337932584867172062536936659580062496376725031761947853797616144722951150366), (312561202964174641418374249372795866312316418339033659616479811293396545784222831940693521783320210142928831041876567191311299441145114267518727337262760202592556647080312362043257528912222210286414689961205171501686651169667562396470594737656674526557756537592537836629538026756087332492162127459195284356673169544035592756521817716020372218714204538501067660432300212823767390300769182756201979575187905772575789782269131854660433145491725403868561062714413119, 312561202964174641418374249372795866312316418339033659616479811293396545784222831940693521783320210142928831041876567191311299441145114267518727337262760202592556647080312362043257528912222210286414689961205171501686651169667562396470594737656674526557756537592537836629538026756087332492162127459195284356673169544035592756521817716020372218714204538501067660432300212823767390300769182756201979575187905772575789782269131854660433145491725403868561062714413119), (622788707473679844188173666368668266768194517807283875079925960920516518088614839289124677547039111152668156781255517623461957177792701018271802927822364331093048977484538472814020832835174586360502148365840414738115771754398160538798376896828929082345126593482445397205729516283894810955907826121886476900986637724501271212964946811274729161703485995257474968956574412031824737601151649061130103143185125101136318055146117862303727632716825753165818752548666355, 622788707473679844188173666368668266768194517807283875079925960920516518088614839289124677547039111152668156781255517623461957177792701018271802927822364331093048977484538472814020832835174586360502148365840414738115771754398160538798376896828929082345126593482445397205729516283894810955907826121886476900986637724501271212964946811274729161703485995257474968956574412031824737601151649061130103143185125101136318055146117862303727632716825753165818752548666355), (462541493345288574323110370778401670387750658261960932716220000303819436196635523329312607313241177909715173198140876367872636253601650563354875432980906739381092902845524226614338677547041606152583076702139783222436888486516598329348338889370752645941372401971664101252079118484815254446897181630184038636947086826606355106575232313252319532909525342081802139824275932492257255845608730620304852036805187603201902859315150208921097170650014204512950089294541049, 462541493345288574323110370778401670387750658261960932716220000303819436196635523329312607313241177909715173198140876367872636253601650563354875432980906739381092902845524226614338677547041606152583076702139783222436888486516598329348338889370752645941372401971664101252079118484815254446897181630184038636947086826606355106575232313252319532909525342081802139824275932492257255845608730620304852036805187603201902859315150208921097170650014204512950089294541049), (269751085029842154579391377576729221408674199933919677713030972387654512756522194606388160519794287478116728889773181513670877228042725387380147173545133598855182804159910316853667683457412103356864367106041126170583043425574133068466823867249524288465707594220420571022413841711097928307630514872073214970306262517604781038904104029166702530345056670710807777416764416269284637603323889760794541693921219527088268977770723269115338771873269192218819903315490366, 269751085029842154579391377576729221408674199933919677713030972387654512756522194606388160519794287478116728889773181513670877228042725387380147173545133598855182804159910316853667683457412103356864367106041126170583043425574133068466823867249524288465707594220420571022413841711097928307630514872073214970306262517604781038904104029166702530345056670710807777416764416269284637603323889760794541693921219527088268977770723269115338771873269192218819903315490366), (607051466971453361717843294477876336459501177678651430132780649527991408179554522005277271540750171786532975768055159866507307300203991916279396663856045603775465163211452592310824386477128726973265144860348223173950890410418842992138042305891264697250650950935019138483881089702417036060913237718924542608832015585406754405848556397471643136645324482606810061366813834103219343626973097068162101546175950720096995805438076173702267152120592596610090416847921113, 607051466971453361717843294477876336459501177678651430132780649527991408179554522005277271540750171786532975768055159866507307300203991916279396663856045603775465163211452592310824386477128726973265144860348223173950890410418842992138042305891264697250650950935019138483881089702417036060913237718924542608832015585406754405848556397471643136645324482606810061366813834103219343626973097068162101546175950720096995805438076173702267152120592596610090416847921113), (666814792968229544130658575746354028277594984383352283866170790381389451431239246695919269302449857155511828574819887406602436131774272712069705603089432977497088945176745656298882775123694434051306244793307356105201331065419349942628546399357857341748341802813524113423133596570348437914792665859592722844219171679036015658714118925481006839406331477662760306413077813848559085845151769495897031430911960186441308642919830878839294571621840525220662500698540792, 666814792968229544130658575746354028277594984383352283866170790381389451431239246695919269302449857155511828574819887406602436131774272712069705603089432977497088945176745656298882775123694434051306244793307356105201331065419349942628546399357857341748341802813524113423133596570348437914792665859592722844219171679036015658714118925481006839406331477662760306413077813848559085845151769495897031430911960186441308642919830878839294571621840525220662500698540792), (627287120329878427730404750379997283736231287020564030093572651263056295626687038637754821023770444746130903464678434339713001401127855757015571424439485612606961844945641465549109639012242069703481742108490150457990773749634162084571417270944941566004532380653743854535276204586456982714628030443888742221751659148511254113218491349823473147069651417661070729081968691964220079203136912933146612785172172220254434420103717065685049095834822764952485385468936653, 627287120329878427730404750379997283736231287020564030093572651263056295626687038637754821023770444746130903464678434339713001401127855757015571424439485612606961844945641465549109639012242069703481742108490150457990773749634162084571417270944941566004532380653743854535276204586456982714628030443888742221751659148511254113218491349823473147069651417661070729081968691964220079203136912933146612785172172220254434420103717065685049095834822764952485385468936653), (537301480999707372770464071760070660497802907492964642419937607233103299851446188129196870214723607523893597427110747010846691103535603365882662645857715058250647570395369699873124024668810175393609852815607410033182566005012755898147995434577275314015361756193227156417734742291510275024767922025389144791280120706343278630072065218508420307114659723984833380689367297611554123864010325747387469719601478929383534068818071690183596004017474096390842156253431268, 537301480999707372770464071760070660497802907492964642419937607233103299851446188129196870214723607523893597427110747010846691103535603365882662645857715058250647570395369699873124024668810175393609852815607410033182566005012755898147995434577275314015361756193227156417734742291510275024767922025389144791280120706343278630072065218508420307114659723984833380689367297611554123864010325747387469719601478929383534068818071690183596004017474096390842156253431268), (14122729555595290927585613935052290344761419909036757963465166059854125351453407276091270389962348656180678094841366814619049606436046536141038362906696840379117388757573264598919101021390439360740762977860997773762671184340338841311632909972159698863851119039763009106110778055859708448819880361187552418623376682558455366777846829039491757936221972063194066720753578996066265259590693444075455030303802883433281852321503541748529441747155690128200085580086236, 14122729555595290927585613935052290344761419909036757963465166059854125351453407276091270389962348656180678094841366814619049606436046536141038362906696840379117388757573264598919101021390439360740762977860997773762671184340338841311632909972159698863851119039763009106110778055859708448819880361187552418623376682558455366777846829039491757936221972063194066720753578996066265259590693444075455030303802883433281852321503541748529441747155690128200085580086236), (364834551113120498088739443914736603787656380557294227416503277375025610623756978003041885788210518883231910912510576582665764088953740816697284357097228117292584667197447583647176278583447322749667293795512428358295564885398994270167480535174661246741089324524175325866791811983975712478991009634564518688656559266758326998008082349073716393317123056445160015996602556287728627169243088654856701898603188632087486086575448085544953159794186436851804519745110230, 364834551113120498088739443914736603787656380557294227416503277375025610623756978003041885788210518883231910912510576582665764088953740816697284357097228117292584667197447583647176278583447322749667293795512428358295564885398994270167480535174661246741089324524175325866791811983975712478991009634564518688656559266758326998008082349073716393317123056445160015996602556287728627169243088654856701898603188632087486086575448085544953159794186436851804519745110230), (94691476952886119368725363412170077540790528735272506433164898918365519346198389741356383500450006627732034452866252369688238828921413255571099024012657747170211687066075399733527989957665594045615362214973706839187525230584160695687073339999964388021249462628013292091238156239945215104622928793355596813720998879719667050074633465406183301562349464336155708229260902297953101717543481720383528235533343987380593614839331578618956696695095779794305103763531650, 94691476952886119368725363412170077540790528735272506433164898918365519346198389741356383500450006627732034452866252369688238828921413255571099024012657747170211687066075399733527989957665594045615362214973706839187525230584160695687073339999964388021249462628013292091238156239945215104622928793355596813720998879719667050074633465406183301562349464336155708229260902297953101717543481720383528235533343987380593614839331578618956696695095779794305103763531650), (79938425006442969318643645899570573512587392535282936632136556701194393837165633836538560001331328953419678961744165419771212473303687061703347766002439471972346439875683635559097155885386284583503121533169858371339783607390469949703835851376737623776784653587012499766011166408475921436474536416417204436839877681891423041990946847319549197307764362212053942672327193210750928542471563780497704943082790447349492257809274535141374288506274458482951336222325602, 79938425006442969318643645899570573512587392535282936632136556701194393837165633836538560001331328953419678961744165419771212473303687061703347766002439471972346439875683635559097155885386284583503121533169858371339783607390469949703835851376737623776784653587012499766011166408475921436474536416417204436839877681891423041990946847319549197307764362212053942672327193210750928542471563780497704943082790447349492257809274535141374288506274458482951336222325602), (660888778251144274718273522914594907720459862510380071316178628581664689023854614501575467689728426161879088038910740999419095422482522788668934102084989102053492106974702940119067743490948466631039228371623657838780185171544908919429898439496318549644012538181396118997529514131816337403711994589978790153294001689456664851909345985357725892638284656057868507410586424946806921007163965457325758715246553876086500166167025400179580268483697699880873759967686570, 660888778251144274718273522914594907720459862510380071316178628581664689023854614501575467689728426161879088038910740999419095422482522788668934102084989102053492106974702940119067743490948466631039228371623657838780185171544908919429898439496318549644012538181396118997529514131816337403711994589978790153294001689456664851909345985357725892638284656057868507410586424946806921007163965457325758715246553876086500166167025400179580268483697699880873759967686570), (307914291666976642551336705940279214328704061471439557250509409415201579323157678877677480719920666387697416879298858707749014588294162297846333395802806174250441906433141777918226694822196873093866303121486323254764859954334446971594354436341969447797367234050993271776913783576362598584144682784776386516482696270795688088488475880602994027639111799390697327542756626544901337857832935721083168844470429956845776450612169184854613867424213009040099627311361357, 307914291666976642551336705940279214328704061471439557250509409415201579323157678877677480719920666387697416879298858707749014588294162297846333395802806174250441906433141777918226694822196873093866303121486323254764859954334446971594354436341969447797367234050993271776913783576362598584144682784776386516482696270795688088488475880602994027639111799390697327542756626544901337857832935721083168844470429956845776450612169184854613867424213009040099627311361357)]
rsa = 736697082354471852889164407164923259257451210939854019873349247012270599852687699177670036632992886301181557866369323819023952232561064113373657033775043448413576585054729026537859709213869999774187342888071196849746991165625965609280615916253865003065860911123090198878649988744184401521325636433072134448858376055294884287914684814289067645144251322087635337306662363739949814257655362296796740218349287642224362467208417901768948610540995409341145450188264139
pwd = b'U:\xa4\x9c<m\x1a\xca1\x96'



In [6]:
PGP(listX, listMod, pk, rsa, pwd)

'liveried14'

# Exercise: 2 Come Together, Right Now, Over Me

In [7]:
# Håstad's broadcast attack.
def broadcastRSA(k, G, ciphertexts):
    tab_e = [e for N, e in G]
    tab_e = filter(lambda a: a <= 2**16+1 and is_prime(a), tab_e)
    tab_e = [x for x in tab_e]
    min_e = min(tab_e)
    index_tab_e = [i for i, elem in enumerate(G) if elem[1] == min_e]
    small_cy = [ciphertexts[i] for i in index_tab_e]
    small_N = [G[i][0] for i in index_tab_e]
    
    #CRT
    N = functools.reduce(lambda a,b:a*b, small_N)
    res = 0
    for n, a in zip(small_N, small_cy):
        N_bis = N//n
        d, u, v = xgcd(n, N_bis)
        res += a*v*N_bis
    res %= N
    #Computing the min_e th root of the result of the CRT.
    res_root = res^(1/min_e)
    return res_root

In [8]:
#Training values
k =  813
G = [(138143979417508556295604335514394117560507488009414917932783981163910057756594716004137785715760419834613824572023919768870227613736091468245587773958670949309857681768773525389549957211701869383177213162531338959932494120415802951083910460649294907414533341586256681900210078675714850935686387256468286607169, 1680279430294380507094669383264114383728499426347678223045822922939885617939), (112626059750446163922675612789269038696660287408072611405245343580144741444523994475120343118926492833401639626963101659738158686322410497390849737317510978988556854625176310682277792096499639242580522824905937587229452438632780962243235670101236276737932887501167720292227663849425811958081393016827763327417, 1649), (146499356690652748078861490731135663658502380136206166155784922775878934872145136289836697803671503909504377677744622133351429427788632917809287165507711694943985906912799697120088929239029496315649736195578981974149876694983586962781215070554277196683969008016448028037382590344764128337155731161236774096729, 1688639519640154108645201769229043060854998988287694510699367732939838789237), (96994896779724272001273296435747999984186008917570480949127333448208330247426712834864092553401673756380372586773493109312894777267949688736394927516296518429476719950113159971393495110403161736419378322639987246686886531574139345820181609773565442833596023317141975848114730664080490394044008710375293039151, 1537), (142610829731775665050136411395903613790541456764498158179572308449502625630249393139222377810842753937267645240172914080675748935909678275795187558675805728072824512215106469919116143618853618017783794994519951452153161031983818135005191372812292606082966397285761141379270161455206911792926482085714868349947, 1643596750415032586887482753364612925510014831160125208494073158393720945661), (144538126290626361764768193398470758458423859669120286313106783243892974102433564733817365561776024583665978268650598862812539283088937514794096298700922963311315977799695585054338557522955968988796716265361379571027535241905352273030887603816364637872749282765346662910232237116061324227738119984475292804959, 344512411528702149333892575497364101605521979473479234826887159730577056957), (97160991081275371525213246807669837925866411382197325310323581858412723845358925030456944298536712885391689834127862536213960895089398763113988490362509189712303546427217407712037984744687003368538254121072690834625858600508328579146390711773881143743883751679333783952228774501806412113731523259288676602361, 271), (128642680518105526712059417695589467621049525184562439411012873777349862102327152977980160820940799900726486502296710035686742738526154129307310561378048519528828820481333508708719441862316800109354670224769299063818572372113243081381721759706135902927331713826806255301296489051217767954119113532815114993667, 1336711739523618833921480900469984536726489278406700533511081110122730285435), (98087887723018066419140228855955247322234550630605612175169380982741303786041734810172962311339369125495271125398803598876925291807857916210370506664162439138539775067068087759090608796318028177865982498020951890458487524055784778062125221308751891658487572223282354512253609516434599714619189515823817356037, 138978651604167240803567793970654045356604818747507552854655758075695034889), (90004727446250923041256112713242637587512337539414023231729134203955994971488226532074853002590900823073889776685098936440824650004480048401669102861174114915262566817282875356757947836118816086577558520625725606508323142625173107416504107061755363101912174348549208923158510969626013151873825792369499313469, 271), (139778671902830016006062571682537705325531909618033405442085285924311046534818864393564158872264749100105006948517477507356366949644404861646016098627102148311378616906629737622172308535186637842697017462925789018040659932980456114087401254121962174074641649415797863836953951750809931332837576721455101698789, 1308293772356041389546610165243617100993629640255077536372828858771600928641), (71716025670517085816132405850925527934187039800505299217462933611911649730492260979605239369277532925232244333690981919162026122495774503978744257344746786768708600457597693838269004000220942007194556619585285332604264944406221575998568424993252219839710113364044005558029978404424823534662038403494681044271, 1335976035873372168653172391564643420213137859091954597064715280219574198141), (124400030158813873355347117855291044633215073744105457811927961185389848257822864415250210769130011243459405539466221664326562729841165084036729959313913286051734023476128030638191777540633980228024592817263400145007526946476020923835153815598821334405362100196400407002556849677277813883530539505190506159161, 1288926069728245199586662784241934677426086739161518693970937936472596092791), (85898486168971602007457920397534773274417865585710530213343232771820772151349068441592226267357861499497354065939062974856769177726033805831524575528051073150775557970601844367607545188524457625150518144351954296628775931810845048844380530650437880155164549568850549962452150463888937506161961830193414748301, 903824520458694142617869799696886013720551638994291780902459993091562720819), (89944958850014783555286229210834777077580943566068707510769029896747281975476133212032139339908715854373509969772942575442183499828500440524615486632772606713658887046562072251458859062689603814834582549645990659872750178502666566082860897398722857599349460886547423024334970731401113443768202261500488838197, 1049418141325976878910341758970987420414694203422544354281250981211433214663), (120706212080015011538340695753665658621130111806549448135013505172045808231263799211070759635156174232326567384576049778258219961712130808603721197448705547743840050279999549298539186732473119561337308416127655313116160615585509773290426136364587270929000052333324983427095334856694296093026044391748467523651, 561766984383106197576211309389406424879488401191014441332926337875820691791), (121510847158278284463255718723169396307942925968584975314385254790120140760682042284615661481442216770183480970215117774216104134633057795932924854209608066832108188705286954426102481436460234175423644116191257290831410880715470652159025984349276933160563723576053365206326820709515847790585425519639024812753, 271), (93484826643538831234535937582551609931553433917652798422093727879463870744326490421336160779468771715738561837950966404215437624607776858757729482029319801708310226404735229861301506900660391702085802935475676997936274126881953538315768744634006889059275252424464872817763119241239991114253129605062236463297, 1279374560236208227841247233561824195179845108004172339699526994864823301149), (112721839985921025721060410739743271386198607289397102901734923585196370680798501751732583895179089006638911912211739844514164446720196152836981529817567732853072699194504832971921552162951963870257903827078856294174605486430206738241730644737252118945371319627942021932192691858142178392143702818000251899937, 1658477614254418259017746777837355934928345941529498760653421264968097961191), (133788657523464265383939970239316969264039241817440520746104716219612645342544701784109121643060341413717396344158104290774219060675326897037475259338457725213164989112009035424792243124571455442019572822344937405689307099519579380421698805917405916987819668464734829530344780363359805250061692356976101895689, 271), (72021434064713858932067067707789486947751232784182418817481230789897234987509544832513079673904356794993912250389938846134867693883243966333726781314486672498898200150960038970863811662631963611014233046097445032677617003666295023862352602621103975714693170347510225573713844419505112065719875727715418312843, 271), (82719335296925628390442513740872287241631842165731491419360214328756418708464028655172965996412660847238666246495155049723363684109373909849265767233715280735858563461042487904091712733584531066366694033570236411818543714109637699693808154935166615853121946586129408092466843855379286659943467759404538989787, 728110164414904252108244141443021258923458293837231360765193025212372936793), (63295895309566771659330237183856737319661167948591543714640150136988765083279707279607826245921541673225795745774831023262596027194468974598038347480647717274222881582142163379274947275676520405431880019245709907191372041081550147871987080369282859500673010703683729802066373114325315279443861800740413969397, 260503576412467621164901366514458321460874337131943472191057802419352500211), (154636138063489560970335888463470101161033934153322024300092052132460551525575357880929906576118835765667553889666774944208573023837613820114182372260398593042531073445477004614443806408782442983876594444176317086439536906472564424510216921108761608372419214340456208573978871879030214200386798275555192884403, 508360009926960949891187791489778493767174427918477387364470246546763955815), (140481678006037043379046118189139942196316831961862704822020249878427015481359092171891273793315414956974630961666713685292142046988742798418774716276610997973188962024362846515146420760121698997047579927781679126436472933998668191992356281476214031487539569394942632590365685728545602531271454959628235590417, 271), (58759617385260467783213965253424700931779893716670700535287970351923914996171646126674155563079005516245742472694532113352612821526253332836797568761687059946465128328196570362687562407744885261096086239183940685324026567643625187178280299836485203990255294920072620674636592254383288701129060590413412402717, 589163657187385131289829260915672032135819801882258080493035802254236964777), (61755766081095342933820295262718838342376556686186145147805484746035014569120398086529842694985188290909567447618814106591014799386179736239739780112816408853279205914403964009840538822022278056043419351478976499051865307237156851189975979647956743844266804352657419358180612246715844876876887715012260270547, 271), (79280014052456039670857463020547031053175966572230300344985003677862953705166984603015495727933376855457417320467716613667264671560766424728795501414596895104530009572306496676126778797698149911737382827812415047558393000008364623259939168315331577029115560953317348259559730662752588869828856885698563172197, 271), (156348554827975130804921024670565001964875950427658068455160661012620167838703523901714719596388256174163582025035487002961235408977135765104772074711663348800958654006128225017311068377613091654655864575029717595017999433193033580741706411100817258383959319539955558933992037441885133867991875911154282921321, 271), (99779274892847805255740338221142604308758433373640795933112032068691681044364002286453770846510901013525713523537679430962080811690633051063602072437797051627035820916570601986469704160674752254466075562221088426007981643604823856401691349574822190621184667039047857548730291516611844214294245009469662763949, 910234177890214178471345667011847779824182805379205137595010611347651165171), (115243436770666176287778323318868024295523143800955068515557927791783572064152615083698097378816046409276911048999026426142102266045114586871191091458002147214513281942482754969870896487356355680825953843249964892972793566281620124745470673535962470276444054925070161052699003258533865497382445768027652787287, 271), (69334452018810328968745306430803717326067946166300713649712862745594487301605822431490751517191749172463654125578706885834268440907304594127391029989235126765042579106412629565793877139615180675119797719555622761794641733974883704034728892583326728352766207559780377402779005555377551977054730166554492553267, 688837214469408453297896442203900978122570588551187568761531841825832695069), (69833877613180059193508872773961138907139351438498972140514713737971081041274514937214459698562672186659680957535389125484398898947679726444314723776848186242180784839189621006277614583399735642128872708634268046240301696643524519341302048061258239934282749157829898655278422073229256454693729895422554171811, 1075), (70107965277225878270242386326054878124966077462709649608638416261749559482300195997419067132837559468464517438843341362743857245669577637382974872928349598685875316185107557249964844069230034916606184772223773054550893158155409611890430575898659022415321906383474837210716557891662973223481455507856608597101, 742263449233737823002303194579281323127205949164643326524565127623027072561), (136738513654534286543728523900775283406710919030601623896158521027292643058980426844641155518021710441782090258690328664755854151231424038277204337990803668310139283809875345051329163802497887520466089893736675065849000818206122294064259897297387679388525942068930464717707587675248997207774496603305584964171, 271), (99464519653092395821881210716322267305158113701928036667892927261546325584955502430563219604530996020108723349852990329008033765948145932517937284875407778557102512781873374498497889173536598436315432215991605895298458796090791060218600440054078830679122229089877493653097814259036491801420539947117618093831, 396425395576213178368313967928885255067342807847809773210712963662337445883), (139303021453962342983880856118504093696211134895265024684035510671672504175925642632172965984580494831822358701175561232344451423001372180599304685306802387574716049163025337950195659505444592016808298616017555986851157866480462823909030809701272614545614029167280980401705393259676858396154802161897875183321, 271), (70385130390719686289235506202598098301739200924392296813292796583432315145715454615863812686698059976920617107187533438300510271525703353838715240181127148082203137536310251199235840643487862218690461430466243297249602746441088066865023411646287016028561139801294473777922973649627236583115575321481325865693, 271), (175481760065520090184638467610729824943568451287613605355988505109188833549142671050681953803729341228366674456826743365296505069894435918395858257011552095553838915288998037593267148565779160013748690815668956912956858917391409181040724728639133040536395913561027368033727597428317895917172964276463044164677, 38407114841253219207492731513627653312035771883450953939062899094914287433), (162606378405624128977483391779229336670494795807993755697580907830417115677774792181298528647692587040993740818264911993975575048497680261991588118524367615056530260050568933267460191535675437706985340175073962294852827399168496812168466518645498011999370650363983460628784230115556998152135568768793273740269, 271), (162784006894070308138914442895856625681861769719657391934319819467713225574642304293284499582107291575214965375515054885541083468471255056641103963568284040963276192101163331318898213116758172089255894281521908110297950934683720675019977056419805508962579303094877753055867677410421820771553472840984274472469, 111402183451001489744212826463039253889585903041070573162675995495079724983), (86799516993015385129907946419787696759225508339881706264174720018863799006085837412145807585192215248909702419202949091401484219684614774944065434173394105170167661643933366807121729449006738880592239232540213210409839574276139676929392399788575064758532866009929420193761216215981371601029535988021894369509, 89908172893821252708099163103381633512924691433820368419559538119245528107), (66318240706336795162263430474741622561125730754483679384696702705743140748832105759867548045598030753049706379147745336730100243960630614488358324554767994357121588344612270547097410130068522708153155399140953643513762605748879663618732424367623032268001348927402973002817721752262752049046427634941138676237, 1335940709675988025982022509361457820503274869526200693889925272772910913167), (57311722538038725968170841870759422205373269755194447992878163158451468190936058259585641530042178985255790302527871752747741233461308554366948821062288335849001188436929725037349746674591252516743506678495138113099989509559010482472032008087166232220303244121294031026742981189219436601181724870270416005617, 66477939959758247854833002238871116367355555202006111818648346422799313301), (124641855072070315755249188610375723159090260519856541591829335024759994208754847840007943863687770714292056364302309913350924534446181859671377767109871807691598843610208175180935940200399516253917527691841108212132964532156145039127564278951743272837713213070979550113386516045667252917805076392431932555577, 1614786944385542896144664826941357129203109949546844344742475264059686175277), (94950853577363529797025142436612650774855868516006718453963110517585130711147045075722716842673173640967983035014471974802458692690970373830024360474806005031805825609849370660382449862577426194195627112469927071840332603518016443911466585953392022399445574355776436436772907112782708259915686406700725165679, 271), (63314121361916593804805592564001263331755175331908636231028012155895853262520359148544959929736467822925857939345945346898123365116618783141868797499074607731632344138825411920719280246770076418054840725270116893323058098825748339069418822253353080335163976511962066826039021015204534632821749771088130057349, 1480178997656306461532267340186057565339446923655205782001616237120437985005), (93372967084383164012869496626443407469988131611386237886879871186475693103739802398992576865682669979513650681831811599185116526786399273140033745901597796321142602635212550849313930884562148754847885913963337416123453036887605841529049864138623897856335205603806155567284397311970816166942582342596725282031, 271), (108373294287397948389173018293617624039557018389138730012369272719947661343716037552227120381739641007816796838256607868875159131547538849642180533697886048376308733561500616360063911457581966769471076414801052204684030603607639250974290675872659176040741528440977127843688606702859667779099848148509960637719, 685774175491601413285494361032207630685370735437302700750196489861827021853), (104896390442982387620932035940984222280770915705584218172757035819606132225668218586124863274436926516615148251732713476313849569041886598352809405983879649427092871920368356437174193661907688129225087273198340714219525083797229604963162640791513096778708900322827005599362847150853491793823125469948028665957, 271), (105242722400342800615504370529992280220752765484563228011403541502540385176400701606861690786522860653132824124739114887850839840414045418192996611162901431891806982690516673017180983487159152066718394801930388485014233789022641725536554442954607057247070143642759805437278374634241603328482113709126098378769, 271), (122879083813198299733303883756088329300001170453023208785136490301940862924911629990224974059501011088130597405981812650762082847516140072443458770673835796738799497269235632180397396779407699687117681654293664646397534077195533266517551138975250069462492642002066932914962304399408723624443903309421187128161, 271), (89246071277763107264269026875406514276594806089359377485965434711273308258809217643975823293332592407589936857879101719808692617959981164744379948335356107358774955519817909961090885716189918830103017004198505579724679206550044695853923212482776380076911370875541734999571607926442339335940716900897021668321, 271), (130469941140234765298343418868541660461583917031291165208529119871841526352347202934356014648001015736530382296294559602900752408427015763600142352650233761604906966003047249687531182461258658582780317366034054749612815923958561713173137247440508906628007134689951470389231256783868952812055037487950741205917, 1723), (77501226437009338656023466265773901136984139865721001652964538433525581193893855746696265038315602534631157659758819755910945982750322400661552564824418876126381917276305160851557199403644814847750451366360525366711844922913782038193969301925680148058670190968309105231005499227191087564096307164086807922573, 271), (85176144114641872670043509388906209270928093390681072094808990419109638712431609881711497127155666600593919504371174548042345214240477475776084031986397205973975228815410948326777527334861616102157945283236607541652843460790422207454365811984283765733432803733535651124654355386144445728528929952495943367923, 1098850628920239913572235836978797332910859326809121243001762399603979757873), (73599827093127326661765448973817128305194404689982207300799965316659369029474548458561116130337960462991256801030763246821886643313886073105984530946638214080899799282227669182452399470337232963233727079090499731878380727428413048479049037711733935846784349035702765032151589313976620248478771095914426878581, 1296192995713694874726534038978882500300951513092203600060218289729864913485), (92824989925655130101318191715999020807489202219863083253912813464796752121558020231202105453241728458205651075097527056065268575771398004835897339900662005695964738569434939008719315590497258449116501848817874716449640832956900965941114631796527044858922596780501311380859434600338342806424749883149850155159, 271), (108663122626379368974488559611801346894450233274685947436403256299728630523327302624766253731998862275696079591809056963530240473543620103244287187813726542976469251971439646171687306738729208782878219903116085525100887638524789679526550150341386120355665836300309382316199185504284163705756854083018985498129, 1450368630316746268673711492184182239900049671716970124244739523513186701557), (76408482680937952413610138845854716623588817831507322646059562616810194114260190022492191403390374918297858935281694741344847397799884632179885430244079742903523011204239677726771768531928327641945850275414683268657496300932877887730116620998540005806042681287929635061407013963866525080449824484084468255527, 582259014644100469065446605188843600858688015935539897549163307708121610167), (155757486823362087898083488782538198580113518957887734520686978971191347814395420925348904974711315695012656448115374270778676449070476412381374770237661220195040166481682076315549886424397729979763232795615879104321278393418031573050525065797869169403888617267341713154131853507213661975652266258038903061703, 1424358617479719161978526608555094432464557519039374648551610712155967416889), (121404247660097414920804204536138346220160445624162875442560989858085135070819339424939575539855303239383993855696633870877528558281858133670909047427350475210102430091370814160119229239917187513476539695983381234456666034862943096778238679748179267429979899311458405779774723084618764973164944260401968051861, 1097826267888323557329342410908702518308032575659425126842775227358903339525), (68706525599288215004360891503897329359035030263053665821936763283125924859220931605473791222470927621342084460515451105971869128964536851163536160521230678138137986807168604824650802781540787772255199755179254163291520668961769675426215854765854865314678508874720425827788765710151862835574682262609274980551, 271), (79956277351721057312199957028916372624407361718428889341314029903935236373454090048401127022564839132709763351686466716046105707760590780238215921231567333461900461549752082583171726601422764866460001365932016847969394401421460300514498202820936159895665099795998896833356242802884299455701282185206234485007, 179900184614399926926885013228008562046991665816997390304603120601605192151), (116618076785195605423471498856110577985662908839327696175432811353610455952306923431317473976325366838198063987205291372343787245911142945096858659995116903360405824711787666412868787098740729163019511155683959065709359663512447514753943797681308843323898423963995411848649445344044080427290764301749449512729, 1421), (82104028584220938218959951057445880349904404640164096034027135341530277745279253507882341977238197671495992690063189277608408727416725407320967048874162711420947024149625512845482617134864679283894335810147959001762392152947034941538818392748541178085161172724915907009035938209992359127188700745498280398421, 1459068920299647410331241070459899262976442701583135776979173838861283445919), (134808675876052884602431169619714721482173839755280329371655205547932399333307096495182488972299604743557558649342518473915867890055120285676094194926687113763817323206622605446350383800629909190375254077436632740699274108387497285304467933295660169258825567339332674342678661136265077813708447945879317970059, 1517), (49113264155600892050301694535784129485259802212468469287765894140055523289024995956800555166282462915500736578504250323734425961159141008767155048179759831100773115097382259530032266938175789036313021197291850258887199420221072426046891596366323582979151469405094540998103386727604830007419323549782683253293, 997970217723268929905058254112651351505517813711140936268848041496354751021), (95058274496671093924941659532566451604323660781161904280125447154053055071557959683374227109471888438397933093100589512426576005025207612723406162679767497546867465260865999087632136869949000615192076126937774388472967342667862178662209564259854124295372694580534462827125323233921749339195932558025555370473, 444678968367024242423150757590515859848761121344898902547569687892534957283), (103059752076768521495445329947662871582946389382486196530730535124152780047559086623364593116385782403930755204918870627836673600572463214953013282533470280888413488307888635444213608920355770184115059035167975860350400682030758317768387338927818989755065532997793457914702577872043174625773182381960433960507, 544158278470412709413779031059950852568582751601126379156320455516924237063), (111667127197064509636121619771873610813929234456716123836329168470376826190523441142569505247767704218332516738667377995211521010758807379725181027081731273008752341211882303890416701735789548874691528296151903421407699676266235956074904178700201701495286727468197880304862241521654672263769663296238923459869, 1687614231683229712883771795234539373667788633445652118391685106856982592543), (86994069287848680699741065474533399431572462424499961355276115474875924824896028841179882306342451195580770869983401661687006511522304844899913662766512507963363526817683648832876259510884294965133577929059118809817315163301021988285831114047456542048169237848183290663241473975588063031953795115165618240697, 580313401753093338466873194271433241332179057936054296315214350977335230469), (90693042060645825878042230034585409202167547344432429321445173109280754553077186208550809968295387616868323522108748764215025501165518729021839305330458618959633087374667246815741394609798089840157836324194150550042343466673342313755233106298043950171197736868767090537657195128623604173769060145494093227187, 219010277735859347362878318315908983533597283087113046367325020204030749175), (91602319212011375449943849990623104987226228967339820093695567668271486046467410106096402647676683467568829039959565276095518903295153180068044591971907191046248112869676137067438071060837687118386134471687143950337729078581399300483549213679044275964100648059286670485688482956696946633782627345173866733733, 1663500871732301713945370767611145476251896368857242079799664410964554825557), (74042786160763209751091184659923011821684317254569919308320524866978580587962735207956864818313987349564415315243576316992608941229818288837535075559541963627838598757252479966013447110576096049926043809528920382235283746459765217232252503453935079288198141557869895176046499208214754876141035754967676466139, 1520107117869709367285257659561583909086531365157475009091842441663701955033), (109546429143004734418741503156802283850970038162914071234925057475081229564843664472718968032465455074876634039826098424043930927869845660169746071184678950226502732744977080155049698524788592171874184682031174655391836323367898690382959018170744878027007443190017527693081773438010466862839054972939919625321, 271), (103285694021873620067263859233420862638276380311566773369335057642043645426602799037364471625628015708446736176231322037120684846614843442191270592139056736761181045301868082129103054528441448323147629051114061960178775094800415951976365557612026155025227411711263504871573704538473778026305941592629522103219, 1613), (92515442417527657141871791812168332245040137016249384622027701644701439526682211120584650156758849508526410779432906207160373999465932812671314564473958441596338181775863098560572078383414000609675854221163975372819250985515738502623053379532144496535737315373956719864057899806698199086092587063659689016759, 206953936768244139555268205252212300357756237414513079512611457264191839471), (115515768970926097143134803086822546233770663821886922621460330158465551327926936553131231086066759167711440939576328041479359248686154226922302078763765073963128890565059318531762027332038798354178488728231175770929453921103202380760575369530615314565710865384184188501224820814909693729186595327525976748937, 1531605464172040206850317311130813887314203416137922213043690831392246873491), (65977167864562311329296549672171839049517863544914930534403134398780784907653042505142474785343896242255264359273809965167785001318458412037623405834081420419768685332128751558881192529762335257296560465490588235013661689486877048159172890234039704825250796584173188819378586725779121580827614877071834890793, 184917843049916622082523721886693199984295549121447438487742310528773335827), (107636370640539176007552871516204613090333639454610943456905843462559515912297268751959862925903220057805001356413633677229516884299859333877231723197048781545283093870627164247587417246373744412527442411283646698429737557849831358398445641925150250432039181455039235097817311947027081534545796949761075982071, 271), (136459743104221430896497852022047536611220933825256091222702105939428909605139166007157301851536216702752321120550960970608797631913173212464552340036069571530953581468214120160351280934011556965321927045857076891560079695476464850313260278421632079658656822523813919135415545512890274950788327173515346535047, 271), (75548095149628014725880699256643486629734720992145511034130914455975034913068002706338254042575621525357763203761840582760731813816522971888346915848942674359209273347889752186280354758154395530242919817144512955016273404464495398050114873892943740269660260765493993680973381257927615480666281014846025435617, 1272334018608856668453540432700724885495472131313189928252591667230207951969), (94518596438520080396440126794228645425117199455341891809615491082077866194543731708356942073164977472365502908061865443341717090604644521925277548114798420079006270101079019804681365118191748340128853989957061662730352962271935738522044624715751155124324357038490451865678588921956670969857800339612926183911, 94782258846477492653978384754827376756818437408768075776651215539250503297), (104049461808614963213681226300186922726998201935532982696786520756358492400894058071866798440123453453005471192293534265220041315506159726038451065744379309831426983241876690202377922922358113700322403153469648499162295760700220521440486146271757063885182028127471135994357065058547000923438186133425680130793, 1807), (104441352096631964426465315084205005112487018289556594226406781664231147622917698912592721165444587558116237764354810921572781316187637876800023183908756376472288359957852661661076734709630800550245799087349058582403590896231087337192673811144862478858665948910292825489017322284945138471900716245076592828487, 1505249637618824778583432453453066805179165549676618711428000414194601861987), (87591538785024719383263609485115972117626868624516364947432615973120477666875744004901300763964072858546522114368431672072357171408370574384459842981059298702976018376107268722719921708920248885284472531465019078049067740321095227229942154759044135958131966376384865295047712833230660337983594482153952415973, 765854574589424562381924039607432013238363729904576968754081513358800018013), (137398789838509188908530326055519958417791429822799718253236354606611004331623610612230515082610439994495319704701506070672828395666803501451818031166468665210695417163405240154690045922253149461413330298616642962356065785709389484841768714989715466528153261689670514239324383517438435490109779464453283771689, 271), (138097395983936348501709020243923461336127963913095424967329396602171583054549868309947647711630580894182506791671119596737583196418280737834572040325635920173682448208100662777418325105527362197108868228538780029883224372932620810641365983421830707566479653213410651427818152916978262524326487913365650056101, 278349246092801122582971355336099860659435244082390939878509132202036168027), (80672546955659000591256510348727134664170477071830180011061393835039183771981308528531669488604585812904706516528496024275503765689689497015136535533855257504777727865979422953075287515768852378281758092402170983823635490953939084266584699860283439155659565411821549951369339120639002846447491462722979242573, 1517461495201389345712339254795504835248790893651561196404201575095991612527), (133895416927793188436290773933772299513367885180516536132481224040054610332231671738835120774058419110152418416933886733255299863640954765422631993771681750293193791175148035969238114399465991030088925403361247887244856249923276254124583150263743839917087054251400657560505629389291334609016538598160124279149, 1200773244341162574339597381579067837278985506008750102103768062858530986715), (145419718803869506625781998976383609008858338070464491617588456527649741574098298289413055377427413692776489481254786652121064842065612488156249877546697007244398959505838048199262062905570511439409387265281878898821271508469161627130146016657862379940123197269987448586867794779765268250877022964503340415739, 346615007749324509291084531261524715557723964433462228863007361889457514101), (95668842110192903938234570823912090518816778105166099225991473776891469564359864731056308455069233144629731840870763572476896025690438584548478909027823828460349475409039352755124896147353826570862719664443411110025471936411103647726839451792696142513606757607357450344196467954241494146445689701061548379411, 1465644318883701104007389921331550550692592609072779903587165743445636346375), (103744572153664918574462966692842953594925307419175217936349694565220245025011539529386747947005729965308722131640982160839859086073568039174272686254561248529448634730312520493778784354121571077612476271270927812685050960858674467767089873150443385678695770158899928329492022151000907451320139266941593014541, 1099287272236367298277778845529017562917637495772707550286903285387964340937), (152107107536577355853902823542727570610266084722954555598222842654768070187397357882288193902499515603078080713950278210414230117312716292634987866362169173583537534527245699391101861977015652623758639990988467643544949137054890221418725363741461122868477520895799136824248968154810920906509159731591139474613, 271), (83505925929256351913118297107917837333697392667761356476345399445775793315261859886212291304086562502457382062206309495974160392774369386417448533048760621321632232571988401836781150025315718529020714355036270203408083086576784845598513897083799045582134188838140303530547936129773210396215143579900292444363, 271), (119336048678403870707592271485605175299857532504814010739798833553429396586727781926983033010719816769094369941794942796651302437881968291292858013979007259379547563568924827479473127164363055213973409495957072523047569074314837845287719738171117460766103180998656204122624429745839127544964434554015399570473, 678971171945333731623906467409590556756127701242434952610762100793968913001), (144210965249313979032228033278236986441175663348131056694752462551224486634154836181496832428621868123257557819338184689032371656608702439836362854020887209690812521153789028243290835110283447552151613253593757768360847843165185977980715934720236267968269558470930504993321591769515674285864870788391037739737, 271), (111842777510098682480803903836073983300063063856194552818328370306028254102822475416835262254180468203947852077357273567418293101217090823629861002462101590344846387345686096661871644116215635310838769055803261823202363012209940840388829158094333051886511378002731393455679447369794842958123912313202031802261, 863847483109230731913248836009459865278926903282030452137267783879435934467), (95622386551874896843930697038869444903771367057714586052249589956217949602519232257380713446161019355721978372363669930429481247647689352768425525718037845381021871608134694069394674257748465454572051039148066655126369245110884160387195353845586338696893256585088099583903425587335699913325028647767737557403, 1486676940284631431461602288841708078438011520563833075141533832515789058711), (114843276034666792137561771709503497245971546063343707558126853430891139930570563894732557962113592299281289319895633824202445847098427245283321164631258165969520205702887830989940696361496180729977533337659731646012378698027286219324459704629668259583920598444795348303722103013203934818133860126430397707023, 271), (82223568205398938782232512902683467945089645521293606667052496010429699214306266968562018288688455251824158486123893424709758230717504241175513321440785141680836959279582122416177624055288918234576932120513541674677858201863317706520275382136439314828650419991178521055981374641708264802121310701773934493401, 264465325128922391089201652768154342353838441878645223215194779951225030793), (151500178545378325292654948112931537839030115067786680004398900872154096505728344917034230431988183617474389642834919009735690410631397798655740564463309045231999414522435868175917173734015244528455315136510330500935226403879712722867811349449579719048831650263662833978155452407608953777926310906464959226273, 1363719279429868599344504886433263360246811765551835759620660212037629753599), (76895314214133883919522985072598008115775822590680557166963080430578418358069157579814265516928953534620329870623012678042933319893044987263375108300370895744697525996102648939604110174839315021249408595940259621715112960267850337339668026975862476515568006231710866830002594868212503895420891252163488739623, 1101061526628899168319558272563841252166309811316287502487524079745573988501), (133257736740188302995569363511459512790946192856066738113836130563249231040362749291033159970647456635111965052148006211810366860080657756776091841577702260172906636619973777073528128851662425312973465000673680578302403357610647323361706668861826280936344288049179672027775917053302579082561385519997126792279, 913346562881140819117386842298849688319645002251465041237591460049681553723), (81135131234669872950598566825188311460517512631680216614912453810593521228408737598475996656837852782334532254972933034856898490750051306551269404282237382792666682479880765532374777346296755098519479848388758458042670627899732606589701285924147143654529911968999433429081704587520580041353613195756006422943, 806859297598271069462683097210139982199372846633510943045185239529740373033), (98288156179523481122199813646112814118503945441991028427562048018445392090246709372804438754635653731612523862683020666214082941745755671488892028731496727497455743492291600304382648691123346723559500938477944424357957009097903817872798638527091784769523782070690170268242491951303874747053908001233500939097, 271), (94446946245002557435663874017735594539461032844246944362655616397987633183574807165275607034201386765413308593647278617277919555060490993313110888982098957807225034916935422220273012257032394676433842679579616575912466922804295147550014156644147374848442288694235100749060310379217332947279207684752854695727, 1235149220607010263819159028833412635214663976958656828055088932425934359825), (102905006388909745396646606089451403963266999333836108372817443405101972537431894426008502443353342999262131849100667682561996670509337845112329158652344057303006977426683983969828661379910108971093688332162715499325742957060888670669964301695240844798461132085155799894485002777627447632735529637802527751609, 279687536682596884996473047415752088558491395598795544675139516271484309305), (102960333385488123483559150877694519288275392421587086668296791895110004719309515477039821007629163460276059217124594010075978110408807622617745046110077616084961810755254543199092519615589367867515281781587652763396873881067994799062882777640498516767025416811581483548805271901469375846743622649010608507671, 843186995915063356174356523026830401921713372185794443592118483085985914773), (59333438622860984845810845489956294935774546317166469993323645509718600810285124504623303424840829373402099242579461689504217487844864251987991361618719376331343251806096045651629131591454738094522314517799589208888723326462318244528826219076015952005032078890556559793814217014266659444160517928629629244691, 1794793941707751665084749565317778587091700930915603157142747898571907038039), (62355165520664580117927982579411669824521510013155445146136262800925901965031617515403778483555532139795223300228387452221414391687905835705688232089307464961576704572011724215928773021123082231820166504019876604174716583108335915304675063268561400109914964820362603204671041219627969983483576594092540005617, 1215861532410349100362771160034623409317052045051981047418632197652287574987), (115562551760588561428986175002594330028687652767637642678948155274017176286843258854036150072112638215533846086264002001721590277441497944833320820275026064488820879111229222651821348479463577927267263343153952262024787687443714490269303236263470017640468080329262802714916308430535725708529715871065978356483, 271), (70491233583154742659567481084765102713618846388739041561702672839250251191606826343749172976530092958022356764413606727113416579471602582772255832764254133732206212476604329925585004859302558314221790003878532074356062275979518316548790813210825081885298963251947080716267086384508613762233882474361328722459, 555980693245722526302106874506130199114744192376336816936024962871952300515), (103191080699220506845371874156325313684111592801861912237945173275065513943520405109031454172182580187681683819163448960570445419158342716877524042324870035462732903453273413587694931418381352943700776274135990917244787114333397992441553593542650437237754944698925627721817573461427443288735519971353742515733, 509314950417545153376137432452456969190787607660948031303893557311514818011), (130774444491323067263567326755042670738551374157332087354413130843919391793892067788078060465538742167018277847666450581449648879599867792513306979105862890501300796508507611644125626267764835824958872045312278664020389575289693320879883349442188865986784949761104384408823329897765188326804065258272387321687, 149382877292557192364945079199938239173588167009137947665136737998828138599), (61827003391541245746106003129490705213308437949653257374719208216357943674354745702218832630825147299606166729084921833446216077816515167632273669737265092921258403865862281658357598115944212477694181190769017105547941790859705033354503693060068871089266271144812526007684409594425006189114761295092872737857, 271), (97921499587533707397372242287374777355932881831726355848797097084733953779221459072167373069413461453107333665483343816275026358138693009017775204325270959489776656729495813135198609456040389439407401865992618677206084509342478361249670876263264787900250850232286445884654884059700500897472148213087349566217, 1400000555416745615345155193966122617128774791717676488268328722182956014939), (108712409933335787928664803530236234351185832268011697772460784743253023869319103059595062226626477567503580278340773315485967247295285997574648450794116467222827373677400078441888395132013723425439194153261714686963833551838604400704863189049748676188812296538509415275945733842632504044465401228192001036159, 1319), (69737183944859651595536757205406761154246487008771984103163276917843679439985337775469690379029442103419690676999751632318141435534348181254229358257515134405232972796022634924595335821973587679126339752797935707955313541100506506967443610270008812051517463208982081243892335884828580498476951265086307526469, 1450181264684928135081315924178543332457640413830545784182982738850222668037), (101433891426689261490831187714942828400737656341479319038463039860999614961495775284285701655136817727859281138419497196243666430243425407835785290271677428922730770054387969240921638643869580345646600374629426247988166578631941237020154872874566905377022474786170500967787682315651559620405357820449517325371, 1871), (126869693827541737557978497350468801654304436064404830612508055487690772699628633909923173733569908556289898690728728499969613707032132394622887528538365351325944182118197148792748887484401964666845958004245625526032096920939053631562226108847410396108253791593958365601085646990978537197661769193626507830359, 271), (145662048369714161732192888358410114065308571147248663143559705228330315735651125588901386584373179276281660131398663728700652128371323166413119150004290526659402560051693621527901710330907584453917830911807082536294955686621589103458934106345145292312574300189989025593068939660512526219308847432016727100913, 1111), (88198341034925913414232290141941613045476038531785453967978484936788502682566142077231987131956527375691541560151551741116215749988794862524690373829664374039580442010548459011182352397243139697539270485762553262896454245564880268132236542023898501123324148623545983620514295140016694561989586917756787548133, 271), (122733284630259609930673052797252962059636422788714179453719043096026022959216307505270973142262412179893076025060229638035633513854310977764558090191101879199604018160006199293909669946061445779652525191138281949998147720826948511215365381573575633543560364537769380735058233919484616754234426560748382423459, 1455664309441733661360977875364073825003123240799767728524948254861255969271), (94575356860938613791865064276816550021939977655944476237454150810943123319163041512930408537342242158930373605671150785287055675960461155186762689416843381126598673127839662857734880964894645709555880154033068834724986602918878822461097100583340138772122806249658905843830941521608808139995941602872683888239, 271), (149761948178273486683596583156225105549738698346276676186103100181680031118313909816286206275529406218442945070248453906043212821089804712275960852092524387435612069866121916390959181675102807038034663843611231131185016422244628571232768123415940853981852468590255933107382036283268805157019008613373323924879, 564229490874728563871076693906597287904708243157344285286857513901352954335), (48931729884324380914737765353399530453983723975303515721311847963820634730592348333409075364054039538900777678022696422214708564830104501523120421215095979223029523334724290666712270925174930618842047629668359057537839150920313826775805171239339694375584927697727522035172796313703929593453120220580006060907, 271), (104850345535766474650158334129465119539092347956763104216591929694789877418038802684362506300785077417487049912723873377322377180712292651457822097792902651608029470098739862793343266357003510119018213673740800120883763834627861756892981224581310173622441332142095267736412682860689267417084236496075835576253, 1553460219433429345204048740631315221835226484449554623382112985724138794121), (60437954781530293217444082369234370620960691421994388480866159054292591054212029607606102405491010720007823422728906308859591160078462080811034552968816198280742488170385212726193097871018227046102436423528989885703301246220608121057794070130802262219569465987755441311078923335554202005434149058443367144337, 55345140457919895514661716575179605043607917145496974541588868109807433193), (75789236172371614107793936522315276625038239748334136269913447624869224252385121762963784701051496952499392016682426199006442881281059233625149935290313574205818091237838266495540015544370616944607700869624241098816375950614107772128405044615294193911304200347627284354259055524038187266218883776222129646223, 1347899256666857454526161326691689134590235649926272165396256373620928973721), (101096925514871938009641308028820012947483289607667847978995630142510487456002946544110589808814345702127950400054513870562003156529209369319886791653727387993619186269360148363164558785003674908080277065881449336153345769703924169679884376197968056440138968062824313075263226852425142282141501756953584109881, 493145681986083382463498526719873796500369297208940186277420567679943586027), (166884458050266296952282779830170022563960755475369496755186133246558526103074937728157905847195543599444086181472793192134354472373049893531724333277532275741837090479750155657739634253694284002623275266265319201386984342481412933224999656736179337849821583842312539800399546680358612495834843677441733926863, 271), (101243582968087787215991479578212179766248065425487539492535338460126315546794322776385446034864657018408203958710480355458336616736311423238353812637565845701728503023001883979954672573248482911011233841664191856494245169101988265863028064388904592638225161311410599719750521619209025964194500284392355814819, 271), (143150059995377594193435130231480408137753001453685033085818308249055832733456218301506530761395644470742543047874082920035865017754020449865860413629059151790340630673069077198173102883551185294511698656472934651086396418927621365416298206355511695393582867022479030664315758595386988440521660149681553891299, 371056035773203313736646470083863743383897414755545893589660483688646285779), (136768440061565462483435600164736548478289078777231223906303384791156865398232092720854176799288304764606370365847277407457156884181220329281749732700368370118416701501316274913798520325844409687996227034487028715250168803865708618061756940690702336031726138079059827506701912828427753489672940386972395083991, 223869147587504428856229956249962965875776544735147296000524841287925271309), (75305752939666533673153251417449189857347722452226493655179925750530573907945857653507614258745942444380632391262196633917377167736095567306932577993455789620804179478506052853084939750214388363591653391392610727618545023330626886339554658677844643980035793629853915320856715721773978189229012898246609227503, 271), (109696335944275761498515616685027167211193243264283133493403754892231062449764444294063037839551610775670045038658330870069424784317190595840298772102556449086263592969285333603034953848031291157915228293484792540945221361257172270291890686595678227069007047494637966793960799795290041135602957688927012298147, 271), (100016995505574508862339034763866219759738799400868606186883737409079761406004694491784815416005192999223622038428714568793056327668279947688376964853620512205120374783812969885498749468574946282737429997259710491173263102798280514879991297727950864951390743356702608827440367755814372748096379781514372028853, 271), (47022994044760735450240928076245571077127259510537725360895201011291095177450490459577799423644313374598088274518288826461460152623506334955677380727231764225487489702383771134144091583542989745178418154145690635639928658286523956341221098098750109800015565205405423439879073336692003313773440116504229188577, 1368435857783525564601772923963551786600427724765426970760793842340828546147), (71876830325545202734504737129778508035802298964022498206320753470016085625354080698893745297236719815158309086997987640430174357816685803290002532219765109193003549988570619834042910066331041892497625008717781582074352651512464231891151082351555152825739161527709735970986715934085706642374227169172224321401, 271), (126287927001805306711072395326297074087095963048493551901463412039112181427681325618796495019622582918138159116217162112923700846020942192021004363488224645152952394742293359527087108655396565519239483819147176641126967067901267243730697321884579532703385394732342711528742230809921032852356277177992620768759, 885063419176021623814977451530255174169576996870206594512478847314909512321), (154847905638751878372133067600749024914110863385853319642421406340996525318854716269214963426476581531393149284915793683571288183272664179413997085166881434154446207173940738200002547890745190684965413288885607461045266495862170123888293175754749280482356787374240782704518203995975034537205540173506247532121, 32062134798138970013045771494629675979583654083262279296879666187248361971), (102094510996403401507411641444759377924681459515505175750557992194125843933255821427303888534860601061111077995629329312188473960521093953317310518141527626938412906383320275058772770454922319261433119099227313057996203176089454961341691920012135630748645364945496221599021933491751469817668752047935683326563, 271), (77497435536419607481235336631012526383119875368352136913083661693920272647418961832940225182949172535465334362770552849674491886230217219334103149878102813328792381978535059764773053499515938811395675179013551006244457293508963005709271223298622928102394566217688406336592587582621447712729747009671266513647, 1399), (72588004748178836922332143269146800163933274031476161007998811132333913770978389661415808943433989283583150581410873830161040295971169687601281642277051594484492027825310324868824529690028179533218448860763900702193222097601043992443238463087785456904051957363648654666264899837240279220067190405874014649661, 601751503684240864425774935613567175035249628990163218445769043509545843627), (125036069624566394641529200513281004141466633008823674889975423872787951355326360554108901473092477683432645796199302181701884199505359561214259393036886692097900876993768188835201234019106614697442623668323954576621758629946652831617863410892501646294730428600118006232372870163851265190861734312257604188861, 271), (76113349077117455319043055646615050408038725126236760864560072316887725678452249872570124685507198082428673408762284188796290668519723950168249148582406261365668623126875985956409367589983993859193007254632734428419894784944072253988822431184600296517426363764123650983898850659404544729762929783242778912189, 1172105473756152217932488882481194994037262116172465732133772481907872030099), (164485770453164917898068302582328802447159188365881394766465297145349020588554345666159734123642047883465467298633582561433509512623804479080867915986921901750672317206128136416720942050026723353072865726250189078494404633684128667365660240132171272946794926416164877542846884585998822940853658795564226420653, 1174523664358057085516810062769727868974966075038392278445158536381243811213), (62914578689847629860814371026114837489495069772779572815829750244510745116380736346743026227259736417525005751767886818985893324824221036852844024590233789673440631472037398495143467509187297510057001611840622476583833457563130473522522753484600425300054829966405521713819351894302698705745038815076556120377, 1335500425020702098888456500499210631402366312259876987699092677820092975635), (159382101608823693769266932131038599656976908279425492736038819889284361504921099067809981564790989120851559180641490739423814881726442822461409077753567698669618966082716721113167594270929742543473602131833381902786183461333206876440054907592983359437734822642156539742924236497517770885291826514528483570107, 1125427867160967623685246302349402306391760239482060325685218297949655606323), (146486372227180063772527584847030967810043352985282727424092458763162919040957505404149677312640502238961110922791943467026016192108951348005378676334935043091286439268071376190544918287365773851848063997659918594178007432540800121625912641058664364922561187483686523365651686134042195498034217696532888311093, 691811566631862511977390777250583510184309813004255712774374209294183761761), (101447539242882344046296352192228986370002729011430114177475051397254803576280866347671488286722582262811082456484056574848910706966243334883647846581723786580968352594298175638020300290572390791483482789244998713058707556386641414813594432875301493869343241587000903359724037353777866221898048455462820666569, 271), (95886814004627198413811359272085006390950970207158933918608320506598495508578721375594576344977068469243778904110253434900088716487581700209568501828015954464287244560974041612652353042920948139736037889021584660313839655827579791918591747019125640894187740635741667814692607721575156574562039127717925467073, 424687742679346715802478950401461597736170080148911394429880456462237214725), (129577537510272671304550697856301256619498618199149658167493970847164694822867576497653236696842706256509088736918217784431342695724942218764054007293992998497021264410200812226855877599254730639669911302518450250562921107221313171308570291053883691453691300036629641376960357889652155302331490151891820643921, 271), (91278222808601295767604214323319316204624351549480722697748248896557967796412605160003302325394631696172774997215670331601872319309500182914358124801213289640931284721992464016242445137826517476288382733038757908872924362738609376642113255282847209958047031019626899979393961607516361175101557502503296195941, 271), (59671355777834603755547132490179089245607762764131740694023139719139527341893536648809186118957111154943761716287537157499602403303427866201092055810407346310053605848291593054487470314581767626893763420705658715369386457783042744431931488109381554215256366767507186699467946016825704621091704902940842114727, 271), (148939475586379239338214600417186895544297935330764451862236020234572450272446038069021576405239421026383960709532046892649296553704328627638477094074712706646101199506088364873200974610812354771051370500527262764072568050681001455209584626952716905108258772238090842657117504909273664453643911763809943419343, 1417567574243389674367574866316911917837073217949131896014894361284498870201), (117504712106551470023246460000044400025332404702439948880837643276965086498087928285975846037325025822988526632098285493301755508574300409077742112768195074138693117195413538508547995510013347875492964948864207989891714987330458631912513385533854916360641710600230660196510928110133057660121602232978515217357, 271), (140971948436851274544177971823634851493850296968749618110166987898564396248254373908935817397381445518563245043534783536495507216339386104234566764848008488863069670575681999351210396623862473105619008921049324892291888183844173870786468005206682951611324376275316402930488222271826680323659473144291147839961, 1055748339798550120416870769775757390999278721060461579567115736948999604769), (78380393430641248117695555740782482744463588870070431276991198156802499052533095232468763178640171794842215603243843538802699573478047200303442537553551652240312099279366741090697319406455640198723573630231991711290786127840844381942522219116442759184925212379627322602563103259362428443122215808181687411037, 271), (98606428560386555437113212218600666745329476448460976709172632988595025524693518022554722736521351206258230723484991035801145365592276474517283469294566909211799866786731028123595968065646134922166757997543179017635927544937728531565763512428827285020958283553343253589258090315074958154051994718983330272657, 1782127732671208643109026341198242818135070844855260586785459116959665731531), (102811657464845733684955445035852403898966844445422763256904244341167922165750431691587619694259454615849542936923922894783603132703832356863466784776295329469239153288747844643092648115230579117755574110106694730087917550274894819257672993705523741332477858171268929782857608699613837382505369721021283827813, 1456629892752210323333210836724639122216178296423954601423732974637796502417), (147385585135319444965781375751989903934884387136224522440778506726801662038783899712492473749551860251155212899379872187726032904511041607642771286412762512859430167568692656601976068324785380613804686594974336084213564158153058145251340456468183161785926047528154677407653822588741705794184627302866326101489, 694146908838499822098554216446158278663546099575696580612155141626463162153), (152245892545678092296827959355913510932966801457938843996070169435419658194993805961255700697075721216946581749760541390970473731032481563828171237197991524891654943289379003974441746471769066196834149591737979020251458996351832172991849023653825658000835700437346512940612413961048935855056890651136359030911, 271), (82523469671288003071534358890465864119329312167529528541036557722588040449232785371546322123288744412733202349143209439623247119464231118260860471775403162159606074992322570081778191204466005970000132150023770987421447101355308619222121460047270743033154711643447175386246069585744424154976513585327993186007, 954839800687528488173359457484096325772756187465524222540774557618656937019), (126202165202493591888628443416094514954896032376821740021086386175585025546567490499454514376744583270427939283870372506501043684572001498877955385885393406214626355910151638429397551220555507888297047987901740707295974297315039210452149521323934634233365962317161219677011985296586453146381699031548598771887, 1240164490052174774700380983421813388979803288124146856569933735625017452093), (71650361098452725843820160961047178688181671021911861286202193902648253192630674392514256019355428846579450547017733860368291528364792038309362516822250113111787129493760296228273910436135090027719779573944932486189376901363197248870936623733756072019517260804629109070859796338119685324780528653077248452331, 1681061681498636522224817316018987496458570873308635214384215860601731061705), (101568900877873335224595744923305760790456954494701879111632200954252415253298982523867187237266313648681840926563126751657669203788099159231041448133867280606700701921887180254544287394710513423617381892294917686104884257063331781598947293617026254234677502696223328655800805006863990053704310667744664535561, 788504451477192199858528685429418068078357739688436005347633421892386680979), (140519970206854644143975183734614484939457491262220323978549213409937541608446441944172834225536059183404931159149752356376208695625979539824486621225377356891466909779878887451892340628894430596948117964948864229575138322739223567541072609383244304624105291275425003343990765259771789229059959025014214248651, 740786068634905194353142615937736483073342835521699717860762462288581316327), (86811632930932942885612185467887119846058383049427437854523155063927217966315454411279411556979520123891548144082142844226348824399650305867284046335593060243184950157720534222596622751961504617750244397078384499182590760814878766977455928958719651969495260794121500892373543944335675978415215110061498166671, 792847388011530855826802182902906753592845462406693258254011495845931760581), (100723601479045233780786760956869002171158129244236960693138241349566041838712130360939964479164449311995810271026170236047381141075258910793089473047419895525204138969903964669879030070886201327748505058690649839764877800127426863709815775438882716345071928241026911664344385386205151997585238503682951335639, 271), (54897313977267395326541195575395802481179293403443686975201923778993553358713303691797281248005937600946290913485293145550069118761866757005902949920154195438675625008843608141831236232169708463649974780772455995010293093031746775046598662243832233256622181214124600485585111359584071833464723696566531799779, 859925403621949438339137292665898012953373626168723490398075941466871321913), (71520846616380314576907694752467991576367900618061563936880934192911400600316004575979964627787937395923811787389963978663960819840514222735602802196755161205767726537423767397948371576072205627887934257909086751328568925359254016163338831342007647602047993517884702479251197024069857158147564052735437791997, 671346707201102188569876271264522127497934941742989601243800647356131265293), (86545285113871697005566809210316486438948757306829321917893462195014660023341796758592338928636662236822884383940304976236258872006704413085950656642955325916234522501422268734647493297464449128670850221162626894781730433106198371345533505993904661383924414106001380639902879075950437047921503700846644329293, 1014504200526000945247670895536089585055045621196267982484740620819366476771), (116668959135006137908599161387086700624260804879301248461599835386170101773436676002801608827511268223216911897969374767606298378620151694845787581628616251091067301028939094677291224695882110840892566225308251889497797626464710221566536346406731968184551414325957879312560602951435837685960123392193804357817, 1320202845434433706338125642358377899180449931490373017252743198753918037263), (91405511961722511549872111512414083740770152064017916995089941476351089143318133416265463381315718770376271803036185503978192521896784589495292923345484214070943340983783107974005771512002957451133500649574798944468420029110939435469584661868366929251656287666043564161766905241310494427372313407815588696643, 205374029756796074637737795797350930719631850236413446886016456882777257987), (80717193496082442988675728673786858618187257264281119494068162568086595238822285424603679624333150565986883993619974965715875110770511130318401727342133250586883041987081257365547561766162770858547028320679706744730749799183486207433191258535455542178758476506072719170855924220624736474729918017943330453513, 1570984704205970997471743381471676180223009368413761834112797941444601764697), (128164482036229390599245910992914449728987042427919882979778668751839825534051032127462069584647089873537745741859174777189430627051578252184728323389555836890791233628290610159263260608813416187696122584103024848014309055255354587224402311647072130539434398192177180660507847215845231475346119133602318861211, 1436034859242869980027317284960412638910455675589045327691090330753137674675), (149466837087240670537082143659548021584219639263994562422097205786851290633862968147944479473504562219860830381327021328363265167262792508930076818845608987090139769325940220232107129990437140332413067977383723390879490092983972305039158798131527191037444201237475921080313883753399830307858809094580289926977, 271), (94054582825720911295278297473253912186933076234325816594291415964472795020289174046351567420543124097717392327628353789582460030806340433027772387227957652581668769670511378509077755497667313435600568795885308259522597279606886596826136039981841532119539632229208732294841829948689334029850392003020123613027, 330901668359803352033018905130121720058543734867915233740606706963976793499), (134443464180431335811609821098161035949765138954304161605314252751340657907941429729115153105437249612356462715474180471471354487492633776786678677314979364536211298928560892305356197813623849693268231953786293513506529936575492421848044057594372135415639659122482627807636768320527504552485514242247040978941, 1206536114946091054017755451510407147519732661938228635395349048987825449167), (113913904738443180807987749724228395625098265641335724633696289424974611877972349795669630545582226456943721376284632395624470866147985836185578391673564185457552725226405672953767643429557621799002735284990188562117435332057688742358978668484559781806670336606723306259998351665897145254736163849501181773951, 271), (127754429780785982274582081434456039162908482803545466842745854305916770415274201065663619335865748499589539698869235044671852885241458899782439695757270934142371576919709465358704858004494016723353240390560102057990079307313495456515419476501303660782456302460124697278452922888046015715734940298513693414027, 1084788901927707389114420732864957065578367138353034693939680623540552947819), (143124141158586920374905223494276564971508994059804430428937202898969436133426375830189629117986415212211191682036119298277630793258922095928971365611032825569095012574524836141785417487405309145018343487677310665697976554462681178131848413617972228163473962214945219796708839581149177079540791458958965665999, 66828893080164675429523338747379412179770680412425652601801331790715259051), (156712961605164285276400497409743478473799254678994571982385814507521838795854490586259699814323982798693519881175995126270647583484100621809399555329926148631338372532111213444059851863280215785705252185689462193717586297722790825134278790749750396442196012480682056851651731502399639235337798930049945093767, 1036817503042082960486087584780083597420690048175228959206561357016367912831), (117651606480247912967661429891184162884991399630180315077174367586337560745160709881153374778594304385917760985721693143339403602691933150329885370521997173437825332540770556503209663595055161244166295298669326572908676310442379841780322500829427371364121585086537029969705897817022350156408710684840864743873, 910549257951200372714760570525794337796005886703936392121803542268450224533), (100955835698190733056030281934865210646607451360516497996414904830116911519318675023618105012761861167511201915326768025008927480421404755728990624526318249330076350201626820204413559967822924461420888687250490392678844961244723880921157783232109105853181003108275729223134947194276813933639290501753120970447, 271), (111401111670472007399718186850679652173983838771054750094093993648070107888769654547067555091953380967592143387160634302742668591589019018101300830685038344202668266321169155219885643351204770034261733477536908843507344152245861064136013297975853597041571014124051815290450013337261066290118731548476715679977, 271), (95751055096305373108444130262289484429604930138923565765957693946187426125328669417474144930989303961054334837878436916830233396177738976048553245973028450098529073914056329228983777948623105545951389360352315514172763577962648451216434883114910356468808549440794642581580973403290576723983934588630871235743, 271), (109617936995004991944080718893673041807983180601078034789164934071025155005884745124122781165674196601760508388626036443041494741431811838248709803940718800180556508026073413263916562173935461506481059971311839781186046028416347170377373125555229135269113053883790099204840473560559923607750221372476021391507, 1580373450188553758510934760747613786113622417391173305784494502885638839569), (143446739163538818200880568409242862721358338708800315846297108593352221862969925803949926913922815626053713908823821168852228329826023527368083698081652137858348838812383584533782427414782325417705506410992284386614192817761840503577542368718822654601283990198168623554396321049526396713247846753440219945947, 271), (67250674142562421639947307955861958875712192177497470868237284435749921223461736162218295900017571316625100064860859539074039473493149776332458820796150184221252077726929120576283883531568849926910718679551985455850269287925222561923315284019044632424278859093355029773456467234642192799370244836263418035019, 997163420080664003762918201733743155983726461169913756052263382587243958639), (84964692730952973049777172951612681198065687497355949566127552217690930519353662988797285567412219785901904165318462803643613829965583459856773599651453081764174995990238154001044693185856423546070774451803543179309910734590732220361263402311561561276248264918759769021075031490074194781219661366764639086541, 1669986794721111588257014873204024524740294544897387147516416801990495437755), (93735255247236093969092439862064153607137260900483612616199973047002012472218223296329756830615688816981369864220903156195963099393827994056749264271355833839609677723074773868214823566260361658686342354711620050848873475430733284651669191681044767599873843218170385589689718285118560700366040417260285457839, 271), (93618917888529889118502329548478581024145896949234762592918359710368753091655790234860330558349419602989098366961695471666370309119344658256083811316612108877854416070826839911499163162868731687264344099354303016951208149705985706716023730504995900849802202218428691844127294324562865392123917085967461425867, 2011), (99333123818605202476489927192580862835772418798867396843250933802895797529353387982676886825635848979604049115536624420500356803993704948209603136705466631168258840165090851375460966680389673705240235867781643093202492677952647015930198217426862075835905674712227858090060900280028972867752419150203349790131, 658681553110167702288082040417964538434120894322773588079917622750094031219), (154391763719309529342744399221410132424774359729906170947399305013351368827170740877249127810947409665619606605510230445089496316701293370124751491329299969541296510263619330396240130162177387562677245552485892176148469697440634368926866918459561539012437719758477213560074730173020169633785490385186766935793, 1439683852531747670461465055436043969705067450958367794102653208968940572291), (122399639888604208988086670519827425493303662633810622454106183728418390207976043590788415492383606911265932677214636140858984489884423683898055644281114282567400474649740348352957296245812885488721524128467611131152903662914649697076295424101423677464098294948087361751066585902322584067381941138103867573993, 271), (79266685369216013529716010967389609386026049703465331122811557948990761484226682236065209666018796055599668226258869039111207543539827380113685791873790834508800742852575469845837983973709708190068173729544659114583466333970458698612145510447233923865561521943822146401561781352410983616323772549365524584333, 812146523825573778169366485823936773657996018357860812681279698221935037203), (55622179347432236898600940123917943546812822862518382734483571057564273409894997460262728856716977047082363481035609136390037693570312182041008538136766934527026405984678809464877242529668107109458335064461211247398953367026715318566239984057677610380464712352055123620962668816570853543355817411006311878613, 1397555941273065044034875549863398760555793157764411215691358575207164252019), (124183699123787767999051803724572251227426491804540042102121244497777985761439612078356157337778027039784783300249093165272065032135492845767185887901746309725803091843436493294137138155926405304943665755026757977951501890740048316872940622531872493760857924684653456075871268121961037246693962262286773560703, 921331233736107059020933854668955003945690730591576301928747152233099926785), (82872522193964274785999603148150185951686872178183504980530897705170270916492217184222579669862336124312170402001919880040067914164398408191398296407788702912357974919393847839376237660651182826984972078985243913937556586739811570278421081058701454956122697485893196020936158777857160251210267335830711467433, 271), (66759698727535032685567519181535313272013169588087531924809375916166653037150164547729162709494532504357492789022706884008926230374901413437744572361856356370894510075204054836522209831414983078194318132647337564070451952619351754151890891207156323993797410220348770314896840522976036355682571727367719754817, 271), (80355811454308072025064007665391624961269956121793704854762373678127115382840622924533905776839204530710776547309432832913232326230578844733796764216307089979351899169631649096385157535266416013605236301371347000598450019250508081403857773596564915099296438031004008667520773163835397029404193143658773797869, 271), (157032328841327497037756922446922117742562078255245717467614906544424838719087499511955753899942118695618277130091304994513735293207840138938976329269827387053429078095562739185948213916418862352571676968676037562635401432161692784752004341880462630652045535657649246409681601514952756993513273793993573581209, 543977312717539124622560126101161868461650034828830254618112143286054452183), (83548057115506524641508977119367244978861044541199043699725302089947734118417950512889305778629155840114259314168391811327034687077862819365637562496885592239817765745528513896402816228285113742585894823414158650126056074552251784245104040077916303107672910171491997940688954921280860775721534581427792792123, 1599006737714475759110675269383379489794040897529229063247320665155889194291), (83264435205802393408424867468628990053912137530894078211002668944023581086954198504428559219261633694267977353598081438882774338824277301184003622943276868955260350202448657302679542787578501067111663224737739635688764494253212718743876807265613532056127497345410010006144414451719038207083573322897247484001, 271), (72845815385577472589319392060727714259076060278654189740799149774910882738691690240234517090932628975740951214931037494042463149965084307218091658003680199303109605858766029534770232776210253972473690736813725863007228922974245005158801340360270993594686106579007401522295320202741716563442694908107184008549, 85542921845744912065478548621993324336603782799961832426920594415798439539), (90570976654895474528902399092664789330487490761804166914475596847616807346446112938686732583414752959632533014120278413681271029327467493101655198838509620985099971722119431806306554719799522227640347962478957121167220841756267320603435773844597279377961226038840367407058186380879183670718420956629212611999, 1510812945775949578910228652989313404958226301329292480306800948281348350789), (75511666740241613918853005762431931209061193804223150350352656458621251514060895374392407723841364483888541590068083292512945121899900562501421396282079885616586789607273152994068451674878746203837765288934211873678513332158447214870569637168404614396022503571316812376564019981287677061838445115090519062947, 271), (100271140828639458036930388682067380176416385754157814039703885545318612158507059397526146160162989255710643324519278242369884001191359903832107070597088969625674694113102176020121110874143630399313180225088085142238006117330312200541757501268821932123437942412552740927444064631406382724997814426178203277399, 1655535663822413466854811435129941836370936207559235991873174975890761939467), (127742335205714241727237236687303110872796754838298014184415600825120258273600071647069542459895938285992815845519340175610841053500058509301191782399229972434158022369278538088015604396814141189887054145633216221046228069726876826801180178628584191740742533246940110088231016725882547509220596237700520207541, 1633), (112300229953675538782795460818513280774559372549898154596785863516107811526400241421621271590578770819221543835346812553420837049660924742859316253711206607900468306230054022615754342970900942830590016668729922982186972714246858545644464554876565660699743841952012573326436046399567707463708493980436862478617, 529419305968635921670294242715821262769655110521493742965118343204199332273), (64686006815369371354960947934106621868151244458783218539279667432355265761614816853991376543161111248175516977691476270898113507384114847958173303196305558773384011725097623956039379190063278929608437022358029302933936057223463077068989407664976315664575584583091862755677452395028315293576220152982617689753, 178623557592795016671860657279077053705301315349227233127800514315624241963), (102456315125633670257614656445068019791485266583203757394557339232529417998133579784728093251207333235296184356059993222697731522486165640048968719545444770352153058130109456700677375866621694365749367931752028773037462207420181946052464002855843760357419902258456059571160242248269113433190088695809845575357, 1331), (45805452611434167823550528900129290515603863330907912614439815323030509213532165005263024272106625067512596090418747690296177156532099491043489316724122540735750558423824653038386432999796563807614867772870927601981045600107090603476911113206572980432554349936715271583730728061147173417551476481024783121359, 1289918140452957992303883774243808683894093216123188752950982006806307678067), (92101032315440910053191309866623148896893852038135651207038477565308979760002327492681339553764820660047816271940223401876964631567821428315285759727189601260005741245406368770709082708517109679738751421032286255064970940112389573929286932457061111190559159797022317670017381400642177738504903760934931391047, 1217), (94953911459971040161533424635221336299419311182559165374653179653155993141793196586205170698268007231535246149082813959305842093984018667666831168242874193631201626048482203635268807681520160035273456659272092429197414058696418146851387798629280705946859914538837903882335364571694792261928606629365099304529, 1959), (83668312457851050271245098047242437210615935897065419955718058825459966449918503014036761165818359926528613915210957073177465948829730991782165794783570494418570639433490743461488712548014098412784537907437737409499731602065554666773133838129393928540509273538881446754098339013276154157471863597909620763953, 7759080927027252597646617058850534930621884036726977806432073493110642773), (156474112601923310887794022740659299099306111320918997469982096796977725888515157332241028425364057472118161861985007964881848403744577724482755670876973940120705437017902090417566850856590510002449352167689209555637304219743189383801442333362661418880747842192196125225736900506456194618539208291901684412569, 922218207562105771075350511312425005067537885805144105199422316706128556275), (151268194975530937008384905120199635491924761487624747008199882973263428686562664903113133116361712486519819015444865172404926181467348714267002693563657417435933062195266873816032229413288781463005725550848656793035303010840366740099395057253150243561085418314933216190549324000268910167911337463101984634589, 271), (102366419379299322750250649511839819021723398692685625625177000050389101991960225314633951381617290450609105709235641769175673350197621769165615780773586667388129521069438084573867887238438563127873347007456327526794137233473029873311867132886911413057848660465365095618158567174137068032604657996441164120507, 271), (121079051647851872096662328930154029356310180684462565165302299194299120325287813276792087406304129381296018452609391768712740416632300205331114673390091429131429551896008214457488654673878912518252837441573190593626485264468769016751966845759719335086664938894310061926707232532955690698325979983655602075519, 1772605514855929946564015398884831181845780296725619767636935700592468533143), (163569308274966200191144057202103294090049862583978752575651116395143537605166594235616777470808309857615559460493014850751966714217936010830751105256459925215413242649540397065858483369112254377672802950121941315687751060332047556817006749877609716483972060542525693721057245361037580592336236716806724237963, 271), (65217614811728145026151674680347892299213212634209035233185719533277896798566640814471834925911395761821150380398589748457307746102764350798949740786710994370908200664438110582304675537117386286351287445044862816769434699302685757415274693255449113213257017616098645323398362357285577902788526194615076454139, 1564474961885984487073390915281654782779375559186340474721172887596129296383), (100142421595960440818675626854105033145627096595676016182778284708258966138786942525887152581075869843922380247532340358688665667682459204286668385015891623946856069870964651544439217956323189999044050930669440471422699077944024338981206664077747787958021141619423952070193666542447947820264593884511850181057, 271), (117190647284311553033500477237339552235856433325881668533395167031548938027279327396350016654735160692865091854922392643469560709348096535929799510642005133781773918635580103958644900563342608739310415895384004960804440248691723708475552987023822328892539583636314741906243283536903343199202328788490515334131, 1433828993285947760867976614331675408255432564602389587362187351527005539177), (91041523099022702841446989509799046115829325855263485257439451279450155651980881648901058283171594659592864317002017484428726477041018595431090278734333152578998010120713964578384915850541310036299179892444116013320211064012506153059381031508184778230635981421522904421739086885875965156355851602981456931237, 216605596172675701595308033609298587638006380093252968605929938118300569565), (85296152686561271371181882811268466411734610872371867252677746588497888698379711706941586805827928533563050109079444425959477458418423254589271479313659019461853011644864913955294923973857768560608479732763502302642745910464279672553750208360777771052665878360459467167515219129047034344230557645256731043821, 271), (133784431674177865242689949135299322809388298002809607934050337442206650063958399383045720582298763967627880517474231040774382329938573689601097680734004452771713292984288031904948750833593008505399798283739488011451984491721143048072582625260154178472163093547429692875951826236807688437253465585546713071931, 1721), (113616146867288728658983275014139514917498497760159529420686036458731475407864561203387315987757054383961459958784079808750482006417911248713362607179902825902074623304501037952555088476028196719995379674134291722934985852601544528082704026194310360671138232222044521799781698734285723068579059463129842307227, 500816002167677895921895662335786457572619450917367756323601584957021027753), (102297873965339251960230120885590243701550460355984856205435720558066412546547753107642021882574808188507601488332749915963514030941076153610126380989847665964002025900838289722745460647717968983993375347909362278492056693030995884299706306485427878428163703749518070939708208471198041557445504872954945306593, 380425102478465048494475106441516236696230343688939027044805297556550683751), (115023547530707790453255957336166476797262402201344547785993421110856670159185990179435954369622108749671611276368848046565648065312069625667429648088503731263950010320046442962735446593769116125872307305285649663218964007208937050690515947110880104366599583252759437398238683951121257922443664567564797461471, 440080999630507984432263190006004759709375955669037135857393634855739723743), (91213458129350592098629269803438450408702717744527383560100249313765868469119134786317663182572115667337090677237151284962865920038048752245208659792450172906275014180702993931728393517840764102610838475681293139417921889647278350573563598760339477607252045580516280436071778231873052535956625056063406122653, 1485281931318081175268753185502599243661687882942976694074237053004025468307), (75841728469933419744357772676230391132238317043010728761784777291170698129986862317662935495381534187377300895073630527371457979174623305645520286255371343632994968089229048702034394768868543258453190508878717544693164374310345965235082571132287855135695383973017327755062787983858461153182549959933952434223, 1736834224730937271255995370210384391508944935816514960266843991784462015639), (63211949021369751274079137820691405357033023265930367268562930817259388986374075270193698483471885251988794259577381128006422601109682907915492140576779221127174596139221450383825018048752554607255974761703908711785369376737160467994245765272981598428693871225280838537992738192957970540297612767420512216041, 271), (79599840166185185728398344700128648434259913663671888264090482681646254467294898986435018412661257892690189727584177252620336083148437254709820217511554725470363378556665688859262677946184273215059391273313624531722038113822109290225716206436461893002323519878399305754375236025910544662495717261385519026897, 271), (84803667610669534290623931723585286730675708237816704587011110657295137251347585887326429357409563097965476866566468965639898930386298945777178874168125831383278566198159260839437560844741168714180124050652093904842469476572742926989393004812215938406433120359493077264802699052148340144710524190389912389631, 1592097485501516315073796591289349070408567536654364817775202644072713525261), (146697908343175863285397798845934841568753668528444114603786863749020643667862382681012889900835625563181050904392777491362348080300381377009393952243209729781768723922994759286348725940862653222733961703338670720328315075538493525732788342087399095937910703218233994168302360905378460039593902451584825252143, 271), (81424181915235413019093817693921586227871730852447540879612119902325900290251024867627882250379346019839916170366430286458201089768549681367509680163513732126204699853079899489383141600643063582001149565507279705029195504422361154196697738428337674811799450620356183045938806616844410686546538877072982766893, 847725041450784096934332304281347011919980727161883158109866789731506901199), (69995092658835555443256595511343149480186626811767701232045174624156973313702086544310172467537445078240252457912927635466516192065273071425602011636476874595555568188622485226455078103470939749581617709640542197918852053330183908895800847732259988334806983330913417070292157827252874953681907072044141591791, 1270747692974936090382987008812121020414383655733865262817224369786044379311), (109651309009598121011139851429464365017922692199882234390766830898750564237634332897696880455818819234343080987287510738336649073820361180075325848399661926253226442769389099313771095268624957893036473632277897939550347758415891284848936829849746818759608854372767012022506969324127281027492210921736674840407, 64291827192740455574407813521590891538438941906396159118634341326804782331), (82355770020406792712306750261662147001465171764016155879527275805157755145208382096206627509214118620373881062560460459975539119477671456882694897905350180929708041161418985096265395472505080411893163192083399102874063836141110766415135254817424329846568235413539844100582032102220054157744400467105619417409, 271), (67275712933780375756309190667086906287382635662648537307883003993535494869805616942456174020472604570056131520876869643899945267047682430072821168981562050147816904639779205958345054948983888176375443870457627446950124521476339321259011171716775957962423342151672303013751193229531914313847982584303141866391, 1014658129589127692080782490822150594782484216775177763824521744816126623023), (81562757930117441857411068174973736370370990313517709626815382687956803153377753924892627449368025037465258467033835096567300928275287654489853201487847580739436324114358707703672996368345738242314415451634691879576671670679536308102971173780003994107604680568712566778159803895898376884680404225528675171527, 1011931627075042375716470525415205338487850346906264246701263698198431699381), (86429714058580530997503833247636209126765996628988236418245770899480742625635120692269319784222759126889341117306132102858048509052551504028609117901065189165342236712257725535830994470645171987573721417007624988410136026990233118451696391856633321029192874355309563958708939768588714216824727941162223721111, 271), (128672219866874227567591870810944665686586542902809646479172720498031399943040890326780837533365935297913342095633030024932260859287377780958365480499357098716276424520620444132644509937258160866849338601059360658400388028369549462254421672011034374569234827734575698123186815297975807248828379001511435977883, 1476355800108121024869737918367454979383744555913451919141214900017937661767), (116136528744790811711737739188733138034243743683404037079817329103589078195181639389135459126890639492483235762479151849651626349174588446546791311680540600235771551832761447239676488961542600392949808745994940265880982235061270668211177555932904693848862764280657803504561366372615425429306780475960841512559, 1730526617479264938595100764150886746264385972858010157260612102323684260073), (133046472904086708367467776648510434308687770585171455254241545557988691284872209041755597892776119097053724103647211408938083312925116669936300664408512100896481192783735206232353813696457980570458124316459593652093687630513246036257911231148138866985964284229502045205509493495449074259249315259755930717083, 556248099921482929249036882533107830355408369253709250666651717396835038165), (70511155204718089504829133592863248674563398593947940588429662532536118264299602478431416091218467067768329068144200199267962154034191307776256945542716023661483823242463347999456457545440668013247889183097993048554430729987358123691400268986006648439229762987784737926512697607829903742316790459001507538519, 1059249909928396317897306816010291266888589838176491962772157351159294984543), (94085673834399517227036196334995092282434763487654689307395685706413899620013326604053864546868263060667484806073432255410467205252317265829787490037155643379352337871396241623272265037864412996835216507537401355144806375681424109178501388780210114486590984088490624922992694801954976346661519901219393902199, 1471937341982533686247648457048867732316979960185601196508698627774252662711), (117234872581642754539320483653233779894690458690287137591608762400835758721102017562369156943306837768499745570401124259973854966196568724543938783567191400161076432249858599634460703360112692764695219300447077604277552478039399182135444561889203298643848872822881156125662845612776285841168603103465781226807, 271), (135374633263709251003400131321317295939578187231302959392680316210825827998277851798453475704560142575877654196716320670430148233185996848966212046056297606133688706141183602544429878699960385309623155285860648100637443100607277715254970699179493183459066656136483424408251957592631843690977653263384361913223, 271), (73443134172437591736712530209897988302090581799181706162481927490197277062663648758239542531474203813710211457972725406104800112203552635189882030322477169361755936519041099152899168252505494803937458848842566783051906002571410589842040405645524288456944625453468875699561621507908034457676677143976198645339, 52789676819059136106665104386587737190765733467995251767884076274846944521), (67675858408887677222377289823961834100353617884892638981607556812090407487858005860481011680078447153509051680070368227226270275150066524745637804685097514192526286848135290124153425942515817643686786589070579785041200979834110447947224208501731028804423863393421159185345363602324313935880681268255391940279, 291664851067645740195695014414204349096170149260959802711535113123611377277), (114804021140711091039843666600280969807980073241481767994316369125659129243492915372865152642840298384972887675136243645805496954760601697576904436305673058303701110470597926793366408799270863996298347436709358822020856378882438817045904496128615234918598681002638457075620753769353819096986215193884958429179, 1080886245393363690925482542736412583106458415298922458800995324366469170053), (115185008469268378206622135704105071110973570659564829342679098193444128864019909167363049706376528748799387598086070347964667650102879196545980054435934504831577441783726757716323453478722330968077778110067400915415968004332673975202556914245302178768980975711236780226006556445627734249115547778726187329319, 271), (141089816524665397332434837261320242998333450454722151906082378655512456272951483978237978419865636480559331314784125655306876394703440750699207956409088523876818636252394606691267751296641936461738546863200293517970173911577360249491040615984442964931235715171292157578784021463232937064860339748984819201189, 271), (120298958746950138360473314620408722004278374319544445327005230426409419197643159269573807749881908373693264082278191563047341851112382589056793124007505435166352154506485576685433210937917310986535239600886051145728859728406483679366399428392922615020079525434317678465515413098092564688549431001466042137563, 403867768843869238228558189488411601529190948333584306036064952477394299897), (122682959782559191651242540072388975793728837196356196052651113383032692788785331646898159171226371614266160241076510898393362602332904056433365897660260506490476060716325534362112711960158090054922310237200863122775488734183313456724950028215622322741481550183627762834829104152060581642535408713625185035683, 271), (100886774576475369817549984657834251567891171479854911073630385637932017873967454727924439261168971173916379089922656206404736488545179139255865771715247000183082923523327582716120480148048199239141822037927685288337221167438706188406806153504964403497871240895578203607868683837311113152480005574196028187941, 706495521647405697375675299644001987600497055823199847941060642059875794589), (163344104999842437985895760531717303223662760235247063154246626678921290135710009401791538193048001215588834924124309793886167750921412762089303392387289846476324594899045827394323074418538021848421619273536766103035112237300955931685477936318525225965360439624864697859031436763424309693857725393660545214609, 271), (140444095243192210961035227090608396564068254675258813174676933192161687074178207238916915522719115058520541138261160299439703962057516394767245860405485015230927452543621695971205621713569641626795277970168051762053306077353565720429629516225875854087100645100694434668016845858609985683547044027724578543667, 201337106060534561509237553366937088994460375859050840433171978955645124077), (135165797191770481490324534625480178468123766789739693878601202699961365679860428417167240985104151722968566302857645550356572476860508621477870262648979412785583961688106359331596150590954907276491650705729720986512150083536732641686299419583171692884947449069873232022296591454374091984777493580588200226193, 271), (123029432650057971456297832875703967247582252356849881180309623759283424860807955283891161742141533661572676578517976328469872608794759499208733391166265760685667306788476352923567555108302988769303773639815366174038918387765778522840872099475797705660173465435485463763473716612934062444659122074606113283779, 271), (80020584821250085012506121837282560492751432637279482941919444586556440627877660315959205239352817475677248412625294608480887881007624944767485906031762591196273110701196936898720706052499367185150428875454758940364714934036621017257833482293360276472690034092690405838403754834993488440028017123034408604387, 271), (121059646153178877681815440268405352567787733348048255052785234334739316023621646235604489081003871952090159057231881275916665430705511337038313605861797644395435575115193619117366824474616091337420990468535725909392898157427046749809039240814294047083309508208294037287086771789392492773474734020156215196531, 1763), (121508962337931930237662999154669505811034356544238489681373273091098111074916825952382768381463776002157093789637571677887486977713752050671173508817456568556604141454994260488049035108995117552645074725511317811629951253166790289980095776706161586477713707365550321978293541204085655879195225013053912048777, 189882939143606368368378984409052511923892554831381510807299433224376680289), (46664651773103520732047978938141948474268662265928239003619132372860459137815921272894581342175386858699419201916498163776629301061487156861478664912514022792230357533850400216956434376524697029033500726740119787708997048327486260471316121299220539193274306006060681051393758554996184319352773422552743001971, 831285604042468282965209444915835023006329118939013532369491418225812183967), (126285158667958550731407007835127675979763959629736156172292198612289938678708280204587837006359364765835324752075827607933096571857076438855591300894857138257658937625649600312970328109784808256772990960436478064528901003675400109008848836605881204025477916988564445424367355139475733936389110743662609057447, 271), (71286671454639928278883293594686932521643369464810434351735050712801658872072945936077212389820307534012284123938566171948743251333822013701543583791673107346743887032790312455301859532205276776179888110518482070452040237460746853418216287597433416935303935446453741048545736197883863747848440221530296364687, 341948208198468004968421187675940694950685586705805067488915461524950563831), (83939784723072355947173229015116840203335198401096384380457502123765670719969474763648458595056599630007685799780328234079385222255530056825220608085091670519642107476440877661894117436521218226208608487489808465363912846390172841791053831984288453601387618832639052777006085033453975041555174988907436867423, 1191051507466812808393850539785211860839817833069281484341407505512931613711), (113480522846846266495682743862392924881246167728629031693849366311740029838618560819483184058802612968002718741923547762395907342721933629701742402299118759364652503050802884720629520382200688103086009597813471037074924266407451118679053000885366578846317980000703588091361890648271760703699374312926672685461, 905616015223017290871259039393823468785735916495070105840828209684938334989), (103636515323764226431258064770085213403575897104685620819617910516393769884930650947754362025911043623813061786224125356914034172611851671191496160978437851981197982918376936268638746164569223456756724763255981221723638573036561944034712626781831264647758704290166892668295582755667534636338783714460709099657, 452987191953114319973707882475423250033824143984070728115387031691955541683), (84233427081720096985831778849714875106188640379822156977630660356808056299191662733290637429471144145920775563857836162394541469828559687814939963291388468446665819687495103415546575370247024089654688460027471250502777617825093853404488124486880669053274804450859937070426308333733242139349111706352533878063, 64255525289651661212580336441200387780866248085140775221323980066265801789), (62919211338689429666495314055678196140760225102943137510862159864289334022496195667230392601017343113896194638640389741711093735332627898125939487470005361356242360854109929854552176431270124784970460265319808181690637437933247554155170419312616082709910810332305604544892852037882164399209973722493024514233, 271), (163196338517578472393580341930595293474834481235004347158803815578662064199704380854430875267296727658975795604663895341312125208035547862965940932083885118705051254271102272781647986175503097646249810914191614261981444575129543423722478462681522114741218726481247120935778407409748562945814795934421635314523, 271), (150849629008953401606742621134436330753915308049508539893576384942986910117629080962740254627997404095243421113793232779574565982157145869907506215998513413087174817235079301616518696596809678920373119518421281251605935732602276334859523301083969392181110399707578528640889416942746241125245714980606087741457, 271), (73695274636901030465472091747403758282618943860352646091624821380164763574399976074565440315623019974546785571999419167060163714207794836175516025330071645102379829758390952761103191305542346202872610591599354483037175866785262286184544667580375936691198146212296418029635627927404086312335797891459727195191, 1847), (61575808058107048370404574670367130160356259337705022236633395739399299171835047976607458682511148651496940772504205380749094611900870448976667143481761225467238603992006838154183049895528363202981923077173661069606829152922690790671113049335177488942997526021031177093844869686311013539652531633044369079917, 439055114843831943454751097818303224396746357643954423381989784572890636987), (135447860088716159398188835733497531009457964567282851410597845225054091432304342001420400338553533317186995874134273221709897182990032766566627115339026543976447203464215769723525721686826082310450138475732640055469878824057737503008389793799639802907066717662501375051766178644455042288718109839277489824631, 614544181910862635583091502160736263270547786327676183029259561979654245947), (75273533030429942828873286019368931822692762623338346460743355935977196180606926959613193545703954942881686167208545581419060425046335674959699483118053413011858194445652953007166370311704215714641619830878180666640929451656464035382392147782957114433351096882848859590946129458810509979834751552898388688179, 517974691198705918518225471142641090001127731894839291633199102852736714159), (97298729197601176961355076341317319453196405204796109131135494864888012393033524417435394100465347145790610130246730259108624635192583289260505356819165210166876791114711132403439274656240357787998900511430008568411021176665240448938481067258154892682010881403921210916080682649628875318545707373545199467023, 1111431023182446880135160215149973132371509717385962182213142253653118887171), (115280628277863858611383156508658610530704961873157553273173062266347817663133693018244576217541125171590950533894177426480026275771640193206608851009893443697651701963533016880714149240698968333877973020892222128936393765746840702738390642028685235730634750034480161372285611163214951056547680165651707895809, 271), (97253005940499622097727456720654000417782147839275484905975467613002037876269123519927808982492775281207779955029394234271213110837496295034214169159572318861139732966895891052772758192526996373677718499942304002501869445609062385143788569727047127646928117019983944842889377607560962866818972031428910526533, 917057874060407006093900098545804949146048647028852442478230826627747682667), (149584706083572364891297677201202096024897689208986078503032266217991448867227906760794435100907237544327554270557986691882955218410571306221926903969031666712591420546224682928951959493419031835665048321150264604145900595655189158160886175462964866710786399212800759725553790760635122805753861245727100816949, 440598787057466684619545132629299721189827104158193280473813790123034537509), (85936474924167948518748078230055003684667250389038175498569409712319824264679002879494579946318515914835752278549930761147858407369628536996197328229049622070576530333538067043751296818289187998735790874044462781287089132495264125196416088781562343267320651536608935708084398626927790978139669796832801526939, 271), (58120306745416229395334668747540752540937785024156114035654055648102253443485587942932336676020432459002473722606237634960663946083585457506612113650900273295728815137145870433972268041612473405472684733982222822634265854411407321307197814343186570318617384916586832359011139044576954863118014660514242979613, 271), (93423033182132874724273923347407676195013060677910975938990881714220132187058700579157724807604757804280085156110647092293356631440857345752951230916558512950358892150464748805325534078381790204488362990455030138630024592843946568371537646263782897081294449589762992155213358880751314088070811547446062145827, 408848380699083099417581696850410266717032407948985863242249520850349264037), (54704446558875124964657246868073394321498490716163222442224892813551512500637500556104444359925677758155695340140184144243176681350023460591174131001974864688247520357417321811146633485974533634601115641038372463741141662980916990764680232190244792586471682500156774404462655388885209639228886306270370772939, 815069003249042666697144388320044022118746634031594666365844119022702553439), (54548288334859686313553936500832107340672950649697261241575917250499064957847701926619620262702881889685767034134751155305045438634555397707623828397434812769771008742264333503421150222861249508193718215237226237548404215474412984786104360107407264501874659672935396917396503654836832084086451652069624597349, 743588523833442487184920877905196603908572428355014115793995325341111246169), (85323453305994763297286537195889137723603803114290638032280184356228430364268301894375217937114829493894537787299522829313227023711595104426240885787204925748703841910952197442263169575720642331651757659239277421961937622742780301496041831854229465375365632542452294707909230939904950970348375579447657047069, 271), (72532717917348057154322555551025899985268890746155435015901924264359155597110109159112504406312416307643834403816752064496911231079916207787633854872885634973192613410732403057886385782122279039275607789112565318738577289096869199333462377658540954480922711235852055464705399845945079950416817753066021736939, 271), (114934628514587847323385172891374393818919768297722990671057429723655488334837732647253915415442035406931996156992556328883578884323363949020300156787193818706080278167351954777126965053219503698569380942826096280289435969924017688619362306688922291320977397693747442437723130197388646451533909713105166735071, 405519209672504235107216111776129978914449566491806948913158340532426148233), (72658207733088748448043947072510272321174346688283359427425370771396507991762956181402078480199807337217355001826794165363410050784317406327736613983404757041897187200744229984365473317561363673880718595097551278861745815645872742260632345166849213247727051041128506436523327114149514844628501392467755316997, 797313133174218214182717850249088749379781209435869136314023108709676156189), (90118183911827909475923886018317455139823747796744031597518651679761513702483529348474889308812722670249363121960252641688635794531790353719693373907223773987359072162920094028477460362595303095732171302711709734912023927575798721512082222869983629789147951179402450503220482100707287119406367718966668823787, 1468998093206669490487739363732044723385016207128320985670733012497921960967), (92477592383091511896236952110447186159655323755604023140088262668571247336915784120432164743647410287556632239316661046272660390973351031089834566181330057262613221604090776709307399217568589941546810176891392365245390881381658093814204628900420713764322220115653910076331357792453197111004235512987987274339, 601299171221622101847854496091984963960370206777288490436560045855391254921), (81119075588418756918030496089886073073571942374006096018009458739778037718970903703972640333651376736605901721579549775352684091738113111092583128210480284956545193154042324738010643057199438804112325720586148599468774000296738728825338724700579902797580259535629035520994215367482036710460674681933670250901, 361272207159324023255936461623341248417900741498248491961654497777206902031), (90306204146169466428883436965814067522200383363634217652515493715003384862065363307705246155829666882272449655662303006515845751577849512692357216713138354400698476499491165153476335946623869751208220031993977547387065148563726695995162010849123551074277537132890781987483998964223499654199252661822350629121, 1611182295412656936517086102776476782613077443621128927297910376429640708651), (90611933163618109517301005291748310192541578869586249269843284768258497878973423491731918572024877042420546726313551980646472086818554940533356526531074144123949161761626885380312664633443278275947802650719564393087689871363446721947447156859676436293812236176380546038035976552951517768606166028446675224657, 1879), (59561865775925737046552041623288312777815506509913881069294746838055750912753788574455879878682101492965410948745724228950487369258889873751932244346297987361625563989318375517042945381532818451020206866779159197294332552004927699013023325234124494406284768288688811953313296885486425233678064323783655325447, 614507100265771702565613399391397031904361297903603972977793600417951504875), (98374090369012388023982796070463628605378737284948959732244632982997317107614776572164246374435351535090123481281032631628254477654942284420908207123388972685690233068093330621284813356945466529150950433848111203302471814215517137121273695952378599948818457916787485718931060402688228155439276677153874123071, 1715), (65261861196772821865763315148585179694371984260922452967330146629871717450117074777137557785311340017496899453145033107687879202509000942238336630596418482183014135235751729917890352891358602414659260073190919995220439792941476647205458965981789320186331354151155055563891436719367729413851693078655410766449, 271), (111682910044508777604791103576767546846332440970545001992080858277376316700471357864015732831745938790282633324438799772707612909064332475314362270642073498844898346797437511388481548560242207961578160186788795965539061858352891776988194831640384910141149389272254015893303689420254682934293746930832300245367, 271), (135657345424035060281857483072738472428904587418435745658041138287667277807894071984987829278422951275762169406497055953740040430352025634329457994798706012973755500211738927792922606949002704649441453192651932825144230185686341175956228316573296127210212446353264131296905517919284288546703278689744300415769, 271), (89153425662002067686632032759544717403078451288675897748951198477109902589426714414071012726614379457777563648583485547128972872854467107265274573777882140029051458524819194154416143415450651997824245741048212000298557642572065376721328045901999306840018729634544685644146012381597901636726272490595020821219, 348570606362352691725502416899258993997044323116790605163105093332643175893), (75581614704235112263924167837733628256311496971967743153958958009667000748322193240779780284070969903528833590745264807266672213151923950242145870021791066472524451140240198970798075009589442390753251691056127028187397639619050488998856208838423651337246924901537166263279571160812042094478414752675556357061, 271), (71521286930055322570354188866374459777714497056276548064282777335734407640519355362248492522645586311942819719098721606319673544168814527076904199650684549774302639938052460610935823492591097293463710463152014350663183539991922577989750073715992915998111354700714204312081026046483756176990425731475304155657, 1339626317006216491007272486809712715698597935637057321594183545034169860369), (64262360889005802842547432437891749283851366379136859294204495285384900852005701975685403245852621780083221314624272186194076767800977695207899777213952746447988349730960989605172488972145967065460874347398747623299942586784014540256564266053593773390784968069482171344124996370702343493234513571329785294297, 390469708691207913749502332229442553947264084269110296721274611887336115293), (158406459235204873574670274144597002568891810131594823059610123465191421359014857941881475296774185811211916985548639480162539924162882110461855490560202996416871783845578648110020601063035838107314731837473710150361358755841394877626231054266760547402932903737822338173952426757266786940715319271079764011027, 291548840276652493746816644839508650619330573941950630104877584864098798579), (90836881641326260880626257142858162967456000326497722472877879130244554265761019646722627213905890836187219648852377477883085893816667972387675194715593525256739373698061204645265194488681628362359107237407540243086770133267799555438697090265649761910235046975680790019294288427170568457028056743646114054347, 1173956201375167873822513870504990906749370290725385035213370373135631056097), (63476153151592612253899962451980876167984344720870288120841632681773419805051547111115862040982187148644272925406952872252754826051643481452496005510055712252752370186853905865520035413519701457914755965272076358153006180796599800040596029988852063422482000860087877408470911848751353784954615868481884275429, 271), (67778092709498970263893197282790117552943659657172991883812015713104016712691471474046062025197973544329103726986498163540406130205384202832399695915832433136762306910379644218770686728697865319988406818848156801917663256359976213650900131661717992112719865832737139721350193056130406167586819661857113130061, 1633230157773536166658118875271149711862403407812114822336314224311124202517), (52985101247066438736927112704618280888566883083125460162752534718497894019663976909007932284493263383073047888408733577589468201214756764717844993305525618470056493490392945810031805851515762370764703198371988202506336782088404471133282190980473640429865668489588346609290071899069437955250263358068181939321, 271), (99908269609801972268686686928947252952931800666278419111441396109369381563797307818716889440069970609313742792537771545430432957420168402013057766659620977608849985725853491503287380711097803281301869142989432258106843490223810351629444273649306980829463642059203527036055600625924843550310608717303929694283, 271), (142692194583741278670062613624263157100842457806247407670229182895867812367592832615799825075695389053302687764568515489719739817281689363137563551242402916667592222407342335368695390347655031197708918287427150204607900715895771994383642381057678665053425717948513787760606286350826793133844920969995301872381, 1791), (75831276095787579699541443723821262292995618670323083979934255029258293851042299647839627982266980088116700467192642989702934777728423454530060736507956326545280584903396731443815143719329489751427948762456616502821161074389968943136174071227884142603330769506209595640780320376245114468769429772513482389849, 617152589150280341634443305925887728055678565552714959499552315932558313321), (92603696287202457025936256311919980939096032504679132627290807906761072603234068189347454688281084035072772454926446015810048975006948596252138836940611615585724713371761131647727118646893355797736328212979573605626796025431874432999329746216578475360841619231237180195318624021751350474298271170015323807997, 1509566013386387081020574517093747437793039135571125754198274314716995382473), (78165979993359749955245717450947103577851973911081657829434612437257656509935940225403832370818778522424854401862490385056221491659958048684364005512056059835609911335470583842506094604821793419345010675953076813410123747407577188806651816430038715770577763607966923589757739073472951337204738955052889244003, 1763), (134237366083582837067929396597360209240614296543295458394959242910995388487559904747680758962780483400215981217830748366754530273922990488666677909830715956086517881404746018835220704818159925025349875725391641591482344366739175436898135508535447858947173487209730464637850789984972026684660635804753239729147, 224288435527375595679990408700714158312026236828079776947218379336440063251), (133618263381598168038343523439387347321176781908446089490473352423149223699452643746868200719504621595770387546262489747995775076254668373162867891625974151843971776871625818357390432197750100713742643731182391474928399922197179282265042882226144566508788328607278482806081355219021740281547152826506111055591, 271), (94401857795116574729786146990054118960862693448721054071214878374417621980196561191230519695226717573582884707704116361247421875757195552802549299856233414548170120780400794783978051759741979889784483508381832458605642144559258250408550784990617693685137817600086688910992915140352968804510136728530387482551, 1007618860696600379168377663804953359607441696124748569363368759691534994759), (89449677738496071578904717456523817643091758628803642955847064052445022316654446637822668795969195946169235144001059622423655831697571381067999237465379329640235019164674663330223685445819531612214654576117042243461030495870392482488406124004394927832054603237782493425328116649609684708773504509323623864817, 644960047214349176717812146787737086792911484587466706028132494189723483287), (139521771105660037193676834104792230226991369793611578166559176476742376464154913026135862732914183389690214132733250649760680481718689921099223005290791321955670530920606206239433559246103758675819726265352336451990684974321358036973523775478616836257097495952000708797163899077560484249626483779415238637049, 271), (69396284336393827313619304884039815841719278583246519301560663306055588505821318572255749785847435152258545074073131400346662911288319166932957951542174140536830809919466248750697468801923630295117007904870380656995850757794118213498901583535283265419776074325241368605908702287421321132650741606416874080903, 794255699782191166382660548954063974277692750169219765962718913707399750909), (171140529770450398809566703883981718167410155959797545099675427862779705770800984476200717098186951839495402206464033672166039705274852193445043349471081853106911768479925928293110549158419257227357725458372878774469956197920810809638942148265612349448291746856617700402989733518035351017055800098086801887217, 1436584153699796602209045295677551082998753333529117470130119956914046786519), (128430085808078494175723195545471102120661790531668996299267606426613857573157183891216472219784670862171432890974868112777486052676784763644263666056424077103852620991525867350063400547861779500296262357665842923283717661744712141915728151443552456724633666249624227855373247790538485352523011842367235981483, 271), (110747283748326521678045055513475867104840605512840878942213682535908971409280813998135880806218117743565814448348702871567303128393776440295878419519202065081720260722853371405673762423910882208264915034797286418805018467455356765249405362274281805404130865523784216213275763576377119218207184825427815191289, 271), (144606340632106518236151504106802338684846228392205170154415929012852693368684706740910378338203308051706103015707318329510914637522161625949488196236328665993115302990455460362652884617184390921081789900522762155898853700233493735670398053936826942872906567005931195587522863710261480493322573992947794121393, 1676173677903750638337242229538984974528737405826169585172994146387196484591), (130682037404088431707877880822629631921275107095919561367128452807378376452797487262870687990326755956388521061051128710579022501832185084169251570397883374968113505675347834093343857196590025377051512942998487728132290506542549225547534169825636913502584515645742178467147638518378144616163282709995721245621, 247681794018519133149464072113879153292917066262074557227519302555825785385), (130282355941046063876967799627564554629496103983656605212461421942189751720861673199743298181848258672499238607071929266246929052032234822165085192944693065022682207833476829365514190904988184298857140308503342752203042419164194144189175534112951319265720361538190332704621322949726133236598345314627341623641, 1750878944551044676490862872477864914736260432153638878771300123585823225175), (140939321979803916165633315459200288513449808178872677807750710624340117962280623068885123649374636319872730899399091504416241477868010507738230029411624785800183930310308216211142910481461462027768847254143586336566816742684920053851639175683122174103532567813410452655084250316378680528484724466591232527533, 473819894179163396043028548636595055228577128510998104290478815724306725919), (89932132460522597157926848433550986824947658587213930156046182804602914732555385386911072563433773751750035577524779579138009964966754727808386632656446672033526478879490832069095644869064224261346611523310158840341039446138220243651746476093457968655466447663127177008853784075780559858103993488285868214393, 1683702143519557282699128423879674234322569252258117063328967596783464042133), (71614360801657209128945306193278668106813663030413843823432564610688597808624802950285784466466158901869884437288907257530212712430031798945316994166397499038515647426041103316323600729078385283932560969753200901699502797901630101426631875419884714763826060386873554541504994979736478377013312952631254495527, 1274159557814471295100720671711619884235771135406135331497861425165185752741), (99913285639000127204165132444286375562666469639245985447203947688730797025695159723127233975889066317872711955244186022983839703549037769011094430749873528004791345116800092551485959875712344245086657378719880218997876905074020638864993012412800230119100860634572374859655659283002072899534391117359913307173, 271), (153070156999953052029291917835639056817547430605960463473656869109955601022256104075273781415515742111988824191499962590899644188232500180571364367698176664304962194900671537932133955531721937983089730149953854611701125090754531325953949823196190655905522965476528564817789893394017470114071555209271161629829, 1537), (92853038209096334731233873179359365646525006549727635002782105543554436624034090045808334424982129132902564421677792541149656493101070068284881230615348548281977192087403962960110001267490726244435109253158598755545094079331456740822825022444285025553425725246829969361168319898694144017234608102430049187813, 271), (141953439700405856307588233150009265905181910598448939781044597523932434729552819072108851548778446760002866719683682227475570289649382100184240227946035131137581313090843337640108860180566080593142347633106824402440799327445284956438705680135771403566068457296986758963912994917213437738755217091575149040279, 1060915379469904661469549045007991608406035905468591002255477379079622852461), (52187909562926659761730203858447745346079580573787797044824348142486421113854145785411580820261593484485569555466158018631529065943023130238591583498156871377802634642416617905687846257220237977464366722216477131663348646285357062087381678824185482925707955040716667021601546601567192336252185480213975919761, 271), (162690411335669459300372464296744991691481996313927073710945965444461866257653594748664842802015043052223453390120715955351488360824003672349469434796658521572118208984650936409940212593755671903476694434232444852313733618392629182677212815592488669627734844440999690395985679752977579784068325616641582577937, 271), (67230842015180127428924986838505549674920215144300328701119922515709245683019282494765803060299868881751767169225132738652006131903247680545802106628961048310326029379242587699494064680998817509550391833364646065174548709060818402257234163937004345673310599473059437490384671951116250871539048126890552090999, 271), (92196832833649948719607553883622842326083947359511815760020383914868778805048410980325783587592277298690410314632146487149470400669204802921064246414984807287745094859390162558138561867529285966215644932619036901182377486452727773852414613549484048055718034892273985044431005940928352488314936550194163922639, 362718464726973185569419649552423021758223063602441800414353494063545751287), (141729981517677635349174060986519695837471219436150583306835783455917308814983104511363917804603013640653335053169823040044898470050408650224924920728776248960271808297825174264816166119286386674589540317868026283313363962865007526272756143473674745048875844100832353393705451037899377585181596466756066414501, 1780228411939194326360925196914354373766965662286892277910263774816976749707), (77717235006525589049096725969011339696625400026819371519362896175438660274539758156925504886181014473866225572700299821518179911779919351949918447299507001951918815124369033454507417300367784077186931349416146190644803969875677967879336637438009998863606664295653356169126173890368442388335409782423113732883, 271), (141567755663956803401267831154542142165886831342452447938996545337402511449644503429546724682437210033701906271518159947178342300687482388819738858457699816236231133717964086948495072047807715194774939261795158316751061992374695802069531982665889934652802575666638297858277433534137460904736397430302144566857, 1253), (169545591079561320837915796663873917873304709501621572787686125749687374029628340599579947373098165316254442761107150808987242347550575342194372797830746080744769754763592150993907114830710551916197878344259207655776637475424118912530096398457469098897968396764683583239034388346018433290994017206108598906841, 922117751603961947958038534593196789034096099772503160510017234467176431461), (130458191295706930204776368774603853280313634980040854137380864058942001492306113952426738415703955804394195535215230915946455076801523620393111470812455093388178934779550936646774130624193949716384624481621771934090239809584891835741648895903921045408524992743824472737735866548962480355646524880719967000597, 421725046568799292618522539063359083208645574010704616902676346822084070677), (122311861142854374890607285390400875629319066085697881821888518616688264064503337094146023160827907304762453044126610647967775355981093928738460435855325666993029331717566825365060200758223997340814629196028140927398859992904710385599691937483481734145431309409325944627752966118933974205520177919490206644711, 271), (69693690746138921535626744535213101977097257433345636990203583708335048380206312509150954525864397125819973861215865824106889227282374546893413985753234905706124173296058795452813787048048695907482610326624373586502781354033371930522419479900878276536902218169842062551493109721908451318292750203157721067949, 1915), (101502296898996753621004667027592560755169931434283853626652138358274773067446145452675512386751113776895762408419754810190880785213482346096094569457683447130100708099558035228648420753830780819940553523418798390577548100033770413338370472365653084137063992399024508427025025168147777260678874684583541157309, 271), (163238950116802335026757720734191624030498146708760362774715477662568098260524075092483443523676628069563184089636405468009267945191202916578583364607451079047733065069508871509493832585513187880333646639083506470282026161334185953188793120656354017087012882881416750158682991807915772920090929744284176677753, 947994320524854724287389492760807310744568447723355404823125018932862134151), (157129034117768794882459738991100714192325910394557903345732623485579313530980918742558290371781794235482108288068446424958435585287362958759449519629491810568152674508742374860178853095900948322214522506169652777180495131242234343175855027299315588249378118933039796390187958501519536038165205102443038367193, 598923104647839249363950265095084180393406918661286959186198238579156808169), (85365349416626478007987134254292778584820374723999673494948195850582078052241103825704478892218254790134208096968265140624227789377810700347853607488225030187679018397357045047898334068731831748010989247634885367016042051494525152368339261009742816608583618055713261137332941537836016745198479380843667965683, 970173155184839351823373592056616885837476436837616154631576261576381323361), (110388959440103919315775540008471051771161882666678783298147886966227065708928576912616528789324688823763920754904885496085971140498712323631951544024887359675126987175661180289983231474933778707774811727914739011085447957965927770615138175234248869822719006607028072346447540874936794923018764384373169050117, 220757479934347490292799600381425748759706238677761660585167947960683191953), (88884938084742795403840870861893636794948442780406968213177851670800364546545585985970941684158146381037771906257516389192482613321121920237504992109421953758837698108481285228361748854653489115345274356410640762483614886966082060243178494093692811827791915561224463239305205274406974285173911211484922011321, 134372592831965668344239741906746871115183383047736237175118476617427706159), (153731276694756875740374189910205689338484340144854972404064308341617058532500588325847021042031848429465421692454239367577842036625593460492914291178688346888691980878020503239953803243583632188403478593796538163949767353941480836018375109742081025513728268285850075851168397212523839136845573196006668406649, 1543226233713304257449769255572826899998332128960925192895484783230186491443), (90240137040670778269859437667935009319246589857667399220975805337527590690585563468327946090467375809097920929875503355974561184038489262530676107007034615970294758222651002893333424822451981919355653077797431162501637008895623628603611690448370855764589001319443065609920630528873605452587152771977185944731, 10722399232396665403539918424300497380517139097601307181398356785132202621), (85832772453192540995490947609490123489256998852610771720170287881012034486408777450512121936102647102554968035039784406217170490743102840428041406038406535178911003993989215330826874082612433156317798135935388844417929317425144775361320551447755393344812068627066291705272424769264967175409370983801179485467, 527732733482977897400347115749123211249837547477198949138391797604186438545), (67196443814388445223274207729764003691241546278019494189139754815871875707552223706076834966825904231027771886220623346575239178992103445096233299825789326450312492625935998009500371308486919588048888570551223377647707374247073797982275860550344467082883383458105823020467457353576558281767766041810458654347, 1657), (149310231212416422536852116651013720601245205040657909928601677797874721677751104959803023755781586162262228848993223405043196048761110493769061345091507008959618771889941072000067905896579368236198546015067372388041772696305200296152324225275996569731054785460152775977809182226157822833499474679053782356939, 271), (118701172144232125371323599887825483588325459179631481268279862839101909421454996419696915352749182176471523001277058690638431511313247809563304040370760437926178869141284575806846168348972512381582624928198706232474022966652989688862409256702232112241527097091250374422279054480207766878352784761278578332161, 271), (76227226612747619657681740418127922915593217803112619529657556772921478517784889918117882036376803248045449444751259666389982307203705146890196297203206961982920700368428109726191362622867917593355294603053284462906098695784350505628086300919243226690347705603400763265341075491015086957648184412491465338643, 638093139348244759164900696858773404427168552384861993321072447390329560443), (60738158086780047534675764512026012983608751408196811530749537095638854482824928689692361022405831213642667740089521219413518412672987465563695002473812518542602890986899508928534095098920571520927210660378805843323410787564719597321012411139681349630820780689044516134154031286621898490592438349511463351767, 271), (57200512899930825514430163601882674775180498239847694728412674495858523161259691950966900570318919825255895227233388010897504377301668700942077124009373561821342836795919682649272044196887579028949151319681176573769461589912032495371098509886219695198812221040947326060558858073137727820982271965761939765661, 364146348157644989927372025680552971660570224238263939165266704850818335965), (80281657910146063753654827528674518845848702196924485531794026127821911539940017425405587791559310081124763621738637891195838736625158816649017100541152632690487546460795029671622870695596273694082002233648446748203381824302563590113731824998061715311265342793434141686729450221748583042643993626959708463583, 271), (108470476629538565692567619521894477447707344654357052074560458915536698898624406534992016104612688243339359940928416837264063935450326129315473504403583062752527170970340479309948235281973691081853223599317127161513254495455018441211831307175702698345467953412039879052370429749423287820972417924263890298701, 1003542987982509903299845136955954143269802548303155435203489734532001355451), (74837123653254399865938586019311425719580017301039104322151923843621434369551944839440213222207515550039170072748170517164626694007097048294375451107038352761592869893070479929745898885776857674201698328123676901054412711973120071912457974866810209933514027074343324204590120558569822926411157456154519792183, 894330882298412697224024413966568071212176659244165592501424281652046903233), (126845567834519906988446649773033214649409322633237320415115334973156017415670464960905786134688151662909774040361112438047926399664301948807867358670666426233991869215278129974936324770610413933295447065886576171900649446335883652458974343692119604166132503741733294202189417086404879744466496083931092466021, 1231558698827146541480580991472103261023368358133392386404604668406245045391), (68624554592028226199843144021146256559574820651983197352273398200110949509840313303190658881766630025605501595647952565545395953542793137651733424404614181119004711296413026899552979169453817186618691000966083913388440393221227413903096202483401684754784834198419926069739758297213986632961432290816885092949, 461008173543213773924123193895593861130727449220008457668716703672802471141), (150495563930209049380468955006432791631599841738791424316272995043199706267262933918534127218924968948447933350319205198291773896787920508326246508291944870436206780938248777840307439555148908190346546269310244336759561548039702792154103124286534715176859188428808711831696892212804854999114906746501068454951, 1037), (90017233841256682677015405307675344649531874560315072627608779741486884188723819205193580254692739811217587976376900521234195485846477730230284950524389593901072863122803844831149637791944475790997626642049350447886311575416815537699939242441797163924221409476860535389810764382004372368591657926520272672857, 1560664769876505969499853975110053365611159703704488523285158115726671567949), (71882835550098126996170051596844891988417462330295909077370543330538526525378623719836092825600021165033600187998489973681752443372930147220233747599441829110359334819018015823201292983971439066739000479516245655074831993429857845286408687301329568367062336779737201873366266565620673529263759954720523378743, 1194937228887787192496672693259124008676227125841092061251173458466802970093), (93112836989618558707146164207149657330226912525070782329183886063405026983246394689178022661514263237426456252174714257768039551543424755177805734428500494554976508671353787991361134278189749176892053295703398840689622209886112954283700625208021889503853647335746755974103646035067031397863523095420331146519, 271), (80872657742210001140540956737695258378174200463456384559778552073155075792375497537989617528149328757003282566671420802539272730334077607063517265974394679588912993000700462419752355465243264401357336799899127618706973519140436515663087539994878357286054339387944490322711638340262830547412917014545082861479, 1103), (60168624771917034630013992713997234431110945566785803912652538693531290501821421206694385373074359801733337849576760729636988199990214910715820699988233952397143615488321747878641724680458254531886694528698370279633987615040298915443771958510513495822626381885114304012600513227432649236356694575250344193423, 1011207502055591760828620548495339419704327427082921702184341984489604434557), (90956243557047887017143246557468094166052402979341184984107970641408349932416826603188693558909729293506350980975543667731291076891030976407347689946244934218311030495978730565479504326182419805595241154155022735134026842042259443510842076570900714117530693306254278225740233360941056903526595260680755093281, 695221471414505029675531277855625999176112516823816514338091137458264808697), (95076179693516868733142770726681879818002157424160503494967351337008245068281774864310113183181495478309383329555574194592757514653436838676924408317992227798249460367779329821940555229719415574203490476604051717049356526065371700033610791869758766626489604683721884161497318267183962772166228041513239318377, 1592192033461441072726120629123993582595540258178533431678531055548719843231), (73877673423416990460725272344580205402372903433040769323489702774370656672970491649685729897184367658350920392129048385165990517809438959636378720523871843417774990756096177792731110096541753123022939893539643746422899666838729537979965953444434458723585988910289313721529724533968570598194271677575444758053, 4588642409497487260537696942012845334872553101871205225186774956024155793), (113568022475651037334379713288903139821838160304605106043488245179413652860702954947774340755966197571128522513153704164942554243141169953548387630482077474454211529178042698233754108844109819765657269259829648936087199861310411287675985623693507015137205411378868109332044662043365603600398601031929709681403, 271), (87281434385318468181540758672263358066312181135758273874491547950297582223171772636434176410486755920055880692137728282410690879045555405791713386643254030262261528147099803346952295958523236855473095222309906291418063230405581416611849313518442733795708298170851013845372212125071915440650345916677576876139, 78581375519605165126104611306305763272600943787007230628278106922369889905), (146771038595964313557642069698793445316378167671568592473222532300601147226832584408286537829342432417743995416192626375801085803891804227665150249512363635738516792895077027509746550353054131743137088101619318551165774886909193459636541319576643749055323627659739877328724851288212888477243811933034933871741, 1092654991403138779826169840357019670359112175005015959899600001263025358707), (63364655323340975041150590168431777110603281177748745077444272625275119342133411367059357535517489353529371157894613570132263719422343582042712428428074791032767500329552454717053735647283418183729543733096627944290841960386459868114026733516133133808664736173022539600831895477408762377799073960744045732657, 271), (107525605673194090048069579337615346661126525247212680389455951532634229681720892719873327043358342890547166181235314322759387900570811792368877653604291001539610710658401348176837910855000135663355627756307332569546975854780179105929559963556703421793488023757083976808256336781864568705296044233506905564963, 1229), (75626191957946081961686901036408605117099333518149999233426154535889540882752600770226836715424364529014499398938956516348151192082001811066171851570021911957579541631858237583125655252774368544748501705541048268555169258054808942917026879037618532742940347035496027842382291927772352161884779080564021174919, 862848800904401948056204855394629578089625724541619947065600029132825470071), (96906971212010778629887152830347605067466105873623339423378049292589283507629734474414101952725878176820807448880773457165897069719089295898474265712439228925516500985829969840672462284149352464271664447775764951846129361724496252402382402854547409237738428704199875798266128201658620691742308605991685198233, 1647058480228284760216416269331990376368085473320743859287274953114225391751), (48952513610370954411643648091148755896024422278669362333439458240799163647426651285294184923571712166613200993009712606767402916579339559764409028944872749690046949581021722025185168954689289210845056153355427540153248989445716795516750396582536359434309771737242824553828925378708921314887904587944402926369, 1425915976504737429231568910532100988225974515477727450265127393362353502281), (113758556024028061074599805709278682112452621242819062168309898675529858635362357482862878755473922621105642612214591908243520989241201742684694170296224091674829518201410804794635749724254569787816904121340358278100235339800868013323741437518866939463209116225295077768534138430071243846164216155673929416109, 271), (91105656768966118200688403277411983491914681280044999871198505341390313614313173453188340977741192494610358874276359407651060781530790171467330301293968414357631032145920785049916811133865299932284482520411025248269159662635393187632627764560748089431845873438212705263682972686983707850640082463975960852383, 648745265185451215125081636254297930093996114909712618269541730027927698583), (168046646789116903429580734508396515989800202180989651844113653337404955050578573228945118843846275288401445581647434180550159598925317980460664590519343429335204533349980824171470725797053298568868281785155489262603951963320391730862181404781109398936521589054990037633848918667266917836924475919644282388761, 1710849319588831542952281375992232037793505035206974277328719699742238382477), (85298347867068705128602941883648580224425407109096367606571888223390787552345347744660976636635216623024498367605753850179166667719026859769960370609120977188838548665556930786004911431244591410166632753699716354626102410893598782067279336842035875837005616034172812077311478811739670188716041720147984374339, 271), (73090704432628376751963362288745837146239836559737244006913758257506593984106621046286024528746912125407392668492754159815486606382121773998084638803737680631411562233136936983864602969479851172012125548361925680821789973621850370202208605777922856573982115724698949094615404951588932548751981280004098722217, 1536139197665324028709471836308865558413988480162360297685502358616417885327), (130603546751125188906690497666452211579203091907051705338488457042115331129834369242736108270300709207263412722720779644139141609046996025117001694230722237226351365915175276307290628805060113552893180845552207689561964036915976602079230470183626981514564556690439978056230269693092372385235627525476104855011, 271), (85045179471858195968195828338542517831625468528021232905120606201995234886576247219287739883072851629038435884171222199226995296670581839848026013620475420247023108014718042619282800479709413672050665730596141180157444315904558996464006867254992981266438108906146323761122860409943103090188420138630771712827, 271), (92892882206954305952068011351482593464560547666809326136851879290671150294813067172909425520753106310779077128193182559159184161144776551223618109935902268937879160981482200028976299174071111839192965879620273839220777378749622239798120031890691558254411352181388800600914574142289954127100028224400967849709, 271), (73040770581667363108823445749537391417906444335403208136891332803230335677907141672751621331340155917098723280391905330848406591201455305801130700713008074099206847748202936533163733644450437070665292660974982657026897425200055794921069187624511959171793268770932469960194808906257987626286550611425079996093, 1439638487632970551038352522426531396183706580179991100350857765632154430587), (121089173012342315145711906814134865760587724985504619277747662247803749194252106076696295547252560438985130299020234963626404364918270698042746504594026970053426270160830285792495088975504455117239299776429085517198572954702358005078860420621668388496328343016111901871195635305194850417174440489569189022331, 704551836191468066876918549611804060501415778210346138191645403945115536973), (114288277226172931587590834879895481570932318921348105241296359911366719018013077957845191514764487669524768669224940231185831864446771505493765348719251192847899981960366260816812792486160459782661128394449583854724342742106496204198842070052496782025470524198595768248321131750275017807296373267945024304201, 271), (102954869227815681593361032771221390526322004355208164830462864905005563214103433464569738774165350813384994841235189229351222601840642289391050941419436755653234118116274288821947838865144502642780847608966574555645345778212963811410689058092992647127347648727190319200432529770056549461226084742779277867283, 903565675679897211508495132682484561798041578934796323411570833336122428267), (61747245413068133195218961091598195964139711338373259505945345735362356346887240062371776353448533940592280352357092698125222325455844215275128648713270291363909356053948686412605753095678080648801222490130471659139725336922887992868531738279613494149922395787034610365443011382778720300409558587180847610191, 271), (119883146366586851996051423825756745314063744110476681924262852915848091707852811301443144985258369333420565009006460960835075459547547503223734295751128111923860444684885421304317797002947921447921526243995680096366815499144236131191504527256067022585062206901420469741695738610295321269488028535128897798763, 1702635731612337134911361331795333815386472667643301354558782454917086631357), (103639936861437623420117631853780460583028800270891538501666769053879985441622041383191573875482803002659474756254845033274128876604871146126947980733879683121815901232260866033104622373029074404008947603141059522232471241312922134392017070904686939040761765108720322791501782322985819667633314250571836613811, 1957), (149849926270771056393866368364140388673588498771416950580136210574314162732058811493874824165355784022749459018305741769519885598667895050644983197145683866244372411300173158829223578198631217459108476096933514903968027832896944022454731259833770789269829666059333298061190305313586413020426217559530258040103, 271), (139662565106333458577126151637903320951038939909052363026602058139538054080440205316222614634320707117906297399238500809897035551884608138807370364965141900394697315929492500681996700648933768668139179632895136140844403308287563035931665750490523985501295305310196076106462414399196884922587213905129379636131, 271), (94109975017836175147650578614070389468743719359561732407992567920186301498433645310731939770987228295216111297775513432442811760444699667606277893473659564657514049913115103883766360722888216010808564898684214110927735483032801514367341527863651909196740451232248260472728174265912781498657458788366270256443, 121018165139499934886732441457700469394852888844553268408123492103098828863), (88571964105842063982394417962158593135362911810099076502323312924022717209915142203182432930908094559966999026831945805523780035705857178246466524061319537572009424206227728371977356550115115403132775416927415738714369616617128831161348259302985623759945315243487682258483190266980978565724243470086935648311, 1115181564713668402040860971801328875845875286480141149446089239740095500657), (55904720779430535925807579864077752213840608620917951373282911921930016625143749597382421037539358459598560506170666793475116906915477810913542215971890398507285011326762006830858530583668761934584889122415314923481336097972004612308229993717894152263904543122850314493277324106132152092585663850744260785121, 792688533254609541741780090091720742904464908107977594459366646173952982531), (89328805225503091893428595370895157875460736740066904272320449166631479324638332762678098573582968675012207817864824108179574935447021543982833380547535413956453269266249854203586991358276528367135683197120338339775281781737935664129104855985124896694888978639865641872888436089207764522597642807159553939519, 1174984567167325237332640625548891343523536585416770544145821759625940635539), (65100660670980398391927815040969577550354963466765705929392884583926620823704042910578182242060589051852364048305417136241301859293878338563709326667615505510424558101974049245762845920583274762782551028961238014331881669988213699199511960061404536466793831255209506487196539378639192384472279331380913418977, 271), (72075546933397231788435229147458601825323555271849436045791404931520427291741037723313854335377172137294463586507371605913479091162540301725606731514829048309222926501159446861227607065242043494130992811568368496965757023362967669537321169120463412614952890634816490196919803607699742059629683226391831068989, 271), (93574283651744789637949702810166828649042783229484087987281660308875515806959811308043122392450477670710412221259206290162985371596990335571827659772146409264157244795928113526852797224959831882264085647215899038389861492634820941568321965510856480621732495120373195396782417750907012006098655225441325610329, 110138198004850422704472834541767112968783929320158245195386211512949362027), (116541286881442852528486275813128996854308369089914306814833498089415179256055654607820129666761204439783438797776508532222730558903955477370519809139138996297667481769097443593599498361381232341419189698950760981468352897601835383598378265746677583982731407951610780448997441135908512480080483203886024669397, 540596285113341167318416906457931653116734752918711269973374063348584893373), (109024131550666166803476401041541302434409830994033115916337558548216339595961091236300339870122587575988504703510731494966658498390891784334489564770604932911483386648719942982226680449233515172904640571091783230929064672633140210112954666901159766914312115488888961486778395736812662224720119940933042951317, 271), (50055857964246789633767489189219247942019544079049641042576288558683240808518754785614842024127318302288844575202090240704201163456731576158908001564834441370842782531365068575525919932613345285111293789086651333884239200013465613805377231432736630559751968001473359834386242968280858840791914097655083258247, 310243045611667693799113190052984307134099517293822326307716446402603249025), (138673035528990853462231605258268168800724456372924746093934821617191472800513393691365522223227004814757853138166463984137718743526355901916737244376125129730935338226154777685076432384913749416097606036183929387485096849607524901899854369265655624416614068839203861258425625543188109264350311990011461713227, 316161381896603592019697671847754460904416175334393235574578947103275569699), (81297506145856567834781118756649370096105934684477937878912552411384541028702210210419228317303289771148393738990478473248982583776938657097686099433139578133313392307256468926829420398174936526659186133558188579267861531218849799464358559985426377981475786009295017015249921307643282553749712959516424465073, 1672048779049230884007568472692529573836836523518377652405861622697970307331), (108207091469593922230516227754900635686911881451028304760148589600877839026490440553927826296850494067437120069565635018931452589157542783822888175643370937792690804306686805899365745344208244373019990160125070562848815587529658457098058014644111344739326755218737301764561395936311654738969761273480103391363, 1752866325960781516227058124396124623780605752283428352280348619160018593443), (119992869604021078841356154623512733907266418864753243167740768607116342318355554569392600403830488153427759501480418927699371560200004388565393000160274751686386502864418684683805931715508733259319621255192096620927654384430170039193818554015726074342436491636603625369333270203277363696919778749652182352321, 603543660584686546737613577245693769819402207100548905147056108535930008455), (158154027458781737381641984542781636478716823656250320787285411016441665098581272863392528256688184581614804452385076504516487027342173892594977297779618385547158197961508807066648243158982757576363164115112525119290672235624124189778041067487992719186144985181358160238737719052905491498172241520193603273933, 271), (95247150384941190890284356204755321725772079572033531131222819011157177071638208670199259156092750035898261616975980752526387800781124996314828512034933799140767705427466557303387998764381417774230679306121669106790471949205062871701579797478473765797255620103808730884343759542156904244872727717663068415279, 271), (93102610222326310019941822098920155193878340370881433273149386009557854580977737102052625132526428657936420425909931121968037001040629058070563829937424247067404653073553248268597656660033842758140864588016704346405523681633266541371387908535958389711533975989767211031605334628853012041383277949030667182121, 271), (96957407505111819783705398761426524144204124642287128738917058791382181248816783533585694839156070746740235701053726952163819437978185696642507305775707775852766171297428963045761476077443556867556708512562977735275278898523656090094398804591562053104564299353099373897231749794418339253206309325305446065087, 271), (135882012837782611073848640364939293033587314063191977397403634250013098236733343219989960977709965645138229863102646992834753570806443872218616201129381392713743844385144349087169920435734048633146402212819682466874687381359056296908655341797141946387429789489409230257304293182957557375912164497175226019939, 271), (86850486988738293875035499328501256516203023228975373260580112723582497872515051397230433264767853332927866478183238375708068494183998707952723247841606649525114770674425074614269432217336074121242294499691892838042660011055876904871465701187156576865236827089409204691939547908310297940077639633688825075147, 271), (157906206749199984938796345105101242164120549403546637607171403386077006366746722142906427396331229489108644088677150684130821801619263695531036833632795132746024574271626102271229729048282591319478967548419837430271156101056929038628376668378751972873788949425282841727820099974471478299551597787789629273071, 936383092889939206365728645074300088308248973945382090558750169145699178611), (62431783241263163649083406106135979764744195926727867351366986120276459431442332501286741673677316229639122881599668201311310214878932716986508853755960528078167381869376265665227138938957306249132731854384238281420516219544000836640304584754347762942934679037099177961919006718517589748396377235527053872369, 271), (116174381592601443621080451002298525826041817555422079280934487075769174477787680587118954821307974460464485075876447454519301302052062580260407930636610597781197685123407407934393537943194629553688393957885215048396797674836459438841315955895474596162256044391424554068665757008427811330859068635617187458809, 1755197327167921843606701863378243851909958491102265208759688859844247102099), (86409030595447539375005637098483391919698403366168214468919174663021171622271009590791740552036956896354040061271012462553500269329549439071431972654892116901076995312678662090858360876257858446785711501897707759745764765066156943927682552850483141544905223593033761927360495268252496584503713353426609936611, 271), (120508095400264347853306060948005463454404918537256057105490433249547372249469338726644408940043435544917682245821311461498255991802706706365116589842789622386638611291549374985757100679217086973963445147700236943657082658689116991461192103336072943161457221388073294962104397299141689352085177392835384533103, 1418223012055956530208676106697822553575521003055290139049542626273867816117), (95874207494287885708220329701040242245261692228612468748775857985067907585857587738612702798636865555345639614306969671583185111788812191737209380569633999140730009086692454264476380949840237519184842960994465864948306343645031575077308009778687434537994135766294995255858467149554159717510782392566696702463, 271), (116646585795762810404950913776818243020286385064852058787477453748850671289931288349139693597368174442619030962038907899324958409302143520369127220918909521242996236925139265670987307671611632945557659213125923156554888536776331133694754537347257013761719617885588415129707043348611630403234113711880679049647, 1498468501743324539437495322200267209224277275586833605606764054103347036257), (99807423397717602261126053617119129105642376833294165040916375550476042471581176544398913487502335438457150649559198987274953061282619407005664360759170979587532815858780785940987144967957557080429418286652983808712570862056590217510520625761893892587851854827930155398845461696979299949010319373151860863571, 271), (90058799198092896247762932601466893511651246762801129812774138860307229414392355980219068264994405704806700798362067633370522158686994630273957626351019069973681014058190674587501811331354940552241378779282711731215492814286293615877683513649006476772349998999007128692254046818273518268472984173344214528481, 271), (105176486427350098934786086601651969148415307335275785791182713545251090210981904192149166214747144090040926083441136638107383913664038190710501888495309724932530293780659428147075764815752596314906427180090387468203864673128221628755827570795564018029986221073212546181450156587714540466021605832522087707523, 590258905202078720232077627658371058561226199678238986364933894564167405065), (99633608690450974503206963707482204404212660480220798739930489530422076129546981788382685079767691203820147693853046892779960512327341544683111170336635688445270886192732983411547283283817640866294577565401026586842096463875921308717080048156900966438055020161858862342901753165006694979591308529737507956397, 1625179320265934608293026427901668132291352886197706042830127502631123242797), (102190463715297765158692741654340693447936617741175418968091603911245669627867270239759198084322436792631101462785641315302747955306686125588986599987024906403751214924688008106011514402764509911977919262578619968890259045939987814370535881287623967211517261799611523152515994248483583819789384523081995420509, 1618406054322003564613640860367395069487842862285241338076758479008426864689), (118918860705232505657008260434073642089617003632681304102070770984310228989033892739115784679463108057349184985642696036287876695918227160064594665883468083865397073480502157835293392087423347776104049438841528844715010499606724973960074104646768427790765881045417139724293046495581370636517735480003037632299, 1663), (59487729724747267351910805879267351955738842583113873436797145563926948019246587475129360313948280148644686237526241066928140919449943943994900072468965787867564694744696114560021076885966117189328241384534401751391397696143485531939581207098214975492663118785785486027318040973538562630830121332646433665399, 612957301792123492511169056944424597418434918103075406873616427277484142329), (151430495658894216364312531307451109442536028226059487180349145219418651610980305673889816343704099338751887168546597188545602500904053393671738105395406973027899543897286756727774475248411152671290771911439027248451965052813887599178656249324046035763480127286979400955887961050652764862261900833672816943061, 271), (126707578320448911163253104792148613604201081007444922610153680647296543937750407578673906187975696687335820495485420628939716003306634698469232454233527145711331767891866860990121291015454951331318944394797534867373328337149093893110305371974343105640298853992643699478465067891919375838693392294146888745889, 1064374177086111090732782667222729342068222238676962274370699268145985802023), (107427195094562048931297479775105458110601702851797965947256573087598497292512480439890310845601282079435400501706337260605344441349279629334766117698874991523388216603328232821947893933582418951906044885828427593063460351737637598343027141638429030613481532258612940495265992280453180854145537114341920926083, 1554367802024651199011061304437073687689982443419636925981210777729735130405), (52200906029791651554268900574700005118556664764099839394185880465663909539081757214778963980949286485089839683762218478806934600598253188045795068123975282595572660778428161670724830339045943138283419676557464416995589524620642678066259852690448924221132431344133574011310742053275790570477589062413971756587, 271), (82151753399203290932068607022666135136153713273774284694094626317514321462926084565551607124461068460225127972001886948565217345105308199177191275981409717422157497112896924394891260275999826306644503337946243201727250345367085734044614641345309599826213803630690909103139661126317339417954683344589417237881, 1797883088036271414369827896047353064063489670349369129835955150213940836979), (103756880099272780565506707149659901411620232275225987734419368154494959160783810470393162582006061085047813756889714890613365804064175566100427556136291719219799008898423952359883789703554860930296830308661128248580495008441043733011094260079550070186862991622051722744113524540199393844317173063076955504237, 271), (100992701323616247981619142283467636701134430691641337342945569032107670424971419367092799555105274030528276413822113869099710034261619690510536151266789157553623848142019260383044427609979187315304562246561645446955351037868846373042784034251456090018680409230108528869646425549403701184342087140481267264327, 271), (82491283827157342234582208888723015556899278992378883083964498124315316543977370004469252678751041087431179847623971896874287254102304397462224143457638838638134774383952733808274002401585031295188955240736850352466257656971439230929932130603542303498128361277904587387227694399749443518926716872274769694497, 217093295380911788139041505309764307146482271701175044954536730217843666471), (130598623898405296885102990682658809991935858507586200116200895246645315391996014530424101836961289504842078158195594570925362680007520144938798464129497897666192474522079434732971225053614431750598457994360611686602154323179470805154799278645005757354289115793345347530159920977293273394093801731409156737421, 693264996109526155043593981728944840593221268074122663181832485662594365829), (74027341336328814440457208484143841043057713790590369947953789661275914082506037184055238956925030979316479492542668565469699829968001061792212840292604057234027240071591043838454684926269409630678935049039609260121512567445696346120648137054377900891055733452774607675185449894421542720440803446520554553003, 271), (112332690386483748512329950322596877210291731601594746908180163493466685933184652115278191013034620042858441763520835145577398508372803686085760544424442506730569039624151973796431507725060033113299245408555514401070082494399020171778620240034258639563209058069648962797351433622508911983159302667284427135539, 1495902919980249606548856418866948094055419052937920837174006102032516722307), (149766212832998542989650977096348163227578036495332760964698304544331302832268370124552644888691060535478393044248413975831315800841382502577457766260332955437345986808853778721470974854496604284484131222461240588946271803342685965358462743499516596705747182836943075441713083336245908050646108260680082791707, 271), (57118356334047815624628249652948829688515617629770183609547106180778242870615364715110547606478189654088748219806256614857049110195696659025520970525490200322563479409990869864842051117189198257965458612521505975466313670077111203632584410056210205159928145457497060637574949778374575359015646652023381961501, 271), (104215442444098521921029782152115975581563388685878743451564841063003731098327263966385779952030367703714971647105937571545546832257727389394251797554910061326791926761711963409122826489720076527990634211045425713782981056820710465039100818739262049705184061188552254414694510813606336849019454393318792275109, 271), (132480082355878071269069817666641500542936058217046588909292887099307778290981868348496883126722720337873115501088607232130304633496702455748436487600448618845420997721372320485043454987937143851290749921823119074248619187618721814792315857013267752971182067303984251278743820157546770844219743059279402223229, 97003518282347282630441078322005416733857191938456402954854365350303079629), (49394341094180741314548716600414602067704567073068634153042643645580536444621280977623236211238137467372173882071018599413914064224778665357664576743021152376914026288226856112479975177383197421250204043650437498215760105090648074542587176082965808267975448944761773247137490924848940276192498662688136173403, 271), (77883132682526695074814525076038066400210041540889311153025455635448176809102887575301431349009599479138869840853022216343564520796967225398738726012348224268183690941859780355339886206810519206298903937798779694413941539455382146549820893885340088888748226337992952259788639578492004994105345697679076367369, 271), (66333460567718566960679567000832632541303892621919104111193331255090598857604763056954512301998642729639742039781117016226029178227211332291819890557825234457720495465168521873045354276858521121147249401228832422352180394930935219290002635595548949622698526516422097450301555928754537560090410692333403016207, 271), (98265575166562683085440376342492103191148370965741219118908352757357637268830866131462214862282388436352277998749496247394668916950048920573693737447434168953435618600988379288569517846864028739162575650364870412382428412877496025809624891146917318930472072768480081659093479451870416725396136016916097582043, 271), (98810090336026587816876126165194208805956420001997813486579326612533275100578560039913800714284156549589768235218668728661212009115267489390820487798086890536244485389584868414542755407330385729721711974935728146828063418980044375900411281151064730429163813397180807487140577178951956368680414322147071002181, 271), (123116358853022923141733996931539870054001997361035580792489657381779646936849712248303370527077211394636505682512352185834188639997484892404201833076504495023548176306958619441681998745030173201237089820804672847226746649318447645765913367704552652430979178379913288875674830546821534072446124869753895899717, 1015655000288953898196150928361526769489201017721111739343113853819768766973), (81337887084711979081427717952406173915089931133605862265780346858809080507743856978536370193752264967501727831365587245489881106894213168782017350353530679737382597876369074659857401162617283267202398556766602290373024200324931728624520826359968659916726336339821974286867863513281546440932614360000959974681, 1652857900751137582677939627780793934380499676969429837670079390720806033399), (102811443031344899881313321805970051987661813900529528694090612324002394083622461480382496441093201304665212672922495293821647072304380956438501152974758743183759292412467752707678070103179185489950280853757284442572348983555446682020203153402949364077321701017890769756891376544727652774259083880457104353617, 271), (94128232623464282644629170156279558308883605388088700517516402455571008282029625072826577807146230160250820420643592216030793769961621197738521170150640633054010107745669600774894604430479021076532610953221549724473309757506991053126109028183875970526178816512799222233854832484848447646562884509297821297679, 1188498508815996617005135600662455102171361366438392189853762463701939330427), (73946059692447652933176490163367337552002703025079887486741433805484490753972278654674900679690338895411052422848933906201630076093171242949731436486688268757364887995293002087766365088495508903637497824418352153860663105187594269345106233053256077796917397598239788523176605675475897617559111269898192320381, 271), (77228066734327290386621776181540571656128938001389390032299446057146403619041950977859035376535687781502281991769476100689462404657273810834051348799145246340455746254862034782529707318163427689203431016611981751474251054866797446534990241696791869037925167918886332914511200655540252843398153667280661158011, 1393106947085556835753220382540023878869935121547211340435443246191187157037), (133614936728575915139592428015912363857282515549350884191452695864107684266537742441801351253121619016163189093630956869352661411534596011474225434799071139095537760945770476079248005198151659335039386640849659319045594476778463474643174331402324231120324885512947985569190604341174288675066978713833032231583, 115357135797740389115544378567726568994832277657096379297443610802400516159), (137201599225508065654990325495319233387939363123069695964357365224409477435093174076544147142114326647243630050798919037059415710225293472077111264955123114666523582370535837075549717416409137892938983346277231012421239012084928975172838101959159641969952325346882244780410148596152937493308996986515743869541, 271), (101202509753231842043106126551018727970711099965569555359890265576374248371773674026885861407872715277109205412591074869143660234340405701968281743834928928089634868214332464420111786131573752630837133393958230130091892104894340086715167787574619209072522973928643202254656859644072543849248611479427090830423, 1541), (85617160015435082794445295064084142721485401924785217766613919537298902458753893604567972815589394124814122718012075573935115592808322819159306919727063434130389880214088076189462289572360953825666138446755431249287585436657284683406452709386463599285348822211371219947554173702135720902931416795561608239749, 43366949694807926944364202964790286443665471846378403816738516110907657491), (68172197322265578296519575086980168582931396613816773455304527046411036216085208365888496647695950215759229554545169424794113952938794666127073505784163944270900048742526998913190053341335312565320859707011623581902975173815797498305936120690035900163430648844042191001391491594843256832113981651423172981387, 666687759715817191488523727370480836056379022298956589208449885878417564679), (102289780494353085215268939076739728104450673372988831565365437773190439261187994046159342678787848650361393623660064101067669934048598931470851750471689624514634620636460617830083298564973230506440329760954564742751077415475415008963861790537056322000979783420513610193656456602316120532441394563512136683797, 271), (61195135475668739544920897198710183116037716379968182300852413566141719572021630212681621378510315248803872346666176274873199449346471090259293933088662886112567976317011913548521539359164722598315237739223949969068460138822480221656005580046200076144955583762374584463191327129874649309616486444858448271121, 271), (102950123575210372865160989054150900495718786905100785525967129246802514725361091902972730659783297551094930594017089883617155657333760320930124207741762290786217901339440613934034523204075465061089113423857073644910458966400581081569336617878313713690144826025198897582567841110320733502197739259454552618279, 1396671176473872729968116559469068115656138185501628047512163518830994505109), (90651714551414088602512811378080303405706596170131146119234655421531911414547664310927293577497419377287517331022980481374771713548003892391630358385292791017647169772063735385385401444267318261397328967477221753821148689620913848085201192228288602743836195854989542510607458720468136091411100131869366906667, 271), (81669429390754190128335150040147830413769983423689231379538672777997099136652364323747993257802104639328627194642190993400026849078446714584986240265227704653939079876273301581851112560331865475606251552502752811914118060876033422714347116007975691449818078472821516449265837628630910605558667655681206071327, 271), (87714880818918531834916178979104555668799749512716165838318523104764480676009445475867732201908754497042963859366332099317945112874933061469629978043626320864688209606344942915995819224279828672937460146523467554432229314631536293975022263039256184506553736334774405606611814304947672874909683631764467045553, 1097), (91647010255853066526295265269887761186989929927257819643064288982181551823859291432689387620869828586261832301569811010989333533627158003488424408647471834346856788902919789195237834892922352005441833065733294225045215376139605915188184187727538722385445096602539831263354884861358346532245313621327847852421, 271), (86938039350725733110891684811422603924518143579190371154163053398088228774257941556656070101825218365934624478126759752632431025790590910265223201902218183494359794800709948104746701343338344401077182473731604369202168525718875169623266942592744974831593348326272775538999200434829174298687056439186187171329, 961758301501974086464822341855588259088043794539497649923866590886182704101), (99429583824308642649026703302953697190483386580218365853466400524915448762515529842413984080311475381836056475360179327651252379781062625024111963990282438465624965746463559587393941038685655143389124164612426669713991241544921244159546958875028703424443882432091839243609132334686570805897013692435554484081, 395919603723714419379478453269971533936345078515261970679671222172085811151), (115697488627456782268161507017767299239910995980656696760047221792651778131517986358241898626652433764157335321990508955472213022563452535256065499865339552509755904999293064518116123406991522342369967644864166902515685399445687793764486677726503864902445060788485520411527879652770224078511882605726508277529, 900149419625302135767306579688401248321965291239937818942672094478046745349), (84589553539821088672745176869401142002870355195513298307652069708738042747798980061809553823317088478115828601873374372938582793617080615506208021340644789475273738645774546606602041799179372392756567998943376325118395976380129142499437577282266872582007421516387900137886614233974087801821598214600715293513, 271), (71281405672779602721610707715340869940598846565077521488269981301525248317554442760216742362531864635889906230303643929856995231093764749239676976304947709582142637648080181897184358506972062492899235232201186043106567785759898862275972697236116013348720040619013929606654080477745005783252855837478660293587, 271), (103766091236504080120054006083172567941503026946034512092243782061832037891123911600938229937570022953815783115378170818840966922820527603304514064159272129355300379245443335640049320748068568724638063101011319684366328689179377356369400070509826384774932445271011919043074201775445652917725706038832789873743, 271), (89380340448253861184442669311427021284622619292654687664583794547695098592029871161412680799791325929859569566528111910658742507994982324900592183557605290604171905034855480406709246991471255685394324701279244684296227851623763745404891683748261473028996756030399973821045116399900475737376851972534980794303, 1764232944656158444492706609035742790441801991581494839865062292860617383017), (92519390206572900290923450630882229373061169840547458083869160715742060708254870075842439257052309467252175520910264012541078190138345309865087542022666503190906084816546762701801466382518037154977673021794488340949293768182284597396427730675591808873068576617037986451302595621368236349433819790807031884299, 941912556670962084523411069185135297161906105813075214974832488797126205063), (67839521452616323066526729849417075385859789372568498882608329962445397137113914676553017948383592523344355533831214303842868834419889820719509624294266534703145161098410857920133241942035672784280157561616739398469734821443605848701112391657654949792964668732427386497964031419355391574944076899965942380543, 271), (72051456097556688672307303146427550048188013717701186311085131461028508713270112683872504814000864016711608123757212999933386382744450670134681774661298411074462767518597090363446183838688350766905483097029634718173739051318370091959795277426674837685868252215572743848674304494877491076632529809665727123179, 307001956267417209389586617756577081331837472344565505226063608038681795077), (111587564832313124971731754004664002028930162464486286959386735111083555071327385322901799033853368951001835627855105789033084147369088518807663384214819676594100614120823733409686277395286000049031359465621530057355467349149467306239301483131696706350928831627158911572581359961656823668919009573216683798777, 271), (137194820292837038591136095645041162867194598709054602669741503358774862959702155089658175517811419679526944614938504037809693465994195672074931700959201612755552254249346609522851680614447225800280768191115790429569537577490095253590801589386118876581023960730414419512080480969741873481181983415231873569679, 761979532717378093881688882453217574929661985860715365825312055892394435389), (100873256234118756784045234873617923231833730204681832415691945778876072963782937629732831241004882434764062520139645611359964020177571106171273508240154359704953307033745743982321609678214891230752323008554996627270334525647628241653808517673402212286634519147330505413262429794599140929031160933810826769419, 1795782273179313592116895715340796492000843281078707551580329676456471870593), (142945481368082977013715734696410972985800175398229381960123636907236217278477613784962309720841814598240862465235828585860140801445577214020423334833820903391156295595579874735408258402618694865029285409547296817793643593364117243250712730659561152263375801034109969340210017798033062523891588740485340952693, 966763828237704153798253822688173136342714114658983911459190824460773183997), (128125611099899411162044984404092359298481839993685755859249186723514035811268713201718245518552417456294255834763269412031086998729660016268402622276675892541737715475789103021184171002020798337659454705337119187168938082423499057183453477029070446696052595314460981041835875356229028444372739673863981013443, 570175365324196004526662058366728425896340363205796907283542034819275106669), (66480657525193045794850169565712938729034846289741977100127177378845885869728836841590376600437830016482669852696510722567324416592477202249392569198914270559708205762411176709007385019741080721823151975273341899148151085272438091564274719562871186861877668190025673668947547712917869395104868461679797470883, 1579), (96267731594547181722218986402962303572928155905755804055388462084347540268170511200796715770158931082519961704674323186930582649764383702638257155650775397755518460897823504614566662615018915121701773819912044529558188478938835341540809590579175905549263558564146564632657053839213040336418648441484527432241, 1008482732566073070630216413218255317472798565668106110232128706037857284823), (79626106476668526253489531766429766144016373097081725418875939581159443255358194061079169665351771165751325631726840955475051591681109873454972406223427523608029387487262822687004874212523358730505378595269581536387069293123306544052049597809852449412755827164035112583040236434216357979982410677799560461129, 1590491805616501402271506698715256328584134014208426083713049893600425564191), (102488378007881847216818578995963446216626947583895421719626219712099038483396534576580675227784186532889664943454638318245770462737701993500114070451674526295063415225285575634248143312574322034584056773387453977161318069846218551062928870890491520657621446654895823751709422163652215554472512734356524820831, 1613), (76661293539701671949010302649111986249214360546816175211480384229152952829585168933529012130756347589542433373543236706765663303486495519139203172589157329696751957524024214563911874014806144653719233551135161059733887405246689126147471452054529979091831660210282026723359706340327884498208480044513825567207, 12415869006427268047169916122064530194113476388109221768914200880304210089), (124093772516867089195386787983470760689726918837314352155295698033639884396228766226281205062492611691598346629402331220579464016278044233964165678478609725949158296903137647992034641621540886268936045491297418009227867994566114642826249294946150614100732693035882792685784292561381378075634634455128789192541, 1242039509279101701064613696574594689205946104252130837022727939580564430863), (144265958660330635693460740612468667523772174266902392736274442307317702930696222045075724672970630641426829145603588350312922388890421101523662541236649444238679040880810353609819121231136806349469985356697187972521504843882021132627003708744147333679330262269039357692631501078709105776764296557195831976471, 1785739817270359334064392001944948349719008915671605088310694226854839454383), (119745423017936462903281758839982685580912783663919775103313431300093294881336067557297369152532460774440753847469279902221601480218264503189627183885224893401278098093844727543592154905521036360852011163839924423273993419950595526611661686965909498783607022277530453145908532029781297257748052256327974182929, 812309880839245944123480677333146395212033528652731138734687532502592243171), (154781285443400242810897569584025595204316073021474340354533830019624744583653977865450350490063234445607326323405772839599783570814954128200546729960252992981168114645423847352425518277001026773110852377970549843666567184169074241469511315780246238801115275563730760194028175941259403264890208715614977368033, 1717931461559810985344185606596218687694123201459669340506573951693210440377), (99200043452812858266689363018193602756933219492554409414924562167733991854770940590196235910182918361803200332115414335915560233287996915091863048923470889887952650761347049481411188317464988765435039384812228053971951945110142689059036011309229667787950813688266747444845882370746239243043021204410443014381, 1595), (82183409487638181398046684979791395828492160012439379143317885512700620604116761910367469850138466343631628123755059226119672696329889627109627107213153798655449268535566926615934335536304074264060092980162869716180099007876140010903438280829613882805014414354758402030666165636170565430058235454912148346963, 271), (134837347824074884963361460910953633155815977685528996606575700995598317916661427445382644333159298357879821147564496131835506378071527863944038220075826843215254666742558851700458038194635955732526230444908966966616666908328700273721989075924184950581246023396599282869712415670376938220380045315922301028239, 1734160690690843106501809269720784815005285902232900450584005145034303089469), (74439991538497388724624249211764928922911034061042512313456693751740467570140358009405690803060332589611213853129904240250570328843329615920985611420819146917808400786674979464893250444019425269878016058880855660060771859894079223241062279804770522962369771006033972071319514319332827511212488244660558104371, 271), (89888220222300674073362810601646769823376491935933888255840749874363771859935550138933817094784330007510613334862626673837303296865765542242347996268413193079622164685585377362811265805448769994380613405165966484332215580310273104398015352853377693089669227539918389255137289443459854875862145295326836031509, 622909060041157729850829235698704115179124136191590593152988959509906849735), (81653417930082495872470667533743132820141973592142157829401643815383539653588766867883011957308845056632412877396106351996435233307422413126882345138610385441933230659453027116509408511247455013368356163522487987513829402039534697900670594720153263233046110679140231305003861856505997896740185307530321202651, 1521771780687816901399735109809741004959522868063821311009237161285452328407), (142210201878540551441592318887718231015625327317300179863267675765124074710065506689473297190600162053093955211917959235874712943518794903204348597022349164259197718906366124507614675161517262591687782582433844564959344804065816634713611170194793098760500115064329481191007488724314301018191456359429452349453, 967828272029258575566604309049464198929570432116422672128127966723670931647), (126111678513297081866317289440272315608400652130688383252774252291438354374694385273767194852794219511407990480341357926233266395878344321520214712349575602496561591167224997162793564282299736310057455926469394658259214353788968711394549594543072966729777862375249279925772081316010614776233431206993257441809, 1090721445865607563368759678387269284752614253545275244270752788580153290893), (82830910905616449971767420996376942296083130039481757898175164954912051911136531932660540975625877658450512467867023870352725184588297061096983698752945610906554299950713911736070143563254771113248943682437901625156508497903020438577586263419642561886029822103670873734758899487062857957460943721938703815341, 765556448619667294871881074155452224173729469236674895950133091027373448657), (57264861179251599002926216499772951241228325604966843784251864204666329043594655829224481085035306876591765800078130273832388717929062207817171749624622049435466188397725116893195130877122128222851721081043291122498456827629702929835932062505790851199295662621119270216515861966469584849381973578966946697239, 1518209625937421726399346898657919269060967782621987189728764015715896260537), (110846594091294003319140685662825742840998739255672007028302518446928140339288438237613015240614172317113797630090754124703320478601495578562891897155257512092373525568799221339668814628088425927951029003510187230507752788603388584923318657639199857890012223295366934462142559306458316815984778783236488387543, 271), (151622829146125284951761532215609312689254295435657119149571589059567615276986593384083284628122318952385452021371612329949629182877856802598368857512910484804685518566330416116353681176471374822804352888302629299549982405731098477111796368214511555630336378216127967409484335746192919324398780478102208495281, 271), (86980921883876143243612086196769331048626085535851807867374234854991694454017099477849723739628650641932947626893608934043123518709610466492946290327107419545478515913489342100672074004482003354784668179371069135168801820512588461525021888762833776761387410556486998352414655823740515047811248069910488271289, 644549772445332183960701759040666844672869354817330014502671660820891228211), (83903800402634504305437576364429390878703716474710801449315228313110783338118875144456890483008698293309049003607587921695974868740229856233751944292042252438972531384587409626986286255735185422585896411857796653306044232088988196938589035693451838620515012701431617160224017349760010114814211088188896423617, 774394282795420110675792139957668330127639770978382382921315485588486588099), (58826210873617050559186044520486059394269030705302944960906769136098282786504852379301577514686083550063918893337329817997759044218530192345180639371300228827885982119178345534537074612561150378930122133648847053595105556286939242024691259038583586900814420141539292181607018055845132030953576984630253063949, 1371295966630153046173591277619392579204773753064510884331011438645752862041), (71012992177679853422364367702143954133902878374755170073796874010376314118054435213568077031328078032023294163636104890824395852628083576900864519884899929228378852634868330206421584068741128875902865659287196265502964100910500453775909225000969108644031221480643656782072687725265215265225219674105438449797, 1715820377813047753647145527220688878730589231857230600385466882440639247191), (73229720443939946791619832968454571227151735387782744452804841437250662427467943772388623538878489874968087238900434894424959470988988851618257209502124965679440623837839112179478817344117755542326001932832166581856978526670925443975075319211514181966553328242938946535923414978399619496430752053296408380837, 329739618561014645106758557315184864159392551106068318199349393157616448283), (110779350810554393154405921887274963072822839346006546778031108145433742834506061703293883351562231522814164607664303105173085017354929116265274410774559796163459660481791520630756216783360211926327684385064778429941011161102654036579285962768249342793651659742249967269257823501510424876335323590465671880581, 14604451523483058826456933085578331213746494338762891642896536513955493399), (57111543422786809201190632481378073026055020150884483360465450994316414000130617282255241443928151664188126085962876308263647958902668673136171276894722913025414202824667244415103413364786677933142753652718318264010012060268855408806043274204249328078549551479051785114461479941984652290097454272704967885317, 1546669547625309301769495234738393348998793762462907379546822550388196357529), (69873640720796894237034115351964717894963382422742502086108478519312139864824893685220027081202514358093251133912486991279532114754748683577586049995837438839394900700863340506217303581234505381353608609109573547356305032606308548485423498798393823182034897343785166591734055271937808726344682512359456664343, 271), (128242191239370519136803491733665453910598115702891749667791640173052382465961377734707634910813714167282332951618828329747075263904424819810135930957749691495174189506389002190458169393342409253651279869375143672176699528417213915184160170825869811874561058326034839356522591802608225956826933415737228542913, 271), (87949763117638280308259479242646127767503995191267853216995415058907264828760865520226901055536192060412192391575457019511482665988199520077921110448193693237921721433775294729975503473037806322962998436285290374142931342278637213372152337334540694618440610711209366646477456138931244638381798633495694851517, 1302907705229811593129285788945619397084119655013737516448812358467953056265), (96919107192544703310980924432756256396504381133863002389806425549393925864470462375231950673531237453764630609161462449810292266011594766480055802217607488559067236231159806345475068433020973718912659119424987065171988293149325683110665726035264860734355487183690676820685574373517522747226101881665307447727, 990390237292102678335391988043505940954048468783146791637459796636121772501), (99305800396186811481850758868273796212767977757133599006917941741410764567567535220813498905281596717641704780169500322975025690339024152436807633552929536138307380072178734735946753359120645692002793812154986810412299162839924735112295809554977333774713037732321798678713825476304378589483938146566798178261, 271), (120778928993627912676192881735374416029594646000584248331521242685520232225084786685487279265835675450961994288125213786711666996012685701261114681493827006365748528968927639300187956688452900815019427793684194364907014064527843808997650701704903832687294682477971124655630317590346724289530647995591807066469, 1274534642742405074389898253006467530822879856115871495121354859775399794947), (47527856608471599253754510293119996178543587776168894946861652791235556662194687404840482220824561321224828026695123391486770010604579532977265978568326738123362935255702086903755627440120456941379328371062883702596195536812242730022012867538157181707779840621389736832631279010983576629659624128512516619071, 271), (94322899528584124807953627670315151577204902518365618373726413505004058490459135499645816563573014017611374171374014435634424418275172391918587034766965839357006309757675173460232021279263387141430414443498384857491283417048116117399697555942171740790082970014751543915576945274272235327745785904399796065093, 271), (123289841147002675454860245374511862827723749856428751513757310506161550778754477828247410577118432314587763626174708343095923054058970409065790233259433053054813270760503607308390356366316076317234747178250678178386729667952728483938031335770148364649616312166203224049910022786203215265758333765591709286013, 859354804186774161192690195073830758982922325522033192180522200688547063185), (86909673606695861528695149868368994494052915596032575064986830809716802714844987956366513237714298410479832941759122880853683276423289615854800928310265515672160988571046597581507301555455828216524082214270066399376577849419677510358481111508288875540879293197850822704334063766716128666364973699303058791027, 271), (103060322926525504375262296820262785317679323286073044222351273710171541890249054359468492276952305637305770300013066545988346786316391909045037068603033849899129969874509435417729805581991307077706308369715881794266194042357364392857058377105293169973804856548144114441313937133960746738958384835499469166667, 1633), (144316788670075447407768377666122062793410919159170921267663150870785988149055395326170517643987394706036038746076103804860890759988784846511611043606616524217877931854031229006071562264761418825454821643428002722740527431486191854722652856538988904380820301023157570691839706974457234196997571293158887275093, 271), (143298390326685792835429375317115968104248052773491530603119624039391738999360282498475401969407466412758657204719907513802579158621204756522702525334997418347510365851342874237293784152084638180706462487691199198628083511842011971231414900715263554330342931482480984400681734660845986329213715139823587827859, 415555496546133271099319850148074702094900503908417604928456952648905058273), (84494381041600252703048871665186915188321093409125594810211590974552635794079610766474134345485682081126348298374683102756186040244059877035462553346183392460204920525084745458596959051380215241096339718884646734129223692038648564558978298743928962574026752410077726890088014097866768425912760471749191224389, 1190631170813150910027973728519231638523093064178023917250112048757209122099), (95850315524939540310169788994852667085911634032498870626894902379324318303527580850422508596362045410741190024847339209655716712377584889830460619578948591163264036604290090685716669577411321716009232928262914378904682335157719292814465218402647995049810833777824011047721353547696087044971605906114931911351, 32983673518854597947935280352682519140622480836184521949748916569468230493), (117592863046362225822015382869722918963150226622515554545425866415922353356971735525493895562950386198441314658133489672592777081498641114361962870787696060000798528884333729460789711323530519222442266241760867770514327684849110843656576846440580550171550988604603173706278249813883341366039082302293034016313, 271), (69340138507010787966894183065824767933547115696315744236464336830774725035751957504279143005898234190557278078794779431510371212448300115213181620258641546470276309101097509604035741662170940042893079996774095437219097871559326676248591081651248770179608332512692525908476171557742701567470820083967237284827, 953567586627916143203728165854533978274805865335054560881003483826225214943), (80132403062385021866028556844827644223985228060892790020118694755009478999127053626124752672075419922409637035866410872676817761851474884937991137729234122767725166358498881479741136773384857795782555258089811246786997139169638932358710141925008340759618980627565109661435903170007101777610424660471396565061, 116092488989071881486620855523172553140181533980373901546752447744274860857), (94440129688982474678018298788255169531527807100473822000470679927932774678325100235559994567326632463131655743249106451265781963292050510428770558800197049551912094162109600958421832087987491492176503703464322683251124719146444816088999221697139056874435309463096394984262666907982360236451628043316054877793, 1341516420555858370659330199509949864372490479849892089377943430635463944141), (152048682755823285735115729295057374742053753561083309441042888557570103340445119438808506542702785475835249567077295839434511554512182427338558597535901351935466463590086610833979037761441759383502784061221155829881433816307850374794639625306147208046909180778979118310708642382263408337726123442447390086857, 1423), (79367599603480632733488199597529500098506046973999860715583444834366713795665191339867182017079098235492204144392190515063636627770545938702389536964221880643060575691681371365773929064744618920570934435130969781997044650305679459943912804799111137167497223937560215048959057569572729074926485265842896590663, 1020957833412517662836173200380775835181817503632612258151754455564616731705), (102995339827122612865630572338196693630741880737750690313833821391657182111026890584854969482708879803485749446940424434995153301826746185124925385690151205955960800141214532690147302159583384826005202148443824265726851414938216783540440277316057671529668380306855004749099893307781698462192025619340578245263, 1891), (74554222285156823313923603463253831463407689874785362184655509101595847806206293890281789406200393640034320813950179208991878540457105068057754140043643925248765289056262167240751480093412220585571273361177030159218423462049895488089056577830635300205671340995308269458166041701818944683383479389889294514781, 85507223806223597751530041296536289436783103846930407742196475095128203765), (66016954393435601485775393167129827977100648704208266043432738857289272786508033462043989418972950233880195384315946715117156606447158432723712733341287031948070743071688045968570426512191111014853422834256017386808956986260843560005329768972699297722707088780490824696374021055460626775455942515054526748763, 1024904042376329704978492773795219593168308074437740305194983433912084426299), (69223257330125548162334504630419575990862850787103702300262179504056934978166096805985696314666112587235827922631178738293694226461339982369167871063747326776409087510930193245830747717809600087062232823300458909819171659519909400034130663997610221039425540815082169867198235245599579238846035616393886432687, 1687), (136478828493053745016780275500543622155299879683789709505234225033787108267495866495023646788702558315664592194636071513096072034584233819480803072307641940217076906422678665734515655126996121824862998311396161430971987830963320372299283598955628349311618318917118637808731999476030064208691106344835560763227, 271), (111853530187475793191593218555981164901586004232858310062730936740885059838547033726430277135869548788719655537521646311951750204642098122874169953269462418776926332830814692213014502374338205174316410376187825839772702547940034335669808454449253782024740962821397024583921484090654767319764588076048494955537, 271), (123693642375765201089178601152398873891588952020628528903343530131051881797491038339836699433079444969983314106289243224845984446808743391538758005867591897504933424527769432443235410962711265341953729497974257802490584895545186566032947157536349648747889423791677740103498486603109271183705648995278634010049, 1479886352161126070752540409539115025583809659569388944719505566945285382301), (105450334531095552130547567664941658999125406934933856350209079317886429349960048020891092244869302878387314635005278027890786807460403983345603138301722462886324864231157077787898176726913583698944676475301404076114624073538611317455052030284378429810178003634951349953581429427954106461611119099483623617357, 468602061173911383963079032118534990329655373938285286679345751685680263721), (59531914069999152782951831413268822484889354864111688120863585692500201465368906552061990654026096169879488081465518074198623744719316583907690072749406535632258872190858444138914026956010302405956007915876269164782586545233893013240894187017059013187381325590607291875750481431965428432645192100877063873689, 1206593848771327521613441690253420152978018036673350554914276390106470017957), (91048755856578958383290008285337569798554267314176341374151501301325281083190451806809862843939479126724632557828155533860881185478001301701863224115069161796056204092087358652816713921944308133562830831012635714767958266219103000150189736297433052039644968118437211775626896443506914332907162917474901832949, 63978464167700560691161476880751589577059372606532428666356803451316631537), (87076657565587184637137366766448610557558736570270205313987429765637833104383439823063669532889452219799676713079950186045371236531557944208875719417543538644559260484545575776604618682263354739810739718656442852324552659770079832814008150370479627960448556120341880600072732347662488174672924144772758484643, 1460959225822457501865209324090888742473021097936968882297259346828413407689), (109281047941428739188099618427231550888536978358011262103160614674472796957229483964094821655127510390946521710432424682276229833144198873024026066539731748856202305874305196880674243110699110177273567425390892400174946535955273874928757953672236729269917052259590694889200389453314668214404060279716187759803, 796135332246930104258301882450966712100037102248027446842592076725515438427), (120998118811141586855129064976845881709224789162917145718688849460856889569735263638032836527334031881248713070772202535175432863336310189105021473948381297105430630696054377701802327643132237690991499086736814893873339885019010784688665057067543321951772320615684845928308202670186229467235661207842764380503, 1612082376154755114628533933145062811320126058784325673260520242327779374241), (57524226632697694911607030670184351868980742937311183532915206695603848866498017964842368321074839573404709160121444862285617576109106820467118591847286699841439771565046625317870973156727646888599283430759923876106872587477694481826200399058043272114837290283214385277723964269958612966639510590043900571923, 381933080866026635112206890659859737826214964571156680143317114090545930787), (95482012060891979823636357197242107651876741863337082464879555068790443028597311770553921203551249272598075401090135963645529094063547945796835765264329271765248022160691473493726285271496268647687268813604507758933874921699402581297305659319614878917928790986609764351715945091958250741579292665963245228491, 382655681324362209344028944318529022585707123219741308292289148726078656357), (95486577008797621021041483272468606238047018431537494750512580737846383151867276629816495596783182293291267904788548246097044066524516916802133187725533181226387427339623614148827853153288204352076986290966222992409611770095077256273444139823239158291485741712497410850460725361783619336411565231189830713281, 1655959055586279304850292714495118225109119454030321380572245574420166112953), (123944758367199580072963864438881585629546005858063451241988488185604846447700801237647449428381301277127190143916953096896730632291518438322297751482894299418632639533950779401189011344582692819557144972454345524150690557832699611288663926530389007963851742768749152727672238905633779307016996102595114978679, 271), (106664428778025638767362563889718879137184073747350511531857658518823470618391961244650482405367434489738755134839390008363868268820709536620438575352939611377134729131045752374156927754656894799536322887023433702325670581018163862666130521228683430628706901887170075950523368689754845676183181016469391586651, 271), (96705188012119779197035848962519168480988186686759293600070742451146102887324763205283105025832770800575406540397010839001403212087738192127685055139633950024478519261001158122386103373890571711194880896026040787893368472201785904948818081036418073141048140399626185445869192091957415178764451542947718238863, 996441472400790480079771122541738188716850683718084931793206869711395590805), (85891421724972676504770646221603188962218807844982271047187849382385493017986960117017542583786319498872985412719396686388606355171659619204050553805143150525639261487332598049106864779128662947702420641097822496519213968899397893592760728692502146457974597019626590731744176857808041992176722587158910105709, 308773645152310390973441519093185010650619918695159711106242115346982591559), (47390971676737108565691287484874270178055935787389972004964001105911547745943494463412676759061427542622099121688976691213812500440062813194724841009804609159514526656899092487696847035690106397571477712803794819834620498410540546406103423519909598541144450992478045489226860999415555640476945398694259506299, 1144037483716423837023376001679566771191598862931003061428768422760282841017), (125889644186859893308310435584456579818453682204549750470722963827513003672855268907622854928691094915206315135907646840338414240602592581310832101632124151805260129831538226841399026047725550837669370964260777903395407661514939546508811989522447458375262106790218409909224122226790859818161502669273668218273, 1383), (46317497262797241464587627569585820288617394463302971800248731240839241276724633101500906085927308675446773544441075498923217163628074992028918173135193817939765511830867570886297681211569181075332466128405834988604361159289134580494597410344670854105609733049503224817340636372067489697707469322280059957777, 774576657449670604234689205565585971434692567010468850901870810906294209345), (82778869383981725912116539196187489720612564427758684124841914055521642745192494810006452425721314020787145523648488981312440118482934883636490357603958658324045398003827383544399288941317723804540699755897227145567209457445361635117995378932699818894342385215522428685360697078622001473126881018174986119453, 236325198300806584809305249431711012252317981205675686045522090548288505049), (89079396150256564200500473470572991914810000138031992090104670493301131086549479826017703300446919096723533399506579520267756601712920336329520660000397369359202381016702214749588997549603646814240830994672746619490181146090604354344947775688476936831560873642823574056423731708795485828447928085461511305973, 1403623125184243053552970720733243463452010163015542766835711200204815427121), (100951321667913735258054552228653958564293483690556860468375952706376541580203401774438976249416411324250540896480680929387472011514938828581985642712436510560428982759908772746689305049539622574046350918964648835337153443609812652754026602660920760665971649204361627289451360301225576488875192611877009002121, 1644987213457373673737359463654083467617448571221928823174339184970875671983), (134419632510124309626342949084001748637780144700378501924941183217517781653266477872757052583828821592243501768711312105676368811409717736122336742200189532790302637222337894407072087725086225035669246764856434307214760863885710882111345975961325063210017087037839072602980087648415068847406104162373055276979, 271), (112571195679864980526924987773220117768552790317367210620330237697135796547558447418226926859537233183957584619697718066460063625084689530586987243187029133724071067652898249859852990760490867582638619102914856240307726927131466483176610375249201201014462446765245412166188728048509939706404699706103419006997, 1203), (77230942771359246136114243957438735182578037270515556431159889842155630169003855659401037894342225707499879543903672745754511779586186108521518443507713526541057339921707983943364262057941447145916228654321403244915922452010503876124498739801949252386658154440135688468519959315234459842064370098489269575931, 271), (106380868889017914002492774946282220388608737739074654269030425958709855868955111488212753640704813547371030612505344961569134901034579574023249853680318757383881725090685311404705985531451052348714000159643460330287514307927202595385020451625421065756394795951415530051616493007420593432625812466718130515037, 1647182990169386739193491082040837546841408752250074990915981728026514346989), (117580130012193596732572106130189079261895208752682029469259163281462442065503987890543125594773361706218431627531639430395298731547714994250422019163082212213999465984808306748801384664934363023924174769903974147596476558081232659880753264300179677446013710067946781433720875971240948103827338014128818186181, 8912404133994020536682822517921204810136409095237704985652801649095605561), (85276861177245602705829842056878481146942428116259880073753657421079130338739445116664953363485949798054807204507194047739001031896480472507299052772473450108666386594272646179112157986137481499873782480248840162955445464758773148267119898615583505865229193581923567322949404767671149545632263626898877866147, 1762221869872676103933284424988419123456001807539300136716386295064580510635), (115784845850700761104205197293823096393662315424472533628327309045645557066551450712143683458638101787047138702034662206708899505263887509122512267967990559967582470785478204084197757197173646432715136184696595422617453201618961417799779220330720356065868547744459710810501341166280768137807741836915577000269, 1775039979438773093253408911904590075052626940096751542617816497184456365873), (149733666159761214669003696379790939393255346264318977040816702709228001860654890609523612842239877156069968698930159073545876094907018587320742074816662304289265209311858377058053886313702028513931396859111531122331376031955871530018123416275216095995112337599917972308794803786583439964460397333986142465629, 467517131125891653194296740072007669482983057784170297650765952339711782387), (73070303116372049281546687474945533887379544385272828256550244366847870528889300063399821384825715157462229367214187355679947491706346812862172530401581038041702305027222695474526613923441716454504645135800283040787011333621134756282021994836206628666260569060973551925328998619845705457170614839242857693419, 271), (82506071026007264943442329726267915595157356345988932165815290931413919588726268157298034495005472640140431808191326630035205884094666606808271400552435989843845695053524733342473241595071471118143392515132501486137393101853548497019902814079691800861922940737705904216659628104972289696224252734993750868861, 480131259820231016127815191179440233709378820927849306786092987681841092729), (130542095942873233087292296617795529902242458114478304966407720611738621713823830036505652602423272610073762049908667426321761132745588620058662940337024239845724827518855095767681263486352568130518895302566167477046948468590729587911365427259791528961917768524697307881547197591809342353805119861361012064081, 271), (125935237959249209276264363922851945565957826968636832108749225870254266015253439610914908511126995480480568343988095109543041590542931827863493748046935660682373897548444445383238015004983893265400658119087073174046507319513298498804745185689910158090242017868029267128868362759211718969747661216557819878913, 271), (93190103143069625932602423194555300167828428389159528193073068017478055583209993413461732355292013520261940032155655845965293252055889429751808053290123688326796600833216423560147157736378213788961936701763867380848441992347575006531064480779311100285238728757415822489300749088340059892287934607166884936737, 271), (98088458928834772090722127062304624170366318425633022113959589405854538284430915874302561455925324240231815707941906703885177080553020535028343618654232208430694946123601961654306394825981890645694310498738836252075964763770527519293175254467650401888322923350651414847288643569588903946859674793489542605259, 366956403845732182772353703181924632461581822386720307778118069030740311803), (124659150324784897482947298849009517780838575277876396976117542182722695062589657011744941725218079165048032765188046192722174841322646237398667141644278167458482395266156975120938530411357872885132810973555495579787616920445071096182886318400739049473298318993102160350177927380673073903479254697268904743533, 219640878023241216999853261796849151763365761100551079435370361002526112563), (97211317121155666608103729962881010845890294289867887112227017353060289769126719282965142188069103855992343428971575413275987746550063305569275218308650083784531251003107890322861260866157425328268852085031907911287577081089430299400151886306797027974336118242050385418736330024980086517047822236317330139469, 481985464407634420294423078169472970664720066893727439290188800654697410229), (130889329830827392619325679607650481821185322095834779590990372676596373444980559900401862259278080253030287957992120899499550651061349236065782645537545300465931529129509933754638515994160734402618174179679894746690272453742831257227007765326456293266872586489031625513866476608026011738853344976937349112177, 1000200646260669547428174003023583509375504065106821706033742776747356022195), (117471628659477879012627826644135796697284578169319221051503873734802421556385463792144854845006115887672028482317758002327855060582672612703876130259795094043912148286249192555988399629723368801268870042020314031082585741200622187558756782722651213676324789285715843548363870521786524741363698163354990720847, 271), (81156517251388269426398388310607682275919499675651585303620558219521498834409204763157589161231320628891590470626135924438832661324831567503800912502759618138812761275431039591100443880281532034114543805588827656632342573557133801285036357586899799792657893038378741787692610300378377711341981092166513763429, 271), (58503399363557883115785299697696159139383346991842146749636663766187127166535986307209583256043007240720511113510634441591236613413895659526894977709561662824905307533805353574534199964647943068484031006094668431715966821014143079846759330024274656578906393017393878869630923683058770925364596579014213226639, 741893843927404455961501068682335347881323381734304450803605443745523819203), (122119123387502384729834012396413142028476400070719652743624910562311483771469776821871729938040732691588527676795679773811687733203832479033997211199824210300452472570564400943182785568455842947311675736634044537020241142613126614364924433359170699842471050608342480920279495800605990919013486379553364770527, 56299775312668522593022411742207138874989343096953787870012833489501310511), (132793394051257124597549481914275987046222087970166199481966091252802036723152769014088667082868886435441832713688979131263624695892553012650838902373151147920275158312754746392906283264444693708732666212501999385246782249575385238876156477527311646197023544305135965459158710231975235175884840415819503854711, 271), (80765533685478749028482970656807872895072497083953351913252161284454148857536712686955834941047934032953594490946020365148864011698742990203462638589349919982194440836991404731531775989286897818876292649632340803441242478346105673290684246730036647383286944324737448535795333641524090883146875404285293881513, 1501), (114312951240349696757740008220564694766348006867316745796281053513619930309161560114843595100483828214096622209629198388819745242298444824233444338072053333452699480958307725370418549111614465459792545032635838971642610788735194569686880225505464315350054363209089641507264069085405624392451835160359975647273, 568457758836506040438469346106741246144265519254275492280800329051956827903), (76577215503921849236065890560469765294998782675706129436993910049045527622275878271201185389485668395526387517939001974233978056320202042837073208582828071761776447427710542973417000526032819646903092219412281607247084888641865059177143016762609101359446175078717093949423656879603859725368484022357338256981, 1957), (52841013710299868878939454551235314543897121130071384605471801533024358678301259927940093604893213122305827739276749117368418877118719343999590225604495505581276265804866772764224446981233656660693065209048757930029672971243512633973135173751040600669156447739140722574274290241834614778457218549726713462169, 816460593490249545071315661600306316111611403683026050442269474776956122159), (74851505433922256744725436503079841344540664317417950960256269792673880758655020068001551959429571736475432079780467103482540257485593729552081543963447514420149993302200722820530583857064479382668553817840900658522977655358815198263944776882414279945603619851575789999540499882637426680901730918846501449969, 725094425468754907617712000776035454994089757733600968929197075555584398005), (169239300248248374810710126625150474661844543410231096202985000636253534076611979189254017132036128178378932300708725806680902812563004095915687189730555947172383955685394081277886312315887936026635743286444232440558225002107771155091857646810760948743775716053219962623114355590139103951314659855778829487711, 1684248973102494366977069407080842824955875580904767338506030096514941272743), (103349194078442871326245979769591686472291906994331976559503930828800279846128138482935904091650380762544153643170375360638770340697563203628222061431478457066543348213251125948810232400258502094575287417921611667888721220055393441923393495525507713054659048087354942647001385375451926052024468330158697069421, 133599854993182637439431129571484079066578452100733044271005985929354959061), (118281751906125312913800817752991300669628830896673018201525001791409232911385321126837419429438338076783638227283984036842819134169444677617896505936037504501565259865904915796163469287359904565640676056753511241794544376651581384695682190890862191572856796857042282577068051869096578837505111976559014478271, 1189), (102108146253081953177184694004967008186973958726842929910288932171237332406735590763345645026241511508921736083437101699885590753255683799574711898988672910451612370949512797245359431408254012447308692017317749128103038816137580070394083371489191434612384361985308367222902830883423173502420479572839833882307, 169248120503879608304062348077938886340462169405568553129837137756255372757), (97107924777726474590694597203248337984568103926521629581119532646286038935241048327739688201193764824085402746592706268535408281367512040890057958077321341465027579790022765517009793069335404993205225894280233034748300560143182940279809825378528779623371361447296348496236905432044860279655374183861161819701, 1053), (86923558364744068861718129724652969796132918095200951417222534825069244542456250729040865645840654456329096006234329950794824894846407131286636643800926973176936502742152242041647230481322658671953788654713498367558418218695795763616635989164921503255151180077473287910653534337322411587865609319785089037509, 1799752364615622983900278092257783718508068192210112271570293469612030524857), (84385203830135720528080965384134442113430685609915009471705148598780722616615560426139350238499946541856477280155657349668776623708565056735559055650764004721104325598062204589976067452080332556590730517290790228025741121747541944447892057106016949729659726243031755932017226323251999774365769634154554621137, 184750131345705336511768472495105155697596607065859316464489852044796464157), (106938850931221649465528988609474522966501202289015390329174630477701359913135509900413710796350599594543601140126647142411935358409332258935949151933828012029788546846535022082644904618451725497688205582310746895398728434733602449890110199265214183228374405473544222376149128474256296744975931910462972937081, 271), (102131403326553027959939149313394725958175678892466909896031870546793615772747302104684181440842093181163612550511036459647336136565673855089219163117821642768166300208220539504794523736482600903405034816020128556915525084274149672071125050073270369473627122794580534349971994904566023702908664468009260300733, 1581827519141818068584616747819159623965053102585694249138948838313781957019), (115755716174166641032629265449114220669034512164722595623139355325395103093708051130282312261127198965642012182403794253323051778957603460284907622810979883008185487986948714694655898909683237640696393305550610210669536212520861947129079418985502635266804345100935547909626539479309033847941172203015566575393, 271), (89116570214865269550934984799646154747914711809620920596530236257728943466489220238157592481058455553860467648354330108957883516946558589221668173208311090264969328909190273180933824172250807766250992253381158367890105241142946724965038785221344023362988088938163146628953262053337267159525072614747267821179, 271), (95044190773068179180918229981585073997122744777420311529674844993550294713506608340338412559180948427348273016465949490182776975107587994000359388052265193813795613067297796889065623122178938160817564283546181083716244272843237563388350242694517742930919830184102122347200242439177583376048372509091991997939, 271), (149284240785166907388845029337813268772232056428234138461004395059308430673646013135606852182479208234849855406434136529239277796060304796151383892896344239022919544078632135902608707150441298103624734094440870094391093364969865810731622412958776587961608060325454395439803151011485110111928878936292005257713, 271), (127616766828427343379998260064377676195608553224618973153847673652038715798426408729735516240770722282361260006856796097641804268955653895276683930361737554142930887159898456578498002404181599221573760182532879372654045175411555718766356303593895866013954252418252898229869672452568867782593002996033196749913, 1248516580656996294970189888532788725040396563801638348585608216840793640811), (126740411293561840424261491590055360517613885189696640748493249470321050274548869465377734822009720790400929914139290063578130300812949228835646770025187868848881511821592702702242135731969513216665679060479611990085715255861089474791617377819244311299269602645233559355530357450858820188395679309588753350007, 959832073890087316088163200587088387921942312119549329083003822660234281277), (146702482215594725817239950800399842628904727306884401758269964264717776204914923521639359496161437907203751173201154937697356806035818287255519627331016371291479620354302431343037409390480609111125020375329447751577206061006469558568203895422334966580868144332925620675507503825649548213754528642878743526949, 1633), (68321968915636893968751166014755061492247458862384100659414762763233475642483192856607290020839629209547087643790732934838755546163900987258756610221772212105988512427185821371638781674776646608702373624654313536624654239164270370981890250544378750481956882923263338732360066224414236787993987959229501520847, 390733788761268701584213179714876990468262637900858579930301197790844562947), (87872304702236968628582571912800146687316437164415366921728115111866306906991112242483449822182311673177160280286929310410346763545779109875129397894816023023646142453260107991326743786290229664998748107636373617516726281960842531602825075008686029582341806073641623129857099031246714201446069278356055057161, 271), (55417298075243351240341341781862093986585746483252961915885066299241213451432123559968544106731186078259804554573062755072999171945800714772861123825137932172544415720539652425499052682819086523865662032244669875855221719977961353793659657406582969495548840526301686537206849494877760504563420736611557697121, 271), (98098431210737046946079165208593784498072071032073001915829124711327351002071338844249167453536712140083818909826365856621231104121823488647424651157887113049902692865478974654271588107884229098428030938019970500931059536639056851310799935029219400612099431476489529850052473608950220387582273967008236341769, 271), (165489020334493606565729185296717878248758691978513902491699657509546732033129761878482023866220270464524303118723098515313016942122724747856371271054136924096651429109745237147790151026003609783119973583442333000492359823356000155228671255735274522566718272482689564262918945420091161758076393211924153715627, 271), (53336822736828055864384538807676285000476550792078454385663177289728729915261713389013302267488500178303432178529717999206597389027639073712044664338604392602825776652986058076685343614665755218020277165063506112875602181272739853566094824625881052590216450154982108343715765193893592582198143764180001270831, 1139), (49277833600810839954547379753798500426964621450865630660424879075004663125484271682489258953406342396495785108411283683895941431138601637918299760818084812624720604011395629784933501332057723099088801897150788371730217663778879078432467542977480207026311657290019999229027323084915272939786447875297612002683, 582790711942088595963767078921132993521957983263915624714698457930595796151), (92522738921438166859273332302302632568950915717807143134096317197541633577807832959816300011161837370674083936920839881654364543649969738444989682548968100562890535959960757742899103211860177783358038381802873818289510111267230530320532934585277891706889670128683943281856744342667566448463834562660040240371, 1732762755373521675221113923723392944549922559126836893084039397563443682123), (162197952091801287656791269581886841993289965471557938644259207404148525841172015264092713476263343501756201105627152023189625745670362561380015526802983436372971749761339041922653399138634334291307567762768912431859985157965686373844976695925588664368656545916020220967232028909824469775376606530360399368439, 271), (65828911828576681829422298978307597568058289637634800278762690240329638433482724903832097039839008375048448481799908700495570839709589572132985638151257009755852648810886219102583698764455986759785888438184775838092312244376943248573705194881035611687350122733388074854993056206672002586200741992472022603283, 1580043843068969669398111441842419349987960178028944789438547647534353995027), (88444289570302215136376371521591061684832950470889719495019729475013278957491989264679302773665836842526724030495152094434851227054792435874707931845437211358423458873068461928653555187046696372711125384540721287552064366661745704531351887283930341872045632090358092110658804688979736999270322612781894520563, 19210129966894478586397661834262410357206307830955853440838057974574690513), (88416328807937621963791801032768755178309970600297581672746301951254350280552047071783718454266711457909179202241519592486792042983211434826162920768514362826016063925615490900891870876890823834217804698906057516601936318698770573813610955895520754901943568091706335602196407176712502091964485642332528513131, 271), (71467327587434017785511848930932057709412978360293018697941415330428030932567562677293207959501080258042035692232227574043042933794475988177736370644697372217642777816789236108708463633012358630282194356883112143294919676798162193535172381442435303124618780705693134106562891398191429590360252028849304235801, 618155891957957034066070975939880357144428385985667487062038407282322499651), (100903199373375602328378409487168439450083325179030004588469556058471090971843025119555731806974052008659083562224316527999483451070488307459097218334066613264083732459008132807670433131504600217940969327948754512550124134264564272158415403019229376071598662780055954987049084468058795855976699308446905472057, 497234159749330743397615957702039702937522856515765485671867430514041277529), (97046983875130893570713026477609584554135226304067553024823445398552414449766184236719053954292112448632842150934660426259352738939807882409007884836109022224032877325051957839362043639139613041647994929732974114864725684161594567512797143236851394601254722397322500624963255204814730920056079704892289856037, 271), (61472965234412458472850612658298046537579886941950737359191683697821800943044816366685124996999182760410315551896370786216494577562364124548694536759365131613456302234678274567187804933454297291460830621197048949726862269962371090890456834594779655147566405999933365801087284105381653681250666411078294402109, 108829893701107291138606281209582255936099498762398863466364736338166285009), (119554264758857632645173500010835563475384328147646798827443306559927705904494857358254215072811751958631479597184272181722656209553408650471324312916491505422757191996996650641806867315940677105447206282614133678763842921306350458464126579861474574679876635116420688262930739816651520209165576986114086099693, 1065956216390080750332144838760049684547475969640969886279231141857715972617), (92830532927704958347571191899360319988903264367101921892687226033538768281299074219416980222722322332356305284542262577130162171903811430800109671373559986062154017661030820260517189989921385775928613890914597892886771080743681420553016621901092658811880462003602483152582672561473619633586058318248526781993, 606509647516844175945300617702654133592625070427713776402650198140659211659), (139572712277655314683356697910051391413793061650537367399962468556880157232253285562298183056147108803738471688862943110828466995180754795652254688550008603876893171567373856221181765693132472388818555727151606598025125709686328164405687121411487060233995175325248365183382295174025559014232206859469248361961, 1485513624482764053939915394704233103501651285068244600860589384765645732203), (127235552325055364308689032078615630495613738405214594121869350629352654059886183918070923065419775372626594344307929011694679346379099298577685999249792800518494512762536633271763327712093074626725772686209629741489020261196071604219876467938822678393462412512470221690979987466686680699998782004671635018823, 271), (74389270918059041436698565729943196024090257557775563156616160346541699740613863385207826380082097854153897537698474157572773700993873570312000984180118285612157834068645695911342974622946533101158003441503967422350780067012918102264767855467594072587501536619780355377278347285790100478414967701261339533553, 271), (93308040055316871396581663528458542158624159584434785366509149715891227656687802858975092977033971783868675760509214133994695049701004801320394704555515438692480680177811033717552559960093793722190133141548208864103490024059698846906111064074250754999880375346582005294040012710067419982914578512693093111033, 271), (110580185770077407344376468186802137916471654432727544733032462227162275312277040465165076422260778343048642185043039865042356545046611527042381079805111418997435653982659781174237947649395825609359676603779028033542594757874135025731932421949036204011483196828076562911591654511178766874509572495993245134043, 838720779400045143808016515140525949002532761447181228589565905679816249637), (83305820705098443348774188290338601260641177415572622738742264354829324920560179589687360694218813694719623196316173291374036430162822440328781356633972544745480662813315098765923476332732335587689415025345633489281542414100361219883135564471239150398397511462631158546528545171234781930561926452747063439641, 717182757043397509727349084139559090768047517061330677660215545295567238195), (141814545326206055966704348361553809455197598651776881473127897485866459705321286543575720496511461823328967462421489417474907773880777132841090245958022352807474394967908604584676588822407901675022895814089537933925690490236040051175537424564857219371262946919012994578031201868533197746885577784728267325573, 271), (102809011483979139478491125718913034561662181369125340727064646705109635626206930928889366591388518679435203787627139467932647475421746917779701158905842823346007739448640027900995510689604595560001817641371964581097801699480303579014299116443985107752532125193084410347754523146779351987381619543104077014139, 1185830950805692503935581234309835403365063862450197240043652261494183362491), (110617578348465709860668960846394880728675686207726473737030295631602407802088730295269887761222514759953802067195642029142559243406233851502576100352099333355635663907841301919979524293007385901886191440007176092809989370722930184342654770215555821399974002916390470662394577632127629162872178895748989406037, 271), (128776850606002828692972690601034791309507758204384202382659331729190737436526650092885026175367147605002231691892259939206929052556487779264770158219048856363279914836898456593009022149136626821395280517189673288848654676819128062840447926187360446636065129966471771001528072637790842431616501876426289541039, 271), (168238862561841282524486247559371255468825131634477948981168517735905380866256921591826534610123541553881391839946644133052909881997131115765758322931315096654739830018146361940690704664413389611141640318163261517237548761726525647414330937533934621786980697973168098783789405239367313185433031543087266210819, 1601531039699457322864140261743917030732495609547710971366679685777231684711), (87704460426335443089293364074848155775263735283619700838137328711510027134238220244334468018513420998673506253960950408515313299062389985404728851393883036514843867713427512734368347852375318979131962598138642071807349552751026979355089951423097159562908669179807619711474821984678349921217012362900251935653, 271), (100881817488325945209645769752904121573615445533073333092808607552200942922998690645088834657099406620133881318222753390476341222471357597371862853946687484074177357926292595813489649109481877597704852665708395230237466898188697002528017482542714095021228785396000293926640603304112723751717441242062701166709, 406631121563560020378456871799178429703130284972936195239813328801364733583), (86018861911955449458174154672040364934760911148796749137511909534908949442089033687492051510187793847969935989091448308202246995750175471942496579066880982693921142590561842063333220578517248277680734418864285127165361882298446933461958888820325885146568937477160972524171675475393553696250038987030383509487, 929673894399993100585333969729496260655709028431303127810967946784437508963), (88822950914255061199926940963773844194696015143932233168653974117671833203752864624922140670173637027564476450563301611127708615717998514774976777104687447781679805110076961125735311673221470753210261074245942636909983337054989861611621673929006600034560570043195875826933014890900347261692978107252824176821, 1313139937868217093247176480270232981144567006435553983493857397953168875811), (168389534816160439713763476136501204793032840178714337853711485347369260294372713935582173610526181858655630265008168523590818130547735591448930812596751059041238973779855193058159463151837018125494190537888371634057532992240316371379554034774896639875927772552382551449140596476096641279920277640931997462847, 1756814502917335431418629322387950542193192145746552651391347794370158119915), (116245333395782029955424432590994023083090468217620175059576998363262948059416563833251977999080044445791929402469457467930585734558203953603267050817666797286357497880879984800843086423233980196955672066237769696165196828159007470671337002661603786485246492221144997680201311964655002241322073376479410369003, 1573), (125496352839990235417386463284618825408778792186440258001662835769664542673811511292185645746376557618374656697613373438937918848753684988869505625909724593295683304105038190905209545570168421272770381368631121818689693940176196042513342128833673431032410727918410694573143975425946178647750356595007729905299, 1626257722670675456654908515470773992144680922364049253109403964755807237713), (57073333695381682723329709555416846176860512205228120455662662922634591924236955227452866797145897675587664476558224671404272052775724704900236059165046935544463548452708442672362236874700004817059665258017177596291810768068245028213408406141380999338687268807990281001347027996069545772082201401376079460957, 271), (127591230251247506064676380177984249119605937811919913820180486010726171833475658295200292292505886304699130546012174840504951437806317251510599184850098475288364273577188385942128206123072354453348993380282350108935835057853603262851507658839797759321899462096012695033158706081906928556485389996117984527899, 271), (100511797429932794473304985361994315535328839239609754020713152799219570905827431695815001654882476209872278517460227700938619966895031285080713807349584286546995715075334788928048428819105814987168752529517457408507187647160103403847315784801371956810451526180499794337585621575703058766135816968987134085631, 271), (105275180058317596307235504935396150913022119594043533268564782719094892857000013336939717912834409284551091202575901517621860761391121553379882689676902634327575118144903885527948204101974052695462690501964561877665795674089506272334981281654292046023913310928006674230632076252682284034518914594046876372087, 113416752305415506792370749077247522840637968673209207884516899126325316309), (113978192508580689636316055443800611430148384891012928506784653092891513604090319297065910606934065622663741395448652827728082818706178395354172986399300921328644136457550436465791362141974797061918937097093932879298674119246885614181858681494133780145074976208509277232492836203774668431736584130417435739233, 1693), (121572743248216320519736206771732946139331556663848403100108018576296805678207275627811802875390772494506443395146315783078542647581932972926195384557962997298562062747217802510546902342031192752643140765200202972951732105140547674659683558782261098804759453109545463227453183706112894700366009793521637483863, 406908372191806368932308094700980642308008836942871847782694317656103372621), (149275621695265849846612788975369943694112692428938386423890715886777268076140180809882562847479220528639036687044701123151648759404102167543686558356000346265849146809908306060702222798748775273347902158813255767699394128133155655578510778010369512226281565531454046662699822799037909433627788018888577761407, 1425274783069954168687413357074588943455862358091131594710482652245745099953), (51769506894815099621373978649236327826174742204620177170621510035658612112199210220179854405795441258799343859356222587813706880114162031195940946484199969577545954884770462446642814410688037032530988108736884153677085642194193716565228680010975295098099551887482559482523618001298551007241769501010089499361, 1289736719253816646976205078122035553621561854162855919022623596720962764371), (139346353854039574899905135254217550679840849469834547621248552643131014041947275393718819046369915483146586390952997525016661006089863234291781713357916479937366512040469800211358041770552428626772750870713414661749193356353008434898791210771978338331036669072816245091497464100141776756240846833839077515067, 271), (69613416063349189357896113880156442628823499652768421974698197726018150516008143193650550853602701288597120249347526711051023586966870199017279320983374263374954703380935831057491253843355584476499529358183492139872143040030448949742826597447410288379420472300218990353314997390277820303771715858239184810037, 1262825775278323808438743660945746461101268952658570480997937362344235093913), (127291855770026445898466193760943198214998470377655305090245222152883324035816368251918117312298061892699357640739790431211241536239944288136701095691166139617699506317026402960588982042301759522489019116001829515082066427173156431060632138008344518206380394410903497629634652330626398495563674179654850841247, 1873), (74296756249620876043312893164428676536747758486500443733791692723872732671196389044459319961042450208845983298327438410246817883706348617833128100636382657267394719445378045641010178960426809014772704950604062276120209408289053230614116339444825303556310295169459063525415946613806311025255232919050962934827, 1306817988435589975000634954469023612222751516183470324963600528354595286015), (88133330893904852423308475214469645136031436595860178107367854688768069956590157797915502968474444478660731651439193186535795699247622867916648796115063664721505473907285331685207429824184258237887563233476064049063832815047343368397813901536142364300640520085675307879059085085262553152271950099765262412779, 1507287715064847423022232628116336310048300267129076974050322176481244719071), (79236148417818260566382682759644739204739527386925690470369554800339363775491704390361654396556823891169154812131238798416646374103081938618645979445707075098538030781460324459706266165989714317376507786895570210961836906551547440521056039005327586244600915238479828414346715682625150190702773863662559962879, 666257146656495790415015009110002471132871274258334041116977137778323816377), (171514141479351742572850657735191319344425970407570935123482948471816475359353351261585828273163223921987568091067842185943115493420380861202285697298835001934821889754817729915576600467428825736783155514096286488051880947723841976211795070270346307696293716990277926185884282977024088661236389000090120797093, 271), (75122793902735680135675555873898885944501724834486980494876712036626576562108026076018927255093731790370732126970243085268402931530336572025169560509309400907830778116250879166156071611653758325776584596308348522219471230495928755838087801895973266824766851822246899028352859469198914771040635695068912457469, 1159), (81620403658982392872595230172028220166896429527687065949874018999788633259952754492605717098500316276402924166912312432894150176238660820743190853293039974739509214298443843183283325673381039060471509000562211614615837088640982427062087310423999714333004387176447681823375808780508713191110997729519042673717, 271), (111661899847061291757451242312868278515215359860853672607201627610919993273442214928492832373587945077434883057598305335814699615589042913388746480696744507789250923150737743015931807128241772356682198854913097990507855765899881943864360190210325159539498833024800167649477135021249169628975393033885871508609, 1764599217786359076049442615490518598177232300352563401570286793079888730021), (169441762406815192075598400123073637744529845731878882744496647838486837737618427601184984544621227830698705667800110008975474911207403571863500936742233788787682826320880378980933396499192671298528138219225558553090418091728499094979382747136198605192220918571283978388592973773723233678423984009836750342809, 580454783617353085842726042831694376260801051447914835695269013697638721939), (103377422463674792530703078054897215904194229917409550480884543055905600417449566767901526923486259711253209058127234809366980544732126536439016191977302186566368582764036937652367810098416403413404737541266537979026741350510369049019274530450135647569143799784470929085783253824812661301076770339499805363417, 894333152805797262058692188914786825637425852323216724894404269008857408477), (120011524163438317162183018885106212726434264047578256760603280448891842436655692626799914493781277363410644839088564068911095963658955811097574314093766355796789794808787905631127181951273457344294961615119117569622872490824653545370288884465454288707059693727959585192552382648108554178444072650948568107933, 271), (64442947797186943656071957573495562004074733433909867585495187582651343758543997991450863516754705625730635849146466247199284284341138414640494085888763636762825622911773432489962140380177412584758726965263199548238461732657884254280538544954404332025704871464461942546402340348157627482904734491481822639399, 13604849666907314503010679826056436792387194580715334469296638200043519377), (116512806887618782013000052659599497380034217067670716055544468058892654434832505046160757609822701003418955217333360373983261519719837259451553548015823911427261188535170660805299071353709543980725714272530616448699074993865798024848842566484329405824191857021270446754765168985581539499504563059271782892661, 1672501652405307810742521462234939812630733365325627669321273526994246409857), (136059526074081510510456461052556785285483410032433951039670282521485553196799728495945129501170504790368575035233101681091983488657969226327996219292902163621279904300303960847130629228186803023888433177360430739327781494949218256567132478143676861721021344744587488743272127484092037708321504309845503139579, 1545009288388049859084656778297818724766409578637909719385990601877262432353), (86785044789179021975832904563847827622970812241533851780523547771920256884128152705845112209611097796969706793245578682875929976334580454999224648175128102683203119864752815627590043393231997636475408507591662005535894855390819509480253829923422005694067212980396749511958338381441058045761864656917436419873, 682475943132471796647847945421851429199999102823129477105508709518142278301), (119277809736602745351606685905981782187765975226605697393914895160709995815094475922023239838403835307384032374626159370122800148697290382645408918168532000602311761451513469321155032330659354348302564687432665272717941126712387289453365759532639556340977744189575030652274383569662783425555607828878654255113, 129621275237961464274864247803944564123317498067328850322889675502004150929), (93438644410597098376203861487396141817215510131827182645884069347594156657674542929609040286999843067658463569691267559929615795261738087631008052265344331036811580046993189094413542834432223037603235606310252996085185253538706864230097023082221493787008136965997986881988519094394093010153534967838971465673, 1086438330644826270544193094952701291345034908881404654830906822730642462681), (89360447142145322824398136972175867495656078483068112871179500066407425644534961015719368430254813779030744403807246095747798752827652444820011053010205207384910248527305362784047907416703365853773813542371365383535203642732310719137687039940775134687089655228970087054319566014542872869633438697174134237941, 271), (78923224775476357208215116272871940892943668890113021172936954448550100701202029721256651633732417034284333701081523209382157521762439658844732142406833408748901047120843155432916602830064033389773004383038707908204948563544935491777566952197812264273203829446392374033124336657852860614810808666748326222721, 1400006745547077976839837729225451298643990314443519075765674724248763657303), (88826301713504972599971000390991174319869080331378762749705027166992215467746247218696871992700538272880646862532218115615509640854456488440636333919254129593184477848986226488046537204076429637329245908180657928510159116072751579863723988718852059088657027248687282621884410990106234855435675013046461608397, 11557785700522248618478563428342945342536582009794652495592020620888901711), (121230004402848249073440720412794511426783581604407934703452004641584931588991229754622063540818889520710619314529319100694213630883849283019652260829949970479303462202020668687014772845525806018221871774218629583273257743339060716924846811744252263542786305655635030331152804260231763145379630737116574327707, 901770149838017268568345177698462059041273966747084500308368860894360717947), (76576399490735418798234504415513227308139190606815681449251618723046491122922989734031290726580022007396100531308144967133681144644976977387278813438948193113364342074795080837811656030944815102488445112270521501111872910259215835938331087014232760161369000362784401998212764460181084913540872195569643130633, 386251279174262130519721807110824096863008462058421054126290670743894515101), (85748680454349648970031149045852181695735838409421295311439043826776138855295602905832913331531304269735138170320044618024081850583535591829430139071457582371986687405277421828664818607798089955619249345407347513951593426672925477725682501306780880241855978738232613958396981372105347822470028013122023741261, 271), (93953919785424899695818531645138257708160591795326211250815870875742741600712183654036299102765546444059653707144587189949435970005953821029396554511713505892874430914483114210722814150268749510607685449037705671098876458444421272605054697269729153808667563801490465847841952454575297661991700628161582868737, 599917864575545942904336758410115577592729592058024191058170115976342589111), (100692090204490041763328515770139851711726261512461884909681565746815934734015697489057133862777488468755281935794824011440747786484707845449496249137519326965968660886895267636544100241126755898455090659949775944923367079089539609815385270505029217434638212399716258526512769130935463096428386519148686951723, 1959), (136406527586046665188998173223510995348145268902796255710093285954102639174712349558903869138757320658587122175392973378431311299028539596030821782452961539180306607821355635711041729664917068068315750213273692188529523207413286040242117255340470337267292684365052622365763996480535738098537007241253487670521, 461085732349022608179651153694691593490060084046748866800492550742495087027), (55056953931531904962703560081878861386742292249325871007735101153734310803930796185007479609877378331927689066743413274025626270084603965288860571555242793941357889974688593097477642018381840629947980163136772090565377249874014790743287091359185202814076149835354718646072192876766717891700204675372869414397, 847720289072777285323324979499857862899097672859782586435472865894586427477), (124477748441093100772966692507987262209241051699797050237730286743351474644962759040305300244308144405196752055383147843578492891613183230693468285769493084720338129789037045621403428546689025994840384723101408413572182853762310140717288341250617791141538866861166345421278934304140868790910852148814955409091, 1320691776685920197446559610251471254658776039930730244096269621021269921379), (117494089994892902989766470073589124754527384701853859491186485508182182355397742117943096231204217697462748917713941774960274830016641599313767502972453745879935607352167832663103836087121725102338392735974131025605915215715005810829412477932575268426542639651997207172196436774952803151833206158173362481393, 271), (105764415952466176138215984336804889521386167007604196712171043459037253279053734767343105583281062144734039232642966160053546383172402280867175867375352199781060456765610844365796695566922378960194147008847374813346617109374180565831497093861834681904210066320706838237748631285137945534127761009947034046201, 115318553122408880748073174689894653050330684580106276929603982596721488545), (141158300766067731115769431524185404826003997439364687819256889263631769965238373829928699570389430095094334555315240831393565634280033030701146790390525074764649395925807671704417840738975248641794418793115621879590081431566394298693619344424232496527564312525120759487253080877520221576801496469192302180833, 271), (113827464952487331404059742603391734372170637220216752267140175550168020753071528956869114662724346185695491950916943117907547319203048128008879629860064184584427092550031324671466567360888342640502373348021031225197205759324747013494774558662225124890798953797380379870646310959888027321634507416365392883839, 271), (61538463865028095385138241360300537687809805430465165714622094068425126466025748759656689380110265108899700538203650979356601647212445374551598930188186754233444650230285495283208590965857746941382514759670948828112762804865755612936906182984928190985697750471718977740428041362326041824508196735965326507731, 1264350740574837671105768487646590077736141100839775873851105404534305060735), (104931245072194824287523980594258554260512074599376732411695866926004603076843079066199135827996287019519295902377441952884265920970523806357389994609136249269753153005949764116420675345245536818641135503788244351275482286090462534272998174432466147097890633551614223770910932857182809096185494758258248432591, 271), (74928558495122068501173715196787784301619765187983299912868479152548599198271863884329251177934162947711660372734279488847057722155182914028045521941964066190472207924705723339531821270438238161455072797162902131098700812971102557174545297374710222621160958872716521398513513740812140368650233602688370977181, 139790244747511869027636968443178670079465222279896084284754471751312786513), (92646688064862009873071812965891277400510854032681480396294649865998009244804710250380314981691124014277717214355749809148817778263944641639367529252340851041146491899645009584066350876991055018703883095227689040738973716922330575163941586988329584994838153212828544457978526283199544329895313192313095905189, 541191891759937385411788302339397329922186569891425860422484746830822724117), (104526437636898760268740168108163599453281327590576784426137156863038592500078905615859431786674445268068946661919570537807879435431314198864712827154963510373986899874390055982085559878560947066514748772240964174518848586298254040011697736012533252412718674975369156920904378006062417154073087545743095966701, 1009766913842870595051002590987895200979387762744811990408746860102283152375), (109619987322469200020001173628736821525150905715358889016178162503621992313686546208052553853885233483203244459854315113427360604682446712381908724825848648457145314694620014832825908456782053214031598235724903637735945023346326240959190442611231738375863499190667193286773066999788350777139988308112760323179, 271), (60972706278406606436036727450832514447213264077912157925244940515612782090424170346833178021766101043769915680575633220143854928958926424493547475181311759414641401517238012324676698555494314071358847273611050995441281811490468016852134775877119710645846828523107342821050088003297774792990348992301848942437, 1226962437091492852525729987968529470410866636081171887270683725685943111821), (74774152934006658750350276076131448253122986137661838364044693823089601141256423524822482962034976493441266388827432809351466168377482299977028265247889475108221326655032800862886790058953467020617379301935725562422132452218287413776699551421251585097890986372783308142680068289471593516029808521922430270539, 271), (82901391945172235081249569070620352693359989607880446847126337920551434802378665400435628981414940573200981770184382334232899449416365788218505539087275291268155972593401645516362929917763886140050556868462841951997044227116060457681752193100867027380260880211498130415562658608081734635961024378302823467103, 271), (150073138964946990920113773628010392791198645873546403288207322516494816862979618162560109437473925681541245940838292067338205475659012321648409723785981724900460185140554610812674801309451740422123490820886978287470836628413041361319819814570380218389360428975554078674120230432719112491113983143946539917291, 271), (88874875930678033578171790780938179592193012255609104977012000248488591587982797211616656279248626082520346045719846459940087902015800024122316837482944152633274572147815161379563007155975042643362393332405302508352759962575773489991498520464411982292456884701289686409798973275977707453117219252860683271987, 271), (118185990208795324913099502324418594207325716086337043071177736472190341711875210607524565759543238443002844912324766767251753239079600880885931683017785680284772502363325143539685465815891325799437843954431680765141278723075970869863783717651774541579251151241224654954712056384559077208078499693094500659041, 675481279460907303213725799660612083520135007450671982710722106973457851363), (76384694805091929768035596440752845476498631364352179145312892349473878124834369586771046580419386234709074245470979557195029125513182896191554073639366909098513994540765518963972537995130323307001602467595193206429862895226574145127171931421280781663755800245152765536219532767342294387140302638352387272407, 685042264969725955374348494202090219334346649995362519683437819270788162217), (66531419087434591937381066704487959607496713635966389530812355538813719046820275234433505904404749857517903597835776738401207304391171793608150259563417054872357257619869533326897273688546532415718570176451952251995472681997720939034590738079679557601792993389305480495065896243484453473527434022374736918697, 271), (118983978442864980543006104124592925085467802119185522121077079646083632060868301852239647054863542771152813663119084515705588656564144281827298255484068251492827744827989710545324578934155844864894437327883972452900389436783121759647472861780210756621620499485458121856468606954862107568295615326727579746489, 271), (79749233122743184400005014407648582335656537761362829527671508032210946129508154843799929087028177676908473977349272005768353883431936402338619780334686227947266971765113419692079112210045260063393558706193934559010745513635053617942600109403381969536616208432510759233522312956001484156209901191105967817493, 483040110313187281129864184626649970461556764893093564040667774877375781729), (133566692272152011066478953978028432517801791998884057400296483704705372645803656505875742495509260969934983025011801851148842025663120196317741641745354389398117047163054597649259194421892057778544100589117768568593324004497225339767916666045559635443296071890929078316997280179304795923558743595937741893161, 271), (108950769462551979961357001034735655123072914743030382000334453098436098253917640759718288561011744740576791587795330588158234955511551233603794980178503719481971808377349208700296602444772298413936278719816313673287158310876934943821117330971006990960195637545328093187705026802003922617372280705918690847723, 1722109667697232112088233963742121400887491138383371696792108287931924543711), (167689908020771638561422683338218916782564227748392996898235099739972814084359159357871840696162469362417379038098008042088639957631508080324289243934848034142116563364208662859738927049894849803682330153311416024458135488102573827507781586513508965972248687361042669714010379316745808067211941747386715053719, 271), (92010548906720670621593454184834840512825067811081655817219282933084170221801649922676394835108124006619283500216682034059008639644703545756761411581448837342677805719496488966754637453178452910633701606712999665119884652245179877804358837336613430608672828526141865381747061174095428893998474957433634373129, 1028878341641659722639091026154745915716876682941985511628811073803316367019), (153851546194990261615075564829276893548040783664403058447414370762095914600705013472074723657002791844036166140570406236380471406545904601563949152136868996045409174386579616306868277603834893647967720004691836965453649247272453253002922659928261831835899841763180032361310842801984781787237594889745974131889, 271), (171280260869012599170815268963032175748921120187544855261364403224730070912305517681114399100523619919938987824129084780371447828935513559874201734741732085898913576486264241962947783971168850534105922457703377633119588875988728634161131034576895965003217433683549043395463748677318596709706516957589650166567, 1583607723038681478385101113558241184794315695541776803390510460787915003161), (159992434859977983903116852963608237759243329299873744737032608155654474740799685993761889865867433438748079812904808456605197518011068745535128248393681514338653268832972804996688495616742778436930067532710284591398340902023970166368687083276966687359550743208408982585091440603213489105457132357256348249011, 271), (123632156803183125872164932226386114815584089800525096981989315917524560388451797328726684107150522092272537567572780684375697481680312507275136465274408160907060104482234254225851602099458957906988852307469989991051068713322185223091772701935940202098710273629679642209371425151281394959830872509582268246733, 271), (104285606032320857776392513415698787109453581420918967064712029555949277389473650171101397894199094090918369019449507066183057820633224985045467994901172742798557534884043420659930575116415918685376859806848558126523953546591403903475043751616274338455575069100439574387054736852211628638398631969042598646373, 271), (121079988460652341508249938758406387710510447705564363355605888571103351213892972348142895517727784765597206717194249759234904441099303138477821025469190884850751664289718239854041799005807320483938512774219084407357089229406677286282048913157403721791919339107673371059565074787212521408343784475322148588723, 502035001804564339538131816118906124380142741364982596206741013881497770149), (103243023782558059256783520535627815576494109145771183300280581824368909285684962087916970856459345757986621110994815279900636317437507275265170059375378893588515499377570138784521628489798924664378213540891989329847066295826065882859710732883256419859706336277690399900095871247659810069698741915270858489777, 23352347280681983698122191029961051780378507409525400761269685391618763395), (110851473782410339020221898946821434318433942011575462808009755733956265386120717253949060451721117813587758592375471665368089244611186634880717435242230776499299082001317939927447363314412849420379119509192678651024026562138163054189783297840371994452334308540675661190270441471190865286129901810305878246913, 1393495698035058232953159653014828551957788841087270269100223429368762402769), (149397016464799913790822530501010611562929959660984356014257969272172613691474605017488440732605362095619671502086998311858012657257762811165144987539271600618367026999294798753813997171655287500572956955367042175206703533100351513624982305438495973955271391831839827557066031603224718878835282530760429146053, 271), (77083306805399885918093160641150923032063242064393199203206423229912231052127720972148282052607059774948850842769174094725303061401809080232408591267212249599188144115879566177127229512610019515746060293396153884944216490911888265017215297887641332882212914844660972042153732107112320557401560909813676008461, 271), (126130418276131116287479076238101629346803926547195830420413006431287722639889250688515974811816292514489706986646176812748297952231381053990207976751635758703335814193843303319379310972049158617677963231275133087523500086739919171330404950891832707918894837397545195665153543764919917316566254245480740382607, 271), (116740875712777514580311683292155864998040308710894984175231083836465725367759294101300872735037870440471921487307331844319408025950490223574860033899379849574600720330711577451251450665323670638235149710779269138569140338193917968982496796372069349408575071192911455216778518773307313617194860705734124229771, 271), (89421773306694782170518515925591834408967182634571287490326521309299590596418289212384257170570806592634522574695010671256515663820891719570511561965131342950460512575292308518925296833430903131981919380800500914889392176572431664291298575708332547325424301851589007055146169604793064141942402094474780159707, 271), (58169768248369063879648791279288525215816299506743568721382403718421518073059047010676306190076849917320866985238379059923323623141439229909748417900577638861841623350664565243705214380203039727394973772331131528525708439688449766134900119266657246774165367718890473203250574456934910968532756155271907683289, 271), (79603209294611070446064031591718539699548697400232660428975714624716309863400853176862815653778873443621477816719252793807948908585569039132565414916290603863533422441677142965718359249766657486856978958569545431357996151037312015951888616547965294874130231134338182589880909970497487655810525465925239422329, 271), (95715824554665773684988759524746217267603654819082133572437558306234671557897315703128539478959232760008625808469612340759687770235240053406464932949708847965874557346909053558769404054966795006202476537749492956857736080031631825709243960758051312234783664554918411715686172972424300882087595905249137837857, 1592619528670162620152123956393907515196764060044399158883168704402599722737), (105371191769462929988741172455313870999257648963973380729462163638938624429958495070344193202410176161920387785098349737230794468375291718214746338563147299233779051618894540783357277016840538887419483122395182809624031850786161013070563926211339203548947085217300923549415794029244289846615756149209681886117, 1807476892438654204384367339017243338291097004037935185609722595251366524929), (69274673691855772935045599467757648142876216565827399504192911375332682609957329810613632871398239440658840346873263115775814381651650963754158819273908966798202081137528955473436841598767927322350535281688710418432940879456407017601039176174174539254356656499100277990973906027063686972197247782367165116777, 1069638087909059801251906496766066008672678507977492771874439580328669076353), (110547123636130386788286330382330381323342784605640382264599712168183290132667089949391899716340768163560191750434945713017686832136977799020737842738159682503004450726282934859982175738130199266371999426920976626171099749470445432359721432357917431001526229923574153312544893654243394018371237001345915127077, 1963), (74420395700893137105817791994203615357385482570900710078969412281389687103139758199986652201866402718962240417258713001161910631279780170981063270439031287473690100002466010730915335282858833841049283887769526271797427617577028081902419511655024466765888549477764146912707628540021364635140328930891496213337, 1283575222884903232108307116957648112267323964758815521326607968475173749297), (108919764665533353065053632180942476606509264500203694748913266048462772062128284656207253228330494030314862333840731103921434272982672902421727017791569348290568376285614748141272241445486348569500000954532708957295776236751604097321790591485477631393554784882350443353460288221022411478687664313092288934509, 1281814512509653271419243062091979749090985124945038215465500851978771498807), (157720883824471053718830298746550130265912605630369354216875232127207186426604844294432949676439472969141476359979485619292485652180297191186647541426299233958267010066102981260857200338953362087228532284781797683723271869291566622441444343583183470659527169337450804673293390840896497353279105619983981984269, 1716154822320188533438926153022315082299975900407829699335441656901118065343), (102572896281172644130866044898500406006430037752291938121374173480214626760951836559042005187004958072041829323249394464354419664393240081158889739939977240833349233171445762246550459414197174265646608026128545972072235946283631044007890345174862129260339674577387396681189348420995388293496601027464784735643, 271), (110574359145229426064987849032308062562689005903177942726416850156027680536945209043909365336545374452987510909877138877699867116957866523345807302551090465525057278539104546523464817911223166152375741827859418887082956470751318765908876436515724877702124942995747340427751967786461282729842039827761355680827, 271), (75224376068808206419183354699515716622478129919670700902068253522722218173048223202722631845679844061895252442633827889630258422413449882542742003582434581815942241148817628600403343129178416584601509819379397252300712846021407694453061772088088905415906173387682766711648622863466908207623028098448626146537, 53284055916772708399473886694280647663521398819456287820860965336232375969), (62120674405931795672895426306219129881364825527396010809823122656591819633556963752152349533295584351560273231892402360746397655213297060748727271896973244700172116635893224298165028743018511500940540388312360832351245618442229416630982004521359373530041380723919546624701183069480976666542684100499290701293, 583199704483053222326842663492061877574867482498094363130961648802435555031), (130967399024340356602413700007877173917476556861318086328368827704567632209995942410482370082969989513964141494621364556745052848122040581650061663080648567695755968353506080903975651433774976227329604555014876902712537710201968303903340332018452234900269955099744576576475858457758884843090936561446572422273, 271), (89327802931874840446646466902513461177924741537986486502482519884060148930499680388152070784515817358567301228631231982502568026615040248643345381319056788244158513627842929753267624995368795751492281598269114464715763904113299065000982760147328014551900899056603910708488708007493393189551952522559925026589, 271), (93779255023168029966087868442222429159660885309635728341905975605844587670352033180069481528756102852736621501269798204137859263826558715577518283531538027895531325422749630744040859890030584516432066255833226165588089282697309313513349577575250835081388529626986724776453238501157408893596752246848818820803, 1071456122837068172226966986730168856788731979459826743216107692222970485141), (84266081325507165829026313252790256382916098740310638292785324691528918744849185457952032174335416384556708582378840157063320183667310406809985605481203662654725592358261271311303069829976440516153851368902637679271284454692785057757603639935551290628945719469738128588471194466167193997479657368270885753381, 1451148501373161795952758940421467580551045024692404061916621909395854463487), (66922251719410149092814516512912026379923539091031577601666979871331853211194826413484334036119766774337360190085401236473314002656197200980510680075651645674595971035027151181280966626268827821876041098378583670410643191119435906394447096659272367958360420561707919892138719629409369527278487598732849032901, 821504630095042847399528780597578484570132306814253624565015886060202893257), (97319876072911287582320482117812999576443093633532733581682885348860495069262197331675248515769738388039678655803344131212284757387461464937131733638954446420369389640871788976538240880539675197741509228832781361655601052014915471108989916597141186745956364817003666108130676070434472416398901145481815718899, 781238422540628906602470349953341219441869011133805069386657405936264978561), (109418440849552578633491746347049508922766077132160227608119032654949818969648012682525712717907551291341924247387501458837130283190025379508200514147202451942146047987328616826317651848234894928253202853078237196236518368841411069182752636062027721781181988698855993248210619780112100548218279590940103115153, 271), (70408418905374769779796615385620810036670839997656814975353322905407207235516359736862846465214293051031400125185830666655638976283190689451604629106503242830452243760759986108391325649914713209345183576669306984941797256702449016847573899733252812793276736717229609313843906136134886463821600136204383460857, 426148348564259380190306792602367605526605964229292786437283547073201616651), (65376577940890682278159216329856237867714223990510165169973805037577356741057507137482619614840535868143813136402532990301609088615758492398431650652681878823521916610266618152950916185439856990788890339252812772682365778814716484570348346680279529651610774791981591083066183311090258237671448481259925566957, 1219232266642162929671631721501118730004073191981042811646410248765551340003), (130939304633433170641604493695973911286209881613821937263888012277111557783434030295057840121450863132410434898674033987810769516682231183092692636085975479024154752713081689462684116756055800916319124890317921137271363361295061880924541476608126622722502375448328457055150373298509429697598649979842615711413, 271), (66293171524885605811155474237248142120826125580823324676159012926167758974491049302978920654672338739090452432149525782008961996925012358078826465073967364351128047490966658896804044697432865845769378836886002683684262815004565080675175619033296591309436825205143197024741178649528630008666587050939252548361, 49926966365288264867394171168302092711571722158117364997386488807413425787), (90373893040147543135167825346086236201008724690356625854599189919471369038921416642853261295888133295476068823561432446553410500909388799858324013503943583088200904327982734804414956219527688204607295802654643289473984155009635244462926355531979494434026991788874748436621418252888916575743884051605343292069, 140152665974709431890340969962974569768443146769845499575366950452327972571), (89329284733998215175152117705961761962624462999470666845967609795588970237807768790606761712685211343100211855345696010982117606144316443886553209508689187381371036621788444811242028675907179896454069044791967454809021249060390401087267908885760743895950714471493808736963965553165053923313145023543395665711, 557455620023558082356716692901119797504524976446899603111634133711642361067), (131414107272998640573736873155934500974085580946939386008764371178792313802097648251058065508448901246177157885172055010643879935806355538125351223097976845637216090540438179706470248967666194443360230420473488704780349054346771972529076551619823737457899275958213568815866000303185701539171207842442100837053, 1963), (104706696313358527187641256538810513152709335782489822246799319747047392285867274180742436002946678414653247691720244918156097437936979598084698482658543451735552021733072265329951602633940863900559392500527400238126200741020344050848691336831557825109894463493677776322986868868542878463474049253043456379121, 271), (98680673291826402453877397552952155823897403972458135524379312685367872822726589664000754137804255658832778066289984525196669226789230796503981719527447020410835594607420197587200662245951307476216272228733375825406823058764438088716901936230128249064787747100405121323729417913298763326719694647173909436307, 271), (88865654125038797888667383059184788556156174386941506786403842251579341000661566555338060171709889575549681907496358236320254399035197623157960318620196317810175251621888507302464890037359833229388965543779634581037998232078547866338981653982348585441211209498201221204231673736379612399607450167511644810201, 271), (114317301245287047713705697314175355915701262466416652034378740975616943047617593972077448458795103891755485648961673093476344150804929024865393499025538389736988085056443892826781454208750574059731544840696053220273387007258485669600383948357037729384904663278877209868599961322121402939702659074589943136863, 1257329780379086523788325872241059085436361349458383775241592957131715880849), (141996400758624038258745047561438758864597798357355382350675246616805178682537994156710682263243630785690647735466472387609096830180117750248229322206306614180885455541855792052630862394183250054175436107033614011650353295094247125204244062835907347641589609538590246364861258558418407986066488788927308224681, 271), (105530934655152882864053441006370727306105852555562091854283611165340422505221072741981872794375300481086660797099993418444168351293784430804251209425944727070839954420196635626857041500189843885642419429201721267996324297096380840905192301188201227678874794466139816801698699105663026477487213268095329380501, 1633006052828401430522961296116240114337493102613392813962673363315541878139), (120696387979652260681167863969237578675660461810517051226218410460639867146321451658457175001581425164159811372423417789743520695264435683253486153175673430539485262905707006912178989966868335780089772914749748500658936981623806628494691924574692533206216691378207670073885977242644330641433794596429268190273, 271), (78908009634658603078168371887732763038088019897437167475251189778690326224840980584924362718301493746099460850760362105482810463982218506413255784414786315479799248658474656205131266203141933603887039672355862695546964087829557305545769311672185099328848648736229325185423510214394561240653271341289226927469, 271), (58753848141307169089919023349087492667610473363483337228407695812863071311397842993135478707136569297597406609247278665957325187239883076452371994176943342168695772083837965176161074070802487011280812251783064065819193769243602338676122961503809385261153653336273368362897766634966814417390652591722089081921, 1993), (149206396511408464532014536233170944607419214103808492403454637792683924988246166807523414228259357631393276704389890863693239913227707570795675837040303692407609708901898059948432709942122296728920338362850523906405138706460927121536428579214819849340974142548659525187289301881169846772005279539840345851703, 271), (131569690292806129603724776903013685388847610523078451967434725093250220466537713150262934571896447004376195355069934790122766459890187935840638759760999284391466455591020431678825380826468666225907597369601364986864276387408539901749995879190901532398774930823357019136030015505858649903905763267308140065543, 1715132901384959967634740694292542514309708737104539958493875509969974945777), (111213039225897552372953068086005785708332984323574069262583854378366350628624576848783650148128275662708723380452958244248822079648171335852629183139076554030246951053707225405191991697189189264604987608950799541489111509908277475721017616307933168425885991913226747478965765385061953653091277240798255305621, 1508743193978582686373833457752619844673638940441811865033740710639751850313), (85326876962168227282982806097028764078634887920797005967096712603306533880742060882539818813269562164455160299165291868329753650035922930082101352747142759261654606989024910639368468463022922006948884948932544712035926088217459625696670385836047322910195515321877956086638771625771302101633381223568734920559, 271), (88294146147392183175981326787317271188433379420106949502474419617212681133005324390278218633306700053302009442735268882198532792864181783127223776520509360307466844266399040964310830101659337082305870828339501949450936615828865697125990790567357054696575799678768267568265266446297600900852314695527623665271, 786321489378293010091506269487458251684733479802918869880127120463344909979), (114446616810789913367811829629321863198269463339005747320852265135537093705314131221423835199407005477305660128362168880003364473046459995406028535696297394041359352763150006838586096349371940837632402074971315091098697084564218610712762233545808735635469654038726525144816429417714622562889740003470935000967, 462573501589295997494321785127491064671567243443230518505743354517393336491), (71287374402876311180398557769681547926752175098984578019658872132147390975750018251990721947362882541621022976678114711948363896466841263051568611602249634217051390551356024312026551016495306440982389222647798805989877199146346585394914233623108198756004585618619831597975692545844207836038895923952687440509, 1714845536947983820583350924614773205540712070371457589635573230512014103017), (119730774490203235107659659537897379421069165703070102675431044042333351109929780582981794053755642378401822007617410331455759245977550479266074715556186418380211282360299452923852050580589004051071740356891161644067945931065335414140891457811626611539963776637972841904222840409772436652092049254356772879721, 134847551231694354656306231678356601380535900670409438138591467421759534535), (79967279611964008566237047372115272072247109447365191828158240559244111923712955805876152387872345556712122268536059433830163957464202380502969823328659297551095346030634407008270010974574859571247503714198774109453254402510837625424956996764254794785663313104978078653509912595466899582615490873235095003239, 239363575566453326102546600098445270987650394340507790697055288206465484299), (64789415103874294178014850393088519653235645231524476473552433335963031460803759692057090095542634893868403141626640571680588073692988802593491781391773989981717410417611797971706353342472460253074683023367272478349858176768807631386000302166540845529509216670046575191269284660578902375809831239936270081777, 1804397306074582233397505521280517778279740138996817949136882384447615812003), (124580271092056981398932649353177234281822888810352353491580939504966818991909671510088985575649146779092168349731078472239262977204108846106884122646206306964780732399019792443248167418646626035606416321295453564660741438334630063474615317631945689706626437429987365847869920138608755798590031128156271356347, 1363), (78561313864755771429847383177561154401017718382120659329455104763703346495399713082329768298157697264469409939255929395817028904435408908973520404587119554967519481721224192086958514136420030190636148306771194821807322000579859536812922092947062782532340841056940891361595232112561641505123032232707014790943, 271), (99290758286735391170726267657771470487868209493931290953682065264357087929171004808070934635491214983116981674303643189592396380022626198444593878385218406415127021073398235092985521408446540538779764157406488505941514446556888389527759267882362464441361439011951624687510356907038590896617495598372416939363, 1645901176699950630527252912791217644355726079971168373042557437336097919679), (92684167473502809054762850095622083546332671007499960979641048962778453595377876317902568362912753222187745529137436106107185232116334595933118018272785439964416539479341713319973067583202094291600857938607706219709214892714281213263783266540050557420989288486044677947630173732673294501537821832413424552441, 271), (125168007038308485203623381711288188085262494580784276522488741531877871362998808231293279117633620329595938656276657461757184596538422438758727542851946764536717515281205855448343993540429853936270890054626575960814142114524067261352735829098708459085815764059051855926014126495700820941682787807194332692989, 1363), (113537788188944484593394882816338263788329835294355579515802124917661032431329641813759805612485187084686966928203620502425366517140423885566593167409041483400486798307758581318188910173553317821067411980243240082071776768629149732532377903330540700268909983533097812959594701791236035313382895039282050112783, 1796689700814454748260789441537086465215576174804980665068908001446221966271), (115165643794613480760557772673962370680644905510499108881638453043594734481184876231011835152393731446722719902931999927891893414051987627383445214245495800644251758467162642882282396587685148505467179021126207510573623744128592293478808747523432455579042518665582075104872573909725825780495769968707182025577, 1072699697236562249057959246302321961108739901400816991713130428883017496471), (106803018432155909833358611390257921118062066591153016447050555386253123295359067336887832816194563218808388923673926803980741823288966342296246943834598228779615449092435972336597791831386452893482537746534513477196827513172298891557063270615977667990142631247920376506807376499053334094175748063371696399653, 1853), (73077574923647718926455334902225518265192180393081396825836061949922163028801086098473132169334541404367210615218499110335023538262768085271592508195367616521734623351489754201005765929511513192852213415553365560519843483570659944201983632016270283791793708350314365848583445208605390410671490059330738451891, 278771047379324370211619396498518892733085447522961657120025951528966831893), (80820138331050984986064508708466882087219243853771583436591045007221191453810320016207529610791666546537930066678428936130052308363929792429274693816952043001825997520987389937526750912630931042142898304871689281681917510167112814518442267864014102858175832798268394446944336948310736378024694105455382055873, 594205409160690071343770104129757977151873581887367153578749546966924569223), (78876171069814570820590473586344195281551684015323575466438715189970769513946510568803426733115653490053647506726795458200516112117528487397958315281317731582271433983677078791800192312559863244560284195035165776747455500539316322270164895503777283035038113194162451258901113928019720097776801567746926371267, 151576836205696130565192283642392352932205041110839712834480019576472991505), (128062736288852636554900201936009852717513818493755742502141751680644169913019873459408562491738198424937566944930147978295084803868037751962229585034170383528591094307288199693722641134721555228871936885600678126136047041574641643198764650901742986837404647899309909978686335366593677189708733388855611235879, 1037), (96647746177292690626575829413585629302483552370662316583246460834532230995753874060053702828568166483434747295658023977395594702041990235530485825185425528912825084997007758001724547116482233822030204428681135165157156179922940732668496378836048969716827371806423475974606750909066385892829178555142803586279, 1165096466662689160184533760454475266974074524089332084190272148912069104859), (129459246718861181252465127926173971974138706544656765895528533970711396541580603180177284403040391527255774801443943304612386201893335426801233328361855232688926557944377314831191647350330944061268632109718130964943161650724031859993888236087648114041298581873600958155239259801119898920222299404015408400879, 1658034964804591173959390935362789967199325422710702164700737208753551546683), (63968557518791222854722878782301022448454914512510756050067061240015746452578798994014705414851894232660468544716226059526904039318434517349375488381755876539855429386729804419160637218994604196582149731967563486325982571021068645552486585064422983294080943433826006568566267904461308547221663089218039106713, 271), (55665331847347216594457090193348679682840828825505905078059314678223973180011568194700359438089834127187220402320208670033154363624863871810696452390119322565873595938722649796477612857789061021672066651830333200122978185909662820029684183229541463102797816158896535637706389077154899906028832973767111060729, 52715926933446071457021070672549949658319620043946655906269941510372475251), (111619989081796083419366138694698618135913474191449890614369020923113424709565486203463652603819665848640072935575664240310587208566576548731785582961479615895387807766781158677509026730143823746726566250426932193832051192822015482630400857305506473282252037827181307325518381108937272636102574898904523047759, 1687354078205797890727058099005092109248209649848948809964172367679966599289), (68208843864690463761685080266930484779966539055922135995633765047254017694018685967068359309621631432548064520396642871743147574621768841782700727282705786534054121129986066942337888568765331556672784172364134494633853803124149041218191966267459047692304682173387215484692932593307139554567900409703772856789, 271), (145809046013554996297205298517914618592289352907831117680685971704116485433126841063380091462120759642835095177186103510744739389660982765127180883796348388458956219988275400795494237770215193559749236437762403772859287221959220744431548166713377202419529589062667806036132984365852412067095885120872386369847, 1287601852630972656670293093357574761117344141443294348829729522786148157005), (96662060642734872694522275368070486040597463058775527732493761402404497260970076232922371082291183785568953495351605023559920947845117897143880284038408501978905501993046691811681629133765905617318568508505214532964393638372191941430435230468170051400441238536071323087430424381533510379616512493055321934641, 300694816523897921866585850968937220787776008220022340773626113658392464363), (93648015483799276591943892844929798348659966452286871907337647500095029814337685783806472844107405685446259158618795377281226001604593529948377578824852988585715054394902907900100162271647847430845240954106139482142277640723264537880138411414291107979803134384279756464339380962861429743400121075465809481743, 271), (50455818635589545173532900052564677393849700628544344727114147496507048627031407376363552387569684155444972467503879052567703656878011424869045011238301320823710198449402401830703128799069108663005457639713769196661270026155294595071289816021243054672447965682804004410410432214489771064955707228301311731609, 116237717700552010688332785458635941163448405678707623422049253224047824229), (69635711458386636550131881312043775221802209928504869960708300812505439499486608060714619771724544763644316165276400041100289509600731270276685171739114784240977153927028881273924919946704615373984938451861936946153564402966248066491264021773749155723033906371031487393080962343208396125559128032936213450169, 1088407574074124344863718151053550528231115796053347224911157591710717987593), (59012068995883753011369266063978727480933117532903396077690465612058878175471096198093314517111955018980150988867292851699256030020544390790524990049888133619815790697596284431901729260960985348698738241106912884892786789597340615201437683120546605786655109198292590495482299335715068049879643740959644582767, 271), (76825414748972028169786608885429790125437271658977323969572169952147153768400656700949755706623070522249438075179663424504195627384492211598478933938370389051963620275254745410845638488221976536311329225775322236061050672704524569327278541794019617000801699382773792868197093118906054768390668293809177381383, 1780309832788348376977496158652322471130243590111494910632568165741617167911), (166104193152071690242076173746603329852656080667764499055077617208814828339646983302342405675125184855042621319882759616232394918559147380721457641370755205275258483457976820803284565227539785996880856299213815078056596492238007603409272984669039259240026672381437835011039024993741537607731130526071358075637, 709961666671692513602260340529887323781940073875501988120325365093379315339), (160456563162315721632886851370639376563004589638414896040277954864432720552423689798695720750380472157885159758763489357634409182782263932227991136156128893437892442582167643791927422996111221811883057809990304076792505376905328979734693050486372944598619347318214271375357050370500422906815812538363078954631, 1300088893995157560411160905219549410330383280534839467967210484715632629279), (113094245300799371048758897375823026515717731783168154597973452807571021384054292852223996932185650811786458296133999727482464754397965921859335296068976142450446571736148883937267612846597824256146037278237253876900603361273146520423228586734759255745123407092744252292825428358611466956593111436357513634843, 1575239178986869986816000866336726320637005795860170837980317571749201668869), (127774109708843664561308442325309565416612714382060772413955972710400244399552447500236672035342516470232888299507070360312555659985476802727848318607524135638555125763163400140211665416058845597509330292117888710866595092939920389142270562509397380000858839491587328586962123911189031894613856686651387886907, 1356716949261187941392562435286614663270706043056382157521246707007823227083), (120902397299716428611500527071737990461109184355234760356073372507889022383354760867823664938532115976681975424113056474079119366531732326638977438758480280838768508492024385223151051922083958657034225645273872399095441130858986206862600030264887587071377089839019681400250611268195496858454318724417488489751, 1412139416221530821771016426548647164462518178173504694194364519411182026547), (93140969500909088905636557815550330413248547164332891620483945517338411398312967949015055481576009324288866893523352738920536076084092514530617222363950348117667918009759707134042342861468114850375693413628584323581638821861295322786993056199968255352659642092424094867953014492910184960863882150800577853891, 271), (103268747720708596583248814284417170107495347463850715281540159879037139742721458149289842191565179732518286184541531981294093960363790097977378993778764179602073112038121173747835509371025070627429857397725391330022304047662950159310518588542844683906631249751820802416485057603816429049522497303556025427577, 271), (107314785793096787405720702004654532897028062618906737670104341079024974833075406191569669472671394515459103558810087456276614399578699454259259511075575474163948086931234759426190334304246678626555189104000336536709030222639471648195794561925506162957398105516377886720421529624030039350295460943118370730311, 1381), (135916626865028829375356808456936790563813723915935987737923395069085996071324890424458058394892141049317415564027033990672823335382884590568174483715363387831115570036137883125566701857750538361244124954691667294590608194131447401813392158469908600367558245998274452155721437345617729582118312823263564640781, 1693979597488633486989740004069454471900454654813785628029924707482085393503), (85879852440015546625963355077761161204178740230692831052002881572103112399629962154660229338047417887844273645660319788108457096189190415044523394680733456478217120437333791540618582878394447079155357928623043673629533118009903971448650067982117795543815971765427590977139336545455797993352565487375856826627, 250200116453731234735740241887769407714303745552339576687960453080010123967), (101216790447249596206887346476833008230627365288746084243095955602905974647689695973780563134511123662046074388367475215465819455194120882092391954669679674173934915204155357642887030959875997582316689901194986589490613252398153788051765186483870099518347978164832726594072535045540902112596247762694810237131, 1618664834618764494974002096536757518243778826521064159133101878999305912203), (120302172618100628577497655191915111836067088441136670398100456918839222750171226831360237252248842519264126790499928012493083202167103667765632826525166520244991855762527347162362971322455799162516305065443819955358516454398659063174927898099211659560318877502686148036107750538614410242993487235650829392673, 92164644629318236327773430473808673046982453876093641404708491936549018645), (104613938052574144064830252486786902808090319422803731558916285889568859044142845029597255043917573456327302520891629312947889797940846973436135342278583934800995458545412169463417930084986785796997623257634571648319582304744840511442228162533340103797822704538252604709465157297419430150199584590619561187387, 223744170084885424913143200558647958060040705416679190320208303544333449419), (77164040783053464295864389414751048410233121973334075975677317864001387743543466086084737296658049302287550620292265724785559566330760340842795483374787174382000825793308398396274643005370797672354790962169326306923459473673545076425213631186359862679757609593213061053007173716480466555230734392008314687001, 990822550463612406155176177615709800630560485391117253234081691282838067995), (73028404118725288500575477457004241617694858408062326059315369591274024438268992659804819596464961519690578350318033098920607876054119926206060021259301454476824930667190936205416608314399835199798852404692166137302424674459027865426910706703824178645447902740973284710088342925469368853415154844972663893327, 1172805294881096227231115397650387862171346993376789318019888196357246355221), (78582573821493374597989935554236635368214204185672386078502991959405001854875160499849770603992963220124052835535540714541834250712514251016761663222867660671878295502233657899698712219666707729691146083013713265040601387114326977827225973505128263281600184062760170244498289002458524713842182463599499149011, 405923094828127681617660066838005364104850001223411907998603323757730416497), (57986225506411329278991348886292721553865504064801641113637159119160510082189503296899106855301057144775249069811651189588961595294622102162210763385159011947259176468640563094003994673407307645026973146656092431728636575279013312976075664988293872264465587063525823650743837645941787990040388967781805878759, 1108111210223456048712219732830695557253290166137744598287884733864881161097), (137495490408654854129516071823547434331953858370096605971327185614533683543998766384379366196924334164639334241951536558763931121548060407678837513712531127507296798202050020208760341898325932395677611186360713264954236025270706857456590786305916034393228789308557584782784008676452204197318888752967567291793, 271), (101443142132888984887420723066357794123426610677484817163960960603692215979670672057546025884916726962100037579547775083273816183752050113901330023426925250377487625791992621654224779046899982590231013520595054820395669147913784233971122134002517834482824621842875512647392137818082240269298064328763702441177, 557411142291347751392467518962307658394647993438927815220837219164899491207), (102954585997776485066607810153464390914581583478348212122639410782615245402234238850825985316148016772457089853691591780908905588489173349384098465817474406664309984716250194469342844257022850800608432810209683102542656592095301723137561646792087516678024953913907111104124823669195274964308097215521804263171, 1715893387807063978733035003624916095822419366937284395650135872542830289431), (126814554911898579060736013073951418209709993623538475359635788261991887380475222821987190686065892971031137286273652038738749740135787709133972664348571541372180969728399590403258684393896616055150779811757891826297928743871471570021824809398655851843155297324038687949884461003147377397936117938298069370427, 787161108723356416787486952779047527051220040147886528338875830961875390105), (128018011392514642134490393278035709569108363002088376457372305116602595113928980510095752737988015150506208701445372233546401651524781262643164102788806716712895815615242307713553410146187923687736938129300171629568782503339093774284736726370392858468734912667095780089003156245484944750087541974471744723127, 271)]
ciphertexts =  [...]

In [9]:
broadcastRSA(k, G, ciphertexts)

141733551052310240523235440

In [10]:
#Submission values
k =  844
G = [(134753244266227559698699992245769544308029211980524633734470692232777713003460495016599085474598230838634190544959736869565603139856636471724684383650653252933276183111861379766074431778888882660012956939069413116219870213863810363999716999092978018574454095295864334646598031687707822264279295057712711467689, 1319026927070321204111377558236833768061080610960133009734153256491180430691), (54324807123156952026150682594215691225288600613490112718503769368304435586564497582656121166702218385342704009599840227313286805169055263291432346134107879682049069943724165581315424507890136619364490148791586307147231668884131111174569316772475053569329441063300640518467337908950727164238428472805174517739, 282765500314615123749136918710211661019075743758064290621278935210194741853), (90131443787496889981068298327040849608031298904163876920803476319787743211399010247900041100082244855406785540165344535428536236539585836299820315761417683017476103597825464882777287936596241408523014003709005090555959490165387481457617773343477020453608831135429636430138198981682245540179055303972557002093, 1691), (83692369910789602873654452970432981635842549070884156754611433515971198110337895495508710933262660198186440847992144248477310492499721871572495977889637039584540861784738237858431789682524072143061401120893385931033651370775081073517415930780740962055385319078397400363181194543965326626168897581585270274551, 1439), (98799601147577464223614644800957153945957056862302797355751286496018311390415134760097086103805484844556693746660931024443297438093138714834266667583958414429246229347222619785619317058586510468525733274051168208874195429744374277838038160892783548647390902510315457147091017917806265635292097328687584108681, 1095), (87603968713007696403279264388613493708024165495339284667351524247851403183534628475313152044946059419437161709296105687877976216928571127434237021995460789540612532612677069563571895133185344776676710613945251767003573586334851918294611552596865090321562733944570368243805140341410820837176410563738031409753, 1099), (81441460214813929439842931689638274370433298869652303215454378004694247675162866043181053905859939408084843186777079007393372331232759842269496614861659534763706862310516956877755280756864443480865333301416554449349541815859502885116821558607195116466827829549783863951985122830257811102615612702276505162377, 1029352693320364443027887712348263800253403393776821658295300745137770145281), (104555286900946533267892642057280983640506050837169722434314173112198565076069828342621950057365396498987061828933572802138273284280544145772023539001001262013672841755846902796231318835609751945361580104073585933174930224173353530820220636308770480249061682019577875494943038850816401923525710360216615301923, 1809074677010261088987314393550096008539351119670802807253962658242888504417), (129371374574217349594493680224356010225697137609900996998898841079334207389948492494992751591256347864535256957236217629523290680299994435648319628183909601605209350879023296685247336970103091709753947758728246540596845483005163023280532066745554039038264042646078524210203103906368160937777141469463799179247, 1310388967919333065741753202748710331708958242591813347761919687332658836157), (104324458796564097438563522272034244077930600538637898122475460567568751827887838875602677780724811835985113372247860399509158211919726635585845589088705348700590362385922969126204872761151910160293184550764032402188383507943019075791925675936589467060454144973981939518869220967918179834105113537232729132917, 1871), (117388735877918171444395742794853689884411363563901766362691359342162755938845516465695061791076777606771162592450139063373396305202355101802566712717077152148985499124232771727346852480552977117706510509981324337617046192572941965641453639110703002980262463338607876823915341260908375334947597170103477291011, 1055), (120131038257917935923676794731847944303995856736820001634360679043846077356437200703285732085736703892392385710993344796949536030158685662733682976756830092147219509171709230438553477683774670072362402085574813993083997431557298074478505648089159073365959633456522979596659082064089498131207560956744718651101, 1664166734631625500723218337075458259881639908611756119330827238627487406319), (73541374093013641794711775327813016354281258790953618144248562797052733016264626725383450027271050855158285090469056972259744694629791863944367287081471472855544117364375617630803616780092068215083590310487454133957397840824342873314013430924404709260454224512723113280495987451142258019427519513006487719409, 1115), (89171852530994152563699231067611194670552646709914303617919072575857165787325630235223099079635497208570930132510178265358417429327717876673463343477604934871578558739352619417539910401111558735945831678578282582780548509767635529919631236552865794220515834481759903455921971504380201969345920783939952696047, 1563), (136649548131436559530542869299958967947104816597139639029274194157805760900346721497685686620972961253351207629770289088487972996860640248844557858665094168322292911941670371319161979382844847447772476352433780138395159126372580365990910107189167799467375629471095116210538716740123964201591988588874602741209, 865844689962811556482773738015350153319373815651374830219140799496309191705), (93278287478310613607916985294727853865387794273112730855732922558706047940348439929652372563533632186345826552203502781145060909623711288666957181260109429031130630337168350084327832574628110683310322905583512721115162178367887627789837228160217518797960216425721694250212076631130858112987745417005902877311, 228142148320932469692341172569127049276790819660798516510779432540182763883), (119888873778097975274880038154865353013013907548998090036268467372055865415002078724520704442426580099430190148539353473832760281962114026949792430753624125375987482837217587762278438257814336334333112778353128996412188169406334850727122203665857164540625577137061552577452714381785601278991143812336253641423, 1735), (79427577344716115969348313063882319030560232085059747429824794401482475441782470038686833291731111931488750895426788374138278858980654696397928741514789659449052037376099971311095902645695341715129460319762594570641595697161142378545195353673279556786909815963014205063046913814226787957308476670554259849827, 1435768543258383218235485974431217138160182545804304012904699876674462787257), (77746622879258033507293821472406266009861369848147444395247975356242823018350691066739461931870158764308164371997755307538515400549643252795305769260229812332252599150532207354793409501660220851722794927953171218010648406454326286757218923173939712429819387038091922260116486354934828848805500973995578001357, 1469), (85393872072798091796954627177985688545392697301153239736628990359541129916311620889905725405037965762198389874238381364100582496926687383024517664686039352307935303067886781214116611499793556805760270907396845524612816873741338405104917968091733487248287927400441003327180280785915098662320370088486890974093, 1652390208795100737371735290637670124993416959454093834647748881517421308979), (118287762330484077270077354593684556026070913486625309681302521777588115232665286445866304608179309465436248472961038874979999198965928804988839342854115270110185534986759343420120712103323515459507701812050303011448565107004675967156404658639124393338849449382699399701880524848048391370414220277348551454347, 73816780370426488664855064694063346793280243401791920725167222616630877205), (104140461035199847030902359303475528476219335993090264588527898005756896433222241054309776075272165210441925013033881763978658839060441714020228413763030623217116295048786513694395394953768349311875742141958865569028912411429087253785911331837108805636659583489483315199432828316451826082778288918774173465797, 1715), (118392388561226341544412426989723192805453467828609297610393524295488975346306282455889761931654836504677205756254343344884240348223923234235604146239256912225032449931407579472726822503842442364019485425617575888302744119140926087363044754100398573423413852481620521676344029245895185426292044442612500596667, 1375), (114668691070038265709100735365200240772206411839042265905386158314934583539130552842734991265522754657956251536451991355985617141487549554264119947663656207336999026476629264547422453275035168024890365093823617176091300322597284860272576064314843817851532485155238698361385481315873837650717707579034946392553, 258591430364038578240540537894809615252105901739125805448255739561598461077), (90826740105861709348033850255581977132230956804960088005340377020954193536684765581799556406189246714780125305586278646562586273871696117155365875007317243569698343740499522066760669981864818962892837309214408076330195451452225762469902462566005118646878667899455422699047727201801414394084616919577262265413, 238612403613890694417782251801968546694719849903546524134079950364453767259), (94301710127552900085360321415100710108828851283176193366991721682756667374329256535753453700173889177652011390924083969888777325954789042683997561015595742283747336825941392445204744806964427802593123145218361755162778682917700751235445222934464934104736037714216642521250879019983851162826579122235421161927, 1501), (74976429522489061341469885345655817243096758647451743988693220561376203431394305468445370684308260259067151071652977614940041949703767277156136104386114433387817349986198956803565897347598711837318629972511499953350460951785084692543498604545515090405239558294193951266114394531187330167949508635926088739517, 135869098911147440779783236901494450574401100215049615762922544905173503007), (71482295178285693134199959557163121725334964975280674992527996067467205272384490484545585779765327408728584739833274544780377407866908227747803295415201113938388008354562858207027847667098912435837496024693766728609911137933179279673066270562208209747607196561733084254264236218593617786549553205021302944557, 552161997760248171925413287329711903784950670994346789001946845270859960853), (90749859361069271894099517739666517028961763993554080767759467390508012581140957693318040518773606857094511707184007806538875242265729140911337513556343741985195289802546057813230896709210927118369919737941010107802007644203695475988621852396537227507784367941683629445264631613824953031752592152154518164381, 695675784604544695100958218465255980026395024990548085186459846875412840851), (71671813482173551916824851800793050040195509674607232027642034984153928716967691416798010414589770510209634799578590598637788101271315158068632932417504929067247530562883339390054415872859220608775597754203465274576344374419527556189860398597632980849824438292811940650986716728429685466984043771749847018679, 1073749927118130195393126704603011915344306242592665370111335754779544572393), (104293328446301523162383067663852484604250003689170690564972842228849778088757038858626431690818422274202990700202243443316173283148273245966145521496405124871634103052816570713498947010928090627047576204454861511216167214771812075707311031316975424545179731868059370061744552383014945770536654490450664987389, 907676988938744768094580983250756491543244582739426793250892362637895407209), (111123221378570089728013154844586277370079682435328860365631645920627373605298213153399609024281420565700651265216149488496371229455219303424010218606085260685407791010869158196984250964174112465496229494333968526008678646218540937203987966351123916175673933571957308358403914681719309624380472129695691616107, 498320184515762422419290825128316291414101784062883582943795084077191965179), (60119291285052766297432504570420908389146399935865985102876371460130513100512665521753083655708741287861044057176124740886248721470246311588563646341463442307688878685460031916328638105194116992132398035016261191475460449822830486825233723981388782445960516503398707030943259458075972968174622769259788328069, 1643), (89708974667565758333862632179791550042968270564438368133336510072917942268067720819699060916018525323284451388762691138378897511393124611903446411077470351293330598767890516256432137915560373218289080389335010684133058758509513939135348903273297063505557802298434111803818718914499967190822416761662626405549, 1416456655915373865624178585473258797923634878556974884797736883474573469423), (95217154879925072468337971231386507022702627165427909505009705055221109997282375675170040926158085488043084302187735334760430633460921110832487772063820345799938668056644057962768577571864698697923671405514890863846723230368341876211118696436634236877636081971422216370903268128410993922313855252675321196653, 459862134179070991781471468001864230199210216424807503781024456700990539741), (143137810343707458407749321495128516427975565195662366691709370113019084208235994455991571546634892086294702811232348181180877185222879699007877038237294127042054767042807921312330312686961609421563175713969912385063929911405218098817977485132611941080043304174779823487589988550384865099041357838443155730631, 348653903591990800687693618640159160404822720282344640516263801974418639), (100659191965035652710550422489821113412975285289570331279902414142790633113152330279999998489881056602125316084588580192349544770592647463152450116611275654558887305260817736986394679739768705958098027071384131102148386310624883601520450530221921434068711342002427631315437781914986098150704022819906925596103, 1529), (92946217571471739969395970314148307861044941485064507364156447850393832667622915975420938453660190426796373761318724075506050845912229144697957928101554072792369239055373595654095759308559254617618689698181413502086411209799362189843348007801942446020461043529706942947166632980572272460436237373834598570061, 621622804747167933456334115511491116263812771964923518211624230230077746517), (106636819005199929679687430521504911303620227766191946264393840489971751668451321543615923365513391570240201414920207274391294928846729057623915552942065724304259508104921024805411853584630005478603348979991968888187677179255024460203900237234676745110040208889722862878730816011811893876919519154031153081837, 1427), (62437899232437461479868881756880581811925078051055806198854191189780460922500080431093378579360191795174510879257277331607967665747530478171136837223179540165960400962781703970292652416801705946628101147532482008687511515198931502921095935949415813560327310699361836924885643164092126325277068132529250749451, 1415437941975955509407562861640437010205072835520007800067844678558509761961), (59660360011422471415650596646086439838704250717867024258877684255811447409112381272613663080350168082494074377871508223706746866588143655463258674412868718669477971526394375815191664950937734684718190708862323545582700783947468210384665789447988725518383935796127697111653154830639559926761577359434073182921, 1275093295349430649148791565861870120013508854061721356521266508941579183249), (87698727003445254007803587262843991891320104797152250485579746712607469198626910928277018407103369870513492076118885757817195048979758694157575201663192544758824011886083014359085314408395909693770536965675226844752616573595608786826822973738124320090089912186685200019777528536780726389694384557233349116231, 693279575944326217762553532194916413299140135350995033350300865817549355227), (65632280808505348072670478963475659582325948323820630707449349637089759266332748013982437526192565917236472000119713264481104781681843065529493451552113250602035250038590360352930630633356570761636078462658694331639048185137844819026162260558533035988367940558982912649932252676640527491101191783326307857299, 1514674472590284833896272083934483849439972682378752527371294312341809623917), (94804496554167616316291855835996154446770562288127456030506199358514485475464960753357398078619524983080150204322195393273885880586384071144613854520678571212138746014823999841589384456925410725153166763968822433152361563572287559834949987518077706540602928470539515076579540331479030193544195148154164466153, 1568176285475465715197011030661731549340811565191242264647522442002033820085), (96539508178459920124846001554319208473759747984850977579994269878962925985445737199086795611749318588266879566477908132405912347167484471192150553129181432806747306377192808775605796003265030069135393549619336153187922318788365136285636914030529969960338854244104436611564141598610195865305266592907061677113, 1645666040233854715597265017642581430465894375042708589046257413456650951055), (95533104916188219205155811972395764927972966466489564605144844922264690340802720837542970303839716385339963302787383341346754265611512813274200192379847258480667590417062739290598263615748579902170687387548512354155712920032477512671827748428778009461504776308153283528835304297748880365798217405551637070067, 446446551589964359496379734082230700261969693941473807670513781172278125683), (130181719372185430652867940284556643827178952830223671493001348884298747908947481152649932731236161484985334091639803837715542298021846987906993335182351085624263895048452097954730511790801968260561520884453619007332942622640206622308390923434558472856048746951852528402104830324229997994784905583409564855551, 1033), (60059488425000292272709049378086233088577358990010503651964189160628292288947761204148860814240945916092134850930342902129651701469675575392310319166936047943982443503306959655721096341884274788214283626659252590152502797530931578721058451879146297360434018866512251794695968663565131271188578608755072286293, 382631880654170675699730475715462866603318875786896534010860578180456525733), (96858241018428010839350625059960903290636641075226622595492018329629057944438446885360163850407045321534422073837882611895461751868428277194542751351041147736565293639033850161978511926312098099837235326085672695696669025929707518721696554130683882312454523835061272489774574570602693498420411286484721016991, 1289), (98163760184668078220402879378307802002088035043034399508888318327764806186441138227496537489341094708246364632533838206335304550676935436394308981783929431797928054135704801585613977515442675791759337696185957730977524246392191210524868904052016335574288435219132993962012817220261201955984705895423870255351, 1823), (77606019750320033136328550565250792664762745765624263244265524325634426763439832492898406515441484883459592670486520393630354967318449222663254491694979007104436062410754764389623521821803783202527387001852083982390141514962674194487712020263664838199844352632749175156798568865963890285799694198566050190891, 59), (119553162440060292342456750353915647451509413651209845139053278014355336467678686310807363660700465569105417432201071958772924407836453873974527120299090438519520959156058091602965990640688598571987997545099208448716630430457167708214914143276837168286161276294013932412250648560389737919910911213142658537347, 1159), (85961682299165665954914154968910618360905702198188175390969669874651810249152947183538705231876561793183037386552907968760324998344145581484987488413431328146433178465866669306065919171300141970528437209182417001641036963734904468389167150768640514949516442206082583460129905975027867098907114787015907187127, 1361), (93088414197551129627299348305133898273284203457698207276702891423233470144111491037752302822034801683004646345561474070012611313359993787432078152403888534610316452655438577470698960025150751836399955954112762705837538009193673050864376538691828516930657327442710142542549152516941257958237497560271543340091, 411665221484240675374736916704351410519148950204222534898337813628161320879), (166462587335435158700044883689063966063075305753332243603081183281508914004678434349775927243368832394813370365966340957613765474171036741806914198667875802679162045469518300658824361278392225865363138854594243968867846714586153328828870114090695543742797357652950567235850880021901610757893415617772945950339, 1414446384901779545739590411920124733604081919926588155472332275227725957773), (137272323608555906053862379689704333754978845257292135876430668042158525007131316644056959283865726142371828976984528310341719922336836563370928511923366585879357375038152560149737961326912281413903112174961813270300040538601405588888653944464945901430423653852069098410819018957925520847632254974869123332351, 1633), (137901571194142300796946857294758881118345310785741253003797194178706071812716610211903088754861343122629191819070031264947082465299254793966090120567532395407296758585267745050393705271963363445972258108276156493571012227812071729959119765425189905461751019907477507360476005805362130403464575437019512272671, 1901), (74747298555792042167270877048501559422485427582104268413718578734424217661557013114386331919367085974972021417947351544932168317263442981630326624269635612463736234770872708820013587051214916672000477645424404882025886260679745689328570148076043800035674514110398086817988682995226387532761832499731680949383, 1280203315954446570041895268466815200316555325461205189332636656642542050783), (128046775807410104044345094149788997074781882040875383927700893149033366471185075837195158984891546267507925077003187794507108590177270189273001854439591094499262271881689030494787362296339653448641614059791893476082233887948180596407561378096456583532338506209057920157792341314947881406960017312781299542629, 739210153154005496502493911424411653524311971911781731751241926271860628933), (142720020228685516392504020041617856202985929127005192868832592972031045240923036665806208327138615734888566127454261362556683927273126485117191549105221345997774989343225811686133752422539943675032715860910576541048650713877442735543483104102919957501492903380887727519401998996472967338101674550363204206017, 104751918929367834866271011377166123824249513317949382122143131246626683827), (70029290011172859071806774088423605517456281510330450637979687035038821884931213263888600599646068792255996699980064353510931677788772375212846291055307437578523601649728303597605935447928303256174443717157789022792316937913382692769364087163933253996365030461798051877314514248865840752026071983227268393619, 1299153162016800335488081082192000865094746724140444366269010684015297062449), (122334957508716615221468447550434993634728607200157691929809972592572377226991130529961710360214170234344494230867869429499195722917311483693982393228906111694668977655589961969268573070516756627658781425948612138101646100887268636194770341204731239652903073780887923920902707580505528111643951583551329952393, 1933), (116373276364320637423769611262271393376603252617877452462499000490564687718292183094433044414316520659353569655914825826588241697439958549914573443957886227950794046285505707034405050560159961297801717188541340902222679581306045545872457559314161955453449824621459531067075984309466374382608963891838933707811, 59), (59849967409829169483835131963991357302495239504099691389466028250319860403590702273025066165277914874564538664953876240397490545449963651887285677297527914527407230933995190254215547784892374424441710362681742865502973948395259755842377799287617344985846100700994898353022660750833541834033630223954986966091, 476710331080460994192532022300543594236721003408423730488148553032067723111), (100494276050650011313070273970621755669217518372727816812838832184922455105941783362101480515528518394833484457596477583574143618384978084742708181681872037347289430311182714143044395545210157372945391503907934847485729212260880741534838864898813645215858652343693614342785312361088492262701877454822660355123, 142784684648270614181363521061488030923761453991050372030237319057061602203), (135899618032162362351534009301202782509992403036858979418306105201442274634764908945126290023213261858388016400204233701864131551965107981076736831961439072284426599070538170759306336638608123540821073901604514378658609021230773382750238212239844168127815139410416351101462386400810219794884940571854180402561, 2029), (114157072194820232907314433813304758524040662369240116267314255595692081787961314265644659827829526464278713101863032412735426894674155784240160786800430576112711092197393009236780873130901008615086587082285008573141278613346913946484878224411262602578180931186956446725084921507774035800752150566135052133269, 508513484490444194385849664040266415800812157942728069057985522530488577229), (101047956645303941172874892759027660940341194649471714849267467908475446920268265668813817824646839141367010943049789980061802249752411145266969029899442926036445996894477880213813900214713095480857746780671910291434965763922152661836183693828623795580178954976941915696737204793410023266348535652271012417757, 1275147321959711331847623064171185585084966761869195094122716377269175274589), (118987884971458913332094239053021421706340184056107715745397211534890017341199786694012735867839345781061974413867341791280220038154182605244385972053582433037446850187787167667800348020871601126754164493383943320313324052022330529838931958896432721633472613327222063739508064941351661578411837835202088596893, 59), (60175831722751699493439697560645464872711614756905604640545818805233397409366736251712186450635421428227350132768837294285041289992966853656342160888335907985380183632163461683108856700511372217475024907984043570083126263189690399931260076860857834371966321994617151573094376552814265932366406366189252187091, 1481), (67920173875609124077995979374949482003722846069315258880465052473017617752651143343360589419560196818416361036451897720413512590948858791567608770805150627839732052479903834342568215881797695421137210071952591475154392751763783004310015030403649312327360968695161272914761979976064529614186821952580541239597, 232655338844070026542355838339642725992577787197696515307462664281692392437), (102244536212375563313059023418438126805715681508053828982164099666590514571023669182171992779381381641298426558767466185999615090433358652120404784193091542870312502970080819794292657594454196707005964029827095344203614670994534875638787728824293336542657718404047639593726049552773596132283805728340487101339, 1509), (121361984996171655993836590893390822865398143201157950016468442869255505242053350589932617905072446393888357539887839197046707252462119050468183733703533914298837698475115863180458722103514008152528660181644143204367296191982082149088265179058321718333445932285177944422809948007260527929999511970790694166777, 1733), (103375482474766753516231335058188383043312060707077880846719227440575080210947470424236893227462545569798377858683389496397543491620778814753065713832045282107594673350886877707504141349775612448341937533413953031854386373644807722650616426360326049759118121310065163039355172146379276344107123444191505713727, 820174630767888424438763428352790321636995651656596744680218889573417056169), (100625754938615279670156152898832038983272938724729072028034140707629971517571699724187920620511767212240147706855458424059805520945036371894034517892955029330187976644820630545836977675609215538505004566501385673099051042797080068690626625496735383800330887805916917651482542740249449760932689688721287228781, 1094371969332191414130390913980814037726741956060632201105974802016024666861), (108938498726051501523668545149575166717748558921076020753328385921787411806414603057650427424410399224278302321665532716905520953404115769763023902484299902824104203017822221947111978752558854716123204418527204873071020134177551525014743515527822322450879203044116179259462169210739620528012563100877888072693, 1661), (143281696877360985426287579407809430126072068140015129363728137935512285956643691718342661043386812281465934565650123305881822006407138104087783837018933817789664401762934321574149954871182536378678362241731108676933711994110814601752983926348936510631010507888748238384677689116409774971194056048119138672431, 220487047634182805829286522223019999240370994476273593298973436898668040001), (79661869735953201273157884474666616465253272733808021497027071931114667127750044151570474276825167768090320235612051744594646470937871326776525128041555326724497588896303884902420227335447273873284089296804078224545455550381048607613841046595779937897701114185212924226640044602165143132344558155071549991057, 1473260361268765751889522467319968538721537564186532194952777974291881560077), (88850114840396667313316379912178957865253649785467880294917824464795871734082067620566695286798510277780165471910462885168707239033435241685710616507713807695713477967953971886525095348500712135118658493692564408018480948280207053962290090795955473111148157723088917270068829974768647905930221712829087690619, 444406841112402015979229443991413956601647559395164090443682497171459475371), (87245997033605393905361112865867459415560307263770811317710720446109583440420830395884492540150480510128784026546635375930044027420073011510059058142389132441722367542682060518555561348707965596068207012847164370488060945015519197791356686183331463799455681903686467171726064551332071183763255699475106572311, 1445), (63739711808970186119796843601034798935071163528271352494946522578106335326863820099990406762282206122870959755214894787223670952528090731521871921687407964397603302430059150297906121646118024366880730917652728560533736749343135256793769921068890732707811245623805837392585207912874868625878450797615601931147, 1159), (124058896362214736839413314319598832213863783947066208332517970685955082545389004697757191775499978988409779610331515944544640675310265630689696167638573634767991005272565817495773750620257369834157485609425632211032005020084378919487734602200589721062003983440026635067788619962262141457818783310520151400927, 395587323254358563927708648867912626358128403308225469285128104403107798713), (96630369776936052871717043418835271446107659417678445288864764869570137241199605008230576359369783199335376622714543316693096240810587105272329316518721517234649751019848942797212531400081986223514392338557473217815057493494292968487177537455547861604938338305560955860057034419543830954968969297061898577241, 59), (97138500076560678323668603783393091702197126925858578377486304547496887047593032501092809467299720323267304661871662142930638024571464098183276914476140924884272667890053589990456462132281757381811422937789145312847497346618724614626970748191873294899951156045031425865337016043823176244895041250353861129961, 1261758419887444123105951100852089638063014293815235786492989979711019145261), (54929922698315590719722829899489889724353431287038914684753382365540074468798251387388849219869821698508940274923808862641598168704775316633269897284891369020945169254065677337540582598816459648262691120102438447372212399368324702134836357165829363160451523638004189735504551157908193680943565735554272758541, 993969051441061741271552869582892376143975355675378300883081199082833081425), (62034565418673397444192181330869890148472844776975699892733510308934052252141146718569218124707802371004827042923850643536292832875240731186470596935072503099867099617648957707721606510545342466248676300131568288444052973412107557767195424981195215986372149189572074953075336937907760238058953038954166033809, 857823495724907248621543273203098362473783091271308745374196815159868074963), (112742071508827270937153856815599993923209021972550048321710589486355105074394734745790651940139755810111037872528827118218933660567585265686097672903905087644277252750377334658688872104405976754210187400245633697048080570325572057457239001217634089837137042002014768541477431900019096996127516872911362562861, 1045), (70341657155470318732657207998103238190498087835161885942764915725785431189564342410422790890300066713638348109518829459531003118789036425535212587086945863105802548166152274879910294617474881774951328857845910797644283542001216620418918002080675806583464244967792195483205202314665247023644717990966196879529, 1535698921116423845221660525827053432725334754037889963846934846231127952827), (72017428762608904720961318527074818216977204724497952196169847082715034936661888809414365947471641154110756789126564582122901540476616267369606874969504178933705617337455889588414727925349640403100355844749848558932366868158146755280196417872248043853278635696969165662290276729666034419656652526640601429033, 459111068650893764245524599035027221502940526725497662482183643018936559837), (90928473299610125954866932870292870947770380597908344959785489102973714910841700288450679757805445195190396424368736518054593374368445450714865484186133121332244256959185917562021422141757545213671216301337960183771998195730656820369685431460506261141516087666300037715129175826056390207751537279522831397941, 59), (61083055412575654494530421218636893279957929853594049437126283178034180377534000064451485988245784858200374479910219398137664629615344391069571768495497231763827226057074102447453368654032621172910706162751291793939818762858182191531080374055729400240926285562783771551558329409565065643518430252544047404247, 1081736915311262454071748148746153526827870627305365775238977892919725587611), (85245783753541326395299678130610989279596236967132727236630331196874652606382882662340581719782783219385051213001452587133436695975199252949628848090257013260862340785792703060241939652307712625362034880469590785014270194315631619585428733387308569683298281179736778021478680990945108581221568641384645088169, 729814941852974215217492537401568307687021422407422573740442990301587675499), (112338207471026608443532998270975735813117243797983979837133731954190777160951703026629674268927572304568313657841688918994121760511643722574763473145529800435919309099810087915786573645465253957601863134276268729893083573329784866916357653466276996478564923966419162431278041309496213537231694549132115894253, 59), (69797835308599582161507978183062785495428796349257330200858904829815900459875055579525506359012989424381768972651053005338843645678543772807272857446876256849596991514168422124226799939669118004014387169842267792275403065674927667007987555562749427020326888126095355566962443537897052027839786396132419550097, 59), (84812156404059433961436752521897621647475289989758532901853223281505586035444964292346321929042394829148446626260278201039417114605262289083300347344045944280645703735523756403938925467243505342341471020589411333488907391341661255986725113085774974440481090438054917127817904345745092982519487275371604581581, 544066401014307170819020053585204600636306532482897797439483752893180527841), (88221587744210552552819473660012980582910020444801451097525433589730136907015088491456953225004900592625351195887493027886514348550674641047366459537056617042254974763192223693579671446111989573512577724756916115443991234416482282504420663390656122468749130104204430144970082661474796709425864046543343278629, 59), (90261186229886749011112692500407591781641332414507644459654678918744216905323419115052745404634822681415898938753846393409300253724846299457211434851634209340881166789893770855878548606132221874925598561159522702232664514013454422809118365579243489649023210336795904529083566003650372501103524191884037525041, 1999), (83927425959671188081036901464393163182101267433335184042350965939251402806636754744501405209465907879797183045332006526050677222804306644305523321658614209506430498834438091144134071380991818865992756743861735170001280899193780325969012258097274034902874671939540015761053089062133093793334913651175632771119, 239172373393275074215080445724933770832847885536866924012562835504874272217), (66244270806037763372730377613943797054424061044598955254100718342403906283096394589985264531740598647567740425467957154726026624449766059391125092557111055006833400456197677784090375907379500405185523514867024361563414207210410302807782788947039702286293679787549847993590169579137931954556399808362025894993, 1573), (64536916531562700240483091489581836574045684303415939701978092867433931230987091623487992962232801442932389406491843712710085555918401256968191532392730358948055399708614062270440443621747614998492107002898862899543356496341546865074823846802000407834833208144141251960134352790787047123179546055155584918613, 456963164329614786848856374379527809011817192962450673039202467296535824901), (65735043213983849725408594531509806386666744830931466799859620235965273354373425246707212455765490095376946289086730119250752886819543267170714342391561289340141458961388120706966523509315168428988622872834367241987340454548059770844873577408493722192564925099517518759444151715369510132844618689714246378429, 466952575538074131019220955434879486253965733011112620834254738556358741261), (88789037078242289295693363794924377126455763056944713874984815401666058888303620215555095227501527428261509156857221351524738023514098784594632475989497843610002944667981825446730574276245837803802983106017222540344656345673297879118692029465923690315955120345389498164865617147038611653615137873519725739213, 1649), (106657063185514209507289862400737791077978863299440382723502699671132774784927234422480051781244314905143962466367583822858764319010479848396626604709247666278239553759234569997109295782723929460971587789605042976625695146385951814334696718435076132785091177252500180639080926087393231566398241134370136778787, 990018712491660253131236750715263513545265735457021149321913763070999495593), (108617910019413378507289132912326560527482989060663054854703934272041671057579267076667042110789316215334293026100503901453566962366076892497919796237567737745933914853950334988947564724098848807425909155738543329742726175092050417708242919409886510265802128681255845372020245884398280589784553051118254412669, 967443897834810576202508540716157598080498691482445937274484726852528855557), (96662502224274127893563167483795566476553576459292385071345055141674366934189931540959466820545229009339635678709642577313924887256515354173693319607990518209584138274560605216223457326598928114719508610425487786328579699458241619425453557396672145301947025684945253979059039478029538991932242243819469476209, 59), (80537175769103330915187002027758891853904335978743885098145255042308824609071738645326724241852410999474446682777365789506504802319129289492498793324749143580711527681899997763049485243887667008275200880144931659235424228744377185289697839844798225600012556183147952236641895172838182019800600042872335499851, 1202031076652086210076959509097856907337757956306527883340656331571289061723), (64485366413663303191948538843268184274774189450427315304074211185909630817488428693385632662726019111089866511206207956592037186220344023074484030784460780843563816002582577654603181397607564957409410709545159616260963775904259804579886796150911568024669886108306192583412864260160031763848965520272176779817, 1823), (66932949212504210244921862746623743837009168024512597362833053004516327868886558009362610140499896502468497741758601995959983234561325904500037273088142078921290808780354520411517724647536549660197355720684086400211786653626999136972834308629458444029536152139282464909386437665664527097880837094611038768671, 1787), (170589531524240629942783914728894991946734602250328402234309074087528919278403780676754617752679878348696993598806929020892534976628583844228279220786212879150092222205363003104375066123064462021320988457489697473362382256502425818454014789105299552267883104706855337065009152839959349142692472810204834172071, 1789438347687432065742607124425210280409129809569247097283379382750770770835), (161014370945273269095279188771527597973735856904038763857550439526723185787856769313488957448836297209311552309788514819588899567254279993015544176004715075594508033018238328836696955434951497316606892588314711875109875945078976588277034587198218806467655515401256314705099258717346445618757382119254993526561, 1100390453436682998263747080565773116047820839940377313588321971582909522269), (125664926261751338084995761159696183771239415276234678531243503889848111097692331552770289316296509839134329964872978117854967007360243569762608277459037190531211115302060072380326770287333146193572336587353289596860136515107941409986426525472615693893580550146266970565676973846081433141625854778154468843623, 1367), (119856535885694942922504589617869356163203575244203787332205637129360573667127652352472275971192194613000068827410134983925635292196549751939541952471418948517568186485194395847289684679097349391127938567591017227696415041274385779944847068877708901448718810927582792316534466725002020829911037712848275258571, 1345), (74743838374255539237392972202255624597556510507196904910171737018748043765817770304321286890732675943850811308992489826624119845583340325311405695808954497229006119077477247683334482574617423169431333922969044402779977369036157179512252942222039328252805894537070138437689107429507234291868847300029261382093, 37909583873438442777666414706806996257829899522585049204599711362287798199), (165534417591911159890084529992308299803186867152903324645850682640927403019422995421062474752513973308502130890696628083946830131924711035460866451903059797925114959825222949839157147807190965966095576194510675831941732620979820021855986124893035984308452445876785082504666473272181873582223052357005727749939, 1283656641888282375780006649260368379230598459055189451768347188165899942707), (103237965442703862187747001254497119652967171127083807375214839271144174949988539331282082938883122335320415027561018686196472737149075500430772875648660151858292682639837050187020134411014385742902637357988858088988077815130456576166889130229389218430086785358483078014707279726132535607095531719631505013719, 2187148796897202320126540714691948493644494939836421253696531516958305311), (54510312828080340756272580877816608780499725396803649296818122320869903229194064271605320441946986110950235597202833515568642293721559129886127831386940811265537015740984375861691483987547071674369444135476694975512392350460580316127074036469835328029032916623204994958472386759269883249246925201721728160809, 396984161352811440018199673774844112445780339145242123157283076796318511429), (144545927768131941147739202550932428933085035889140056317980225033354318769373836244146197648302658448737015276492190639629906902659833605136750049625824523510873393076485259507697137461627321347596234377602245708304178924412376167879747682821026124451594514887189343203519640468090831193273972655712393634549, 741346292494063954874473233338653939215118082188452316604326222453058429071), (57981810979372587987116993016747368884689755981619166630223176740560639645865504668817311922742926488352437739924523676232743624345288849270901623681365501033025492246316290390230397173545476867048162361505791623912840727980761195052643151429945507340341058843725680138198178263876049266193079257788292491649, 1445), (67413166191808011857185684629480951416456698780564603618658565976634673744479416073108662253350047763426720458092815563835676664227222718101121014767345245846030712870269207650975625648557426428234957305328603319031726885808829817369131926392565128144785431731787960811782350841590702822193827007353607574927, 1403668193541044081583156179551127461148817056140987321039804308867557197499), (115947434056519439155317861467430678758037906690514705558228482690724300189383094489661990271989642870316065967468366484866567783161232093295693719600100012281845147432672727821800542051230367113699992101330730520750751144424496232816179011886715007613938067285925432074508385186257409428982712697078945135029, 1441683146514358578168898422181998577860609056745666243031588779560863217303), (66660589891594793506472351103297744347498902029321936658901118928479513691425917834240317952991499725951239300447931675486690985595963190873450529072178434808334415870199717107746883983522325579388633614031037495106991731711949125599488265355767352591433155969992802382311284313298569785681748143547748883251, 1132138486281369010057392014771181171305299038966600783560226033502352715859), (108028383769833892741613512514631649211294525429265755139645902315034379923957678040293201023695893860166592624299767126826503698258513013872028702215775928365282705277052126690944643463012757741990647371880386833468632440744608699115986618790106001149800227495971207869550921635311250693675424996623299089399, 1483), (97888840237250386225365677035517956817412441984964395541495337066248998743961807763567750045065097477032465512550764238494038172171833111074834295181074448253851620367788774964674246719718700095021445454097850382644428238652512662428879590789494350798996418020495887989425532941398787626372699622558010844717, 1589), (69574483953772432252176094515172658003682109348591357671442013395612825425164813358753908043383069405301592975824737306697705209301479519244557260335934765482346656654644165300712854379541250008775901826192215244627791838873268661466607033117771705994223400183941024708539694955650814444082781016420371297441, 1541), (74211767163194144363025063359468159221318906339895598166287608871821224606581813493093587118655967319395601155200587753183208840399542355890358598085028387858246843084005558874255297536068971463151890624242500695179893555293364431336254876262549678158500545591260600654126283095233771167003362931264619943987, 1113565120998016539158334091747828921778283071227485946177372202765816286349), (116852008654190858301353127509360596228545675658880052753461490099460214750765107922457728957902010075735749467443902725714033922247963959169557038660244198395730595036411405057191767793285731022517575206792863121693162216570535091262314619056725711074776944628891368933670772054248026815318592938694115093411, 1663), (100644300183243288539420388287467326302219897724750551517832551061089319590423233769310180641806056658649445778091366648594859395214926451024194863691821188219466398268753292841612333955830252549484871952039402916171863371190521239974351649186050795358269013004989015468878612254736634833269010954892024417213, 1969), (118372280709424441241290606330201388031494229722859182811470784839930096810963574734563363038813420052244474181436480504449975921401355121281858587230841777100705926945448524102122203321595279021179061028211010329504494440689842002146830868593186195661187188650080460446102506593995764168783616903752990481171, 1239), (105635667914593057862494104224561884134548417408286433368630049602992466355762794105597658097919167408209819158923815924562619168453508207010192886558390157768263487944132991368041781281246004842694651648263759241825098594215920296720620214278422767313393783492199579998358854948630905571189321487983887646197, 639235982056819255692021601774257847690504285020158696730415090922681950639), (70328545299581249443941543430369686212120565122265027831043580693986236375748556361195063094316344746074452860748857498378494507849479190515061137284164876112358294461617404649034406498507952820979310209329520177583069037352719097048537107451095200309621854779912724032402441636810470901983945397179912417407, 1907), (156532726156328291992991618035044155622402155188700334025492776310128443975816365706581344609086034844484276596988234483827975283676848989746758008030494132530758764155941015457568954771694565369436960709502438071139240180434421022149462273491863340574307938401350853516797811895164961083937302238360507992699, 254147222177559382272767882757146339679270150996086558178534388291987178323), (63733453276555167005357049450687244294594116185007854965676513991493750446249719609926921752247535588957483989583878647778134826555277458766566696189572892045049847402164619146710208660478885406227397087500180840517523064387132283146639467423110935651320665420643484922413933807336768017002429195407958493731, 1739), (133081070601456355577512804336626809109661248702967645742589927666862094679397192222759785716654081779222210754916460866283646915441662403918900606506056965629568370332156663010681294378331143436221853877884939527972516558810001847284528546045223992085930597385034944234750418871326593343205900479816553264359, 1503), (54862857616455809988961621426465059703345475474078224151759709155940820608453386678007103102690572730955125052832855974840428627475440829583981619653427702505732721968607586545359520211663645120922300590128953078574048374927807060993361484435859729342554567002662432170719095060027282052756983033163925390733, 1312899201445004971388111791804693218484124067240497563124344974350633062829), (112099312588392765443758988918880546894699073244637827694096032703857339753517613794004900964801543919435614921716517201913592196529005056235916247580459999534547671744388597096375062178161660900451581064077559726225291643834723792127381235558386640116887423483306335707833451799905337330888271564513137622421, 59), (62596088486078178252037340992337673492227188374608106598947056419559007487352369051841264727259325814125443459968216526353242788350345767709788233872465377404802594084816615819403340069391022495592869768362576319862637386923989813906025267225468340061470405380990595376164474563723229520326915807969693672813, 59), (98971810712282687354322284842454203001221921525458506934925401244781683494751936006371294302621771000791550212811754332984730426620987494712527309102152842106574917151809525358825678402864482749395300443913811713460819074689966701506556952929850647492699264581017012451014216741556096955896104559204404593993, 1009923710182782307226595520528258528278364002186569220148263658296260895143), (60188997296461572745018049094751428095205827435494410373119876663770895277751495870559305102982829456134317358465025418864819842567261576195698490823059605948102426231126391319310089159599799583364198503064121029548228461281032010936474701562212029032095218592107835728036499356166357438090955229355563445011, 1503087356029873928892464298096306207156296967165460331732277307808159004397), (68398261784108979934424530859698312625624227373416113287172476238340144546082328480903826506009251027039543591613285128718332742149197038241758328104984948389574577764161671324669384100546191625655069459607689741835787644942106741085877317464285673861632504744982114320080145705789755034584944230742716293067, 405073723636680781255423716188867771988040426514789408490778809327300079567), (108240567061988930127692019693181423878160481099507418285510528708336478532528255424112211195193424301142196992855462226895157523984926659708780382299133742017676632194827270226152936298503499276727315555312665853221164648430448287838252032989886022525003763285268583983413872448215231733958769643411291875907, 553282388456945073707689370746043489122917077069298619732224538454705585841), (59932666287778586388318242916753403096054069614087645346406186838366297186657817115374906732152428908869108114159409652919387029242495374739665902601063676166127372582944288062483821691255273412272863684036406826445306111250529420609127050428767095995127605292410419922330755166079190187704607775594516972131, 1171217985047548863175272646124131117365120309593523741142502255823034819189), (90319372181490294816339305439930377748536142789225952597914907066009961355812642580712645821977180597255204119764328330064630005175718642830375312578048531825459786803522916131872764235819155207517245790824179732447611980677389412534176521623609060286496167789589192766343523089429717091334490810696227025651, 128944131663114979706446515219421774428022593485008364861726293663892919221), (105395956448629850310707026706023501601651996042633327251002296331766130193615903246786641089482484495936862774669179235767872406488545060672009481953898447913801728907707613376136960146530417017944870710174532457021327567063953446828640859806907936686991336112399288759613888940048850219928480628347500020851, 712387860555501220785682590799215696192583223642045288537313939233274630173), (119808387365285222130194368564969515471127168477677683028388579258290663947170559441973770289476628930073711412423435314039148707492900299935887654987474473092045997580821198581256783035857569612204011723399932403406382330710197023006941960130427146386692461293097496692006482467244961473962945494895915917589, 1601), (73959765413804145230308720905090945931324334439471721630219543767525734290901687650813170543315717940126234947763729394154963074681933858484395863822304661011281462437390579045196650985164394267414798412568369367088520230352605861847753124560797584983248055352610558848169017194082595942132961147472681027229, 1637), (84460513812843403883072144388209733549127595127002922447518499964933949843504971217415315815893712299883638888737404301013370088325449928007446565859611769103484419155960625930090681972739389741158874869273916898099913233200775073071593678977004479840800353083380666269340106878348420089216163761118859396503, 490072322151035784089576516643781070201940052037452649345135231505591521093), (92881702320855772946668446329751089039075710991651536303028625600987412377039581458423740136736461353963125842819510300870294213117740121954830793466794072420885072152623532302821111089572220832442243123451768571433086035337771628932343163183996872352647155867967609010124875492215069901777467813795165020547, 36737352536549649593499879830838836566492708692326408097158047599920687287), (97951976138616890829667248322143461907097019048621428174418026937285884465797543406843332771497001132727022993817851955621170325093570214850786642748816805790778555700662812529742636402720137225557801989455710055151627106651883325823481341099149689329987450867467341452132302776317767396213148141155604061401, 1361), (166433171302574358682206799077719895626121026420138958412707365239832431648731945394856696858610705593284882498115690339308120531681511853323629458873329410093704689105407407804822712013465578473850619964850483581721382623280933455545029161427415543454161389547624113703192804909858747871167905797352276139167, 1684649659776899730738618019251926117095337843026234551646869101531702484301), (117692781975286665324113552158765629970638688906314886337254324822857667725242111849209308575107663681418414161326046705089632268546515183765433574560467417377352691074401454618624517171706275063415685328463476992183514120041810901930406970825326681482753002547115617536912041722220521548197420227035592938267, 1171981181789302631080912144215989886911391541953090448512840012218773144769), (97186107067732030075558813406293312468214734974746397926924115292674787675952833304193358573008029515689739838739294761277500296374388350641208410665830335407929487985575120664558740259760099992559563103622999705646822164822984401168450286098061814310680915855748269109593864025838351067237715736058608319887, 1078169947911023419482343336975351113191658773210769677777699343893797003607), (99457879415514218398825989136438766446364177759326629459092119237057507292892817473973701289036158015728949209198940781079761484770034760484670609502879463560657505610748217672331490485821529002688522808724761765841468389668065696182950946686246368057442630680117908035078943673806926996961325326255978569119, 1041), (87880481958448430004238844753259161260304617929024317270466828105120613817022069994503631165723038265668080039814970101235647993680899905306050471327709064308747861516547138064909606627593793514155057443031935258297033354277103095944209887161381976055819263501813475410185492392802246803343674134810439402641, 1209717476031462239504886479294188057576191401971707394963575035872126056863), (91803579768628509313156033032590584943937114364467701222152806893231106669341178764661181052865077325836762994302548286464724011483696966479313288033963732284298219504166098808586207923945531642120134929935973961953670561365787066705584585435202950715717568200202134525405750463176176642003933947174453989939, 881052122246207569457183957387417089802024411072777780554315062467572657819), (87567401478590396159279519884873255762195973719038207421882594720053929396920815827507423745641761907125758550766652617431341337906095382060789091402864218456098445241962347343942493610981550347258186970397970225718431426474188593752965951283758917672956183029203135648946908020178256524650002213359362954931, 810066945071967227574520778168227696769059804607823908592836630301990860725), (112351760664322614798011786665712186513233245945675792753584971761047754396729176364966288261660087042573341544920949813302785022604446509073855362954267744840169210318311559118701329758098258587972494205436514016823369311876816426411160930820502391402333418136087209937495920991070365049884540493482069124651, 1229834805502800129591971345779431831937585590477745549723345021028321214725), (71542125832088286457426052010689647510005566995549969901122767898692042946727234866355171734773167987541549786436475092167257029411324686791325274348111323912017260557649196357280150648735620557431599935469972332682909044423251863206010625018196584168682364812529357535902439599464142031037208245633188091091, 59), (90109298319331898908811456314456919273958127620578490263637353293192963411474634347117473948224166907996822291490973250273805942643613681823662302419186191889250450463094453886115674094580416975426441327611477856807171638704853113438214154913313588974787172734474226934162658167764730025206424278992916136937, 361536681684111999045807605203157645606315744033191251097147789456774807191), (97906778884937363399561651686600613913552984059469145508767636836044945564324578495005498229151489951573728011103156765091178150851921412694228841925766048770525951239191065396870456714302442924815701644999222314772998281433890762030701502424042461037120393016290907419880075348021237292882018764824655782801, 1748021543364823653145095493472982415291575017418747188969441894354223971869), (65443655190605933356487400113862679545653490167249365107086358527188218888352782624673774606221154569946864063592185867363556190130141524530437150312562602248495701010600554034264104301370641114516252143995965379843486174071995447069786165733278446376565167116548919573070750763238831045073134070232976507703, 13148761658615909828569937936059422018070830877405605929783427118673522119), (75918691832951670918878058160342149416705980533322988113685941586893153402116970652139224146814200475427036014691738068621912696622223181919730518385785301925863261174977983439231310149605208463265936129545306556161444220479224115402735977958621580414522476734086739714315461619022134683491290479408562589913, 1085931910662957338344955987505762918080832390410104026413441143924704684275), (80320252591411414570803864945875897689824473787865202439242533690329450059818025850803385830852375730961167174924072478423809829091970476942078451590328641184993558242043394214072150191149416934029535456079921143071103207498969198944092522869481372574852534251375625985675368602035666899362407894566270782627, 718735218006275117867032608170572327507918759151486316472874915738331765143), (61938630905678494129817039383359111517034978232875902716175299142660271881555588772596355826543608371583596874116860767563641972971250254694829426902319563706023779467010526815847716994970939492241508224428624131264050789973351899121900159006970062115504066704920098588301124721176183250998175637563674809377, 160801775823027680545982749571848252334444494374598685656814259469464008859), (72935257968550272234109609702858254867485885017998083461985414594757591610699201804814040830777726971894437300540560521782332658216420300394084223859673673687483638789545292332211819856809664608356960321604194690185163569903144559733570596789032926255769894503331717081539358871362318586557654933037094489199, 1813), (112651939006024290932445767703769151744943954548405971796381153315786311363178732366660735951563897386319391269119065693160906243965925363105435040067153466862349116423505347305207803422953565894471752585822275953314598704256360444778406086139839964806376874443359239205095002788143253708983999372210155762299, 59), (66129967165030157315056248333618456606570399233973802851709016902564352972943991636461360437739903459823925787322772155626185268127127545099697258307407139668951124909187036769855341897033398251754268726607976935019765617815681086020432912605365815037965116312156697201161760324209555756003263625457503077029, 1453), (114041456983104045532775752430526986643753625687872945869448973312011643739465094931442699925484669102781448707230729815035559575811259037523406335767992904371014637117328031652273923753575909496250739431321679460912104467863925997709147118232539198158667483016981756810811184116092960371060382554219510071077, 1259), (69051047538284822590971359033989129591633185003322243337207870199138098631877155004236468085240836543559574228729389103028495556390955836984323073798812856247934707898678066202776657098128052637197613525530545567348719054995675485362600647045110775547052398581028788379113893774570307720785039509725211006541, 1110857034493367497716026581864876693787319122632393204301181633349318425035), (50128144209807590124645786037085948476979217462738641521910034654282156922666090849273795148330679591288099325626430502267580466254749082777135478572535672617284956583499142344860837945280363229067660231439803544095258712184095113747733800344047811273371495032021001174355944642941609994972700648790357710253, 1163), (136429862151412149353259440646415094652520397689727176635289846264425805635197001554114509200555861755131795014209915718869665104179010104495681184131671793044745273696090282602720990037306376131560430919271717111415290992449017109447199809588333116944515631887744217097371951671320293646772537799275345331659, 59), (82955232624615424244763180533146047986897225117849001037858626661580881659670363079951170682130704203280339230602375872884281620857711241475693112426523447166876229560284878924747073744039404807329278590085076919075700361160483506861851973992154590919842435348935184711960341718017604981371038514276889499701, 1871), (92679976308612440764242687475763200833080174530137358924061789168164173951120021724916917530665993546447451772104684532885368388613723277106359059586828345868695277582658782021091302248283187162315595015127609205039319084316072455260469798951943839641117823291854072045736888174188357292145432230288203632979, 1337), (96094233029423005955719407433499992609438574306085599019600784322729277934342150643297921975923593811649871501084613862916372760825176170432835718336076525256201278068473476244881948188582160197102333341773365149974371456724808265813921789939849715244628881658532512281970727342815558541808936048023941301213, 13620356262148592759937619554129240009487702573565701248888289480784884147), (152663590691087882063190144459758826286638553214298990814703754385687913118612301432747328004775826081277152383596772520624889763884388095261876010800488830983455537377007890662576973598166722542474473656693284831784771886964706970721879549647806671002819465051070676511171903283917597130867036768578377649997, 1721), (93466751135025613391007952768323724934465445279097570740667154382407887924583265757685580833663659757050067817207139634847108084980198167079622002381873020975052555550451361023812196310341409515332030496372385019260727460737464311823220320299893013542226252560755018673181832336313785532311005743857156154089, 1219), (173537740556994854340094511674009625716370945374595124703263617870643080619755907042930550774507952431240600434099541890400281830430202942415395436537717452841121228567302211742171629416418218096912564552716655590149673933063081295935442858433358531689489548860065592553519199679214196099340411987231336263193, 1715), (82766574920810499829345607422551726637402279985217177853479874581391190613934456245140397588353047132275870996869011714483778319124256332501544299132142492494258052736777043429099944667600621884246136108957451609938005298798428938838526045665173506479991657614197840839512273758402250108631018641237587267089, 1273), (67193781986934338279871176166641344094740878744160336253235180458187362652884093758109340915894108559315130338377789155162700851484428163910898438420162815313628506925351311274214283345367601863278097738959631907651757332175161999088278375405164024172435429639196362119445433319695224945628730203648032041821, 1637), (125078670884959576572392859909253425804621767526336934087284212646559748072306279967309713611522828139019892857728793847861104973576370443580572215914090440602578776859754929986858511911356284600726293862513826997491028724753813172086926228039653184335984036363325083913788971340501545257653053768019786595589, 1554087207234899617069528605696580332681869027609233754289538517657519158787), (149076104081490237975785957548674493771603903464879427498594794909796345121770868658209031843453612317038025205767413527414115861109733357887545972224466046090369384279710070464489478063020685764703566425984896894963599650040215556328642594359064755576726657824023276062913941639895336979381052773396481261901, 2003), (112404996078400243178526715309624681382667253962263209978535767275164634910059066361700871773059661351457640156444280460381105482366933230606369889369379145328692208254296177876663831794833229524551536459068950316433996469375470979236537156879645324337005189123592579306189928964347373100527144681014956397157, 1219), (123859125548947794520050683827934783031745653774203220931636962933939645569908012657760757383689073023038314607598811334628519820286633531113979374719929582704732888901781818301358419612248028756684953615529142298888303031824280725118918927362430918146734008873703338502721977269113847003306947894067770837311, 711182416277839395218484514320088265783469904226122995282802101471810673483), (99630513510943160047729888206409913047472970861426842529535491433982189764836502624118086047698791589475769180449780056575412109853112982299202065099035250822758001020590946529145711703894257695385876494997476604129112955045760507419319513950056718052002240902599694404414565242219200315663659109101540797899, 723014434902567805671109944130332692871683929960843850514693421229660528165), (110640500610774659362133000228543476185065466450625287903119723375583302198038030191091280551374261030874041968759953847078777019764112304130045351038739755234112337062875637981262413580038803629487476013740456305294860129072187392602071298748805838742249272320370545524544894291810051932038626496359834404649, 1687), (113986465970081870644028664059771410479151184425518018822105398268672713524776010170207614588573176673259001480756825440762832912545931685072104548823788164280209417200687310493340141060530490918558364638756609622815186976538680881532273587040681986147948153103469767521573467083960167970677793159140768177241, 1061), (75762694024663576387293028828151964888725591162153924778490775812559989832753851027608833693089765887235697853161871086844403348616200775284937135259622917057330603946992562017761689633783896533326009243979977033049157977364588238562719382263784839568969871294465567124311381708517461481975129346860841923729, 702993363318418656474085460841044777886653810278933460288730168987017334991), (123917195012251270859483565432550038776917450335620069456182481830500909613911311189161015244552560258292914932462999070207342211241338372805528509676009143883606219941598542088201162086011556817256282499112166058933172858815083556642833469563159826303262698237788943467392249054122977044553089616385322028019, 242040442293486423121306686464368830512970518466155727122975584897824737933), (113146750795015848001009388979261231000561644748318500601246092149223991406898324632726997528059715921734985407796533209600027149124181164500079339355422090183671798977326742456340091883881344734823273519433533396530330772076047889134548902845186814847919230410105667420484652705536721837772822790430543397387, 2039), (80081705799713808011719297014643871948905966232221232718821002266747359744533849275117291275348326899476674722037015482054505225584174377129171018686291391914492086345166808895197862305176833844023098575455376166678862024610224091566580665429632002656872213332260554269671257414339813766840231342215747705771, 1427103328567902923811807372085847899115759588090042512109005156362843622211), (110461420858084996961348218056009190321586538777525417099862447901797217901591457956054998631278308945737770001257055031030202130025034666956737803587358153172836174851850615415005944683013640768422497559244407807845607449041446026474715799428760775099398925493545973607681404691418616919192462157716730132989, 407301850938093816471214865627878439391638040838728635298916932739273234939), (58511491375715120359694555264051209381748948246800837792853264782645687616696196735691481143758256052347774349547207763664669420348972411053437710437137324269757734370062820032097692588284947483521586306077155058596297711351983861922379045251180945754166308243111739437088856393636164796464326807107905523213, 1199), (84101031709376066265057885566075473971019525573515179592066386539064943000804494396712623043414559588538630047814302733018525517864436874613306337538476808268822280220878681811347713422936447807407018464576477061210972796982081127293622108733639063776228474688246096564566876587915637198168127885962635374189, 1901), (92751725012138625123047447298642914518713599866679219001520586653066194427442763605391673927259720718662190818016441313738571300912716781992672616815227132429175002740504282586292174277433125508626885260082236514216964893436081755726417848582336263441037678948651982427987549952598768136987227399296201778479, 586944960837598760024646975683815867526126946785222192462718591130029551815), (96237207973547831995634237892741529799436053981637266389295423396541280644416037756761033720827803014574727431417985973328001061238458727075163548573825093059534003837509824138997414757356001525294901052498706861364038143803224465972798672268496240164243971346498981869653165340638710089171905296372032184583, 59), (87614809496217974190535101909349194646282679046235462922532455900170544135674199367023254247202767469552978518418792419243972271181772032825105594691927991243245336986617679135277430600435087060504545185801327506117942755541879108826760426472490699564333665854332883905851290816892113751170127186054512418857, 1339), (111678336446745284408187488382999294267043170499869926088758836630268119810088329103644406626221585757047189332134345721480086521120777181594881105692497631598318946898703560316647034600898789068934940650941366216466476204414845885143877222959809764890618836037811491697007949368395274342980767860915541176251, 1366442443776230423093812575707140018321841810450615497140572149701509028731), (63505576805410966732572212950858144739265323776077485828050190160114024822305684258021186378504273435750819830962563636147060448276248497086115759033235416743037146754684132623081947545704931649252775159530022703374390329496073779727643258265533569011470248448557727256893622237315451921444056211306695523883, 303594857778736405928857984407177947085829257741432650239358231556968638039), (98475130457166070342123407096651900287564628610551803881925006056484410350816941102876621610710204602721326940580963929684754939520442618070129273999545229620393661914500147034141250793478323227737076805337812750834350342830895834745181008931561684614075841707458427912273594923282951064548436333731326030957, 1031), (153880897891734031939150484243083474980522355292910308918144458829814848483502898914332272055228160362491673340377270890561252225693425305598341597220710567662432455118584291479469484395483406995261128072604134240347884815949411730683892245727892947004519343883378016783348786920642561550498210504024101460713, 98961174058298224435341427743214555685229557514140663923627571810993705713), (77559499304591695960631220213867223095340910582235373484897636960876928977010279422276327044100077442564500899096803495984236888343179658486934355481666111545455330000499545503237398378242447528896212224464873722434866172670034198906437483142333457430856385375291763148774719724112358342390342893520524332723, 1981), (80558577543929334992398932333683084608933918892805276390535774515484136044293056465628184271514370373857565977056848911928480043255402676159449829914048779422551019277881436413419286417039176872881654359229019564986112165963243158630435230391426007142307969536552864209308725484270490559892196087520713415749, 182121099498059895877942906086065454452308889758493981602726034666064832747), (78943271694326094244416440936138584101649398368457954964807775978897188728320204964286754934361029888267567421625488011536996279896329500645449416514416473169000202338269117648249700992686448645230189687272105705554242424336608437798818178954466837953616705092758473841645319210199526358384187578237508277563, 1355), (59965772104937542136194285652231739107981155634785974578882795359084802759733659194792811057660104859504960689942022649817785346467502733788636304716441172361247835670540760523645058538169298504190615332742156963945836319711494792450096352371822535673998403674821326019101635199483947259920641093461282164049, 1763), (79799476554517640985432255683468157908856540976674285359028025520015662190907868714717371423105531544920696692269752310950522786761435243135400373986283287024300282419016698772801327561997091421025750913158258428250774347732983702306159559965906556252770620990607051922997520736496441030317959527850930193587, 1463), (120557422378063091940873371199636902021588545106758766524140150283001032730853422876309635829029848798430627823097241906343012248381055660313567120501629203149988983681898238919441950352072411882041551437565252275044250419290010031491596921490477186158402002378537727873503041475970348699584654949318747437719, 735753266963579877625385190831804139960998551581104828079137366993110795731), (145964190985806241992563498298260719577119557280153237052862585604605089853417111733931632102718128040209757418267739735792593838199942337089179538698432930289614329272540462823275957489653348400099923779342625483171472872782801654041955780189173630786855840016968723420528015114647162396496711751334930563649, 1233315251405804664698712116490151748408871875163283161094326057615307978281), (121736836587277392786562285629093889762034526069614887498032320501796690995883959257723598651452025022839865236905415943102241561950462014247976257110882413195481666743044782260291912624206842258355780013450992675740249777296518809747424544042949345160014515780399642602199773917174540842333507203570023469233, 708499181829408183949451834556238360176570933431580254759537892796856124443), (82859058810700424574998449514269633630800930331775531707205912118655738094540873319576530238475903775221424488992882392882168308974338635193289923612062838564394554926915334347726405448740489837979731712078756119231040482989432319022185487898533390031031717038531474405189714040834058679220343133339388599863, 1139), (123340315607120455516747921832882607142696501496798523190559170610360495652467144262894249865279599121829329064967567709664313958414836561151512826555762499191292169384953771892654827054623188871549582369841756732669187535062487012444932864803053793314046466463722180846855983134373658677711677374283737447917, 826373558024397673104921744568850726470174083244813532260761432768884957893), (100812055093869665428038483217903111257760421090944854250733426115866984354548083419549460365193132772550611066934843937614246143630117199368508832993088601315064224496652095748357642673105739750286063138882804495561082376660675970714582904261268275364011472267145539211338052453663605537204528730667646190219, 1179005582225639292349891495174473428139511431156622797086863309401383389451), (68780954502062932484354565836945287531455546671617079140477841141209530079050338036556075482349555160313387711681619892581760747071418062765479020222766358978617342167334509961818418264572736615614975445446164351771791247743576293656246294563729756501158152503132367330861423917957226097226197517469478560121, 1789), (138751882621703829551137389309301664348383165147116180457625102793140277380599442558760350141463239077115591896504563326966776022144727435089045723941257978788244911413568089819962596051320995019493000429911830020839254768029283673144545829109270663626352429622925862888521021793705796043547144926069708141573, 1249), (105737264107950899855615668335233943729515278082030726596857690886792267145504855736838822691804967483430322418070270901741020854656869081610043252400518160293332866743233692386461815473159138329413272297317553173087760487303437831182426582093456228398890602640436706666339149240372688577356145346687072193609, 1499), (61959243775014500421875856693968453701149307930189458707135093926685802844870080444214262048419148184669146736801964002244645218065826115971957047330248719527795733765134608978578649306591534423352161514122100837775440849544720211968999383304950651970919430273064804726430484929404022685236805291357628700397, 1671508761723865137782400420697663568726600603167985519209314964653673243387), (71685787618683389333388102960219072976344980152227045376313990535956246741534179809395832165798985185578615732008266675696377022121174628269322859465587100253927023812424202055669062414593016964547256432655553114224253958481286706687845425153884383729879824849958857272427019206451107569552526117936671105741, 495966063223861598021141524278719370179518406185162059786846061112574086271), (58443608822338623518841698508409733529941760504321622318316580818786920093616289690541379918256895570533937559522605599468332705504284168981280879139229924301160700355503585682513782001626601373603990569008870327104818353529885192887554100443779119634336293051245366475586248718786621233923110375857018836691, 1408580583916300877993811733641579742988790903535826350797215692767902757641), (73544442734316183838790628133323396209922646472211820151877038449476136807555187229249394760092309084057601856459514258528233161437767710211591804523255124189176699586836500629241365952151918505658449702632592678567259649215244629948371825574021197777430175997798688238947834309537855591659427950871583496353, 1801), (79593552644112074781087151656377955236754150727729266149271239182915638110227225745508164455176699007254972925535832379401470483723881876310234536728117352603505091787853694938492637667246066078641178321784888133921884711024050714506607348558326052122325584585107070487235219396159213755322056026580538153699, 1140275454126063178951045732881824681774933871009556193728586130031707365213), (97033659983036950850692166630458670684713806156913920921448809552634339781811587775072654803652662817772693526886165697993101768244068572660475142152198714873869817909639407550584369706328766609412126702483645804884166389265479476216272349630896623999577935716933534643856978958682912557522307720708307946729, 941463966676337583169666816132318373280512088558393495355007857655416923781), (110135174207532622713568296816970213287879999133368484034863069759257835480546412709950558432948153117501396865845510811081211511779354889928560586110433305066826660757381361941155375737032344962018750931787195602312521762560213649167810595504101965930049291239421104705334225233766827356121311955525233204977, 1789), (98920488052177379457198688227796224375394141348289090137432483511248881650904612567178493927926075403252601516570010683479866614617306926154825356586493247655347577032887810521734314089470423239834555721616730255634303080737196305650758681886622281255412606203765490947498638779310303433678784339693627119849, 1735000201806402184038811663211166457699575804537940160698366539180344089617), (92523547226399013059907320202666504452973470740088631957782236871022946623253503622532128584938252608600694834874883049494855102538543068729986096622534883303432214418147501808254490951545012508979511466015183154215947043063882044740337529160397303900534998721565557534621981274857871088836212729879266123287, 1255797179847495692757243817783331028129984499919494700654546564223812324297), (131100227961986322194222910740926711777518687760058411124555522682761372279048363067314066377776208870775965314223836053033350027605383727771595156053673170947273780322389967347131515082185513705229788304270018008730500116756216721635741599709354821915089496997772461035460395273871767428574187358064161787459, 1421680043756160289911941301469974600849605277187983070403900882953335409343), (67080527072187407519292729527705818266474145943325518189616105390272979637873411784736738752295082238449884988053109749632902814322333478284109167569935625254377162813277161735644267468160446134165692440602504247121301154799312699536349274675022661144107252937109437947836053884161120087363168808415738081637, 980772057026485502953645917726645689467615246566480424558457324484965148979), (73796395523008013583037745419821446709352967452579901516992241048579994667489934004249193313337119261726376643146519549266025785823496038254797355972765773150451289705824668933409732873362483270494820943265358723214032622129190846972191777249022544107919025505424736852404297599107009435427117652902791769481, 1903), (73817629632424331002887010026003611617731001090298567722629541566663187704171092841724670172680316276657440639735252330931160792271287647882080274274000303469832243834440506405570429978415845514244482764064077217073218610317421457478234360745995915707832425854830628396185403341288646279647993102259939101521, 1107), (55649009070556103715508555524180174217592811929517649002637033034398230334616602803959561820001641626019318489755291014985211993834536547605869652464944946071577057376065844582032417468056085553050958252603738283224922526444144938286926993408780542257542881107641091965318065824801296383184029582835366715279, 1168203183327477981200209692079333523460942573099900317797109281454900558799), (139870795778584353489173919153576561478321060502955216123775361066556074503053388915822558168461060663947169385371324074902882811408851130214802433026234334017342588446287572937451498377574080559193546753924611135887706030807654438080767038915725478273115186265793342118417856381089594215336593615562134613649, 1423), (147718517514136849696696403143899295755740966744206975838707135752284808892046696882402698695518996956866321353538385309968302690303030915666079409460478647557492419222512761746942532117954752114565845409143577861941373863893063109142342009052737909716942393301386033255766230864735830742704839725825209112431, 1907), (108979946484901564272390877221662117568768373794239581716356608198308872547000562482456624791841100252230113074132887960664759298016102164052245893373362637919037499373606247406353013989130276994646385775633013444649022567607593261480251542049922802174123167807865042406939856082517410413133331971261383060899, 1759), (110810541749305014188787304527527398166735719989027528907814800325950924920500843450481321634804754584625115877741761135440935502903389511256495803132890454366215687745644605195432340122856293891517213318300105345803559877399729304161687254253716225035804066705460126426839024047174316716921031908406355635923, 640275445931127209046879811475090075748529594184574598883683302815037699203), (136308911643269607416758582436313447159313003854981970982316486499245261160857653119328870090144796703745432111060369004236665533633291598309400260798546780611115630908850617384955512409740278032881556505135723881894563027721824917232239077833706764506088484605110843866170205583239465609432119997183881263639, 896260111635059350511526059315062452761971806847108744877963194238852743405), (94297315679786500110935006437970873154327966699222060554320240540532501697773994654161839460539989755944982323176334408977550276808884706709802469087954750098926605436224441610278058896588815191748402120988843343906354781786086943129523673104921887418525354559642496886733004816692104211192263708920929272417, 1187), (123356968776819084543835560331129012960410637007501394456583145318579852091274408048892330340365134993025514002233286995815385655123197166977759247729969043573085964727961821499975009287162819482651808780904694654348969877466744351088662755458598723036546664813809701075674292795373571458292217931588663907913, 187418263533955765088299138049496155425893918706022814596598765575694704707), (71130896972864073984106258119924085979978370406382738433799876426379761419851586575859707060064347524120346513147627323063697312659273030846611669285378643277938119172368001831926409497973568660427172709725019112273727472187577259981250941021275928548716056319357420309038266621201041979620834416471460105433, 520751359672291643830077939092412765059720941398047316984159077791128839147), (79896402472515295203001380337529905578183033950017809978546856411152573860321694458239416461353787340819468148334611821894436025588641618696831201659773637274585951650446778296508943303572011564079805764098960306819974428687957311160855977686238040678894750855828844909904689212534719123685767711940711816727, 2009), (111148241717574783275996991594263245505033786447713597336581460647920769345813184934151658236707418052643508470583546783428362769831749395311704743984505727411030230295940068729340822498543940932458706762308880185081201945254337645603056640080120823625730191870533626368356791025946699023278836960300566565913, 548275082453060094085813439702456590673870265197759335339877285108942435153), (115280221750615891711474207025411044453844144716766207787211725552581113948091336413185913522991701761242434738257417512473813243283878907685148733933920749716073057008785222803664487468337248745769890256778781598693866021210506993978928317476157586671639143533674529987802219767277849994582126694723709396681, 1875), (113251412072637114211378458637075438406585091858722930573009961779047212560268694304742939892598076073429017226177803306447978025671285886377157622288241573100772088498153361970988757685536960046151378820535787048076465015894982240604648402738966136539068104576728449454691005567564387490689828151946577712363, 1541), (95825897383506060030580336510891351593315579256166828070223713806563112822809202728584198328940103845853866127051439100546045853466147340608877262980455023767715070711511349747671384717594968818460651884572217231087915297596009871805745771986737703527948389617356502985222954732120264065692764443807735691803, 1769), (52283119766546377750268482913477258483379314896991238592285434011717857439699910935686216924697021842816090187917212288248375980436519941686055054717014969732708931146281040096212265265461014172916852824957075658859115697563389551441594594285011076390985671227746687398846371676380210507661078258805339194709, 562626321127919430047552314793594783314397295023250546711928878280691160299), (67848682808459358970795194512031282128502099188270601990809448932465345534984165130838647787076092864061590115974211196030305643527195713628238447648245276500635941654000295185307758451074177272384947149883453070061854950653341209949364374101792253065103312514583071146032321991103077603771085007771992112287, 1533420195756820992782834126942044894470077150061477349192653382133807403631), (68271816730728825387600060192285274597441361536230694548039642251477113478384265702829413621837383466895237721877143574604886319617686972649434895252459540210937239893910175781377993798693088634985230886759139349706405346531722774038683223597477688601987023483431828187834582613145179811983123998111424194907, 1320681461012312457381093527987768872637254878088127915978984947241726032089), (85936846810032452829391587887336056261568763256170179815053342945919062545625362352972702305189760917026591737950632953976025072092349542462442041063532781655528214401584262117420181710496591051664124508786851758299046574683360656085936626763201321463649483207067952388890293814319875552693021471903422501013, 1103), (104099218527628818804982413674731266691787914133426724403500227218951703886235867510134692612232330880843677161433854854674114394390599513758813137950728383453113757088383546374328334722287182213993532985541845770671591342859910545416073091479073649758429067315526808501372250628854801767893388915413925129147, 991565512795238014046881981292165989017479260861971344991665835103314002987), (135765681548645302423625536214728687857600672148128033671160316400663814236759575142912357151153590499819031017019222316830018460543959219210143782201015641131775105087672785835466846843179703155120378520410173167361192647713693498352992959071976170887744840729874426254994265887184182793403827302740583608441, 296930619867056846119285625020313662051237352580625670329849228288415560331), (62151054964064297123015641075861334010417280713437634267849168749851062073807873327690539487142774507377633237899555226934699338288501017469707031348991797401985624046185745649379989771570518577317916821738506764018694835284014619530091308738145776022682157441760595755208750233920404572026372148245957056503, 1691), (98957915052436362127304483944939455267788112719364690479588352504729576935637616171765640886547775486867207671082376093359157349355350179099149016402469072782767880071505440517105488541979450506469023315729917700532939960617417636119155793684767660047409525491189434715491683482638911294865455960134013205081, 556957499063583441463189122001050324732251817942162276692692551863684076395), (107213807431940022220646238711067861772253523353319906453410124847296951968557114411321908301539371464526350027438537151485130820547324302547463813628978062062272445029149748641484685931816448840537439837904945224153710314500062808702264571207425866325468366433659456699478545829758938926154684667614897757171, 59), (102232496224730064111356619450551056124084690700915732731257602803288479569060977409586861100532478484891990987257709873898505425865730882525131812923800132217687163079544704489126039721426915340865507376897891112783807376069187928024954218721315752132650558202491130805623255920672706635010719047396714371767, 1531193876393355753306323822273828762619737759580386405518863216778323212813), (100743427052477032747927632846778603184008552120176991467970095663473422398787526905681175781893686054867774403171519486290195393630539672184183329385876261461491199586234300265188840145317402307804612846040332442230277998918393797120302857914731709342990594301033269106142735908084972876570794874135157303739, 748882959734976852495262800983217821273054770427814974313061657301301262595), (68163620819037570519167509947060572860186981098893483325308152603715872481923695027811524384571945219975155088952875069802100379590031382038534738493432261260734960329916415008896219971995987563808776993386230570547439902383401091144014885756818059915820219850528098898022953743058497071606512529893461137737, 683682795330345977480722224834334805231346172793109681410082192581894755271), (128950002369351129557990235753316686264693060973798089927011992057473099208647881949592900044732215786344839241576883026563937574153102382550855147420376155614863545854539325137944172521260867882004912217473127675640277185015807489352774518715648422130220475924383316222093849221501984927382110787338133196433, 59), (97242184278161619080102556106796857181739932328429240266345652339602528635314268507051495146390531825913077109814499059604738002896232219209587458565719127741974079558577897588652809057633553827200669768862399554582279644247639468494474758811919152656443931409876453281459177458228934829964909573662166975487, 1799), (127979563793342767041303380337625860880690545009953319241006212681631402667929559652131806427986539403280128030989797543332249355934897436667209408096391423531628539662898635923904882280837683269485840877408867332088954329044175870275988864268368759973051838111472285436594139321686460435670087793455954715623, 1237), (68928881555988629831383302835041687165730833227365425017562251824409242160924908620440302728753661253448911874353544297852160456327257890723587627676056622326010395414324510339984783158220228625816550312166041866463115242220272941854192224382059063635388173626588982715902519220863419276423951185796969292101, 1125), (77699472063825292253170888910079448439423875878864147149518759124474061177393494199891701392107278571460881313800426007878519436323340646128264746243751638452338373236981527235621551563161343960472266152095693808302751985097419957857505269305840811956892694162673461031181684895799863438440907985934535381671, 59), (73331211345141238587666527593379955164448876742365453856233672086653236138906894005887693257557723132202347754700865083049193952655260981660875624196205610705898625602689236246109557580896372159266353600422542531086525777978560334779792824460571333421080656435046326640607708160481711356461755298443677352337, 59), (133698367470124926888304829756351339002507129473867535675480081046475493591879262407235735508072165087556602610072173868482633970507970838768177130681149198733796630158366833417268571534270529225569598839784219832248734752633846990413602199463427267800438619225425286093468489836045443732895275021581666819179, 1477), (117708038735871151276944025654021542102750124165375022028817504067028041928013208186423961500283685424333932816983105300385710267501015814544325238821204737517278485633554257413600640451716895572117864062226399652688573674741870784601287749409589793073816152041352069320840989314520680621391453385800267699337, 1634968071183790717434087496839945154385741512880264098983763215376108033721), (96782643188715303651636105741130321199979043621290876934437091679862607899435115969405417538252734393931067094292274267323638591844090189176849712388771413967801415537757315789188161984211224735381512997883089926688993536287519302598107087077277578597910233366697079264746489147239231047859220274070105178553, 1093), (103445436700748924366079759072020580546303075811715437528154343607374518592569085617001174331836129341931245778328853712339309287292281496063650766216326305658704846917065164836054334647484481374580644793534783921175027003428142394312772962295259161793206718911661420818851541624755695756516226581139266095477, 1465), (97828598281374816038313686822848874614331631422390450632001665477213004560253533577782529187378029219561198535265588593302459755537489049187296069143727808348366756880929164919969735243152506883744816969410563109897350379929986839197689030340258505830468590809028608363376915694162581269420341419361935666531, 518043093398439516469446941713656440099196227447142696736547489043244023599), (71608856619615410375389268791173299049985232048965832826327230380934959506819265474303379085502538099301215860914085342125053962209949626817396393066372430427706684005385565023050900898608790031607703567932078941899407242170459200908464573957289741709361448480511942951417716525650197254561515859900331875131, 1187456241482073262381167835711246402377642148062398754673408624382270227465), (117905652333386823143258249006836122773242085182560185049446376055473662881896050561656164450805520810134243875320473084227577918574587387321236503247965666844296996730189541246622224299564936307815474623112102906201006720035768587396755843003463672039135596362084613116582272440986459664012790764237943624949, 204986866466472612727220611481402415430751891001124317231635634109312432471), (100311510015938145816626383092795115958182638534375125044176873840060812693693099764806282833243517149068347567654791074673792752397857603332333968097586138410098889448281191633485342717682295112980445663112080161333577340891211164959886602962151082936792630917687645412958495191374893717715851786619152781087, 1961), (117907165523929758731333092195864143025429736455969457895713535425366471259003912276524524896912766175442695324200711433656912934550289184816602803475092964638398117851639675929232812901233813940855490564661004980172820525581314548637294042476093107272817882383959088083847862889635644439227514501657319060759, 1891), (154575052627704168679993312370774715524239434351825761079842760467010312315245673920962343394850325478699099654425846334545458691333997253540127153047421634876816122464018016870026031830556342905174742187363425516752769167990186517397377863144227287088237398902992035786092620013624193719128626726180688948659, 1213), (69409118407961939614230923918264451441515145683702817486734803307135893518047219983971234190606682033535478958188413687976818320055883575248197165576856233650499276076245994421987866353682441241910861210089994448609907343494686613625206843572415709203876378199221037121597627259349795630181224699644209018417, 815402691886947931046280230963977779538543532138072026978768206996141950051), (79997932507471486044403289398824248687455722028294709515813359056590439644224181120028160914669538905749854624266774418638593820823258136555730683559407187731899743482138219901322006386604583510383591601346295042517706488710377916050355429213631808623815571513854463107779502021883693075069534409949825741383, 1309684630485144891421084655891702551113431565495017072214175188921118605329), (156351963012422984671290795924148473472393520310678185138233213394914395061521329699683714892995309512821379303750271731576448667829965466809780899526518941804837957356293544436291113066687439969776488259908883392154319881680900095162559025665051477767420760477046372729860921632842069264668898113067036502889, 1763), (62771620268893606940161939249866657034684696564057256022165029693396922002340949177146354551210177723757365993235801660542081712292247202986087063052422154212126930134515861230763833098048568014628572379137199233035851260088043708827903411632103338001292847500048079927618444474494654323811642117135653002259, 1829), (77142241908549103361376027924714138685823066157087378936947575518518775890780158049532842491701765910170771706338076862539583193571951787156218586970084796557405992558263714614019896662098933719808027586529964190576175412375013303689382563946807372567150966014029443778540419774043162720496204724626131758777, 59), (150200992132600143318103275614321382972743601497686654187429305563534995926512949729458963646777841802642884666100360573300523452766410625154682229946478004068086664372899456573442838567079870766042051324473658798238254375506286488878334058268696526178014048876866493914031746034284212653630653780747422218777, 1724465322616381171834169341948275267229780075170184881301505201116301814803), (84587312842513168034611571257607776241676379991605112354940723205353618995068972341110519505137674171753298361644493073207949902339987650704347707776490843280615483186496893585819950505954442973030097824820540157224920299736521151785539533653987966342221984909047197535131034448365703525459203858149807466229, 1391454586678277678864819245993871155980448877997348436916035604941764190563), (97629107246423206163333884735411462926846904576758376666697342887725349155398866639532734441679247408833400228800553716936682192639024117588949984716694656617160247167727148253374079466414929995109974102274310383375580721213726337439213219687501707463113141998402222835033806426617460952157231484493378615157, 1457788870580665118380077461771435300023581355123402865810601614043862081837), (123185243267784522731920331411421375114784814448603189133456669913083420360563416181684155780121178078099086836005331021423982578551421388414495725675287873767317690005829274269239127212538620433501409960162828288997101080284888145845532107821902271873276149756110406962320269654303192800370742296081474273389, 56078560677264621923195832095547279457684978122558957584310456739746916403), (166610159800471974652504948670549250008323190077682424132588519128878223444279274948422868379425811760557195811730366778372731019284622174336491430808788740527586337900216768801499303440204167344089229025400807561800390722214735053790184276435402613499718402959229008113391971681438001157876368698214130605221, 1502571157069940476124256769631167474253884155914846385849206327997674710579), (67343459647227158101885674420444015240774803269823865299745485482518746934923864662336167889756582969025896528192842162959966361544274689334522057134836581850228282747853001227076361818578935391936912112163940294052872702064717889166162141045755108306185042211885364740816617959684983418594275766846297853989, 1311718070718330354636109302508163710885021412021774772401752563021991649845), (108487483911496649650261833563356572514557268165544939100845521534777246259359325133448885537077583802985949398432201128492700176607833527375553634995407789676436477440526385079866196322112944325258589210148607494864829217790749973728972697616901962618929712057982726424584922652800284554506367855578049677209, 1399170377041053749058885541747371865187544790339211593534041106708892409779), (85275709257013310494565630186222765308307959204807528794495098452760247359950356853762460660451697919413142685025434364138471955013943689544307504422010758468006553825226590081445015432031704197539675881047682078190595152734065887163122014964642844650665432383288860645649410794018656262828205085670559745377, 1781), (115627018261156422009126518139632132815354871979124119246384034126405476707190320779531072131082740175064808600295629094499885037931779009249368492808223019433847815752816057470784642212965669074463345201598736643364730628356414794559886261880607505866907358766744365845148166193682172333583282711094679482631, 1474188945828612570416653301126854420828992051669722609816591373841500084665), (165482850746477616860523263263951211283729805955846332672149572490437339162410977119278436188259932309114413143968853280279546879365023469627861216479858845397201644749986877839471387613379319246557555352073266982976897856214773103548770485116984393667176800127128222656424380666443992202721588057174302649327, 1343), (70052232025375401510669003364572097410059630756151608283007266245569603240294694970508863533438921897338365217822405972290403874160703419447500556204408514908156117764030167121767458879072678307817633753411866131389249722949767045042971152692366328620033046180172780244373382681071613494887734674303183158763, 1484048329924105181267199736719646343256170620057025620782184445514783580663), (142852277992321196854256480164245761561490103213475001974168430826540087231682498840232494193071486401702511440240664862869188838172025174086605855101722561623560148501266364585954497623990266681694379581723169217865126328636577151861259639157064421498421033586916804764019773504062070031148719436923466955019, 2015), (85879647784053310386024656178839742029933689376162701664397699186180409680579870737836984362352902426975446475857600261854030890576352661439045616958792689528484319013956416119037709729360275892809928433492384388393598328940029949453505854864757822678565323277283947990825017165312135868910774564328025991389, 1733), (115712394221687687466990267257508568004770306990511228635063059801909415045788754022203468679183621347052076652790712881072578104346766121745229610743066099451031930098822551934218304545919779321195592943512492121531297360547202848833270248815112698306224071234530268842180674591611555963760120745224554016561, 1329408325795631029540443688871509910344743014583128487543721669778482065377), (134002739504109226099381583832049361837214265900412806181255936219111354903138080511136798132233583377884577232728153909511230794312645904139604485802717231949538554978676037627917379116465528019855715480737156058019774398214241164432440241706514164301601368533902926939867294884214219206806124162863533796379, 1051369733638356312654188011824257832133665070604471681184733819564436866763), (63969771284546984752481257019836973528295608348923577902448047436948911394080113762702789193389824883991335467959232098023395686952498058422862361100582150127892507270840981662632642584444310961976338393876351392699553321491094760979560223475337585712859191666445444423466919936902024576287346496252445216763, 1444539267028377155525552235018915111510371880713724483362695588438631636873), (84888038868440788412739534516360114751886670355460678484306611221594631758347969689728118907969644279920727039979979168266146426133860719993493291682190863185186959868348056473458094778271382632349454839247587866767306527062262363244703319845991915099922184944360852620322433070017316280184044250623260886573, 669818247496542657106557341096233545385698062834945666393539863190668548451), (153653569847145047634603243307050075296704788633191216688518216462468695131984965220545029667373533207700966246492261286703548397598784861157314136921710694934901820441183019741547270645680702514185648319144604160394802657537784900058675205635835990237964184124613521410172770797925587339010059344340024650939, 1991), (86676551765513259682494935945877442273352205502165580533108354340851133389617430885990480264955970503658646467304135640978856473479407301162557106987807652938660380650937269333743071041321777123569241968907850078292571725972574168462178557611098347987523268403042373976404116129104428872753810899123558979917, 279163250207992940382031101039912869763287416234272945544364887202751487853), (128312296505427786339972000194933660964689864780882559014239535387232369492966330230113323656841294121797489507618237743306520641382168927296633318770487654236754457638648207928975680643294405158871046693371545202685408719366158329666929384095169656450001623209251953255214180008346475115812827384875361826367, 846757492714069213002670883063313972998449452378894208593284029718604406091), (161919353548778927997943266364757843237254305363661788231496475520600915630977244746153917879520175756828367885624339307711576858430889981451075053522162617953645953238933445182603593110016242504219155301817992787990040879666193736482348473049570517772179881641434295081021231013908580729962777295734770679353, 1632974994070284085121570345936785579692445249590325298002911348507715989359), (80825706335550557966387399187121378288019177897466487853917746818400503099630464701636935535650298309213521339692059739444398291467171708705596351015699450011762452923755451364720487295043131022288704416122717891608811063751570796658867715479042699597855016564563306684711756016130017526291963786823392687093, 2029), (129406867102947705017349179420729249114197557999763719807199923899818233161518526958525656394094214200526869537549862638241102567365278951730089420920360111468585942113289956641115170184943867899752898718829083866234679918530526461325238040158678874846932608258217777964689434247350034726059022741515990456609, 782026769383551188000418862387210284757813544430700349838234655660366372241), (71615281955478715955525063799797029129265066053167542126241072614021311853146542030678720159635608867632395460420580472058300598079504593220039412416620081129093515535352035738824677541503122952567121513986696983930700593965601460768267652179077431648205581687343654354517657356214761769557521218036464278821, 59), (99386588429558461324690743347897462985576425918900484390031411075556164375208973564336633084711111446009526216539702471822182330943628595223170704601900717260230834131855469980170150372591879042878579261468295268603571392129682147332489306479025562480275406128227890539206927181782291257433230005908809072339, 784955373138660360286232824407446940001579244247446785005685095779340761839), (84822533753403027082047060260004757538585149898210531345080788727544728479166650724710916263659867428268003725732586669059018618220221312909941199410537214032425841463597493274226637779097725827884006777432097257190325049816908767114477945358854190268446120462932409963197406885967610283630263856238181411091, 1093), (93259394630065796079253704186012885819830039171570965986475874606467228954007872508518747810482266620185607450399895733805858589054474169353159722977162728533190093674884205846237817594759621755747971880599561027459331676314631019720120517829914172584398263788545084560281106946463794490273906688662410541681, 1235175308538982895034251214195964617024629891721525234223694041483833309479), (100868186174735624163803179368124978900033611250650005116666098993817348267129589323786161962848509869335395783069155278833700952333147631904539292295125129730077424927357374860650650153242653631305086021662565904920869105272847569130504338586467744049646034763350819317012469735933893444236185835539642608069, 59), (133504884931003487904994059538608623599870633463483340516139791374494551022175669217451929752987241830532430653178572844140614986887419164063351422618241881170527849504550824774325838615727209912249513405396275505712770144492893509436758965291324474870660878833371189916477980810727721448935249592852056088697, 1229761253957951476076966549602099788808920326841451161518529322271241881389), (125357519700164770712549376308654527593798357078586270881709658162650825694039580612859465029293110165451355612346205462990233753524127247428963579123331181653729327889198969193234453823964596178377512070585903891045718839811538680816824497180704636970759853931683008639506677645017941001682879056868653353043, 1289), (110942616740831987569732510181400357694533593374643952864477541212760130894046982494341606384074809883448623677567349317164826816833559861820601594969913396449147899467943600437062862261905894970868979826437978688660724348659619404036977624768401241329794567786384750774443574630945392090008851837664722896317, 1833), (78969912716098651981698566777082667937523566366656523195018711984152697871683646037164688295043391769416053519017546197784957919700380188753143688976469319678630536735303441414485458575401862971499404988256727017909617435662306342740360572518730797616704542806507813756212890201987838949195528555735684186783, 1849), (112622268213277789056936140274741770723471885510803383559898177795573805628424817832435342480122154638934162911267703225802406586435376014080922380300903149817388447309721192869443520512025893203271568316960155332266440270711321256434733669401121337983216089399657329043934624241804132127446955258769604967249, 1100341488100137661459929003363516466454797312614998379134282094544659834577), (86702611215199406740438920643156838333342175998087577476474077836558579097604710056854270769484177062964045938661007109313797992559966039699109801762719273292649525067574831661393330896085518497370516737372654100498973512476625674981092293020320498876408382369640483388311357943608063783113055479986043751631, 1997), (69107684373267191940963925713988887265977738488968601134080487994450222258164416136013089993076539928109291330180479122427383880360725908367192381932692474531749442386913241967151261458095416380670960070629539884170503419651835184014478576067433098146273810410649666728266951233398668860004562961664363343411, 1728233201039399458007075572430110694196255430099815189422049117986316431693), (102300406496411899081373175172364487594694689210401597235870764702671083766845554971608087096491722599963591362998438438572241214575125707574197686079853707158807470667360378892284696039801120551352316755795481460241924212539270516565236808635482421051186795444436315351427980163934759366057953142072504187877, 46419809549850411022585714699500675595533763319835748462111295143405584909), (150297548070396812312526519802712651757311969434075386157272374333642554849540090913760721545322513658706335423942816082693286970061660996124961812874208085252750219725395741864460427216825699284964754475803342343633513808665566388999077594816866132221563924754159982940394686791253074083861938405090187055633, 59), (101980284111985897808229715579106688413632028826385426236746025829290095719813485319785427807607856944887821970744054664372692995313282084146932482912052545764490002810992307354243933131009334166232803361996965421747812064517701640996856246053476666456161679488968323166949241533345846574382918742912162233719, 997614546711759394085301511900283679178031849951029620662536869324276762281), (75789555904431215292534054700373898599634855571936164795511219881422993024967607422057874245681753791387415045146952582422957917357473511440172673065794497446952823278016655442042308321175160202339795369142477865105766477094596065563599158846234044771551084061607199041123742170804164405911111516034190976051, 1603), (58005111524796594371090840524646849982215268371988723516808938625803828742854928483732389278585423884206066732368732342023415783117164770709347469743001554632399556698188375877819543266570396919941793956652622264742535431993542684963366459312906601170281476650123028934954005468776965715634777638603380027857, 1527), (130130675398381319531531594477903630465792596129153205094227641905052731582585363218832246369827457983566279174292661728633841274266749319747281698359850633326815425461995219571648774038740634507722984217869812651095812180558569195670547906890038108767097651976886576117559986021967480115102508140307258524753, 1637), (104073953432475482729198980500477180721041030534996734615521059181384994246107933062128182001480372421298067733662210992831436216190572213890932633289831744767791363813773996301455522817307267537778046969366697537862125229075130441434763841520329968706281325036607666865360576160462286283226534044265607232747, 1762823507235115206627428481824869577425418963069634657422522428718768718499), (74926155478372840662180378229616674549852538188849528669430106689842058516130206303127801201815813717975731571455992875261902198882640038468458149197272888973355544235192989388198757165136340380413524885313902163697271547301290085050715203775745414362780664252091016491009755696362840264958802071821655278763, 770711454819186073217097314734361085800625380032521145259207280157308944203), (156033724283609615893065431266424057687900599144378908248074668188280321899755139817880591418330217241995224524035208377932814648520939294834883807190784144202841355804942667145161363881637706531547874682783372536395327607734021419664562233969700636093036957065135270933702472310526610583151892092756292849453, 2011), (82750319092940472449152091473371528418548600830935383941277836819991665851074820045677671584952790634808831066540430193879194457802083895802808380035102338637645009083770018266399965306787415014946136708222899677628779884714141143691074012051228960727447162185365661050110894177795110641578935792069330822937, 733714799429096626022142440381119584405879660932864727901397493404215657331), (69799457020946301028688332001802126616702905905627014033814860346953155440387682271040158151347338210662907374708115503358771551246971326318232193541146869494819363366148820299480408269227805787045417569566063730088789039655181234029790224142471483807162313237631767201554393970398619772239011805025857639747, 1339), (82542842910802741468961181248478587273564412799859029662571460566673264280457886336667035266665857902319596769970863534148308697922240893032329355592721230401722548187926305746635528640921206726070000670194223117010663488674388023616212669642616387004909512688977811683304363663121320335684815602126333822383, 2027), (96928616173356881220885138069594120395704559212162418783362668850894558961441334193022487270196481989287547763452438594177730713375587536201774834059777657825594829361381693391933236083387018397130906986937869506878694494121587329835967634632430799229267601055170898899489581182974180450509728233400413572903, 1609156262032021450914291992104049171066262972616376443362697263618183307219), (128234564818945540473712488275913053744818985124044403640564767522085996467425251932932570332545122434638645192102911848722654816890658599503126943299155361767695556406287220740129073987563645865796379920992682495954127532007062482358999058041895774106281579786397102219071317030543293062082796423130483304617, 1128023048071466099997754537429249170406009517503021542594374472604869169955), (115085882068159475862478292272348940232132340033042571909140804941212747143205660722683485274445216844527181451836429229448794272912399147646187945773630416924842602191895259061297242070213480775586620532352558458187207404088513868987527899394228879065265799844462805531548823145385736759383597582171556483111, 1427531151608070202076237988595395730976692118737198271003749316773730710207), (90846033242899934058868219129283074550929387667912657363768568063331681632308642033866501986023527699030872410693700576920569158831195403354846295716123113963477204705205293870797554919422085498651808539464048499951629013310483327032222316762613449956192598727657730232999377686937234482945531695134681383009, 1177788731469999640432655705130205819976929267194892751323741185848928397613), (112483209292071414238085573873187791162775579420982979149374574726790804934835775989251324693979917316774168677309049109551994218214803348472694171040013154724436539827897580971292464582770272720817698856431955223892671062514761003500573107021189025112699261131808265428857084913560106950441831747864038907403, 1325), (51916295543944229553491652712331470334345884945941522839497796919373434335000883027760794649870715024782167572759186172261567693466503069769423334235355363956904894588766983689030927125373874539247717375116977056821414749502511977715637474722224268777582621442696734111566706539343163318602397830966389520087, 161151216709152136528885526092103341102477033989929479715007126939354172061), (82557297872662436210798958240189544433244307044242291856039101737269710594134396757989634505342754966521473798160645544559306876400822742929600009482324870878802342725387549588718497925400428184434817360531348408801684060811101483891935827693640270167926265034456358625024736701827441064819770462909888977321, 1079), (79265680459819855592285076356540842064267857696053199010015404483722586828674145104857291535547176571106509274056663976180834244407440618329032929362135658219722725021199826910462972370803536768481866739458232310123549725031260703126084347037756719570298477346795826877706547487629225623183035680740193340121, 626405556055642120480739827563018073612197864684195266675898859828933020207), (63738197751584131080662870123977558867292093427827918711029160439284624647920737798744008172504237791113160356883316344139612464657091773191742791419926310581868985251032927155385263142815907857933860025380689028809372824501464913378870991639368638604994079188009298168301997838933438260654238748176410384709, 1350674107274849988185088026464262450461872691463206829391050203777834554871), (154296038396869754154008697878980434104596254565584745304743033458418190686308470596463471000429255247822949460616426681084515706756114356909963493732337503933073649026785738699246611284842241098676200292169949664336261908606227545334569089214138306630545288060909902175960953090942966044017709563716324354863, 227065921094647608970003181674859064009759352616948501268655820964949396801), (130353248501262570631771522094432969075129445919593928917310847573338338284964414662079848556297658399381693346499826159303305080061904182584373758348288793319572204541803745590323966723701095296261569620794724423350584157680560799280612231715411127866377169510401929303162917082806285239670470776648185905641, 1804378906927558280615440009677270536139970681414665567078571483400228718727), (73675921987878646793502340548710148580719357866725751795162327549684058994519965246690324990607402030682448561494106524649184207666135879172147772115459848951293293445916179826062689205675392586905992541200640072146889411357475273501794102890443264550379493181317781898241806155106863353507773218908234371949, 1706529730386495254041559029910325501851025382764322327228044884420642667481), (115738741217770814663920096294935640792793828990143141289064612487947026417745863168199269145539304373253632703081479473202366106366605482035580260109012394716504503875512009708267183981245264637787146132194855803935064051100342671166244920039643632785178674515277010663033793564427561301089423606659798509249, 1273970167587479856816478575709181666865520242794337585196998681695162326489), (61533358945550166205137155374950849810344883626942812975779530107530609784983620109896806267108575799704638394993292992805696950669026418736187685087246471539436869858532274439641765347526872415033180740203330960220066148575627498257241450850512551645406076156663649650367300248274581048223208633410259425207, 828310504263312168043888339364919013174704974035473157899897045938958226465), (52333278308320883291427466117694126501922563757900106002945678480233010078130719040739961895731051331918956692244989614342006949369972925036177652397780451542908961626433670684874992404318391095546269668742773549735350860442915486136593947064275495126364615356472452322212972804951207421076578103498578622023, 898458997947729304629177020453094486969071931237631579691219333686068498067), (73716477019701771804650737309878830928270786930824990612503584883878376872026318111123258758183505788265923811512078638437909593386593503769400388047067700769802979715750695223225318103065713334122669205460002706024515419656691519455916564182872303646878161234497309408937381363420488044001297653731939569331, 1259), (91463367336245272875458998133445766062969797916295990154631582892959108844445919219413578206429276034927843711322499486446289187480244188405839390331642579024835044462278017353163300074695753072150988460201686744120297464862462725074396622551827746761980863278538744759816242564646119780422359918607837587143, 2033), (104572039813348261606996988193392721326473060858590374748760900390001692962836531422855224988071999399842181581692836127802186209497523715585169825442459942536691861774008092926151121656239263956335837054731557977067206318163756725432991800537536864237715213605924990483145692332789298298718089879807744925617, 331669016726375658063833311118085918238691605870744950448226245640698133743), (105248879018856276194107693533458890948941144374467923412949307498210023121010286105291610058434661160447292484642711191061459705691079118263621306034504872985683862851512153531585899804203887143134352233582846271716630468510297487033848366285968317883647802751886417189097979407877854801842564590663449646779, 1151074429514433368476113272478450743181670061433012622030510702049004325807), (93954811906854078681666641741457701296904795729422275798262592330631997635911165590674152935811014482400417993652578408248480275028833263212010050544365650559279115412428991066913931571961516092973061460670604092380679090691960446615099293938275883668926188758597531699585403229156858551481801619197592622033, 1385), (84865881060785836176355354929999143253171639969927768245335352433369097669886361217847585698155535862118892009850630261491271391061885821610923105261121728701353487976597569457507948508992049273024462493915027609505744947456248396023271419155542088400646892712867995060729286353329147212248735069698326788171, 50476506834716098649784627104226323517892898398633062017464090951366002743), (102225473101909724637350063980709652624722721892123258357050204324241646543468034960118335540986690960460357557165156484693369024099049930195783499015010743749678988078397261277954510493484968930019190817759104477191756970134086867693658770940980086734159670226101746564300374520117119674486112658653006001593, 408311402518320827259112203524761258308491519234774536840656611244224335637), (70929674386391484075667400617046701878245940387233434735396983450780816113197685506174688150020518560019930355886719567730053335358077211431504071235314656510892867015686873122720853688307342413818942570304744037402644558492789434397691264922563051789938661329830414559984430729746844698995086915330105139717, 485521468286711467398960450219807441722596428322763789205964073836178020961), (89054967275330176687148619703225129392413863938540201689840216704332471809148751010237255597874485030580164192486878301027554000774354224081080569196347709416576963543509628269092117980517905511079079789565232494515649280290801491201858255473215464351745199530425457516887787174097248277412122932797196332303, 59), (104832766622608176177541825540880442321422932308127446313450905901954454632562332446674464940947546946060910949167258934364492062673183512879245590304594325554145916655096130249164474418578747722558044257411366596280565104710641138413176491182723239951406449498837735173589214685822404300602477541299939931439, 1410791933015262172212708647740072680431497874305666496670647280138940379387), (104799155493727983844118831256825588556227521238292468356303634204690259254638506734935376597543528816699271268097390987610845478339350791608019671140525701358514492420459063732526180951569813852866123446884159803178979179618622557184775161941580035038539457326740039174045866028791291522404484596689437749403, 542440931094075743325633012126493439719093174202715004859562013719988653587), (75595957615552494823320410816719625098709408689871427571320764383572985024108663634643361604851956623445106525199760035083912556635632705800365027876110304522635080486620546101422410516435066067431565677200487819302951061691788036511598135148280153006266017468741245985571121459169357039752300694010761501713, 827837335230669656201862781409185009896108489387447338156615014321781850437), (145477261549484887529305310162234479934176389465407734461192839635854877672486275129291488281958212570746232658356692878603331126980124954044372192185429160626337126902086642787917193666966379359030008389172481350900656861622150128624947721047257146923539602529358555711841449189670607399274805063514268467583, 1573), (73691358058999200891676529367808459667346856261441455908566797166847993379331561761424895974856309227331690356733344802428189301925099361556967238939176421504346462160900151974200326025408471141541167250588858137279583271430145065807915504862337017984483618246795698397106242259579490019640996756306121903209, 1647), (122745210779411412930293805216718193364444319144524374004621570256231172839432607960494555184433582703129866302932128472359542519868630335314620274543979897787008427183471398284041779012492644019375919371328182323887228676129707654725525340013233211448738312419674864641579767482012692241068707183824870843383, 174867228128031152272466138412753346948981910879716728724018244566701508519), (100302892774131362117460875562377653763946230360173825784495411861087807708296346048108405462535303057388318662258592689326204854578689047633485626864915854224073093402890844320515805814391103386705041353298248062282396061524017216190415570784674242942910217473964142025141887102941480049513895993631408141523, 1339), (104757897780037901500464427511417918182730554902229409209265046202728184500732300577493023627342192291597262879742194212949863309795828332407158794985992738853171606650577218179867981671143928409398147892830423800320099725021384274069658159741730304279409396602374039720150658914118547080900550226772368601797, 1647890193463762861752678096299118904831050294456476670717633749660593608347), (125664225495130703757942074549832740520919073063578333244378847619479367096660567810045408540962647030804367638540644061959444763654584556320552588830760316293870380217275578847716900449289371263287442654747344963619622103763111825415728599369094031260693174886397339919770389582444189804889947337586208853597, 412234042278760575517271137188070079352873084571525969575602197976194507805), (84847146608611818457248473842466676657713440510900082704570928341596065152658630784975881450532543698387220288553070042027644515728111351318267823822305492682218331807902463295630731794682533272573923817297169649613791178544233174601431246192414417918406727516981781357563470656368802135309511155803788775041, 59), (124690023231595651252566673368638621853160493659922465577947649644342045898701693249192677935472970259958678985675721625355079484914281806987549295819119848598514140558534163455098050114604184955797286229545456657616704070675818400824675310670053251038693362815723959232634636957895967252586564345649269662783, 1999), (94098166623401877252660798893647155830493755365430896289798598350935969762244846453936415881508610837564426833206593325272923318275915303871642603161892878604255180446042171117021384226799831206572114140623546180432450226138287900620475047213160047712624523384864169603595157228257928449444045193954832925827, 1392203600654756747199619014180439712636711802201678066021913539206600672411), (72844316750055104971975722820282601340093297924046073195852207743269874481412471531097525689107542170699083749746859426662876971297101320996829961981311108712857944431777723909276718054306351743851789052682814019592246591290950433775763976355354068853300028816383293035465698233192502427227746688703594448123, 1853), (114694664726281876999119840893678987333415214103871160750339546819580321390828483375050799934721962959348030028663138606948602974885886312842857243901098516948136835172943951001445666733876844517178639062396713849968839608525674158853411308502303930962780765125418815959472765877350004844958796423829984962173, 1289), (116999870400285021274246091112112839884563223213608426925393236413998299459591254134349493870655338836293009931836373101651646051614243361300459670430163184018280851217473388162891315917340697948724707247847744727384120260838241410987023984649738351008841867112755233077827767271493789925532451621508106011983, 1367), (89562287449711521807420538347469225015330426411883071735612187629426511741448537643401959493248011273412951961152695954975373335954018855037282935207877747305898055172232146115464336784421347647535519491870434796059644264780518960899295508940362292257399400478092113881094984036077795828435629226784940142691, 1357987467313935089808780049407130093755256588437202002902124240387735720613), (142141779430592679121788594587759650310759274915443993823548345090264498092535375636204614291533038683479266990490926842556162959906472568685930424601181167787275758874150208199408736685945711531484411333338465514519910689856397928144918262892854505017703786138016132842652893726508465687124867574867583409893, 1626105076730669349785955275037855075662002698012164594380915388854796973635), (100620668248042150458270968245909285523448129400202603235321814974872720104626889896123735548117156546240716607558004266964218565887834567138139811956665990921553186315609062792902741565178635168647758388681319136236822682968447639412468183154739722571496157689485766634300164383257039239807196237477390712531, 1657), (129967233151055305921233172643968775264126176518759911773566341163985000930813189408895837542764878547792640604194564620784703516941820322123563632733115445758780933615469749543510949778399014464991620308992942191896496009919235081045601037849372527599797081803758596351460985257012628050092151193755360540903, 1637), (71656037467897977104084150243123012498177418017539417350590478905018029365499171175269725399957001807496051979367731269002798159651387740924490123485881720428305120264346016706179485159257094371612238367184564899042999448789210398235432549935566718996850699158235956255338482377419203879885975762861886548371, 1407038723812328532298854724220016197816142040933339364356353514051296732541), (65391055504638802434097812760687011934110427019118189038885111411984670061699419129025527400314102201318584979917775952496882646497148783521652888771034187914670035552962433313047338506330881600894947271457124803393060374593391184764744255321291935639077308897042695393518184751857497961630979571022680055643, 820077111437293697233018387756671612086310113343706184267617102255378394057), (101945546452811347098326434567445997566490491865949941874337060081270890365049882966821457183695214114535565045982503486564072100548352696530842952497336298445737867809003192312951657785440784034822082337869348681975193330791584183046630103093709160812705796416050193968829867601081372153236814554573563616917, 1521), (96898589725202647950765130968571742406751257097100146667538712919815280908848871101278991958873122872733615836953264731922133091478131688879821130945450405973443697448053152518479644085904893578539420014778169107892397442310988073714483711242549227812667873992601073571600422999950022524414960873555323209347, 1877), (138676414275377881219319626634600417961947096491002437425543890288525106040185088220775072566660549962085709070635970040099620097218634344654327150585578053124151041983107059154092150904358589923714515088116318692488333612955803890905193092382310181586024578763095874126878204670701166024173683760751450496811, 1531), (104899643159219728857809025246406877170924179481385522564811848694047222527780568657832416758281436448767123535721463093149177932929351884526352404147521454319201339081492640225924199143578537027876495186901534925357817976658012786173209313254343217484611272400601322737197775483624632823116300162169600509117, 1633), (54483456244921480432493548093468228409988596917128531353260111541094636713595657694132610072397419952698291776158405801167496281530371987256810866643134682959932383191158531641404087516808692067517466394077012776426614793433172823332486758012643470030402795015415253031001070134562675093072113013977562369813, 993500921594373160333204340207125394220649284284508262574839358560222590271), (100497970919352544086249554937621930246072793384532394060902354450143393366084347879587025589505945555751268914861991005502876332768547472409184606192660019609816547053698826722775874642965530133466556624762349314968450640045195519232524070745836720158251168924523217942165852121575401842890081344705799585377, 1358193450029729609333647983408432368648438636214903742368956001942791567147), (93536411377822255591924989056817158079962982512145656661313685881866548870744356927117106871940125315915706479037295770575670352966162498916973618191571442445592827798596149567241320683790816375230527979312616282195590248121239555902293913974910076354138398525541859463085063872165275857440092273075998892541, 342410806367813055670175875467883518379409126551927027625150788552387898443), (96721234448764048847377158291482351553605780791584934602950672338651149548060576387081478628207280563861417577268214261522332658812577629074873778633180935090400223551004562011746793641935062862109641725639255802655635096698698018581088819246956983306465500287589598052262012462044688665444221394808921111197, 1585), (137061060369475654651368194264183199201667359497299400847341374444819640822264630516304337971290451527499466795824559739437184094103609323438438176223589490228555792681308783150315876558781814766344723119590981133254250481245567412933198588172677332115664024482733175427285967874315251772421072385464209130299, 1603), (64321962808688906369354895275630054018215572026125172096868804352453196903820680579695480082396129390810053232714445754311778268654483787173091551358253286840983530995530770902367502810625315921170307590580189096211530683064774317030530431934252875570069072789338526127666360599926549255054745541906920888461, 747786462387999766223025246858050506781456685513772812715962426406104659015), (126901246422868089018115203323167551601289218312858835918984264249024463092104991681841543755632638977005817780625158459405377899556128996835561064012215212775223799669582444379502307832216273078624304595413517954775930523465756616733164533008886747550149087377729884901131598925438079238828233306822381429733, 1305955237695821349887077203822478775475684466930720062975401024185074163759), (65242686263626210106727385202754559011790331318849725731528606620168096583302853588315577764926873888886805599308268239565646594788701775895588313901567204303001911515761306992728812226029596580316682353417957540220962875736222916254429001802639570224209845135688533860875488482945328070642334821433166495271, 39259502423724571578532637432487909439578263587564135690711248734743190683), (126727263391575590010212922930060897947044438293792814813939873328395809535376476060543289931411306092008118916928569722875542610857153533073606687537119762180095404055623562642940400180726337980067454853013032864783198161162661094988129917508743936476917900398686938610865341843537435553042541016305870815199, 1853), (89754891959361521660052287092489137422482952336036962916447420861425554041639358741257280430737973019527527280510607840592107001063896720452057688828885970774575581130934038661410142758328635947567110849662407504926851723344728292752009321779507632191567891294711990670701875813132993740062398396303520012881, 1061), (151848915566072743743519107730412530191752321608452950957590509626746802129193134718074522878759428117520962666832089921036437936526319810018773581272748922424943023910805698865305362910919722835982237099811810535243116136965782456478776812001119091098994551704973169220540431828283167203752960208399441060641, 1504548366302665158726356640956022792362325858057863498797207227701281885529), (89136142685377978845520227338796814245486148122862206192943190960305259553248962251722976778554682399037722494523352928382644045332963849405165100430579854367797827134381713243398177933932724059213030558297003103070853979517296678885118704648758089215753898059515792687212566901190915586342830335762629422223, 2021), (102808575120391266699665252986377111023059283104503658522292918310442047716443959463638785569348089353015756730365955134817223846763796234150507866435463202945408303490754983000511158401585773452319170982607881395879696568971211540780134813510233043821037607522692595617024816447170782143298675842373108054151, 1957), (94816267596118570360338455986647734461831716399320959182480355952598193003629859668995180013999844991575991212670783105597805637570958741130700202868653994430579646501578821005191421614932106566108000861647514844904871742941590528454827993117312075424228990702177275575693680794078659003322151855099246244393, 1481123562918794477211868061269289814721809829810417538954476858364496976281), (117447353384822035569295912318903048668043714080465227331937890567830668097494909309623683795188885859734230279688232267229889791999043364585406842363168655575383641894123133751168353519366768851408750348738885912915539211210323497453074406588796471800168442476309280711070098805806234271648504705993486865463, 1717417973835567615986884603204887263507356225858310519817861955306388026891), (126879024269083978177543915811371533352139149516306156587708829174780854197605302542166925406778423502880636269423700887993825700494648580421893716412597249734208179237062202125050157915121479587497651833725667629871897169898134147996471686054538893983959980056512448801696548671522286808118861831669806317969, 361607325797230386900752113836086542562053022185400659583339949099958369433), (128349189595557011841059036085234856329702637107217482614781792131324365726840180839961288428245948986088505756116787643608692891823751226622160875428606910499298752106052909017558273621402613220935017505925292504636011593089999260690569985720378959440323381714352986984564291101392544204572861829908042183119, 1109), (139935689855386270634026366663949550516945758724716994908295183674375766815338604940556021446337092169404350808539313618348197311827259499249146977909669186089458996557061424834334773856486156797004543346067502479675080844574249110379432006435420679857749867601605895196667275615565497715701264293109795678901, 1573), (61164298107642731513770723769587673239944688869221742997969034426780804703322585539788828856478533272958943072829471889606037670611266601817248286579101288976543411630059353462148646241572880522717320128526275459101370035744267431613523240157790972069808374649141535048105320388691883196076931590437633755687, 440142189515969911699757285657441072993863130203949704928645295015378765549), (150498679430413956983248706558763730688207399284325220277830669821508421420736021520857570133366407488784871967786740392440943534712098136496334813304958221887574232887316139397483240850427513088065043120245693895370062460109846881293167074991004548168015412395445331470301644304886014209515009283072741145151, 1513), (89083661403613027013599841947264289912374704847056416294535536717621380792129133523913244976160873514629628732180418810278290820889777202790774265700088768970877074189801283891033483876153539797500071789421814104028965016144910746267042983544899000362760885116733458614944553327220473529382977352065556672987, 1246318716731537218902218036262073045305609656344488727190374493340684264191), (106354132436588500236473797184722502048030843103061878233804557613352252223259047213315217042457605666418677753650359083604490288904759844092969531801529568926714665441603773584409473784964377364553145835194042269688757865072634703176826356927853577532657002521136296931158115085042339560381963244274141177347, 756028310286594640816021212819279575608718890150698452149463132798925877307), (53644959584615824161627366055189896455879529222414057705662011113395904444751493101257211107122246784515285677708040501154499516815784799927081112733197977883589649162066627208795729266537346306025054196779004578726826808516419265997268485652499811757069070805896466025400091638885585751940452336042514726773, 1526185770648375785613985618632511779564652314754776208508143675970350587339), (69453479062455971676735187925511669054888677854702069403964100357212914792543468938099036695453957020341044759303708638594777162562789609365093325961132654194533924549261382495547270774806072761846539117887898803512821660587196140916976998951325621302881899811775046080162446003976552853874449473945315635257, 1020839158752186153824587205383026253413117280817720840391875810313848264873), (120996118076802444313043694185135647154082958045340831680081036018105027546751720084687683924642324982834462700575989661794136025283644616681845141072760183095838351117287657184641282394095220775080746694188344985058518605102166562728138699146506721387304799397412028743869462036502375378732798998001086236613, 1366019940481864026080129008663110484013798815649820740244458319891062317339), (84957823397409676944431944586383531678938262350703242345403096684526716097591604264788530740692238843513389619622084651091718615081304895626070251536143957596746514053856932968442734935326608158897887301576712928953113183106279409896887674074353581474030051957429984945148447420015984104588197481807927880571, 148430021148171320137933935574521240308027687618540243853678907858165038877), (133324695229777132535819369777756177225152961652415479376925632597975461805424674667154883462482114405292746032403291620981646902800511334685964751912095059922293980323007172873243543864726925677775005015528689985761311559770501342562836973194932657308704658257531888856784119405427144395120035992479026619239, 21438471909862170910043757471699521275558275830166589678834636149188843335), (97109525602478252461649046403309028545412695055400169183826532736707219483755435688788604579648083268569097456770723821760803753173630223480355569099991966108897788480051875662004720951657543598423856571070804779667762458580974643869223400463594646797057662869370285250593968928010117827463830689978572548011, 59), (67561110434990452635426720167308588006950740217257673763066652982595035292261676533665271679248118490078481898443838101163530109240116852282811447394446957080983851391700882799065871389642976911656305142843648668331486265288958017606883279638923215862559708473193217953517471382526791033461076839720035001719, 174181599725176893130881587841604026821529912029673929542655904563558163095), (107391975934507125240613723725367225986481040106129708287430925001482718015277556752014426434052809683395522478460093033653552262720090397942378190846708012363034846044906784736968685525699115044328164184210089337177138926723872511818546430022394743481915359183739567888478994979699519311577152401884585454657, 1668321804381842606973891945213954260553244833464423322655741598660662024021), (96103698698993925970312773519525532220419545145752272811238827594474662531641984486151352505219613473686544546972949908468023991236174901553149201901336186995037998964220996315389454321419493089327652864544655509546551428331940819376169700431866113090966258649692137720402284167018564867807809192853738199129, 59), (103012106485486063401526291941717256060555618381555502654517841814883874450856859756492655620701130328673663911338596122437948699699731429830500230559540624136476954270773676696777680616410589037200924603896982264942992556338217937126376402122454865244113982880352909471210621221142704929236854378944109977111, 383212795240651093956241040060715271959785152506352201217897500125199776851), (72316458617167356894497615368133208296198110227743729671868244338345257541916055081751801267384785739795146617346672428798113760878825344034362860641658375613040659670729342520912330988186227585608226045628513809124819618619577306629774033950489884603021142922684710888390932820780374933139637333768503132603, 1179), (123976257831939214977564492714582475110023412372775669296297903968512832885794151322081190298890278617377117532672647195848923386816306176330673151994474401534542324148855962892400746220339371618398176541402904786089745928029050674492100716468803839227262942355666295810565934978096874583114369277465904606157, 876567443104584153497509401843842353135824641439015564369988934025499228775), (60151835145367773854445477585870644151696731416058918554110344947980514741177618019734329912026988071671395842697313313805845359002159428412846492615556464576576986561338488374644427468506134008986393676556525629045607272645886292759664093372604391901651635324922992669568083035666502129302669811363792993063, 1306161585920384067758632877522793062738845368119628117394792062123848837869), (105090580282312265316788416991550306906149116081574290528947297196711687749302689473376851923396090639644366848949999841624339715460171992718632734196460594224684476508102414788777440384869207950826668045394843613290900664946397585116329135763026563648779910172048309997420022998911436978959300520926670180189, 923503694185131167203015855457815095816588560668140705905758031692241177239), (83902513794352976868516450017788068196175121241620681672536389072469656098520636416214528575272413142944360523146986248571531633835321467321953190708592146797547040482893775530953128147610388779213189398983382400342726630148211870037186313372250363814353896083564414875682487596411272590842403561258608858579, 1749), (105651502592379887640209487913952391867210877250978196264491351928248252596195037796789912313883209522767281371921370302943484015645988167072040054099673652213738655765840477735058865211038279672696933869084011310749838849875856784417781311959863906996893137272429429802789356464973424170611938767034648733851, 1195), (70280868498855309165375442100983675834724590634005900966658705514930518584768217331411250796178389265677464281224353010395812317577221313772724244549438436520556019819436552920854395971133512988049132249959371531055559135615847272070094339233278590846788719395631002964040229416475879130893777917981555586411, 1115), (99858453066230376743058278366266297692962015672674369245211163071551202822736506737606329366735913466498064735465798391428106229156267497947674222947182919717200931925107852710346325743406907485485625508113274178021176782297638118626142350215063392204011903047239850228128390973607410573782339270989391628683, 1080774603917587790058591559307064323999795922772574859937993376964986745151), (59881976321343043244175786033833718912123901766584132441598142473000993654516738312815252021989132296955022548728047483648205644585225654292154206606975931140016848656697317298601646260112160485063579716895805874021440428303334392491852503574610058757120280172337817388763492092123161398733649712803616298157, 1381), (96948219576821103502717567190285055465984507717160164296873778196837859685466564202416829211226710222559707584063378064525049939280740207149983469587922924153721901536655235304308134409728110675651041084128637031646981870045324241032429647761566974090600800468254373597366387422502169443275755897109166372443, 596711694131082003334146243932179201398605686330695361834872645837497850119), (100928202847808372573437511012253577162147894380632254792749622044535023033995432622160614650824280193643738919292111258795569056489031072685793307189150346926191772108183352616953531885045530184721622237246882108116653017124935491792041932417701016264117683387077222146154319691344560184562085984354066852607, 1395142474889736174042586138738940529664355994438196388771220882069217081903), (96458789483653336992068646116772856506180123609123137664734579255151845494433407091233610988465923278921916097872172010514646486853096011381108785249958449222479989000120214027023528604074086809230129869304765108651715409208416298279541563534000250477554843237067329709748869896307066967133992060999240111387, 1599731598347508069690075475825440851785771843669034067860087741848510566991), (78726052397220504826365709463386746860323790785595291024557277045276657205541625158593808508973902330421637626307430925285897791334348066942966608041833480971585518694821010473390205343946579760748443620332997233757416451929898512644914255186808677955904854896425121319704608033357262663826531966175081088081, 864772282764745616751826706303852675379672283584414942946134165058009404147), (109437023631496050068370260847813538397819577114895940690893464289545584692535302489622907215136519762459093440622605502824775243889794174099933322579356709929575648291051727038865438911755028442437736823876532522137083677955485385627073254674332027084707667515528032448353669915564144654364321385684049727641, 59), (79048735402265698875259885185552425415800746845056888218577854967694501986343574648264535612417798379749379323145919635860275066326632970901231393829187510709989817630669785703080524723555157389125934337451419163116220341135798992933408027850042846446471268553053154437364600968425826723472431317060674217939, 1093), (93848463382579003732427533500728261432753693186491793418623680862892407779728644230601503429514958783016498304544573115263119004847761564636170524956125135296943752619419593334122278208257797159965124838030095879052385398942631040670714950801566731935690234306619066282103143218103301679498375685787301914199, 2043), (80688148078389732421272867903743121527896926490191945341107309533305385515111942836605207490898393836970198747652871182150949202569587155719552991022094572502630127597209745251272424975219297030756012085996687144974330852675816028296256522395656066469935378454178941018560539317302719659172098931249874647863, 1723726673396744625945955251142367708568579670875950853818650407981027155221), (72082427107204039015388727465120729307256714120265820796429898983456362469462888011486044835727926002434964140893224729569618431005628254535409361121805037193900920551005790736659567861903807530043831775628559466349185456752827662611919820042492412952589465209545469468777931248262563975950687741299945338169, 1196130179636434737041131530328071462847532578989228596373889455266393973511), (91491180536925976623350534699677594099469458471250781541824702559080057830404732910356913145477828161705085193487890749278908159959245423870620987918267531686501563432019324187321930935684354460852699130952762122934840331531891084728117158051356954243149846863369785008358528457614871420828040574829744052773, 481798901290325111219909273277868390262951629318387875520835826648287603779), (51910045286504282727687055614026644425545043643049487903103394324849000529369845484599671113023635700823406702120783422363969404179511396160931536224188702765307419764472095120219421422185788438761324395619270580846617025420545275655067743853202050297838626974939813767392163688651143806729166505076999474343, 1217), (113208842012537283413508909873145906397873017459050623200777500193417315305590412533941591204950192950341774404005951398143661914516154933703374246702053915141679836861163806246497461229389776014030936971443087549169792240312548756604230786852499522557401898304384166304633255517686393879855148458764808194201, 794996693869771320650055461120150058963230974744395103466694628488735661897), (121992073661537134837908817668480104560276539024846576881231403088169596233155709283583111573318080020651050600125242564025642356558670772272828576946974497494179221235267287139436496416381486276484912626321794267983112748938731106953292549793377488883813256598387170409749810006710081972401777479202737723867, 643750116903929545836709695473822324693168052665455310850561346862451545927), (91696886928377891828123330720006613228146403851597403358413841924056019191316799319315366911702691335992610083289478343614174684050673780029842853024834205619681186681669491738263674185411436837757516632626989245040365344216510614754046774047842428273723145280813335641784747950303169570220585932005369343801, 521683434145576448871583781245471807880291289968747294778975623158453972985), (93972667125333595006649425046351441061118432549171022744360474583258410519453868403851882677359542863352667697023332843633132684663968545647527859946425207844506571422415428535846781280253397543971913562121023149349981757570356329117831499134795092226498381305607017732197027757099398519795854583307847986853, 1706187933435437294186276315848773750846869679510770693036284334103671609621), (65890521891694297607085134609662512577468370108978375902021588352541947806739534430986089553605291232282618021432295805538453802767658884517046692545544435482381240622746628336233471260619133194515848697508668520577488604006704868313610415964811225631702027035250080758710778064959705351910741234886842024857, 862259529214886652666717434841716682071327465451594309378675608182434235803), (106113943964990433961069077663875007035881374585170206883865786305058875366544961221504923149977043463769089624697918743963890260041937881400417476492334355225856538045528515491748769151433130423527431379972780924440555530441431886920550427581804455283780137134177087810437658281054874579502982089761176773399, 1165), (110882830475571694705335251896399539663912263036998163181876187477101235584355159488183567931017644106815055431695197371895967328986422417732119927520172390776969714253383553228096164433960117288210354416983535808563895048260224514526036233501245897939393053359345312528260918170868095441670681247138663718727, 857098549579605120655471987348787384864233145735638296998097385166551867511), (88245887586294244520497006203390648975631780556798237802079549964054938978379200204594886215303866419912024246847078515961464309014596537953910596119467088805997264595592482965301929585072291150794587610073628322812684780975328036355227759658136143765594614476295991391142326724180210378490627107267911710307, 1009859651365250032549018157222292254085499095774535656172922245930057785757), (81235446332467439957057892260011867402438285882256162890065554746235546888013074273246876921019329133725652738131702021358235696574228022430992555795633708040914426933068528412238825564217903648648675164656862125828747711354575637408598060203713452010933340514402746862826097169234163355705739976459106329647, 1559), (63056493558811019498801708002255444422310981375675221618893318351492470685432131701055668287925535591830950663563659858256033637310234311134830271334174098127764994734221753972427928877690266586314544750606601478592886770053542878894260455797034224602182120970722690227930453830611497807800997439343663013373, 1717), (142785955380251281509319716386579412021331776028432172889540471470147326069612558250372209857731277536499957608909611589861452595804486489564413557955828479162684813782484697713598828915989777229224163048910245324257196238631065387822165661278431681021419184528554436564630843406263186996925084001787825841833, 1269417019481459080449588453905541776676597577105158329566695400143671248221), (94395870384955299988176794848061765345018951724458227896725831475071962273254047773447774173178745774571663307225761790022132658367224807883573051761272980408657070962607326482282835448587677490732158419044610575771204686468792286899325103114484631697825147116227241331240713518055047194541302577457250995639, 59), (78834459903289908216252023928947177035231079924914749498900343222832536958612469272661188596160712624268943519703456356889143252949499026576302435061255755109144856683910114579377966163185667025568437736828330459625990938076560538146438678937418859202826280759955763600474286920309540485014839498553980315591, 992868583963367062723913351798175437786713138784025487172094326093962450049), (79594417426874761234908300160219601860721548970854283325056045237054772836131019561430065353107292238112383095233196410179460731248123638038730186938614244116283578221660354103033187486713331221616433087979186519391610334502888936263867538922507072932604067713316909035684752087700425146981445106840175528337, 1626365348471308291180067603798000421799071027691113716735561225168741771399), (126856895100689874006190877386826065955420960035362350253967293585766382561391452242096636043075610233798274821799043959459351640097507055416841354709110972022087107483833816449426785713539596466958289105448779580047054879085433970479036939333794940572078214414803939962990380976352754678845972646051117932621, 33178006475831165748499336657019556464873131464846310608354031300012941383), (71005011309713981738911444307301757380224386547203577422204952756415071290442127509784201292884007513524969679168178078153790535674847892181158860430316724345997187232849751131946204857275270555946583732897828188900448217354050052526845052936104479956785120857098809213247181616422718711659913480890432177653, 1188466770035050414946265187554983444743396944952986722896224994561029664081), (62250719937967491508411053732726149772517057312981195562730993008162548887785225900961677173819329987197284236471892587377924666902877392006932326526432072200547330295464052431196495064922639941165890856497524801279424307560367048326203842069168315723396733251950987433997032844127796565146899185150676892273, 1273), (107622680331647937260790906169963126746857417840982456756833161618178928440878944142147661839817527370777739597741439095156998843412428920215347093150735714784844129334250108251769617128703828776148224075912795380071106163978869676653364989504691435360291501830549857455704418828483332571737527919736396096607, 2033), (131951692138684344721671766850514010337881388513767229365050601047127583387249392298857855676115376285711603679081597763707860519629407641948027213373240177473558944226565462129561748710189989934487597273322337983920140013754411396643875316613528851698161632475729965408503331228831137860193116451173691999231, 59), (84538420343341093181751826404018419102856133804058669860056028835237846776793987417215570966142714068270779170544462507466595622227515788434420660068962164543504208931297938935618825121544445451910549956015083783193546726764106503026264750834065656210091383281617545290354506292481619470541434112702897798703, 1907), (62337123356176355698267625377081889545873348681868700228370037395669071235565886667141738805568717264336502006476164446552671126046811247174408992653314112646586570174699059804981585495255017545958024565009085254230509437893985175472107673048393919761362326341323840230734506698456875743223201898097476045171, 616705767249710353097075195571792644329188651068644799456831174067723854965), (61753727885893231273853975412789984017613345644096862532376085019027408480669121041400798320519412484326914670875365565936492608137025579950580181992092163095095007524118934800065644806596326018606278383046575047653312360172693434018526169550240363282616510934614947801578330406026173958727051279215950470881, 725687098463170748801039454571324601568285208263705464416314956009698334543), (94522598916627338063977540413397624748021097066634429947489807343187563703248872414195390400734321043478632915698772165439325046398103320294176510129591205713230966437320850355940040149358702467634057071817309112379266495940733471842280643419307653431677155537313459207044165943979273183655614606664611490359, 59), (101346987194464430093706776360176827535609733390202154123979562099523098253428638056155733455598651919447855840840418387537763990707088198492807899633393467181319964377107551333610800592498495633509746667277227660248877559266802609126312378870951638397112082393331469072461719281608880664586321060296479112191, 1265), (76487708788035022426136673831345700508810634103736387625006415329794992609412629625919493476232436468735470362150314655735132566179578554686533160366982619969505846725414975357917882649018332896138154794146207787375893711113647249355625201956769547849164950045956495336472364135022992136509755138099563512523, 1775), (149291716064717153304805799137168205060280127854091052450585839801304750812165333744330608179336433117788769081780863127353030885521801307419584026028928654791626297336536787748860325725350054343981335440444200859127748550556555904499947329305112691612979583836217147525930705839240338855979434767417918033231, 689458267871813044745262642754840243358494959882385439783949611189450402983), (82023552646975393567636730037818668548979723305076477549999024532822468005387378896517852074176945718858085046681477939168215363571693659821038008745407014520100741317092394062868108810343286723166534039865445428324374628007188279586088534047654734110664192908752280757429648903281970495061999908090778784259, 1175), (142715388477866012063477089142562152235500603705661902393274131171317723753582823961934197430895087606432344297353105324845857554300189372951963423050267353175972241967488569070217684756006954071932203064151171832207197563937818747689980734143576806683066697480040146583773374187220095512980019539059096526229, 1153), (146328051322440139026890685638617237275210639923946481781878497538933500224278533047669611498024879223746347006407638318242075259497373583653121564830688349133684511493981698481786014507565314563553664745633110631005261090940484445620547524544426864248829304904477845655201147719591550628285871682717906287313, 1756057876070170473476235363035056350491374219670075001410471577252053419891), (116205384290271413878561179461622793329770527322604509650569919797214678429069467221562977458212104567859269089091937998083140433262617269273247777170935940632121777386395390261883661999451388677716806130271789887319883242954444738986111454630918888432789690827709382980177031185483738668532468416227991516583, 1062133016911552315374924961473205089530083990895033169224459307636936230703), (79049955915234271498661458014423514342370240910085380666446996630439273977784562654918603329660419741501259742803714753666550381070211683524859386925267576439256280302158013146839255517420774744594180145585737486999761264417702503074511523120490497108042850977133838704516623005195098263695386270416351284337, 1261), (132987751656294063291295609382921167479374885067307042484156128455551730606847407125930659049284149998017981765361477014597750567771747736358411117291933252776010120227980485706305267503342567954237488817542089816312171455384936597039812223016338760578023896734397390588568139137632798730481986766851518851879, 1363), (90071483025723517392546711470449948110240501743974587469615993995154940279420665955991063487867055168709667815429724291544029280842047049871283128156120604285533242891612223128634220962075828445476192320313080921417575393933545469931531942240842732938038116946588069251851179069585213307301129990240211987937, 1412469360917097080382591163573901771733536707933546496350485007411649483459), (130551587346158221565867937252119127540288712753219713339634008758178790223815645048511671865254291651978347334650798859954391630599852521134402167919462197735896667766973897225678316516448428105426394957926290628142567380844205691688493176207571985342516585044998278845130133776355754022535352248098771479407, 1447), (114461676422422438458436766926789056751963641737399102234196758219409662304074984750672054261343072442757626009095112096333815802427002609845155804088840121304442504481900174210389487516753319185212469123808481992097266863371495403443542129824738570558940921869211083516520303973756828615048057149930535809089, 1083278597981456822667630472597978584022475017216192279807085389389499823801), (105151124832375159881975321001940169698916366619269200551485634438315504276200754007539706910608965666614136504249042768369773363384822734368985383199261387130723050283383389159797093105349355181090755279382191355256576919615748096060018242884085434220407906515277152014158335538576540557150167900867545135073, 574527740997251664015854196266985663905370693051867663791970481271868387671), (85581955621526470600782518815496127371077378324764282379821834927117682627961121060925682901767938701437968298604988617426784767874320992051194180505828339691960037038149162216095308885665775523018563333331363797047467128957042012735043325757494986460104562224747576935263919615010901946801177285795007023481, 1471), (167209084131911437316777391438467433520552833845301645480414075117913739356234275213709173882182773779647247883661041114745116546081055440428763166596687442857788868107122114852318016525198684983156483151259558032136624463085502862749033472418264651420392823647000831986605087707832487420006476279613734383089, 446052368071700278989609848072899970872247236892355851798619756065172871175), (107366226899457365062236175535636522893904990027016833566405418880832798506068286164413623682340433039584363859155735164250219752989531524766369778120614251579190724346955034826779796721855071671298852920956139222779482588174495653647312795529263190489008559875924542354215320735358423774302967506671313322063, 193617744474294560380221733139853145057256351585107198463779462943327547947), (75371568300427130041078193342223833318199079287343349459684400083241339903596025899899561466998653714966265593706725958834812222107854883227680626342311887421419500947369786045178090132365872864580573508418734808192631710994603645876604878239232222180927904327579889434605637429341722186360771231133914512903, 1579), (84957927862220537170355575751833932312384206052456457127314365774826555496174652954585135116029029693081064611412707630178586363036692984920174580791886986423665238737188416238544047997473862365258446318835457262139060019654027692039046299405500089333305070988598394114139515099263786463518470904962239131961, 1387319888256461880311649653733921823444427782518873734712670699994758915997), (81474532253040277287230511881687563384248126901655806574188441148948598705394283299319836130481916932734558555191555367716299927042268583410176132273611231924759164055076120225957780216033895367633435255036942888649049594212285205182879908973576946742097761596400009696953475805121530889939650449222866970189, 178595781621395930907538761362486789841692039868026838593296753363658913439), (82790266767377706323292882591197522110059320725467253859288968370496100610969969669806745265249222456467070068394128925515734777766229639230812922596990006698818332381457869285645494648597869318639057286309770300684640465675386088291776478488233090968981608267466130306356486900391184039234692343391224745503, 1694927280740455438309533549775938868568722703993587427950748506670905342349), (94318197483084660584383093434947708536730451827798623269097756603793465210433327853777144374948598581915359241829006999211477473652449925488869476659175033379653994507107738350138872513955226163640458512496863575215069583876694265156048313867419155360887437359426752171355928874383586126425883000217619792667, 1766062000949459673283497080540198218789774141212060819084364337859479795933), (82746340944919777214092533445722054140109137136160720441382891525263375917391620632450431543185919280759489456203665587032714352581292900647302858103736995612854928169341796220087216071677487604530173453004416154108097834851964231507843494144497200862028056258451891142446570874150655722959972251347555227239, 1081), (92884782096063715160003539085470113562134158598815052958944560329104152766663596383828357816229854939891314646265369043488583948730891860298659175185171236804533841421543495430671905860038947939814394780428573039852765904027766070678083123085200693068713404086401911382462272504994762007618551153105176028379, 1410345844472962632262868354885577201602726139992374880707052788211645524383), (144723101072065772467083402460218015540313374893864883553116740570213576179899204671872454541796549330331754457421916331098708430051102588732422803893143237241438162619032767381387359762287471370410960614155416736332482751304950072147757665014684232044340634200290702609073582877417297963598088443559260906391, 1207520411492692024264993208952107436835409489011763082672711072099754156747), (96096079233759363809987646710145921686688225860761617217533849741349364585439161948207423988240060309894827383214657975600532954486744762200847753733635567613290465780877180166546642109844339799334123137653499632615595068915546053481270586188862281640186470016048229060483058662336056980568599146041943448079, 471354236294305062104424749011287923094365031044285430602108294787265242511), (74394624060997591599106435528980420589355599747917392053913223147181823317744843662084103626691208214537145855398965967422351867433063124210931010396522661112011967254950843613882908031523678938638239002877567910724650502507815189310467074823526571982115297541101068225653507288970172210154255693890135184227, 472857791146222699888422784681936225325780311266571799199923160269497625945), (67167726782884534659336862041672906458164621078619756355245876887904980463131220672516248988310000370649018826005736349273878109695725194066105834128961945621038223192869214692326315800870621722557494678282333319832975931127532161094007635510561654732205961695938032186648156378399139799164502952623770223967, 1166972688289921961419958123546217653470801135450123209431425628560930338263), (93300579299634845050503605064047344420372011449356258005113492057686927992373038581765522617333710496085706580116851357544650234934695805274409542650273702679568425526560865060544471747576314098287544227925149149624416094556193247213463618343926300487740138999265735694372040175229361149913392243941914027539, 568970805196488232274368681026652283056613482785233910417011837431642246019), (100521059986247491927827142751569819106484282787625527869046510785539845428135675443831908796726339444230517073909319252557571232469903160879174348813778588628778688035728129849750467006450939999550342995552741764109125390254989763640728581214090108810502907178105826883183984238601521082804171490843908854247, 339465037533773774400144497544797970856118182762536747827223386275608073213), (64218533404129775383659848448709944454696465554897754333101510059475926815557706357013842813568970548138958217319522940146882700457517081277827818755505709498927407363250733472619540375887809035728389012600689616880245208059430913514592487215502598202901307677513688056194475179128330093839185402357374523327, 1470622864779064436839745110412653330805443188231139812454364704324877375659), (96436441568353970354103486610051119751535024731846458503363260643144710059762029081087362045445127449787615364322784795946397066008732575947101867402598010281032807793851925101688202052360140143614631565475320837684079962828405385983600608468576065859753082178781255992963249753908977788109668383056601502211, 389595317599491370399282676172724190603370974207987186225609685654657588487), (126116471444618361580309356683506288051177105268112544464998624671111916316543509631416493757690098800687503495603324119903663551781186624946674140357600289814578422111968651412856856273248950519148659647001370728617439781430286888482318010968851718007473411469379493956648819001666177962120080242065460984403, 275282234672092885601768108936417958391946773292123808447325373170461932449), (102132932189552497740023921909242507643574252225547521609344094068446146367611135708804941141556338320898818838549728511472129047674484383927354945511004880540999317183328442791975959093257126995328571544481466535779589426700642038587596093803197167520505795329630866267798880280259315506928667064180034386819, 1631), (109927332393762486143674418507654365214719068128083601290537719235804586611665530141404427700787007234040989493045110612227559977747149090094180465741495148803101586046143597722342786133854369347942725673676425077737471625118136157069960621203250130719698563817612259634737522741455318189164180639836367541713, 1523), (80268986700114738404105433585844385582103140520541550054964558836844550463868896685172143067587335881082361857682725579823789254228987492178616419640748083920989853972369407207778829013837113151891021475593343867400724951939526728874455413856089744690381134228371943037304437847708104524089762875156438855877, 1649), (79738145530577574459172143652377259252364704698903881861961484522074114668959611539147941324348308445063833613767460414260263708791475904868128377883875189164475444377835347271069168994454095098757344900796994440023841994146578552462481717380861903359453598270769342070255406010172202536829269047430201148629, 1281), (53406521889046243624452883975586023225474284865402383203190863356981346715736661566796861865693036472707889024118239773654698035882451856023687976068533917041238405173424454181276497664393348279995876852477756196911541823026905732515261553350583076421661447251284233568561578372831907382083215642952753602879, 836881608892005913592253338369977929964056088859411561134248869799393270849), (75824810637536873338534441275851936794611099828375267250442355684356632691090614064113870230861110845974326554123148614377301255942937133622468335477426760983592917027155299141460942066946823050369412957439232595530911785469259521957501932852871978477242103980585797641267642014101081266021617191661064890411, 1613284669043706482013825528010900140113540668188033687039517353513919814797), (118822344595937290678634938949486937850464938908900447591235227454191229485369074464425887649388475092053008151634698088551949133393333465575821290820324505025748732263700588777241879705761683720839095463463197700400050847114990198851794173686915341363489886459202484549308362433519074265381095138241695091391, 892632252596578205061378125775841151914134262082946398235771045869022631255), (51917846481510650514260374914528594310385019917310935781243726364065851877749149377864402453346988591451784149299047933636368931520221417813211298386476217476434323413515381235885087441867539649608931737323785162500422658058282796579614267878316404696142378154308274525239586736951506871859469960758450230287, 1393), (175629716645204484665849168939842096964651477548906230036057752393578203860783345361379582006538083255428368893000210401418051819884094721704214466524792909693409685750026642222167747194398829898957613125264699794981446917661346689899484260709161712060558194982818478884008871929887501660485497835089823195609, 1472900370297882300621289103846995802824573965379076878256046708654436873637), (64089166540087337806935290378943141590610202036490926269626595861646424865929930610825088494790248379815926583414676703676150655980060732056675311859343149698854940762846711631974649575277165972730318840469073564152440919993436524456017087956031122094576566803827733539246445092502510322315580483016443904309, 1037), (147893203445690879839780574505088065092057195354550189891117921667938441060470786166030354489611497648810563023397333632712669554491085598014731975260507758250968364353575225556932030231148685063498268201774091977792371416873494226570926233902269660313380422324257385645433924387893821168028323979884894074331, 760819246517883531323549516125491342781056611775621627062741877401911871111), (63638304058576134901481387694538535925896494742371784357583269403198263079236706005549998676296658304454335400979783575338443019898886463857558601649835255101468076985578705213609316041480798663451043006990718840760590857521405641565380250748894712806948460819313604872177616231440134645079278551900796908651, 1627), (61918741098814656630341971202260688065851560966190563290412128100031760532587104013925528366550215777397327331903253523069526115103603084034580060716368803479556809400523978405678222572533971495353723894707294408550976648262221527402533604002655484326956356821986703410783980931890094428189399479902487138723, 204730199570882885704052394829414797070584222696973977064283588913207838169), (105129617942371433369075097482722178002404801344749588395504630222713832727511311210094721745168026920219367136452633415594529495725038880867203622904210610391061662103008748468896575828416286788945792332023808421815800288348480262552809014805483322415021069909165525232725672998495571014911561803466104905531, 1941), (75492465326121101240396816441026617944856096951236840544380569184451054806323955808475277388609963448307084241894817948874223079109948183704062385327857386954381512150734709757341988414879953876379416894151324933587399680669845314098883228112938932123232019159059764695329184444771195849878978301576924494081, 1153), (79213991815621432078508332978554596149498888173250231375473029115760287066915843525579297778939143430227371322637246015466393661493513700002213962168002353294068558361716397362596870372575176502650510557592877178559132163317885828479924736715044060262276833719009427057553753374153049708088890871896708341283, 1613813057521982881444990318121318308061212318268996226282481342628321125723), (105845759090941431568009377132951703165871017887772987794073782045772060265029243207481247919127852990239194869016376655982148228715429613321924896717885479344159060522837663209810718002194298798438622552638434396990286603619380076436124114680386001483470607809298762735469476336543370876023914079112803455217, 1181520787399728296042850807734169118477867497605294380351745917588983223173), (58717826103482625943464784687438933010704962726652205235473173482395624116584588249952114593214776893186856610604115819209025353108286705259805054071896948195342753509138793684106474888012252631227549587451560081340482264258532845052391780643474897720295585152084543930752100566496586150598815962119133409073, 1787), (96863126341954884462833560539789544786349620928919705281652445154016421660234127734380528446552055403636335499489753530567594663771252771473588794035116132561866762556262892102337214798349374374105652314937275751244837840531581929554351575366035111123491165227562843968016684736644849817981265400050955096423, 1433), (147061788896198022987101385989605633018877200586560827841750349755829823598684159290029988418186488220602274521498706617928479346231862154258777744945668517784456784029719809066717851390353403355948031601606004179157013307244890075794389271495781840316691518165727484451381402733232386010655189911993240942967, 14393665997376306167822620735188894942082266614311090150831983431221666027), (69032196310713969819309949498190223178727105047439912336818199038062082716038385677560771007796147919018777267764261899826701543111535277137069819039581347141713332495068532348360019417549511030781476885424964590859914977288944762940548747676789472505151184364424993950511387810232996540236623496579340962441, 365509985262695706056441601434567467266078107042046350447057018123327515605), (110757218716470933728163932474224920927722979434083514331423373619327628516892279826830604340869702434555704167438650835974793674341847449707266546544020641194709088789594781420514857907713347662104072251627756674980751256009155942269233389879625806145811991684496454032605179184632567654374016363537368920727, 1555759159134260050626801272614925709844903756488809209306586836031584580213), (100508139663924167103914022054378124304812672914021239852158918626190649583322296340600256376473895803127662710774202875526699878455288890698855445118810807677771417011851400867944689556941230999369974676135511377107502934664374975733666668820514455793130438957167022253080095384807145952059672581549198420931, 843820010742322162042206007816189391354545414868366298731357268133422665459), (126953822735410828740605917693533449750462665424899616254660301472014772101462589018917917436477783335058396158092544808922427402109204312657374525640642853246963416660879465348784942830503675261948052567833676546871733699658246854670055459738588160510441003244874073656943851858536207162872734353797804662991, 537955014022419511342552872994987645246727604728472917919814523114423659473), (56378121263782595494192019358761214648097839000315083865257568365571448886561053778890378067869772090487578811486650247620071358525210324517196765405745631261549857744658147989507117415266672451087233088737052880609230274205883894818454893854031428101776850097607772179413709176041938622666550885153957362053, 1291), (99042277816917755248544881765494736067853022489880513917643875795828367980157972382231556904206510322094630848757050564546477021218945863700485447521388536693089511253498548548700434882769209294759132135876972082749880645934142180463206614928173782942502688514986676683395250724051783929896593135922615002191, 646032339365626933389837017564342545442726648866436555748769982745946026745), (80915106021131531645678761130037185952306503844842994713164741229233014328611895716527936331849130875062219042086882482866170401280063714053785074760902010236576911779580004092051554788952447855018533178194248856795732062472760554115252535794922735341051268758878307907570486100199008148959370639466062427103, 59), (63882019570661796020060812289108142248267397073987380497628059028767215345232661915072957819177221863060665095837058746508931884189073297534452257815317079341835422317621263818641753496967021254538440201418809942078086151336287353294455483125207186602815582763511838985747648838516106050626534259117810499439, 609380771260546414418446551791467324220422573540696778933631589011389878559), (99632593121809383851188171427642174667727828143496457371134991140639336226577190156745929265930484235292876470609588928655858870146192018842387792561296567868608564082624553423803926222989297212330014673290839370078891647696665112593824416110445378827931533387142115447910944414201125410947100737848422472157, 59), (146530062040604954672602390082042996355848385778446773049693376958365091100637018097132769757891484591834850782171895441976188079059095024375280224074141828978634999947359814290201607346692035308455025457222997384290208898355247414428963312405067487254451150736648266460587064857191576656091618560278490375863, 1296288333282482526170731105154093425391632258591305638791218539252437336225), (80837943651368975246397299285433813372455781936425615880117128764503751944972568460626883234281008145059706631167268774790915505811814310712404530140189067590266716057912612341905286608701087856089711918863811182207762171059696774955493241925515120019381967631923259654397398125821666826932911106896428043577, 62058737019815514854451149174105657091414475564331571155991340170223364543), (120932900669874719707857275731958083001651785497422899493183372129516687047229713473926680039603096271739673954330495115672390081168470110999561873790727366645752913030497547259293524824881094423017909987229413444386608226039795200379240174432293858907937799512289326789222973244442828926216314580020481254257, 572216418220500321541415765896134185494188433572532656677370697694525332309), (97480186103857642657746477868353034017868564576802089336628331657676689971782063664566369055629080365299979055640044730428468811984445874017425770489461084350226793426616806980832190827258334031993685876648189111990965136866292130089374509973488222215650482638402149024424825078413141227326908787884375057617, 1259), (136715018291406659724598342830256269885217556024820161006436133644201412631222198505477727954292048780316756761073904267464423394914363237236851302483252180823579626908586335532544157230707110847643351320394009290074879429567107854780849644922428823239246239561745038083681597214839787071451887844787637417787, 452045084289224048592206105372941994519124521949928133176522205658240099137), (68103118557459920160300036218836292752033445490972777130149963053665765143673159184061352542124597095311148283152353685836724480145146483698401487982048579156643235406896550652131844374455909436006581556687061779621157003401450439556890169664245917475386064936273717608546266429827327568973413597194072505449, 536430357864994386842745780142055314093933760922029235841179406268015003417), (56333454340712849943414229424845329881483117457034573499878921819792850330397355149753328409180571352034634638847571444021674809198687424551545575993632070707696625981083090977833009538738295266607834168088809165758132420499999435456135286173762326296645923045236714864768870607273186632771177040055349581821, 1637), (139409367218074787666117370880345626140894770502570904421172622316256185824146169049126291305700433528051825952726179986171578372659372777582182908207079084736437486577414300062051970221902853064028801643489632550589346754511986756454579743810119911933054955484379988155866452551906385163431327014360389398407, 59), (105965675682131152068892749825957105174973175397214939920528124447153568419910506039334228783090688511080095529622638748001960947166045993900220808672909275385731313561369047911643688090480474700373054465574285560543572767544320234123970014429522603958780764121857704024765710286157017403847789457686402938223, 1121951390651328524013946329456172940739058469393291562818320509223245245641), (105643309704158656744389690331773656771707576855770257433499719581922171631769835053918009861544593755415415300505358600571689860700386622991927029047159042136984937005460505329017699481489353157364891729324564182617841053152132855147124855442609093100451755322710759310851109309504196014894823974837714256877, 1292147247569074068964617297703857074436332392164545563054608975545569070989), (145233502681727420301231200134618128798430142378715103152039503470585355508587201481992068160095154683089655551628483617104336609969034539760825533647264184286431805445795333476648702671522709228021240623676404182113809546753296198163900387306774638337737603530468817242788026484600640295295848669776037730699, 697541630212283384474503683988194146905535547150313369247702782324636434437), (120130935205284013429035231861428922198960543186488983601864922025683407820688712119353226546502838849541988572550061962342754866576772331040489188754403859921629174138697745171721996528828412734264622034508456825107795151550321908708761731694688518469238499372353910656950454638086165412286717303582680406823, 1040062267079645537707525505732177794092187691168158691223192340196696347291), (91989744683924495056190354453805404308788592367111777345228841730370808000687674071789790297940891896739510139987276575728440357168617205254332431096909203944456958904952964941377007830837100318045984074463158113825127110883858573884192169188695580673458090416797136198456278147646559472246716978900314297033, 1632158361414214828625324031285212103075909919600075816151751245983873896449), (70558138626180494544340240922430452894576200054073486824887625403849599116160727085864097608892820600812656490424221495761960157292806762008178055314084434560376778994628384021356166817401624138133988211606547969452753662560482733061758899422662312865112230484142010294969190899518338673810434822061402706809, 1193), (139494014148046679730350304842336402570516363882837771511924761367933871793664835165140862724631714729008307570947256084217045877985534124422880642143257243844621096880420735992271994101831121049808510361890931097116926896790859590869871275301889822616898252542103602208965584141023973102114326705347575925559, 650642198275734239951922462767503400018456820834316585330520737071796095583), (54045961581162799589512578400981792109688127632249054616744753330752268442346712195683077019171401970659071095622418500765451659003535967561473519380101376050698869604755225134621632492243765327071559899131469761344714791302486144438684886608904550346305485162167172332934426369969151102613864819160107820499, 1933), (83853616713806513624072879433263770792626397612510384584050809039885239424181356226830749436854350677748252165138172241215193169333066373756405451726163296700347347426528074120701353963464904640130485345428764306241667085149127700165982267869910780275132906932217877363192884846400096122251191355975096774833, 1223790439847966060130396893229451077725925098075155658525751548115892138181), (98801674955229346282642514129249092633862040899487735297507740963940526341290780630653556605212182645097495719636403756864547063363701445958040155887392596808698224957995303505627263743483793090258262802517009754087775424850813521368118558275173190942754524286438743308774738182839205979676363875462918405357, 1175088624158542541550744889760479597006124024116143534378872006376485080225), (93709585582831594873682978453567517165589254777702696678343480523627846643471502915876953612504125542978616169199116784495927718867970687611043699090452826643712947010704219727867830998296733833798420248882540872059584223907703790013157157637203217851477846000645673682670499074281424129499103781520320799487, 1472919106587610447746526483369920604119035113698479649123543684103859033369), (141045031214278678856620420896943772192541491342156065670231754259716823135120309941604545715934691500346973335344632760886860776827007936997423597512569834497931529755058905283788767284309232823463293036169492093972174349842697535855244976875887963411074444445957466442941593375214501852988321016566486637187, 724724093942330681966299435907330001659980201457077784834645288958222412189), (63419513952676392722017402977801663978501869052127021128403109127402993843101263055822655235371997449200118092339302457451347652914029991725017586316803809901928817448665953755157639435583783940613934811086469370510964380326877995363689012163910092992750147862177403690897034906521537225801708299929118437727, 314600664510971907489941936433300523589862685325419929472289997333920122771), (130491648446714263329577180058702972598922687240782484277655841989866151691855134774702229899446707715992299084559582549644868999095067497284773495556999225118916709953326479135539600459077317367701591503831024036459085403820459697555885128420156950975710582635139607908460733265292862792216992788678967509477, 481977482735964564411250742732635916706073455977860663014998033838614282293), (170698977607471801730406554079049751623863271480723803347797843107631878942356804272523460320006666297569762151612593491364493589361988012435096887384056799624496998893220772842116224721652119844232612268291045127042925242583122231927380021445013748273394984567049542397016884776361861674783474926224575154669, 1789), (104148587311618957642291524219980604028090666627721345271797923679764217726394659475038439139258608595754453596465512378891568423459070995318847769871691251280520918251698184577077647104420165191335722379424469373559382013378227242741488727784263300395922636758925740423553112412416207772201704648303696019823, 1701498881867780454692791390083642288573753073469047228240955004499048676185), (82299708574540866707004484352876486769547400218428332218851946184189002581124239267975421608075475841784490811943762129737063545260787001796815832460206675674843564976929063719615913842448807407390904533655432733282833912353071607812462951200682448400478751621102243328819790042843483449870796032576182116043, 14819438487296313108072755667123971372559567136621980863396929141726704061), (62483611824579476383181012672078017067571168969383431364512631296030438251431434920324662184385385664009152573386047322429812052362391954974412503155359229618318219448654454004257107063938791515388916737529502818835016847340805205906584952601788790888750227354442806290658996525236877303151103094068387240529, 1957), (113479266523073828347804866661339696307943061313340359924109994651355544746168866349425970122486703867023743054055355234672627919168825208763916631695011964506257086873164390594495494598042589296648285533425785564345653591510522521167783806302645859895374705316840746608171653088617695872253282981963154281907, 776301444926725228928829934397078753670998234100992569461873548532475763285), (121495603647850630441733320036592867341638603538761649270660973949303036528212542948789344793085567431182149201813511143651455271874169353508448073713438941592056578037113278184564657850312496451853974863964158581660014402475345240708037921427270834963162897154727002878708765875231396218251926826567274045219, 59), (138898200484706421252706210850704377867456582570440498137217935493467745162694444823270683079706313122670121426959731212104276731605348202233365704499821166790291975907172947163622328193153132174817583788568917860717832368705320886267767597600865234374434194567695470858314067538345273381878126046952501286729, 1686871926217467423583713847208333092810717344043666429245917975885940954371), (141990751014920221482889487728047418261619410163179324217395809574120775940202375264234756541520387350643472984878664400548035862480726103265783860571566351144273634027254373035095152122808720028030480124549870005898723457126750477125863866262098281601086302361293549465902890524065590226860393625675113490493, 1769), (54400115279130933199009100823801757505941216305858210582025659656569387227421269379227740056072035485146844732041091858882991771349910186732596263188181175332118348784641340108220841615824773650677804943964025596041552914946843857996955700314593932504141574547633496200573535498667339479333596846809751335181, 1163), (154432563709834449419193604523774382752586011123122452823375245357710888714008820045124408216666849258564678997927818663289811431755034133680620525361010009230877449469313721073762269183996993400172687216199355274208842370809371706052265307831223255424791318343611759133985289577213651265893412769660216025833, 120235270010635991293102043281238439620996622305668861861911814657609806425), (160306003087741385373096014112778334254029163113220687544013243569101587621273974262613463260451745125329566882785958600506008089966564657777644186132596396816201428487775340938165909336027373610961131712641387434789994361730954150933494561989422856341529314901529506343297129469177835447082527808090417965907, 864263241603056918182319571255575850603113984035152868455627965864416268467), (74438100413339751685103905859502487213262242964654214424372710559447584963959203050714955747362276761079027802141628927077384539410062282082242021300927570512929360961381734383820134911593497654028513256236505496797675917842987860376869182530233624335676883187479048614372346028984434226206325234486050557679, 1307), (70274643007961921189443921795091501875619905095881025060033245947463419720135027825102150347212336319487225862728814795883387679893263245920940796517766932070958032651327759021269313909257932861208602900056995972995687249131750971922255667472440832384450773074608019339239342872832033111720531298763686951693, 1099676824209803003667418839073233000371201718820291538999048357232028378989), (119853983803151050866542856014394106914228113428036088350697639174274510846932585836185279167272030266512452307312005929573417369270649042955160324883729403317512313017440680304675546767071195141108713393613333104343752637599969561883267180856686711627958963165437396407853661385400892272663199587447484451093, 2039), (114541075780232633379908692637407247864535348261146948144169054010114416610185631458115143292187391945695904306006248964804699774582304280346470902050806007960524530631967573772796232016831295489009899045769174348157068772070127260087742020842312596881040547455064393268184322465254167388262588749757335322823, 1091), (140339598865143975615488662748205671052022155706583087400284074227528868735521372627852761260576680634732795051960824927266919231973570034814590099678660469799613225471519722646012349827002757163788983286071153956795265812353512899195006367384337246620696139724147870487755002360892785666605850862286510226671, 1036984925393261829421135023256010180577177055546312713520025191197369016861), (63417317147798086829076121209314411588243146352353195223261138976740328571857161289817357047078138739520633237338107462196983144319882504162261639484849582616959965178933137171681657995393831709423954349431382239520971873686750867252001984766775383473249161106680556453958871456101778018011048004236136496291, 169792968411517371184022088099240123508560961159851001141888157153959539983), (109133166796655353231839572407283748147262359879896905352118934356292077108123473982905144004438667254341609993504318944319996945501801601294227949394858549529587540702394712923269081438141652964186905258149754179147096192809535461966057247951227211305356141591011683089188911499310003071160360316164714497937, 612828244101928522851033834776236668103067222196531987381804856831256663963), (134984080777393767209521068271158114252385910575057656243638866803920110141861317223714295291863291767537698683342234790593290990986156713259417532423426186214533455650950468316168929074860338444610336274559506408743894564185688324773084329573315897164352272382264458593522445529812711459639843705611770618411, 1084450908999994943615146008756056167306616117581772285251969109084662145539), (63008721235037762414910627182299855347901182726903544600702356672378573695154994947855541935749674328830400730635256321225581807015737238846472619460395739240055717017533573392882001198287325175457783442738375351156271886011520804973080295119961047382621802304096697585894990305070938408978488709722288510453, 1891), (140372643162602312334659984772805436322543014593607948192377602403592756136849232894752567248739855380408816499566460152626205807507885384626512687668838479792874609165461919814206812965518202440747462300335032558033440668653726315175919488948488957052963610180887123948830806677164237353535133403591722333147, 1139), (138551767561622808684809400532805987514260521000963375668563030656180090771673943129087286420960028802601000634441916825334918393686683157386105281353510515046402152946940129768092009734265556049283512587599175111242327163856930145663437703199940213677039696650432637035162843737054390522517773831025598567151, 1529), (59233991375511086346376020737813930530823690075748948134698281160138866615781375583272919076157733147512557932881727144340886568731335934769385880551111133390324541990030934116794559963649898694553863260393178823852758550059787706837460744723971759808032257341794861913159494788843087052268993304037004496389, 1127), (87165326980006393476008725530526113493762570240659922309895848587097670506464368345028496548568949577971317835080105438314589961776189730474438379044219773464779537579102957136180083657665836863604847194363798608248139386023836685382371372925098498063241437634717831355994975160435732793734536693653751218123, 59), (94672902811130439252418471714998462735614418498923432405751803697639405108647275018447418716143283211186891382207465408199230555372910157733545147572199525118419299031229312220125935466037347338803942779668263690790191582255387059011231643065356051035166071621803532530732539363264106208988282781082668697233, 1405316023277816257995497078886093671102376529483919202352603533748334965421), (50623692124616855777827720857715515498396534551447556055117949933954207763188290773625219526930824587061107073707204483631584725871702814783668022627721648445366224204748564525450773817178560306036486873430200461267851134860698254463927073618004823039712992447312163086657281809277652996559157020479484205343, 1030810740043372176158968333853145450114687543262547786581345138376690750013), (75390511549197689884944636222712188734154027419804931017780331237304222306426096164927499424763760899739533231317930606167791622760394486289277568369675480402576556746017130474998875823827364663948054588537340931030209184652271686608862719353695643491474095043212433254600692032280030555439115965189332469973, 2011), (62736893445824242930653235188929103518708476165091294451050767929598093403961085240936629596705385476070157352995060507903089958417697669543931341443824230636448810766869481577058583518020316005536433508490035590784563929175772210697799806869970025706662735139379500313987706420401453646932770074020784389063, 1809), (92586608932278882304652981485033279511617565834949754761728445339033797713935519549776501740668285872938275559083162329505093856253440324232658425621072784095955824424677579133142691627868242456016514448886021092096978252846057339939603388670824825965907859520709879404001445334272407766209648523996369217029, 1067), (82101795992388925772726935619535993867913005535054268855996123805176891579651048402813358880207386997015016986659619684545889612398862472064251513776012354821144767549374560239905168506480169583631197845025770271298073281264663445477695616322033712054191262895723773488878875079095840827629714603245447230863, 609160897153054793383352300367194266017676778968723460043469728269308946497), (82248882690566529554830196668595488302903568237920830880347391959374578651137841881720402317229558519749170326329957542868149923424013686281201085499430692733567814447493500393603858156297046803917650915272566276588629700311386857500800611843460183254185090358861377387433060365123962210136490725087338488669, 196580532211810881018846445338325508982770221985328724227433633540383949801), (124392573040477441352381349979371809934136661327779076759147617643546423085655396442806899982693483227980752426677405286403485603502268664726767344263448481724130549897719855073076984067942208949653060127207957906461820398459015302553100738941439523075687252960117839605247307746638834368533225539974861789297, 1596186083654642693959242783522880746732373647253261424619778034584137104201), (61500327911266186498347231851393354428141968670247384252557683099326844233766315507009324212726664754924762762145512344836848404764216529799024391397035580211430684773314204688078653294115154804807512248713589484785793996247061570861271948348387032885549639031423043240279576288589896194977133518961459088003, 687366219396464394194099961979053039446481583136765129153444223116773028299), (102041178058050915511654578852315037802865929990741644513615424562925339938703770332187492470991873278524858885088995018749901143016990682110831695329465463032735684661947316740038888837832014040096905372027028505143778580677914139979366868240556973414453188576578474860019528550734213856333835551483260969299, 949597542325937527213318536098377020790408073209379592922573054311533219643), (57822126780905251366548037639406206426344866921929554424869676699134688510213360738925432901515722258740260245310659998422458738913673628040072864915365930247873337195490408742364162122088698251680901849814359197917550255901218768047576495259724825475290782535246071376986433451331898302262552444676728341739, 449687905138752390022728698038061776158810559307664735783192131917707614903), (87991182441589729363970391570060681346979225017384127378811939476592381156678630158646579722090721094073317276340438505200919920490391355812663775414135608860217959035561913935392640904065891927041005692052006878588499290618103857627891641685491989007609909012979377534735497426574809916690572061947942021341, 500581268286147484406641846136490017539561502610921456792149650612859043007), (75549808767476946850939162971865532416817739530480506661107350342402881020016433671638399296267709762897053339698187904855109463078273016497171230605470875959680025289520515685518536870361139431937541315838659479772107313091452064132085889461263053526903935621841668436492876675047855309568311627661936262013, 1032500386548700493101107729102912631540753890258745933752906466044214707267), (91957749060848870389295137265525216995886885876037132947276757035293201911276259275167381896000979537572285707929049018109115477275664269704131116117229350048836573891159912808833861838953099602879636749950177009410022752294922652281042585356637172921233864735604050138851130769701400796860823696108195470859, 404849550144004982607378652305353200069623330414539117528635725556538513383), (71822902648850436312206519251769754278925654439763721301096864378055848406630652522247486994708942417047328831507217155271783211980655969748267230721481503316158462348651430870481472374740690646629057987924808759504070796852735914028857522115803021712141933745660749494797857450912312708803830606905120347927, 59), (97875456998271840794627565798151210131535996899794840061460929562745033187352892122605499169312687102738715251081829360124560612428350652223147965823588680951419097992957441713549192012283177305772389599282770256768745244435901468797316223370213121931365550162413113915279153305687977404988276672814949061323, 1361), (99689764826162224277646680277557633167865476395763182144282971747187178555851001276057490754609327772068682913657292666810562760006979068056492446154131738393305968930597169765863103319257922053141641094216067905641661397825378018821788190454398808816537485260433744902172213325470346775147210106755525683131, 1621), (87470393138122598262485215648396649974098759789978005967031860537396224982055674639320951167556381965971257504452741489622479491258670077320059067689376851651780724776889511755257117247123495395711988725213120042461722805003857666207284979183607468595285503245023052831548082972920068353874229855814885274341, 1441649577167287744638798712197226368426860115685313743157649193788784527367), (101223729718652723555854700192311310279473711022616748208435128126980717609996104661503651904059642663046511162221713352314347857701921683821425152117094122112124697527647652015777758624224269809489906265879426431299380652769532273319774346619506653239364672648789044694373308926726193707008451864826692749423, 731902675076263851234136349964367767041935823531348685632668468038675870461), (48757776293743122337338035730512526468387582089001263155745452550481364444478417311281437362406822042108617305730985147073148777398832553913145362257155169101711045214636209162647838333605821408234662697383127064229894938891467368995176388412510209635077865958974705777614242572012089395146070719956175424851, 59), (77618978065868435327066481853354661960663575374089918162197211541865846277562570551653065478477721093884452369668217169809464680565786400251860494878115359343311179178726969251447198959425221467706157964667753702224404976840030024279970844260541430490228958248256139892923406739887806417216749103225283984409, 1591), (100669495247014117436486160511467949378974584765927351356710848301746192051636314428676663668903254820950294949832732917383680289159993136407105445645778331571856605548779559031687756862345411274866649551283073106992880305115347766095957653456244187514547935877426012245638375673448145485443259726494590233791, 1597672651379368568119965051126126720018937884569868426357219419271584147941), (107915785601569452963854720608886384278006860563977792421333624631900330892046392295656841208149045440103628824083713645022404541302644421539207942799714347492884319298333360004694179582531354223823761685300764395570470899485064312930787103588070748812182621959538166546787551039305703369041940812102910345591, 1307), (163383135959643595233554695564596584717945005681322503356278705418221647955372777363481975888338680188071789134193973250597445871553172636744913799188400681733849184512884618336094035367568043409527516209815490793332474829601807589362103409625661981063361407941385673059638886285885792393287033169598789370647, 1117), (111449941723556958879707449047322630845443230173857814370731530487803362690392245954020854582787296239953383262334395026930084423185238390015443116106009545592731275024832719437104664800859970270277776966547889412079884269897432630689587589494651471652112586035892127498739712889919900636061967960727088698473, 1069), (67876812031626145729038179272305842090333549540688133013581008567590231818660102365401992458349642100158458223828682267257012310964656083741649280980378390630620384650381679166122252630607273720173439054244150708367422330987928337623284230856835118810090522139505031526142745996257160709059713643498370314717, 1187), (65503288381188116016822109440622263562922034735137623569079969339950073837920135801578770517889993717854222859803264302886137681785427964314805049592625670286135950046167360847656596055860141205230930486800486797683941479165170033748641221838445171732997565800284481486319229461791838042401509539534703998713, 321686763378267086973133835368659398842595637093804262720384101395205470895), (88013383019919140050103103705975286749218009057377862129424290478867152817118805467432749003417124303709779959896741228927017356112563748514774873092254905458558780846589243528211279428862953407464533162524450789359134872291267101968353668411896481095991866347219465530242220092173222773411302711706915080531, 1026282013851808851403500004974681334869587491668488323681138354643856188905), (109714363323976712974576981077381000102042448634238823193944849219447857678559819154190534826834263921567163720668224742759954924584552006686605723596316224026867818657882115536575049501150985897451796324883677383199013558555779488283919935704790873445114851133130051352323731981973543586555482087846164562269, 1198108051329669519393316078997434408067345028226941912181804463507727804679), (142787964583568030547282395601073698270950295389503420234250011470862052433663149378768559839615533864657334707978701114557161539725945589666241123759070711214056332911045915260174589562597779942306528677118502070957198183221659991479670876948946930160480962475480364407568940602037700888126958627345226487713, 1779149546649172889708669335881542163807178778121070769707274177253407804153), (76952030254598675120421014870069832865712396899459249884312877961352624607736309569459137272636981239380751023838308485539129399632603997110300420360971378250653559704246390105478487865396020502819786687544785967750298344961031315528667897593439698621929458792587816071107645626421589801834789765673167250719, 2033), (87758160089425712866855602579945602008763571666183945340188182638330415612243945850026314598731245390777607719928926653614246923011180850547239961013906973620298064553540216679278487102173945173760978996687913701739850462955337894285010131496664353341701446938631282323253342885408481399623850775180839575087, 1219), (163147226009268607065056398025130349908697446429323035921581539874647305185858311706633316189336646293422043097836837465708864736018833540983544056054857719448980668398454154531672767942348213708692805457052770169959264510016326554224247229965757786706588143624629233059283705130329973064023091709840288669647, 1873), (48758129088380817725320894749238675234660575604234397646716226386215597566041084016706515718524766871361509825244149708810249834283496333719753936294789133723691057116399998381102307074047909837379094878060940071147210103200499114813572066942327710217882341026832488612955193723351912777120823225658954110211, 1607), (131858501552342683855077303232373083040838241497653065977943161650549750388294111138003342353780282403407230597941347269092619456145977312909978578322072658114933540184370464678648744626774881619982347022235854240592748024990544084191380992362170011522733093750028782946745848748158797010442561042539285101483, 1678866865307581212209551382835879237999288797701629800204401379026226056521), (101312727298761685683828615331984918240150272063608980265775989937707782191581993744511284850352867613823558419746138477670058833915972095818423511741909163612853075561489599145835472095682715292993841356240350464816032879865372065826724017317966686132996047027874708201378738814918485350037590880449627772463, 1277), (99964911369643532919362285220195269864005517267030404528168889962726099168873956781173845249339777473211129756652733716499570432457394255278407457841412618680449969792075957400378197339931486916475435090458908094584209519709543648177426683926093678935889292340142880387479787629177515434725823959040831076973, 2027), (145528847811210663135459536713144652332978626731668432837497260990086859548754034627083915202413136825515868351865086173910709282335814502302857012579132214923274488169914493310189603159829966864620241464759319212366752782160405717414022390478310301831991878317719728075916551322678823540307883628415692189333, 1993), (82776127502696422485179331204315516132047465015972235327572737346808652756727146362417102762146578139971040598273446901569172395475254218517178198003837078629064181312549989276236725016076596189586147385156570000486611712705800808691025026641682607930061542817960543436760210008270421432049564247045771720699, 22240141414476106350800052717549252105119811925809120646840774258766749817), (135007872986637724025314188139477222959713176278814667981310518905867147582112190867432231190354995329136774797558926797578421780553634824188792247386240185591174326056052816373852501049001006615888794102774872688242194931746140151441344523696424533319403324745339081348042612879223216135221436086378364417803, 1323074216984896450954632696900264233166549069030207647321283951665637235467), (85410304034908022147596386615713288098194284224516663896564500077173258768753463111865386409929546983716838930566423398024310239451701876133749822298903561992006054671381653431553973678321298417335673147936536364338198436737269006407308567366389832290243309611555963972399607874515598379499199821716316194531, 1521), (81580925505357445457274091366844386820515933055282063096134372399313750187766003073540950888761755000699150129344374616103052671484126772735618236061107827446240733404320245955867042864399163862341327335224557163085497584767817798908876448658665283199701706006518089781911244900696146221218936783809216669487, 1213578745845728160219859051098450330487962127385311704858322779590402251207), (112378866261142278417855760782004818657361401136160297607335165911521184039069815278712137628395626746511495115059929261176592446727667664690474270306591937241986408188713680780839697893016511343022525029251282775893457186689804053409028345464973274851470240862263335259155317724544421044641344605601977131373, 1361), (75036628392924366966502582593340222409568795110613201873062660408435937420243332893135149958568856611307186580075361265150793509257779513035886670065351770199233785336582083431286597819152206650543694918873458914064567589527562735336027462487760356150332344226581383772562522431263941589870943098369882444883, 806319480871657381494088258763207271198252903671179501228538676115557848579), (85312696768615232240141826731192165099507979137631733308696756290671666262821753307677704271879159837807873989118001331908025832183992402803684453756518872343848179815499377822740396589460234277276975689028254240085314056261015634030122915940707273234116254526897791210491540328433780712837062936177142014503, 981069384588199056962415025059360649050090807077140320873851325539409276011), (84905215936476758659992233751721506850040041987581436443749917818515250959056439244794939194033427160286430171878889388559521456276476796124815421206592591541679428150607184573489151781215525110773506873006379673937572867124250265442539369704583942536130962351365193007612172430479151485735954481098678211929, 1108390119224146342979833440965962081206081415539529661690836856254042790051), (156081883578032340382319369119400980884475453146066919496785097864416175543588215570690357052916244794322198057284108866726544063183282623958461651228011513475637788212824407283825426061591310345380429471291865307085739349626133959384509426344418934209176896905657949483696621376181180087295788651673444685277, 1362933861756151193720718018228741538706171642813412562448606173594147754881), (177768749637796543962591743022954784398673679188121538499465139164317617665158348275705176374627143870726769309114935975170368977217153953404838500036750040881168575581247794538630321218874672133406363222676025799540205255348208411244325253789297942318178435812134809535765848162201566178101174149346885982513, 516773855380417862635855346610289984475851902379622030868796237115907497351), (107203840253434544112162739006684210485612977830468424888064632515795532020355763590050822073288912832449744654487169114705796441591049334539768050369134583888012607028526540764978237231107297542189398413585760878615890830444820888902219493664723716714157733694946836622733376495892139131988968761582244567967, 436606633844753515422166053171213454217678803243871904549473663461295403589), (74869708407190588704577172218749642378141628755835176528863033094607820718962042771920657087309795361468278596133533858329802355814805920597019866415335198365734595651933463538078043830794336241607348368171719414910124398782252507646881736224360026217143537632764656508515093101502635283090635599124943833373, 2031), (112374715694627368918147082666928561413887091950008600591958379069819897363189471903578073142625995608592745445751670077956740495453597843286716530962864095619106977403059688202533375760510320785013247703429618115111429931347789055765836896605804924884397336969445721204922125097900725209455945395700763728573, 1295), (77761291643663072572999488736934753592490760017471085193135814667896087324128179392585696000212287205128520600041386309414817030331419211708410898589996735202475810452881922083820775011259415008293815532877214500840602041632041063147755535872875113562131600357807638878747744099179214625514179201747488768523, 1081), (126759065413696457549650740295900225638961059809427384735221055241883922104874535522703130247899358488384887788109931218672429708911919452321478783151687031479054264344006618694895060995309936601984677589166671849137943746358808987023331948515551584746740318059127307460823315595099591060825528051370395044101, 752812959609587702199976678428089596354949864760908643368419848771883632953), (112578421199822858202100888296387018043763726236001560038303727734014502134787345161107546483936354116185413560547775545926401338024649169937323367070093174657677386365561496098636201730284966835612917829413510826987087847780352349740471331656304691103857686873088001305332492924078816077417166517185990719039, 1628746546075933455156228997682295775016563890549651633982311927758822821103), (67871518658734996297230636270932695263202817598284592451677033870080760047534380655893995750618698727539987573630615692259024291801823205042578078977499108990698775044164102036905474412567323736097539723503041845407651657702978559095379461073395674545353691636729453210704515787643631684274476650119621980411, 379753636935898032213517702378498758395124904698383066410650962518022813585), (122831671209318892746039964694653797264582424735481273619435212843921785452795766469492760614237589149487062215675410719403868836974039738045723496773970867428378429129943530098666671461950782896621524375271907441752667607952518094646114648770693536035229339465149454853897167671316872260184251888918754725071, 1471040001320032482546433903432332419561753548212254558947023175115966610617), (125110049575459296604433422016326424698802428693111213501982586836716182587891503415222682537590803936160882573082073322021929695594089066905434989448109486989359948089164575831425823332237009514287249153315600531260330371614562826614145792415225293529072496714914586525593553937993060669137611504883329404811, 743550239382185762677211507243728323472915353608838662181192605429119448125), (91343004359297159591482898135602027206894897692582484792797101075572998712598790974771965187935645090543158111397543497223387591350963284966738684044144727932990342516224996420447000733315203418550673104544877428547802042715751966341471953002355487566132057891731495495367082469886620335223557460590230157317, 1101988090655538538029133165891047071958227600302204981417719578019510265505), (75571933012989845466726835570435244711070512078822273583383537783099004270099323431855177910850045913079790922370303706544917448223380092351181569128596831656356912421081227235228238170582869578516138015710873414589674205707276596661063984321796898633122317977849765850932397064793560717878462076320861310651, 1617494370090584534399155388151786234295729598181906687477999074306412009039), (142068510616125630505775738141529576974937219032223958720628992747579600165486297449733481774685140973634931797568621629694468552739667749051304760325478218941440435594148907443528794203191568494225803776181928795138583855405237408236670151974472301048197477454431426145547897324057340562798461762945889352653, 1217), (101087606935912056483117186757740329775880835390898382357026191814475487675436533608685092319394813021426842289753916633522625489799046888903789896983119839451121170520885099310167488283727212821470351873867049251244934599691754950875505446174888570352667540892734787566189504867297464965548979645148621657623, 1373), (101045109425434142229553656374448375773022416189487937305514655612858463239449219321992809859719746082728952808283340597327787053006144957619348435765159443767972212422564963359762861569516211035296408416226430034613848393109598832134532868201749779576618580350936967689246287712841153519312498888727510220437, 1501), (62585111634571665017583858208558510305513330061837205645197004901633645777846995820576812417589112160604841261048646630785316195718404243721474450217466868172832302598833733110146936693080483689913334363555874616877859944679336634853521928733400629190085037066280121792705881226711919884419668610508125139647, 894255230895727269613230289586725880404979615116875809533631744463433726873), (86535021020086769148081537262421047621462840585755512655329924476604726229975034175361954171091460489923000920001335726173820227136727558007232593710421534028759913003246802034396622672452042284172725877370340065216108705115390932709584052300321799751335976593119746661503802841835170114712659661216388667347, 1391955565606679348664470207179769333873012139935993123014181143044990686257), (97893638402216939940425401131479259052155588451984972903644559440057705434738557575715431568305758798840560606730823900759750174842708904891684450798797976967061917193728607451266415378003360434867428457523034475193139007226130837146637927796201056014650342170443619544961628693990256496168512823884239870447, 1891), (134246057391254622834499233167643823892949049577245417468450583883191550062416278364078200903861623159265871755212630714359645027199415966187653833917071344911181596884468508760833695404152688144284460933102354201237742521712893037550511398431369104393653458218047134160793805262871589591267018275506790645899, 1621), (82925257516285753341987668348407401003305475670927825664782644479807201467684809893145567664589909361952490831591854186800298300724876496317509288898021879033340680499062798189050869557352387504771140013008146619579838009122889809004112463001991984733560596055632460157344395340111692653917394668913245162541, 1117), (88702596081933719080428255257698267213703176228930616009997759209133249790412524745435406125499726172793974577122198701146330667610271482616248802538042004056133172424122279120839660774809898270870184540406020030238634883278521838782682563351943961704754776797851387859736314858982158165909249372056510607203, 1775210819563133928367877051815735168689226528007235436862210829183745171903), (72306044257527190840154255127021457746648732171982041097826310538571575363834237598963724042046233911328407988221023735193492257517038390133141695438225545737006820464943303108727734292357313128698791382410296996163426899081822935328956921023315818197076832922564115074145674764812159761564510379935890560177, 457985468800629751854031067897441685033216852282806817678637004056540325189), (133595510680079061957916744616316056059107107200099691148582787605603610070465691374453127437472269080812564928446777135841020223465774663801422264910689483202316144807457533198475801848129552036599635844791066875157804170775630515287061478702241459129986119117972629233375085203172215244967921860673554124933, 1178360252403251170140195380055823857974806315752536424287078372029449963167), (145418032982264118412018478490265576438998397561488237016760484939103727258489890957552954178739830691062479696915896601689761237050326212243305946275693292081914947148170543259621491023086714716699501437656387528192406942568018851011588182379440793075583683082181437080245303998175565402919429162377448148127, 132509812445651289167882308537341106900668144736651695048137959991422983289), (95872914776588688054184852087878990634962850233484633226421732757580017434085613941641627825243902452376461674362692030837414625694235715962137053936039412359096947344012801714061177095711053003315227593083385587754843247602457235502805319870949736927224501381933970154535605438576975783355932217015447155209, 493274389695776605946189865438508366107084609475265759161339121249880150329), (90605736062063079092193196116399030930916575078560788437397276166107534808373898366734976683050732552041741702274510890557843550013754939012433537836840084086390426195642913850992261153733324378815573581195548335738936165080558833704867236348677601352761070346464359937687408486787690619827767173413530877037, 1709), (142211867330471929652312539812749316523230044816720572365068079111008609611073305341338024807176667656126846955821102564671397044698961107801363341258002000629058970309118357643677652124903427211124353552796090709519248050591711509983696327162981676847098464838013968837443097456545880678366308888429741996313, 496034279578604361562646968378445360714046579734341299082806246158288442215), (113467921090146932100352692645132484562215958018783770728942669835795161373089841467585380425254213583143382657994835746444223599632989967805650362360205219625413178662542775233272237234134705773071210039989916334634257077011438275043304610579383387348818257639142396332491783880607841737505382381415565636329, 1344951831683063415136932179135345822066850132816211584845405436808614681221), (99832080394660701029484574011858885054674095882979867830236843076332240979846670535414656785587394362410003306914787290800428715380261223288317152119971145959141003914419434082579278316584591235114331227454816840889385330416707738921827710039265760251919155394422937227109047029696160510794051948072973771391, 1447), (135566166514257234396763099797556316663868370168739679178131592243822416254522365693104782508614136628307447186900831754270113068570214105016324193664594858649928373380112330733802704270303617322841402109956186855329545573506667551501249770476120573808810184411027418367829440580994341721883687371820443789671, 459151085308447045669402951551559344734678682927519755519674351794473724021), (130904555523586308860372289021995720614584008954938382181923859836496069678411659580410066601343750512892824881926203384568581338678093436037849974910324636223005350863263194569501023305138375903262457571062697858171658652966012826231917442040336583147812247477867988962431087381696494446573759549114647918007, 1073), (75635361372259380806130058312385291186141190905617904463487709501863380666229978696029026987044481605444655450284714046306303963307029706422174766308832772666548959471281705537642916231229817431921287966856885280438356125419862053887432962725435951598117054653980275726104730541048537977746793980986018021851, 815967647177779900835620504789714355841291423807842331538887913858906118657), (70707204764962381370327380622702990227439894226757941714497079429458926478098143557557740191074790685119778847090843862361431486170464758487387670494705838647431698071383547793074321065176412137740188256266784816194885414455972023882722399776310096182468471475472041354161888929434390681467135304478301864087, 108107275792964798707342218560379287164526805379768542846106174849989942139), (56261707692304081474065925882625077394628825586135037030326487929393666051172495080942099951829502622582057186687694235911071275755251473284434179371290226199725199519537469226793568022870915740957892019176281940030152968853931415185662856883021091931034207242535913134731761754321411417537975388604256181137, 554976657745831807604839730426274736850927935368304869267807229523369332949), (106039305194334748303139736481576254154269148544466734545073417982130745778304154085493051436965405475353820428444638307073850072250343950015774461788960748083532912312106764999409006357886276937661071865676292026893275739294013573580528957277892641860275429261711253772809322364393995570525517761476730821713, 824101881101822849853061865727539177367975124356273200591619366468535367047), (140606652597549650946522481820505732708416611644244800280111977338736214843653983239346200760491530888318285434294965098818910349682653356556483819826094380750478731747540529973274683618576529908467842403280486076996737562687662236678982956330918220782927612055652349777300943881034126548673197121495635113563, 1319), (142699481919401260557515982562965906835032314420157009588683109888757313941143092579716361876791429270779817224131121062206152756984593362208544576004745296462894817088851961118996997355090027628661082483194931280629805006450341010092145853014681727734941323394858021230679386367049983676348739738090464946477, 430297264740706446492809787538790629271929640437014428997044724557879860535), (66773856374168505071269104576571750140406169123260303485419771772469920430704626876130765176720540436516227445748427870865563185355034210980615484648195327067445197974296184357805010296702652560604522010556233356593853958536709204694864176014869050420188954839923253135323502995972161736143921114881121942887, 1217), (83232915298244385119235536591753506386516086617592424901288691021366850776163510746476526659344194893109321414148783867616273090609201061191713686967942342533396734210238575597458717892945278090918818051093978069837846464643428730376283274035905895876524622013196573757093155970897399324599907775316862807163, 801534079420929368413409407937311460803511090506357960105463716892552585171), (65403205676067101393188145771394424917337328259262014401369345695707930087000644517084168701177562623659625242323651426997668432802507749625869853809550999636530710820000279154018772381556391193965968065674987642850763733526393884681638903670670450522727976306346647169090222821028957859893418822768739049539, 1121), (64125940484419365031992270358521144248556017175388524771722453084072704180551116855503071853491637316347096024452606707549772308938974210241306523967218904472924412367022922335646009737945626994035320258540852806068399279637523153070602534060156232152206834580555551957734147703496750755982600848061086827203, 1999), (93680954455363717308116259890800158015832820932877143647443581750793936393097166763386191573319855497838644020866062876145164233360320098404862729024385302928275533149119652468207100900779541675872050410907007944052400409437602244228561741990645173122578682905626815417164876900235103968925905827498574627191, 646897961974705176341620947222652464682853919036301972983033895266482457859), (150666246094469295930360152019656716115487093529970371488009907535131406986638627089997260719235886552341718604162512181329844761573099745588424849202148130142326508231846197752144738331157230852186636085855896814140231080520357211158583285055040421234847390272755888130899989572889421063948999656253551259071, 1090227343093209061621566420057475958735608613398150290092391330545273487619), (110038680062989200268206245147102309916898256111753297218354915902377604005891378123333381795093218525636487882403698797978619353255575853643556169342821159337048658719422906853052386878074537332559821969990856270434007183848542764502652839101958018601612640690066822603742152298343604669151188198884061183233, 686748928885678843860444151041912982262631298951975908609009512497291081829), (100999483486702537477506300640209430239933046551810179910149826168804243673711076176331005446328367287726118387238061444388903783898268540149109959993029839864612892136894493862815058111994576298934062492653143920113966281885691747276790133725555886362234308912326537609668984551036302657386838315933471600659, 59), (80359956376847516100372133167113714320222740430839630296704351449704220213265703431238806385724867646339693195553309387422785862479042713819359302562955085261151564902081017686075508738667450055139808847486969379030309161374849818401387361558467805769996951922786981455458740187782955118356521062996943116457, 802215763317052690354941492902951819864604763630247692478510337141719381371), (131414066419704475551805057697433423559421890251536861586160748144119038830943036380108346080802078030976551786920190553041680469328446623801748826341344179730736101223292822590117037216872278796013581241266295833545255198837661373567007388787790957563318450266881053452558761290918846720458074147027825398453, 1907), (128306879333550153320457309498458108144293298580267393188424887658158441997699359428904326626674043236733176781504117644723663981696061748874593794904447188073403967791408098972204964046833160989462136445505287359657797623853034932672342365672230037807929841961960712967758830530118968014751653452354472027553, 633343163783982517693513181189206749516533021425844409988127198262416538523), (95493542635589398960154648690357884211879887749019482656278687014189611471469739470166331022384234947592099014464008250249460749873024660781383512228180845320800285850044561007225989823164880343925925601284780308838474572651745279170562051905848266595904937227113281487822446563283681835293019369200126445341, 1421), (108170700567308539075308514111168982960443440637153662442476684981192285278287693328622020297409123643704672306543045385178810545891643513771049363391190731845528830468845973827962476274175989882982631503856260004415070315896429090428369067309136303059532171581691357875551034601101615951679726175262795004151, 1247), (94963567679262071740055974099392366596247503142424470799958393942772887924223242636429880813474231089945853114513366994729481695337700801683125186684923052385714794548026858797938145026586900051765913656697792516826192791875317565450012670900118436565972236223199252911007091822063866934237362339028145426713, 1119340897838659162469584752093809073152838616170921759458075508298167274407), (97091033248640263988048600856543485888802279788760927084618342279660774682420656244570592935676646110693881480420629643610004509149138143224125615103088984078116625885207578913159125701517768575354663279652455251868192584286535778814898194609159729305786780982635006885371043521690744401968569025293996611811, 1265230396695833037954411632516257188468880137146865153210019096575388883641), (82438563431918414032234333672106763180976168682162605786051031257951015002718333275518442740605435680815408356141050306243650097971346938370884182746521354116132206458327470853955497130372417952552739896955325681032102558015617075985003504355278428603173560170355169919411711792660738158983867897526229328299, 1613), (61971684289684149496119883363919671451292758187938995767113801200187781952316054208473086570670982058064406703704982459583756500820533784859023768570472430507906324051745375971024225617120855897629049203377167691643052622230947242009379124626842998473971161885375600314885666393352788510531976484949524024907, 1667386747774004800893412543485440834466374258335966354109663675215232762689), (130158535990928728383848983415745463012618129762503854977636965451853147858467267395628256184372598465988037912961327614507073724095206191889884687514437664322288199747000489430249401627039469822167965562928081467641374438673414707470912861490781351873578287924004399633002652805038147244663854023379297826919, 59), (91455813104461875241544884926339715274553536016887017749925466400617937541681117595612948196109591959183241497346737364936237681561765079572510630055529102268977909185099574269157047040321764851626030086672442862024186450204689783977046451079445275797076716797792967650637063541821206924636817886053560096821, 1811), (60439013882627206223263246725578985745892079711369688558150745195589492978358684475327902867885214323597777719109276875210458129357563663560995636158031451744974410717490001916805781353339811011668861827117527540415868257868520322349611167262305055305448376424348765747272014539344817192647932543233261033587, 59), (86676821634358032189876329207307540393532243772895940079139817547732564815874628823851153912391566011957540500501353211641733118135443702033811842093845803741935085511188734465398305680721739883564572291588740875818787601042123206624310882264981801945931218539174627228351011600326718025447714250112136361251, 1273), (127068204373171155784206204445074746328974175486825437149184509879501429585765692156849395753149047536343698572858312769395637027309672165968728781728471738781294882154726261479878879990110808929176770213880892985242684388168766336506997313997363841671670611123762731085734793325902449417436893929147808400167, 264151192373973885846444360530864602343682329700411895626523376828757054661), (124279114711221797432984794049053818251684233589458393959746444893949593753794114844887358290024842367498843434767507711141321688881590819629893335173353815667288775656024080307588580415718822391554874053143621495703581160702796753758320842014728377087967429030484806203564334152817646011089624588185054577617, 1571), (121507481175941063304084606101260360686829426382171938652022579657478156999921942494942721771407330527000134582008261815401222369606594908537172880240843764851206782130330343021094121078147051040257288313978863943686684877520926995143599756796730046310460532996479044520821410711954361627503748127946189869459, 834367856027501854146533767942606165443232398901104612788234010539134974313), (88425705229249963012053941174844325289385343741797524405655684442449162002862629135293618383951255036086668524173354512929064250178129190606237129615886159626225364595089274893650673560537494210693804674718915905010733397152949303136349711898697237428743808286735290431435522707052094781953825634689479851319, 181996581876705742321371979149605578551413264653659451075638357670759550821), (77922993597713218993584360172618723587859328600781408981461489994706759110237114295436883480995955165659181794777036185740310027023520078315128129317136372785288731909806009581124488217771171681459806476035675294609459203215524794098648327491192853591204844334402042650313849557278863544447315136728134397381, 230472025153744939238214921378251219969452284324643771585104551350031827379), (128466678763743695569228933524133198289459348180801163822775298246114253404323483590152224624327564428052156276899766609708951531717246145385172021654735312755843481543068343549250408390932945372655764146154333171755263872735458042540742628119619738200956783634444493125097029001981328220542560384278935646321, 1165), (140914314831089822959149836414261096717582855177643550235618323115217637643369692970970335886522350553144375543242855121773999761428973505315847632558966292921073196710446416564692930763176111124919045378348572718687172836013205031536971701970879108048736399534025994925592658003816405338015778345611143763413, 629671014162342330327084324115671245241908040066084795934734922163369822603), (113430282571095318607085820991333546612130632723537718170572223167911093883871267758975358512429099998572139745846250127086559090524912271626410606296468931993451720068679550322793452252871983082169949314748830803354212719354811475340458501118557766304725306203351798480773749535820402592887536028411188586523, 1690464997814031064267774775754126156263353728899038063147758072571129565203), (142258155254159528187300249054166077928310520996545348689958558730890488866346311157455211310194040270593291806172545041721546108068156608531105243326036026392872631194055210903269270162152560676876419165087010637391227487104993629843691792978528826629677178570395796218015328828210445928428590860320868544893, 879676235345533343413949408699631695837955581615644297460612016728052019327), (66992063690105976580389820165560665619623513220593492452239115301178763829392949308397446949935630133117599831047794670913674141698871716200115258794668567584947839965162796018483803110571727895347403691727068218188029502531802838793726504751112958261688214178694544444450873056515086046693350404040723261771, 1635122553200157996604008982150801462279348072693297977448732312851437684135), (54421389082526368753827655434399093497348300815615451898962367765731449273468957258016227543257312831414807301105015581010256511181974916523588182067980420599447131376232159789940915299768475973353113945111279611961963015704221771763665025777070399826749161511962099496081927874579313404174679424781576032913, 420280326221687134702944243986206403537581419992098445307398810291594656711), (130117707427628866998872487971938369728099800134488288017304904725697038430761588458695122365726097243903169049184202548465394525778136327434527107929982866367028218805683946390730843343344766810258003055533366504390066616147299243390042362261479993177002788252619489356117493093168717673700390464377384665881, 142807553560213304196165524748782296230444354968676878708338596481935399961), (113803990430839784844775077175015152526990944002954465579017046990217019468132135826068907848381877920226759313454569059747744936487619583720259355156644315185009206949984672515220427470437436095627038908914257211874623469587617677177708291319996320593225273002789636371763705204029922326488820833657584984689, 219656758511671585420543916874107120257174604897377926118983848821865242453), (82303974272495812892568076408769518873838621416321327524428877886879108114626081743704600309308320899394273588507163000146789328793784492311770985141951581782583544172526586111175015803222193842676664031776110349077373878304221984370092850154943642505410187155128111228375397999881602160185854979662925613109, 826006797827522140969461351200327584524620216327932503035308397605945457987), (111620828189716603145976067863538998558617028040481434278999263071390102019249462798199646669273673317244364464688322890042293681661769194832286608109697506355401074871639224689365168148101846123798157979166688241060376821415062277084295224351929838348731016330216690656597610270784534259771740554521015918363, 59), (87154932168897336634573160940146823989259403863220711555801531412588014538095801642232157348881039107802793361929446347397428083537588669344007717963188269770345545249128519698289413848928133095849714188780326996800541390138137682961645223569167259775149004925623923314585274258586328960985869263801461564527, 1775), (73174304968898635051607322531994391166653581341293972895649739037581695549313526133399466870352440399026383699098306683826157516601253995675197490437643134932805780275050215425770388666710024541380175433269616005763869802499063690820888978289845996531460137559566166257903343257038084245773785504224839281577, 1583), (89846457038221464356336729522047092474080560975595880677395007057684192462835086304172152196880353171017527746460396791396390979796395491391411924883010733104471987988892043657393652216954815171249795779094145003434339000273797249694232989373195796462419743637409811504646622591759765831431805897495720940481, 711540887569595065711333968735879906515207538452636546606027197452289667155), (71855054039303142505288293310724367386273964712632937006298503798685993762608180860655061105595673050590734164671733444650046607272797076631557752385582364520423368830987164464450321978838738976794363874512239400664917205081840002418069390422898833569529471431171554444027002218887435289402173991870091410879, 620508655185040235749146989644596028674173267561396349558886521661186332361), (79224204538391787722003186695299396211513973373748427465940514310136049655201166863706879717493702935969193422754628608571267928961359804846299241487188034275327894400143657454672439056087865936358749689083293378303618768677878475964521360958874920621411383620777690218100364071868812245902556768841122404759, 1254277879605862189878015049465886603017231217529243781495805246529565986953), (106846200329892373379368175535498199504976557516287319543133615408971823591662689597968484996339792536520581439151343129222092401177557310357160690188389499312523208361784262921015338627884953335649603722062506868188420269915637110523944067510295556685421539198068975445714840668863336940531994991785705060783, 815421727505511742106557447727883360902250471034512911350425596924324487373), (71566268642860846344070928608792152838561259989188635564171040847243788736447352776515186584609062443837210025071894303404477626722250964551424225002205484191159603846807458977076909690122799539035943914385361507528182022957419563542772252444367193982363940425526536136026290207424418467795889401103926724841, 2005), (78761349141059859275180033295040610391507414132106609208345123178388814225814831371365612124730800843797733275677101456049328451137646771933601565728391070686425950423085433096309410226358661038558081004664528842008542682529258370413265937656057604806289276057459423103098742775595271508700898104315661046687, 789240430763280283825106859023233684053471874814203102563573599289768218323), (130402127241157772290421330979057043391845456529989172696193693901291318680966090125516282125741251998430167079863594377864729657316122679953225984128545322915550827784223473272492360014132740830689755093585179621060869686058083981969180707375371649470760680376399310016324858857626686495284084644651657630359, 1889), (130441955878650045793545767381648173508006572706723838778560664106256154588101962503217040290600015458114972902337243584427926688288252344940108743378639507090181746953010431183879090242478012941259834012253249423424467382695469485286976975708996449470029438848711800775623986784346230737874765114086067287693, 1591), (70517079224479984604556053221013708702099678740483685026252948133175818136517959609403952668450545699336691959707921819626056080400832297170856199115833439667366576918522823703792545211482201387459377549218856933937725538941414068842372427451549624559237142135706606696038373569512970089457257872163255234041, 1027), (141201689450059721217261163799091547963631659947067673769692312861681273444098846456379737310085452237664455926674610000346101932121318326373393840221716018183927340213653613280587654306106010044028978204375202975522527920212868329957557515780527934176332571938596488213130590794165306428816612558001519413193, 122263354246595686705889874893667669343220806021875980413972527641031861919), (95378396210370340864400401214929229441922773735268660414060471535749870119440217173970641107105961910392503913871044993716471201214389632792574080450132558593658941479662321226928956260926841594918253493547313645192054661157601418072567947437167170817547982533038277408307018359162912553301707514855470746881, 1269), (130156243520814897574949596241207520767550895397324988803624417973513223029215778162608502917218135842828216863736045782889832567698039133823972265494810718302268174543478799383183543233162219583201360351447496486011552459785013170536642768890037586498476202955709997674606870293718791734034189989522647787047, 1829), (68725076282620988301722202688853257182448002584522042787902617864717689246362463857435945737715564396160140834745504826891123764035622364972591872538045697177344482972213252217577825509737371450958638482483968560349403547047535812721487738940647055354660124133045936968174893753337973967267396076747492720407, 1786735641224057144120257473263720444351987057105537026243234481845642634023), (133598430739651093656794982711365476785178231796661475097138050468519496680433609080173162133787379000400965279924181955960384477476709004064739643754915379779626256446097405749036454095798829418498735670424757518358342734554992657517612486820671200457410737724960102293464098806212437621084850012714865221929, 817011914089910765917204914095088821970053200935150027233266904741720271779), (79324772735809323770444686236436994714046103385514383792378958297811948798394090973485890208683022121621419892592229924044836614462473118249339237405868586436667387839946819796894110881666858031680370632841439444249626924440850315059971157213377765162245495518435116794907004353386961626649282190666973312937, 1632170334439499462330624939064570899050290233176339418417264463127443307631), (72220138846979443209925230276459218826669300427450927718041138365038201374309760840592164361234751239220939717391585277349491182692020492366892021010134639482098284861619710890290550860752019520528333031696898172157652878645007526981637787231558734375097010969205477471846838153997297951159890570130770841649, 137903508717734304155438068618910772601303432520694567618052457325416723267), (84652882853052799918801137790071010579933211901314437758322239401419107850934091889414950037349433534483487831836463300851840671793928797510387192120342242055778333481211378676776144460994263923758381525999134985257806454968331723631023588791585925337339526215863094139315327392977176027680597935138112676839, 544592619516690050290396095171265802162727631516178219466829451998641271253), (55992920945184602652339831617997568279682361375783799393989729263579556496411072280872973952160507379992648010189745365067449673150055087027200584578398678092624529761342019838715390417995053408360628853050447078449490987110386212347477393704102695677883950518626990373944432720647112482986641981791705266897, 1045631467444892532228707961512143036981731373282812507768238389184020172759), (65321899795291193209783053247597266590551064016845691527211306335487270467650816251495045685211637075434099142910678595484303451179414408379115983346767816938084976363412621274797131597495536571123578968515300482405502374877199543417146234945215301443686192288306340723097177145335256014781902914834494547559, 1981), (89274614015079530430507981702077241441699950131196618389541906644481825955735688053339231936028485865445801686101293413840776556713918969445865298515579464219417987102716300723245864452760397032514537478089248325833818040305786314663852997495754958327586660041907592289316193551153103020708296472436447019029, 372168886124578521508678051578006996671721469014581935294608970461917752477), (142690607758773688264839184301864057113870921555647495654427856921423802274842420242870413521505462538642042273905354367062097969535250190593616255398520310123688613589392396922924108205547411122497225961029260821619379959153027394058556929002912908584028092604809845541948090396249895218363649574441925912201, 1371), (65727961933505342299208011603250838239796436351986647611431491473891131029031816734028554634537306916012970954915556092145497397224117618954433772938806344348051220501025455981253779264735743949340788834707769889501973454396941739048284484087715863467531418721430838789953368358128257535497500807422173114971, 304782960355898364145363138409278831031783290097241516664189855398804086507), (134448541233397332501754503458837712006433776779100297088582948343755029742422496113548572811832039856381431882341833054341186431302896511031625533302946144963972113884912135272945313222483886130130042870757149649521372605326902869861363966883408852953215804033587880086883479403072726586040272669286769487357, 59), (142294945281966475424823555778471777510253842144359246475860484033754165417225079674539137144859173887931498792581855816832356670099709102254640409204708325912495369091210696949144162107072292375726294509684949101180317315179573640796063314305794500333778667576719763954913669940263036132505524483616801562207, 209135988395063776499166620177475560654133222210504730590538762111742099797), (73274412411242970390994094075054365338904533229178371660104901573947428459625778543580214808798168681215064417015656964231350095580588733732381130025536007620743477649751975289567519908215056465706203719015289601440947928114068458359878981440416848938378094832799044528435023649486462448562386064641683141807, 1765112700128061261780243592214850900418965925028520104651604626254546869769), (68772549627964059307880841089773436729511627746826551917103375225055601546720990025600852778286217823343906754388258415685304022411440687798241650430559984489548308479595350164710534745067519849735164288761580731633443053767478430249826855843706498787981716629777135360135287775323935716030836357030776576303, 1589), (69904623511620254614377705013059119795461221437821227320838421097563347046617347666182339861168256344676859855590697521076991026264068219465279217819327427895791295608323981712410277045787929874650442108727521589183956194878295461780051749413809467718541115241743847775328196990894574933427023933203163501229, 392318796124916010062578152715992556498693512146452923065691020026182686587), (84873653605151496734347914284770949795198212535923279179273864320936870657163502768151328256954229336224772633678920280320408405872844443814599719558341900676422201814932318385044300396159306121966038412318560022659229273718800084202302730361222501672691989852742686084708667204995915078608784011394647867283, 1435), (151565387347264795114370800040905261916222823644235181609546472914874895836838054968980205882223870354268421182536650874965502983645305099879991326781281855510550522253171590395970524768825169135230043741479805856075549091571309362432678394074790929184144823981626239070358997116367564439229734730776503659347, 1933), (123416064454631515031056750342384972661429746649893191309686893443672847684692340643514348952311496899528451988123422364142504066268958214100848094229160013775829592008524232315228617195382512999772831708887787695766033821695630378946657927993877895917707512159131076316687301046698812912061411322596961689323, 1576291187841684139276726821006245037554490791261445351038424581961368091313), (130206675026702722584835471916405461482950108300811130364033469207071392134520807442793889105523599522073646079682689191698106017928046982143029312126166445864434439714535745353928812321736836501133989993192143484445073850161224779647861310830595087623329750215262067421151528155146747513583499938243129722867, 405884113022480429667495050287921726543309148547009122334356519435542108877), (80905242983311705698474986912191179659311001208149082911942190095006126672465475911700946313055270846508856219771435835486744090249483473505247284644395719403948750916410687668848760182911528753085598274532979892920705432347180210190477259566023087934531685433646595539388519683187507998601943498402665893047, 1931), (73503644277438893006956169251985471863473628462382548742854422077056641281792046463381797951814989295296581388671103987953020056495283667931114820149769299629266223578993602596527214524407729978213108841312006139777873896507697459545562608888865203239120024990244138831985051375976426046464478150180615201997, 1985), (80144791045451623125825767289482260657229346853523903821885892118146926530662705855529168346816959193254134712562540650548790667977184321495743770294030303462689248277735922571186633009007830880532289202002002828937851617850752273241464477824141975344285465497467331980154207520170417779058721677196978199671, 772501919794436720397579350998669277917575582749035148139243774281448142399), (98038111329052918744286795760248795915896350870044951304436357808429906505976579494377369882730053558486985061209027925964472401295688415103201253105748404278021309749064473091054688408637436582012301887740125566948809297083432894674877311123542465941043220107532187131835269485302892099379569453718630099823, 482841985317749422178214291662907200396576354706810019134337110303392104373), (105897755580641056377655823863844833043341918398213846520172651160453700464942881960950637169827894384258376068647366759799796558278328726490999541509777282251366392444066174729026882818336061223732925852710711274861535735993541328383328406140062306773090556112352449983593020346573045154388729774813278719351, 1613964942830452913614926242177977601541262866480207327835772609480085066773), (88531951805332708177759394705259622212739381250340382692712116608264145575764797762768240254781368921499939984656713347255105703926973887168014296282617151848966011266092886513573916474350547099479458698069035538817169395099063257031123209349267595092158753623288140180695322181417211338233049891125005027573, 552366126937713818240665656075096553866063904869833991926280570031664461801), (70150042741081628255850913912253964808476662179487540152362073005100462922816216859042494808645040660202886504075248529440169515012871062544804744539401291515528169443879914037838820711139945549174211202290823586056908929462589876895824615897218436779474899932890856042132281704098123072943572596230922342449, 544448573150670969696225834544517813240506918134078650321824711913186663371), (167414840169022464350945787491344620170537797980299312841302734363432178544353350831567220416984483719871744978984759159363801598440437738297213453473556569048029263012197513069357728829522694661877545006645641731124193095290229588570976909169390348718366432448787998308149247292215982213701477738492427792009, 59), (92000720951633795540115960652630011904070567441840307614646561696533471070343281284599992236104042690646850166462611683567664066153573305669339142009436372778157989816612532992602624071447106304581470480184074812599975337923307614646415520273781744622349692671163099348680151041222247525399758392982114730607, 49119633858336935909619669919124551902623588413778133060050153191671456421), (104701576914392296036565721530449457079458915979201026485665879367883198819629607584254862107459268298180006505413726108035593932171823724823350734416362319971753188145533477402934701402434032435152775219507046373323921355157526582081740285678578371475719549824266467854754479584204211193164287924951337562011, 102300067897333439710953230218154262742978544844730656186545199775974234661), (116345679792296524386298263975102869979232783944781103932816626291732907674828823657085314320202151123591775015459629080582747438222322786576608873474479095163378875797497967953374720678716053039762442575703527545030211193788281550496428932463500050449771046472917252687563832061035002574350492083218744979161, 1706658650052518855586072197712793968422755117292280684959272653564473725721), (106306896826266265605126855053189626348426079398986175162412409496805431231419110005099720923624946638823174237953482679732882550039903904424442481930376800397749631094261933939554126135990194837225905250160434193010612026028647390635876836137868200449539058135052304959781220272621719804190884599944210134071, 254856006976771535984460476206094026882621072673558466892570663859595045853), (98841545537072921886528091642026277626585310683735429081818291345643786527597518986502214183302936584189837074205207403937615559231830435164643419714053459952978453361839303816492118332825547028858081720357211728225525971155371823643028214637623974215363219953127560331067894294773360444571301321782931285213, 1811), (52858318705297002031161861055959357701151249464633348829449088064159174127798319569412823890434153231439401527819767747906122858479307414146493792213010209223806478700359930246971581921762213314757137372074183360658800505813587566760104244857940271127525756355403107662657978431698108316425625375284125339673, 1793163699445141351348119472837704652833927411030740190555876943829272557629), (96420736137598464302189918751494685692414834251544780218056198844715631666621631909574657464700665672089088054197177434338814264249532084271129167660855850980145917104585467497715081450794090946938654461675620421536167452509763041151501882386288385683184982713020166050210563922716893493043703950480539163279, 1467470173445013862366963431258728450514981627155850878318612894808646043795), (91109313118714906148533808034654909885500639253910295906962288217554701184957867855354380012222151218377822101670689136665061115868359294235188942668332002611113870969814681855514014802559663702319824187502740247964322909300078132470244344300669446526340251952470265692468618064233546701788205839215909140863, 1212444344111844310966767718336143814073444148841514524239004682590818455533), (130198531925933829276235532864438452135475032024266319093887427091565000941065475368241617689882942413927385434480045237423147742067022975498724070932952844945979412000923082653608041347038785685945287779061569680791646276177915197758740086638799033964265895477391544756060099733632826492707556514477718228089, 1746678969900550451232414772727941339243271725971135624075721598234797739137), (86556740529818701999182409331125765523151710428326444415737649768844477209842768081131578453629929344497832258858334645408256544601975658974374270185997001091744284613653036698721234808609095812136243310508748242174545805899963510937812985990897622289991224902253431089218661354149654314648071811645401980203, 59), (119771207864663048727344169923510015953962525325241274667712607845347021896043073800055144520036246241387458012278226080479082689967319252219165822216639137059908717400080677489401441162071001353589573436954508763682267839267483473415440062664443291154409898307355242414281031419869165403008294197134436208349, 1009337940678568280073361297191684938168309860083649154030283320135910746729), (88661117832022521006018166906753962044724339928574111113581775235994360287575454969528315245665688807933565915038357069734803192468687473656721906857661131876420086949288553493688257203319139236631589881544367315562821890325588764541382767571064409006229961896827766214536972666228482664635886504184173434027, 59), (99980884950650791333694179306252468537690854917934852063249383130562752960223243199306545093156724215522241816187732521033776233516484616238049927541269730871899178674678290239399469264878546893857410716270386349727205323318376080603792326823022181080067695834691297793720561264682609422052987424847115458079, 1621), (128002390749904470305474237501129570770205760445652210058006251980271052723461547884438069571612758666763899987814053808038897780686953983463875988152951524929660911458930740376829380205727843718871214530304068825370648899352212458637555419222234813532977969666939528221453940539927594655126766941557120365657, 688557192327725474326176420122737062730943874228056876213094126971336940017), (67535748173109552755636104481575550689823003936247940195157823939011806894860999754109154411910099570849131350964519281897720821687678379419621158759935560739895912261114022277037294176719097911636829555814209229244898904158104686665025585252727278009860510099037321050262114044493255594866396783905810938689, 1751365111366656452721110528809018891608917703500721028173442666058887940869), (152075355428651783086236896710038583955404341736998264277922441884253491877908803666527433182325929164767079420637432202296488805413538123669179949867614358607532402800168304801651829736345789181607318534562294681050107013616383307355066121681563981812037732279890818748864289702831543865044343116696717893511, 759044623759600600188381958345030765561069242043123826167652750835986460801), (73354310505172789781389277574813493264803411740119010520484480803660046739985960508264101046895491303825586165548274767776327550182215979871727963565263483414028979190568457910022814706813315615245516492479704466916132926165959689086754339945707758167242593998684718886281786446408102440332927558019834778559, 338868908865525724963558058227059740357037814323716420797525371661458001325), (159115255554659038147935938223804693088857768307224263594306654005923543200526359882394150870897591195803912933577712777495351825329207361853712376378543527567998255291359331981064789419975642367390511854061239500520340384661897895279585747458237677239800351189708205765911825896517910085529313191295471643033, 1396061670767132348787585659809555943336449244085077698027120836478135108567), (86242159029027328398398917408480551896426215928027110810436734685538545918773527914559836069361187114026075548080974588253244627055199879301029161072220348765858480645976524318297788832531016215861888967026706859395094710950570331852042090940870263255252269332212442399443716793994731902141121087256222554803, 1919), (127629056981578121046697886121752506990428692952031561296525657859211817222653227327898425676012816156465426951283457060538122585878701696530092636276567304997030468705508883424465142464586021170764316683666206334118288880387753226457066817649761422688209430687390542265128960282171166028077677744101929043711, 1301), (89752729907716355146678902152774718825068432984544121048875864809106555230547362172628901689072246173409152214131932584777232491716006710252506254414927373309603346391971109087639399109168531564729390174535008718550825290969846396002287713484280557202159391463436688921245204684112376870607020840659683448801, 1697), (119395164639509454573469090735506243376510716673342542097795055743437035632838444745581729355340978513479048979902928270607094741033810402999543776113484108373301153253756550975575620081743166078669787140064960794225400363124859250642634070206031677151538229890613745918453526066751818685526047439764979094029, 881852020550266334625898865089675865492366951957643881074439670079646383931), (98875409571300076450340687309148312144374924727567564508926149261028899501317309778581946165963641052783547952938833388392440179716172754134178697178407222877625531865742839064327396043506556127995921778752662672470847907450317992673947581483962514208867687283449955558055780120031336465542826114579841082673, 1013677727534509092784641934687516419550057872181641804643352221189486375667), (116788339409017828421440904518958975760564999289859981039724797225442445240528488462849253793518928394987435861240850653998222821540591597535156789241251239187675053304759261735991114402975558712026787212555089607105772048755239680158081160679587788260677242445398701940275383155293668085668489606528376377351, 1047), (83106249656952494170679467963581877934700780049600513544233121200088777628828775528834356551210285283529042338736493241842572995504831379925696203844467971451317838291039480041763648304954251296817183960477417911218360613314215357477949911424829973549991366237993236657238379194557394258857305481744584799403, 781405008152110926295912167004530795035749962789969286673096285945116060831), (121501292286431552931520167479979297548242671326458030217486500224644312822905656607411846460336620321090160727962499652565705548365704625392545919481636122141886202619889900796895320432150279433856057707150942539621402450914443099033460247919444019221078714395526491400620786565195868416155820999458205701301, 1793), (145219785251864171123901243117481356843499189844382004227520651388772173065588608348819537343380378421026590855957849417741111161212436481745979316542277029050157484169478848866675191023097152641081912394990673635938175836722160187332338081473116209867832485610282298467511533370670067043982588756395630912823, 59), (101595145028438268779364972417598813022418908500566814728175636201502600412527404918877366183716347738515277682378804490248127405456625668160278680057173453913406800521813372832103515633645968190135658759635928376189798721800767320972748813193614659709341234962532863787367074578723352004124846155344791213697, 385266544613549266034545789709265739650958154449498625881019436179627473123), (95765965132737192353415208058450477759302836811499274283716843760587887162887440673775661657261010014861906280957164203891443647713963624128291709256550556034514255724589590039493439894785845391848122522532345138153811774037910991199755886285816908623063382673910439212395078502735775029410719397425849676171, 2023), (103630524689653980227436125267579320663017263233470670985301736778135880113953419146907289004356450647100880006354582383012289203011217954509657025435334445475318436668807174553058097789324780848358999203462086258173935557660167404313446110880267368883008565912362060510538363293739751550749112008229837453633, 308428578817280646888508233080598508050521076503059548683752315295944680121), (96983616941116464549759761212387911081461655032748648028339479735942432804763867728309029485050060245626568969609856574301749697698201528278367577966058738830511310478822989127500288897526511171183991101288286784956213080683752150729272725565461979499020682139097008866234852688647494974739802079274674841979, 59), (105367689826123931726216221974836004802904152180295377343555149912069831979108882435215516419627346253993312798937580556445298408525960356828575360492163308555587129377442938326060817674878566357311890122557725010581117883285088455991051975155186495131848869245729729798786357060580176978370970231306350502173, 790121921497525645102621189078372443306135790969270217081945206780693270313), (146411706822568198406934655273126905945868812353868658876468606049773871966503462269091135246678561381301962482714825469498744405324212025185285472136441049210257577803312287243521507867922796975982073028095114423339892287237701139901103141448539060377737411781842135080674048232864239714729437662360222283483, 1987), (87585995937771342123315607223138397633579716761587465596604131635003403017386975796138403507851844630214119449642293197843062416260906626293271491334578735176948561066270509569221671445791216906278274574904158246415148207966966986341249141841783653581936372130850628945927473253867415002028177159735419375763, 823890818650256714716599624573107650158859954896052328738722916810999939483), (117450787002997436708025516710149370891912112545163644822901346888405193385776667216347265093224960778761789083586589800333263811867125921215941117312317025226022658744073467694246896250343353136137333047080869276659395345257379189553810215354085430755920219743972607304673766644041260324538247255298748480849, 168293369240965464115706621660048369572805347153684114419184146900695019521), (111003454174940075658335621455776892239501477689886676453230055302859184918691082300390718547283088614787960441448744513487165344999651999577915028178221136683446938506796046673692050529340493908382308811572525526842444676641783507196367227219077307116992381676869337242986779945402294511904606114390875506959, 1294692151517164277303938971247265798119503359557187002452953991407489609741), (106038269193814969607413885112327598349872289772284996991083907196971999454415969515733808778862002191199028976899453120234014906357445050477537121126554135876152102867417759241977171849556351822312627166310785702096912364802291314021636019738416707580511848483139035983032802731508027181306557211835764840753, 59), (66665791794923963112795783953276892407064559489636484285556735701475547715731420306080349670955520047501272026441045924337946288517070024790468186511643533557152802261476396746586991426172945637911535594371203205982098499782946690936737486666161539783010348430326657058414392015743003334950117779883105880791, 1529), (99060901038551675907096960468311125183894429186185157112637612570177677486136431129206885946179204370128119733548885808472060414215994765053854023254144719988755768867551528860719295121861293566489141562172922815363302210384976986240386536225149860873400980431852429504802994223219975522377077702380229801737, 398638205609943749017528682565256620774812701386406629595618276545519424183), (99309538502266254049879306354320724011324239517250447060336669606500484131506108286521604990178490553511433366072998706315298663402635279683440374315013683576663352796052125002566234332538940747350641397594485995784911185871638922589506759674950153389703502722251409254054369266390995543282749413044966719377, 599028504807486137126353330714957471578721980939702738413022450149936694571), (81666373409988715705659543613056473020661815201564856323488302565531582436266304746421452858778770445633215054407070318126967755829587596900978530348988728339073152555593422276794884981430585810533633381359694886292986770334521267129198029836847944524446398165666785618493321970171982684192490334977481021507, 1731563519316784187030089143846701775267173524064923692762083369855170277225), (72868681363161267206264301450516648107158368192537118482874793725152709827716118083719981055226123366130440970609055540800530586780309596393350328374339182359595362926209050005677706746755301653363267175908531448140437083346087995059679595594770418988883375977375817788707961201822615047999006312209019642819, 59), (70498138162614588311662155705263933559926957636315995313528355118887629209631885828403337251148653259857452736866639308500422989808968594957157724038434316874986990799645934042448618204032337385629502409178550129460015360802464898733928464123378008979499268134133392773414453654245419596421581873788235319289, 1042839604804206305285274274852572057899873587751885161962284559824923729321), (100771300947885349088642192637439352854500847570447866355290419524302079178818408244860775349914380745644202465311650888306461946793833922290704724837492558246506769754782556775106214337896804277670514775967202160382988467189586113855178927198001935006006448641149904771850633209551909831506785408969404224341, 59), (115356812720766015903330283832518305205200578020853195721745150932012521290161121689124042113349944200343794538993113645014772346516009188504577108979072504826670207305355703443947358375915003850978615720890979253458302014124343047488807285456835265273548080929184189403969346257655708917402236367859167713071, 970272572109176533662366112635099445579052400795279616996795315101818313661), (104141915230739525655198114595724471512882563006096856017968031435905898690335218391149869199655329151796830665635430651743915035347783130073707019216290582775439293940133830042722856237921497896829056680124915576362097918255258692016428208877063172424341234504351048954001070234205736914751348795311261758191, 1464804927453662518663899118128443418207781617251768383436374655385722942381), (85704479705655538983063228390131227822674111590714368180036812749360635516771143999421791804904576223576102053219099943521024324599319318976567949687619902353430944492487424513657247274177871071264191118207677839174779030959608525111452127613619230963458625556194683599222397494629618787213214865269813792151, 59), (117628945455451878965382291826315269838956034228466722166407516281237363696976218617627035152216632462764062421794676665964810666414458295881774492751253246251571378037806558058826884874707439509872858848836265377594293690704727715154739120776490755191578297946491183207067813752561692512196547303260379620811, 534135578773046524934356635673010295963462657825440986017979458408015471661), (129790376163449434969767659265823434204365057212472958822006997958677955334275069232760308559803942815385578345167790947541970881829057715736343149722985723553084070450119454430306393088418208409089960000045225003104935786583475133377729011399754913581321712716656219329338461403104906704781542608022399374597, 1399), (157578115891374408665091819500760463300438205805294010122683554207259621899099098158593482934841032297227739906522276882714409470027696201094287016341468002717046247795865843439330642800545420652748477400387182546111069864425263853541603602824783147691994755141763753071147542849506420785717486197280070514783, 965545807769033895880046901754139594659066711717592812147757745331519058687), (114983442830486980671376711580041293231251643000994001669194778638225295664761161986543395178680829288481450914136242863663654000309714166345656712892711853009257908728177045242275534507699842712516970586427334981223978795890189749948495487145547640500113070034183311539233411196082578454876463210878681055159, 1541), (142702234332216553682722848172524188721414135755500442749015737329918399655364750258635243831373235704414445040600061460024133606302437374887931540174610464771899349175154833367425148456149646693862351090676907237093762765736901590643331415604377632978558269386982539705104359244851986553430662563944112390691, 195553381162238721659569419932391378194091155313403842010334145417967983007), (60167430270949623981093185900311348566572741012296429684920821789611402559805833138375057371222740196948088566668136349888029983485933642061651880504666519643217399061942234960323569866777076983285500345497670039744587909296294748843502654893588984218530344614886405802671047198292761918936530433929198129701, 1811), (96364669521592239394504046861372484774102901666711926727369693204030384788112328427598838219790110061619562415662117342964765205324958835787685251254186409311826212214304569821534419175658087863028305184124210316127154838870455686587528760051875153550462348124260981713443418849479982333672566766529518890687, 1283), (121119958896090457289284860612175592648168544731839745085130834168566625393289442280282916036276657510503852463740390006007994821632511086459055564358358142393418090795886591835868184842845477608438687792138746466522796565819792164215740821593289407358782646215450337966718575956963130239827331579304458141411, 1807), (121626290337243210136926113562144924313207201914518901778310651851604397073755607859385680228300154169015807412942039707432441666721071009174631747444101336579380555818932652746500493434838351005112673891929499604085905957818687576178323128131198929186803306222051096741379057008438515174933612294188495439617, 1447), (99309483008323560704973530543559993775560167680960468334976030959403010025043026278933642582146929604557452745245471233945595155350286744815850767229044362382062653732940038066639111090238879958925643566152650815667194849953254473127117401005569356152451848127068152586279373109196285867965729026692620490753, 491179006875563595926399305535986603676108823244250201344487788241038152421), (64317995169422123539007545696817059289847806032886758991601534401061941309632766530218075969280492091917744668885273930170945414849263513217737165641762037983711125667163150118469079602760807174770924104374071396148911180500740960841381722099967522978705755302998414551453044596886888341192255387782817638039, 59), (86194472986076939471889451228596227669710363587774538866531923175348911562627791796771389855622586676512975492624374635789076818777494229115302593926035486106509616663562609806234441660879909641648823431847066765815913724488994397109647841565209163847405097511980798419971100702934394724758024580049688811437, 1083), (114726295752967967370929923626761652492005588038425186823397300112865214074862951872417734108239010630613159316366143651593173270512084700869621815031346620380459711152293128993212701858576917242141877300432682195098441263407266355545717090992498393734534844894857543675283628204364487704676357730609851832079, 1667), (131889566450907345237664239114001769060154544727924076738869891935975278252826988283494606108467212667206564928451116850349896872724371401113468744189966288309602942156971802831114313135308122374839303348420374390311544951750268237837183592401012539405225467019772217679889620320593739602028900053827336561647, 1909), (95031302822381473327168768466054012229881090761683040224811285763046893204638899130755591999146480876135083665707382833176284137216619909578798897332996460823088342322258828214340736193479476099487025653133573242812604098882179588406218846819238645416443259258067608143025640438936007279846122092909102005531, 1390909567486051315948484635091120074853337230923901988666542934707700067423), (111960841506974543995633939273101933266829678784084692758054048499099206132634423056683201427238662707389542772656599831141393618412955298565048002987576949041556051499206697584376606631462059250864660385963208061466944934672697613366498723534663814948629818489889203535085484384265929823234353948116223926623, 1363845698009918436716019369468873506907633492888261271279087012154676240395), (132176047738371451039943791377530693950064352865486097611848761965076657377578000324393013059145351476906466410696447762023981111153478681337801993445176985771224806590382295364729486193949676190356852048668445222740762311122347241140235180399191206343975673866469981879169357070974467864675222757467825157571, 59), (158794473368821446736030789579378937264753291720010848710356498647712156668954939038504182698104350377535921732687349495412407900247026759424636317966905174266580332091306833532228481064468050907325292583782892805423058749673506355852019892431567533043910560081328154078012039153329955504554338612977071762449, 1251843284504702227100012505772955876430738961598929017696407982982169486749), (80958562895287076583386144833574883057290285432284578531127262432454043833329569256287898144359237412390301994690433760044192287522307938792421002783637626072228005801209504903977316079674356505603480559030872967309217695608196433262457321436687015046532497136237325688791570419719655878352171487493644827479, 1865), (60682332081774540709905315860121364709741409471674543191295733183338118526080170521488204017970745616426202670157025229664590394629170378390615773903773214906582640244711155459858414136307398763627743492132841972003073070999038857565362555180406996074092584020749449526208737508804058342234302148044536032589, 1617), (155658195802231657881856302709848754377376256353903838883847216668817969868244782407885555780451326101633484620276658550251857807355754427830791114882510957488975846439293042500799214347866304133696202601701372145772527325159175729215911064657322804154155425114203848292562698366229790902947291025411443730879, 1259), (116303125740561900914481929265852436587360467508395440975999516510631777228671381088906573791843763785205690054093161020689735123024167667855882986370419487049081955836205169034630193675128596001237881167880360137775530228097329878883903832785264299775500326867527440721693123412603541855964203554701712795309, 1232025539212990997044569925754622670362157029703376289977977121201758237223), (132720632702643019734444222412736969272035340345677722052626622057818467173693392894352507254214187073810536862117610848005063264142510546131153619690314725104458871715398619262822585506490855662990204390610128245039236022884153154971165486219243084640076948297968326629546536915403572714629513559438415056903, 1673834762096527160237242603230739673043388672651842675234328626036029950633), (130505767027245051247099230041144508609645159699144171969283844572812519354954237948802149515648345913796393706254357320661645278930970780571973739339771664303207580708671246020622008141237722640404986955443052397541753242370199690438529031705365781679443361827166080848577405525430612085388588811312822298121, 1807), (114962594792937428020425344011475644065678181368501881613860779256974278434619846078905535894385283755167130484381982479752463958680964339177489882808903724835484706391592546993401187650358311666035566921234500761529107388578165946541864210762420198404298341020785355030669636719042491895005556541697495490171, 1525376633665495281895933903745731344095806014807858252593178130271386311787), (95294305878637725952804892624343559545726924392776215810280610709947893547209637716166351197569914910961276244617153408156145361073994516626392385931696064691443126356960631938471658977257218847521935519127083898346345327102962515144522071142341305054472623655297420246999575564689993389703605483133667795029, 1921), (86729199649715724426603060115147307591619932883630592311444130125986888430135077120372808062562219299786086500247921247266543257725616726839331573526140612774630168696307380086467155143986883300261894490346105443836882963473611992297175490654351419022931254467376647295229574849678016008106856007316920341113, 454370940774007198055136624564222217415066749730016492964150798748168460607), (98704217412887966238430879672131703402523399964534407725980632741801035968098434619000173223344833406679104421870303916171347803562404882493568322854503100889053438059856852748486158714614437637892084126232384872770113449941948520973076942168422892127000264497616155287225705363951887865926505632786174967129, 144714484742003032160689038183328532070862186139052710651322748310114874835), (89670784122858939454871073759234158855539222998292772362008857457577187597454722101154204078208039438380012915598038142928222104256370476302217906827682504262178830886131149112776167642626217720810507572522888604570401903959989980421959818765169389776819811966631576142346018734208887455950202175920088665903, 430211502087224830245197637100551581633561301500007841621978829082890521849), (97380184008015667823879067738627117482094179128856763482168986653849739161391010699374185924253650421407510062427194319065933768470609540665295928402628494917471148648532421450720555804254053607546670298599155487098239606841433439259641344033308083898182841395809426859261025628347333546244251936309437864827, 975787719283649186348186239116255590790609586115108286975353446120859750151), (100900391206698675306242156453618298832787044095836222013166393444990981545692320146115637180044970986188355928310333949232135416200507214044543755608254130947557579026320019325639303573835777174147421594746826218239516714011817735516169927704249561294090322708528278584123805746536775216072622091437321347097, 1228013452487721529380375697681594557570725822814473609830207034297729553761), (86215357659735283736295163284971413833241158103539077833702813166400567804206126931267789750051089266764312557952939042205685651130215129174968946603891017397172154791337209897089915408089860321878186013376498874047058309049121358583499372322891635737456273980855670704522506399417307936177751478284470654201, 339693838588250543163547300758532632742946964142158698130298678259235267221), (161853870330856515356943271715080631412773354691839382756640058195914636376455651684543134217263562484627985808665266238922414358322065948999404708702283874018745057331690051241958853453538140773950076732284932911770976464945033020158471138379889914148151966335314783846858545465092579098737208252072884006021, 1957), (102362259017305867212326136112661627297076912732001591900874050847351473820040197583318338888996608015643933889813617532440312370855172374152296454062101567293661403694023715918268468244480596748230336130542840154797725815594258125047606622700678047929367590135363497908416074492102644511108615254111715864301, 1159), (75023639500061799388900859964530964940659032174661617233992772117033254348475065836390501951079181702465495603581191063266980645453513845382278066611044943085592461552054709291634509289573740751275055290218286589022492734320256932231128820889389689438228121344662856944338978561990419644337009228871733113509, 1144757094099751836963779683366919335974252567005506612024705845547659593803), (118226088106109928369338225449911248388729588491839043051387254882679439583281902214829780775430802927880982577710583854655590757573061342986837280517140103366027631204015680876567210699653090863649037529541279331849032691873876187584135557823485977600914372463878944119461147384091401670682505338708288543657, 272243708488352051746471733482873899442738931519432218846790400736681655123), (78698090268056679804225811784627064861021041164324369394631672406738731884204919824984837660866112074854483956075133576318971561957228309061426680633239725693651257572607864408766311629553199390771255708960028456590919478272670147007265392841477517549437924336625824950725569973741179368199708140011042287487, 59), (122453960246971299593970697734143911683309988315913011874750648841906416506061330338653101685448693868123299376509133525453013799292488717926121470913823147910570692737890101604378777557771055293125827430680578772973287203877528696087564166860410212286370262713156368451312308949589835075635231787303361758183, 1433), (109022397222897410429553298607718800459765099086981999135800098177263724774294816597608019666340693715652294610208192958166120252921533407951649759314908122240697416584607215172904592948615167781629597161955973458257973860619048744536403350129976466192127958675552022209206109395859196959638973038159971407781, 1611608255798656072972634264766722562667476449150037710985593863550588097541), (105091870642372502256207900796397515925391802413497968727204401247119258150805551621605738213963904477576303183547807845571852984591195076020947693944217700825125714647724887301627593917909653940636765340025773541990642667219620127118883553704927280476637442892681720666660318549613856799086087685175028334447, 851981814052201146919769327577955529341858632574877714370814422905461462193), (51449326488149482673319048655359715336144061060744177326234808351813198657560525388653536223082030733435738584278666852890094186044558983364777719993255877645503355347743514327336508351282015451437755128486620477549599118255582044925402066798378470366068475512572184140989833690337093681209567838482201393009, 857714320043057570970566376904325796634235923101505551564858057153410851385), (54140168372419793367305054736668862730903441963287666555161520319083297010531328268864099811496242582863150160281732065551440766626331707206908218077892341177764513436071198079760056117840688054814774206875311330843392222081234774820034914577817506387162934683500677856724284056595845069485661161719890918097, 177703459527804114936129968683906779111595643619563263312790243180903498951), (127600534057260749011726512461442874061472234168869867367670440389281923228687395903496189251069700932562167964086042302987829147275018713328678495861030496641248184677007128357250228353891035282324538185878984221808656308522361316199136753392807539115984733738270765495974567282187212930918065540332399140491, 1131352886363474872970689681414047718177149230755204341262027047202280401897), (83063758805173349954694250322722206747905503844750556330512181034808368877743643592914071214854134878092130636971106872564835483798550077518958038981677036204526281960014084496265388788819619727252199812706150997191605597625103183304176412317286911968135200262136653091508913172242958750852306915751362107001, 1211), (86018962544044989044849042529163988533871373604151941387101501301567186527217762288987835722082147100505494014014165690489074677839064631651326457245524801073532670831348149558830978329407694102279136893647036251960353914383794965980520705501906469774674195479026990500458648939833171807300616054713164245507, 1723), (56782896736305094835758872168543926475470985732862954813238311604658878077586530368399434754799543367659670756493008122110464240447532450676965632328425304772792897654908381535580205656740861524844222516734456682591812461097172477718448094297387325624909421259001118019242907925091616670875810067973572595679, 1489), (93727577052205125152538092572081514675042534001987357962249420240884617634107796433341435683581587736235431362705828765099072871252506177824434701724251104680941892404980907565212719210832702950157256144189364154955088590070344147176963768789348727946878723681346924503567209893284946149252159178441615093803, 1591), (94666756558107716689619895877328821877400734401030779186699136106402836971140549355061373978921731028905006464689567507717058517997231838991294170317444521927274783123201893861663436777533428703424972461943770446643261433177883934559229653221574037908501785465665024805214547235791257678028269763018855328229, 1511), (125667461402634401649588472867416118049229497934243897125714735408008264355133241791439281039289782989598334673803478945153907165323036657888803464358733904693039031029853049989924967873845149939551705166073869459228461929444855674410652320839170887620337804609882489473804112485114944467604354255731022894809, 1344552000751077231676141055716386296386352138752344803049066950534077540621), (175167084388792471001629596847682399774231163479062051181770820051265176305277033540174230039086290954731383570975021133183884789659440624052265727554794685178731771892217381170747974486361190504357789467087338377314994136584248243412753666149360217405109954742376431196339702087361944493774010993275547145959, 1344012629098489092166907061080646740058219961426477718671938991935199818713), (91565329396732689912517890883734193234094882507331099539671313754345153413951341666548382752461789975878395805140234187707150388774098044632163603558497871534745880334181573810577652449299490407283494986353689616986569908144506047425114349694535524195786104270806509209262037430047857356994765178817240676163, 1619), (99759541309067901675978504798257950924550237545415378255659883665097083455349649819757563520819602828814895196257870107906585545987230520857588723831126644654881313389410639211009436322096223353860308077768166550979244308630815490150274996884976828417691112774437473981475608074400428881874466042251212121223, 1759), (80157896868276837203698409100060928650028506868735055432332447372221890513322539211392576114773673404396715071156639852315607010647374311500324487127948195550144877439342398237685281400914769300900168475738481014700821645775604934145714503752049189321022629390319364683146652392880609720085158316190573438231, 1375), (61320218047105079764083449145122267947295605924574662404794779028231475867517060765368631836510433952965919479164362947965655345912289093410530798811172115798710347456713304501780981043994003878225480243589181050668343220550883607209343386506736458306177911122332475494000169058310333660672139837089733599281, 1489277018078262725507338609909058038904531445542342294093296721701862148019), (84669164887278408488150727993460265542355823105455865595626790743753168059187543189003308920578482237510774512965336961085060627795799222046059970695542447528305065182929630433190157957144532140983677570745750048800661745692189509956332097382258535564947621262682078140736358661230240042567353807520955481211, 239781170966092312596559159202327606831333481174754059105964049758601106467), (156579270298607613313571124759947718925446514745554284147271191640915307303414601862843723843772275731181818753786160510199618345928399979749768853759481309767430940298601578544677376627676594634251099644029118832047311254134492155505422989096334253685008364791155309690922447594583515430548626675566790099819, 1029428301485044222307136230451476177956795562761795698657452481113260067473), (81530968362103826941782378732043804528308608702876513074898010616961793493589086243595067385573811318926041776564151879488295648498402842167569419077187049464039233119790730217604778462304825391341902947035856053758931570535359368488006794549791390606682380354447624891657550475904468293539493422006736375127, 1523), (129607846966586422339485097868706381772076567994883401399461959896248391689523104548561440597174832465811027292984957025408810811191969943484303584453850788207488437340535528367199338303585159059475029868712035896984939630153991247763719142013640070754014384209494206281164056687963798657123948912283224670997, 1731), (82826969687216866628061246015209193841049321500607116689487935376266062784993741797022491188550130628918226427639838917206086121107367965963621726803129925100063522801360857606978268522328886265155402184504800158387444337858525657713107177117183797606944390974543525958241558165934561450735377766913531668069, 144805843315343046376001343277369150695966117239823943820112767219928773377), (137110339457862919704461462050117657419596976683609245430598902722144864952504325968275454171440262708756885380360096694511017995468445491954651585054267664713466385286978465548688309639129012768262279536983272017092461767138901013710986278189651883743089924056443465673031478296013616028851874179715187846041, 190261148202533588023960915687639344515572079284059004675997452782658252957), (61770464750877604617770627169181702617028338782621975847230657721607537834150052246038941986271783754896158377832070186238586156268780139322041684343793455655853691991557446263035957922231116993676107630377043891923300401828237541174576560992353753109719266791252881057771930164068827257943476101782397508151, 396584388098576068002507124361968314234871079277198153730073515846171431181), (111763002007266092277794290175861000943659888130511591796943083837873092415911903742298684672555229682010799042764622543799565289565135410460925448439405701885229207800333753394371317026132704444085239572339740948047938154844914101069384201689896761230526050962504275518008823611594756106294383927513748226223, 1769), (108774159810815374787013400072074594670303635597632471703354858968704948115595329543855317249817732690444268454628964149418452369074476529292471861142977701560286100269307145139572386785093136695888654374097818103586357531422755316741374844758091894874996211774931073529284967933380738317735354415561593675051, 82212033994905789948942614557628740955547438629101735235535528395163219759), (89312805118744275154234174092644451517608580486697048642415486314230029024035464226903123284526596316113757166029219843855158547337408813725521653886237272865331903337041836574973956105884322150911348475854803144348882163370939681938876255265318885539918139379154383695641801761552337019414523399606602069771, 819530404616433890668351905878721794417316248681840990021545181335114636637), (143425493618772228695950795860718466867929577183713174109293004944062952235663633033543776180476832410811652242300972388287441811713621459112000804154177999618869832403174490622128018630143252957154617222156929796975316135522439310249408316499504051514188135734661257627864819876445291146772914631927762204893, 1301), (55005848778196994876832809021741428432978489854160811363525680325456602620997683609565675026098216149273200612278100660017898519750077173978426363742219880153587416330758979387819463881540346370001642516901979860043772774841776255874003739056653654912388197727235279122966875615722113926069595706857537618681, 1491296348212321362889513926208388177123385451755535600818638142494293142699), (100399119112743915421165618456285157860280375125233572299529963020544554670929270658482952667842583964257471466898354578248927058183933735932990118730842998194835509737740568983561005928745572651963830639006560886706790773504757635134543187458827293349836328950340139859492116205662779926330126034907977355389, 1031402137675127005349223649473737376125579760884444890684093646906634673901), (117919081685559311271946380735704631198611445039116031839316985039645771087122140763575439251750864573850424911681042110565610168403420214644573023936270581929352327416869604349176611201695172497375997486331554894382568492230174654836396120576681688185873159923514416586224492794330657354129817578809341283057, 1592806806097538504968629946264971579263457642002751388615370346099215786311), (114728970820852760514226947595559666733911101436117518995605278643289058858970717860829874390441221043714951205338699113536201592382281674176611088718153478270691183316085889338745746497081440995047576079627891956974925164768618750174202396190036676402159011578408562321243840266337396692595519403434423877611, 302798866739359233229530375640318516112827527397989269365334015747955495017), (104705580678118544108148697941426469910366978696343373137803974652778230527210027662640037572148915544753918113608796709499854371413085752850349257500039226332599229787221297933560479809270512682205477453339309549859385227820888447821416988731467119647551762630915634952474800471062753139641718027364612910601, 656819199425214528492583657525139881992884681302717414353641601608254730389), (78543948185739856947312289321312515396390399832353526478638116816352385334514554435834568638078871874379873683804894468153619863636997181063922202144681202851269871191945489186032596883780206731848930598495030905875327538874085499539791581655728001158514076263507116531543467736809137082616409612993783618501, 1687), (134461183435579539395253893139880964224831718213840286211148538489694524529810216766096946033852618333946280103248374468858319989087655919665922795163449558020718958231473519743807647829702610840398539493735376715567724275382925270334488913281776923107849911275593835544865934689966049243212441769233752625257, 1417), (135076695028681868208588568540243552511015770659649363429683597335416988148963816870257905527506414254610731986724020607882358815624484964888229440491418981910853654539777386888081076852441175960635121822624086032603057378397690973040238116020244334568002738400501311859623845922413585048078134985770796655813, 1673840698158659255461728277810223496775677794142423414877648468023465347423), (64460713470120170871688648120467985776568419501678109819335391946736336803688471805328163308765782988100190075434996159262186646108802209244786237392716335242807086281989721780305961558001512312997108915429241070149240510220566258738739111204321245631175111849810552439574708126586218034282932271629986260739, 791576488821477061281174925771817804852525859475320897134623721514152378527), (133077091717546618781590395726648355060198583863102196837014771495287246115568894425854201687799141971426982571774014458820149257742444047478985174413985330685687691130500959182978541285542071541255832752179369632496833547758502198096621008546872353189420199657134071356368614131507254260910413262632410966219, 1603), (120785202853135814847072329081054836951380241019240711856755253749732152096071525815173912602703078631431407382139295502074007329682454726505504921632771348495930270199274379734611168487882487451309585365391363603023193021554575789803692032967913808497930122099496214057227147595983827947488257649611340636289, 457704343574111627499663978861695235943711261631436939657827705831111137603), (161562556833105723907569418391691496646953070242379110600465475521611731492160614583138507614651890205849522736240839955847769493084805966493044064124972732711402354509803669967031937493644121922533889321226953984108715214016366282350862234661894714448411100059125232850599593866079952305611855170947755606451, 1445854855507406396638259736092748544697599086292071844797821343498828784393), (107820804571369695804907204135094849498040820224336345605306740878964128362252596534769935160191819539408452123902706975150810874018010869383768629865812443588606685782641520588380422230863895565696765931200385345289425215492619932243637643275106862520877123547944843459619758548608457173755359811721882914631, 59), (92309288872286847825834794389177666262023356100161970996614348395776133219113963388705501948088827790851928125398455704746326946772989476003207247972814575779416993568010580030688058625701564155686446990243793657513182536247392560395878709487494186275626507392984243947678308650001565015005456294271904345457, 1825), (109791849946939324348296068848671890596182192612757152784762047809573045640477958532275441159750374877227918718663566907793086073208790180441351021276912020813963166062302487521466847303912289453182491949534006550624374550417603429465432397436355211774207495511873678236525386855710438846495369364706479473223, 59), (129266217218244299407172523188299189580708877031996573363776820370346967500745922731100738463441072765394082947204869901955032961531969073920903473279448378709939107554756439018162680392680316462628753616729729516270122165164609768521180426109983761871809471342872708307257947603648383930725610571601288059519, 59), (100574198413704956369776830372804329658251083537794290869528056945211553556516347137686635554305864298873966098661505614893203745953139786398091856596364988270664215641292349853618847963012119302429987472924574526923095848157887285109989778423989280851641664431898592446747538814112252448810556624518800472651, 1356357308083819072708498245925612201974108550944586019799481876111416585079), (65267108280758553938782727417079920620160644618802714611999427535895769104445753456112996172553885140537621296824392647495399818929988846805903632706095821050221725154360832478340206041019273803310668990173169243487218461039797707356493778581925977627480829449816794592141154772491772372918125754692540764493, 213746724098516558781236364712411594384226482515934860098719235645926340573), (74060563702058163905654667648469052452363192813635484542044081015567069332764471936776320820512505982023532321648252138784251488215764541741398564238406751163133096326455637295627540174246775573010761872615703797809114155254697063538478407965791049977657744478662850352874921809806282757812228471765677927761, 1559869920505748539630210003277340725798309677953074782640438273711562967375), (157737439438877324956439996897386860483268496807012846241464363050739328628609693598699347361798329404895198048809354302511344792876872737558940975152177507927041795531743425469543034657207899136938541051496358231251484672722215004261527980443834115390039588924096721796515384962465232810508158513719327835777, 59)]
ciphertexts =  [...]

In [11]:
broadcastRSA(k, G, ciphertexts)

719583707600764090225167901

# Exercise 3: The pastries of Mr. Copper and Ms. Smith

In [12]:
#Using the convergents formulas to compute k and d.
def all_convergents(cont_frac):
    d_0, k_0 = 1, cont_frac[0]
    d_1, k_1 = cont_frac[1], cont_frac[0]*cont_frac[1] + 1
    res_k = [k_0, k_1]
    res_d = [d_0, d_1]
    for i in range(2, len(cont_frac)):
        k = cont_frac[i]*res_k[i-1] + res_k[i-2]
        d = cont_frac[i]*res_d[i-1] + res_d[i-2]
        res_k.append(k)
        res_d.append(d)
        
    return res_k, res_d

In [13]:
#Using Wiener's theorem and attack
def pq(e, N):
    cont_frac = (e/N).continued_fraction()
    #need to compute the convergents k/d
    list_k, list_d = all_convergents(cont_frac)
    #trying to find phi(N).
    list_phi = []
    d_condition = (N**(1/4))/3
    for k, d in zip(list_k, list_d):
        if k != 0:
            phi, rest = divmod(e*d - 1, k)
            if rest == 0:
                check = mod(e*d, phi) == 1
                #check if e*d mod phi == 1 and if d is not bigger than the given condition.
                if check and d < d_condition:
                    list_phi.append(phi)
    #solve quadratic equation to find q and p.
    for phi in list_phi:
        b = -(N - phi + 1)
        c = N
        delta = (b**2 - 4*c)**(1/2)
        #removing all solutions with a non integer delta.
        if delta >= 0 and 'sqrt' not in str(delta):
            p, q = (-b - (delta))/2, (-b + (delta))/2
            if p < q:
                p,q = q,p
            if p*q == N:
                return p, q
            

In [14]:
#Test params
e=16240841868131927936529703689646361045295735490900215667605528907978179985174107508569101991611313158603944071471303626634336452095209650006270423971287428142814425307057080075814052273223830357774808618890351169960476788245124379740526021174674004560847275284707684451282241248657365059795191444267917008583142608008027300758060196854329872656392599138819535490131370689020918172033635453857644541060354372350270809548878871988915846480967473628923619972565853
N=2239783540124318325186426278057339977233162046093572916094754341241671114784064052662925986256946427951631515634474424773483608928313595822934308952806484968480409874916870579892261436768357757885966102158051882233419604405188863715293117271506551688407948090769865431063219300769457835256450818893671049090001760134145778540500468081549710547929163537034057890459322030165923081731447282383340840885458782894486323116977342827594059589206653432775128275972946001
p,q = pq(e, N)
print(f'N = {p} * {q}')

N = 2043194400436866907930688608509160667079872325576514580918948737115101132679786967197586350313116908242265862025850333032739225987930184967467231570400884630987727231126047704813585252728334111683115040419667272767734905681893573253 * 1096216561500666596160022030946708220398246492667357961214234351438368468408438489975307490720362277824098126111526185243306179417002136274635587135583861782304815211421347908656367122385694729952979062571853844172577127319779341917


In [15]:
#Submission params
e=950444963346993466173528339678278663259774072783324840775599445435472906462895275152893253420862481680852973827963737185455893125885669605000768295709367835705813437860363800537146530929818937776018456668931321646694954720457068123178453835540559426917120863985800377864632107314830263304146162766503832020869496017724901594580225581053180750353784445082335805975019185004633438543844980681424389164529642677175372534396244586774081496710083643407144345404562867
N=1019171353005486725053144493724106088196674539234177029255440987092196325938536751981454737873681454718380047734784880079789267676579662557024084789679900745855532243125986322944449597543847933378955652129624357677946864414091357758209831039747994128419855317099280612775611808618499357160686794063700713999799376468421647560689796451828766556266995607927960740660534658258159632681953694661076489790656817886073292825857283924597105214851006299975946665002972977
p,q = pq(e, N)
print(f'N = {p} * {q}')

N = 1269319976881473485356939701153978001526622746302265251437924794883709471616152167929306647003517382569046245541448049486930904627373031571429843014449422432434061587995515558517999482558018811304365134388542331941872891703457767803 * 802927056666543640628797008910740998255498656229936435851544265312468757779509502673272361697309625529898351062611391504611129352003664781967156056435090100984070640112941600278579151528191386178046617632978774221986466597543304259


In [16]:
def ringGCD(a,b):
    if b == 0:
        return a.monic()
    else:
        return ringGCD(b, a%b)

In [17]:
#Using franklin Reiter algorithm to return M1
def franklinReiter(delta, e, N, c1, c2):
    P.<x> = PolynomialRing(Zmod(N))
    f = x^e - c1
    g = (x + delta)^e - c2
    return ringGCD(f, g)

In [18]:
#Franklin Reiter's Attack on related messages and Coppersmith’s short-pad attack
def recoverM(e, N, c1, c2):
    #Compute the res(g1,g2) 
    R = Zmod(N)
    P.<x,y> = PolynomialRing(R,2)
    g1 = x^e - c1
    g2 = (x + y)^e - c2
    P1.<y> = PolynomialRing(R)
    h = g1.change_ring(P1).resultant(g2.change_ring(P1))
    #Compute the small root delta of h
    h = h.univariate_polynomial(P1)
    root = h.small_roots(X=2^16, beta=0.5)[0]
    
    #Franklin messages
    message = Integer(N - franklinReiter(root, e, N, c1, c2).coefficients()[0])
    
    #Decypher by testing string with the unnecessary padding if the padding is bad the string is removed if we decypher something we keep it and clean it a bit.
    for i in range(8):
        msg = hex(Integer(message*(2^i)))
        if len(msg)%2 == 1:
            msg = '0' + msg[2:]
        elif msg[:2] == '0x':
            msg = msg[2:]
        m = binascii.unhexlify(msg)
        if chr(m[0]) in string.ascii_letters:
            index = str(m).index('00')
            print(str(m)[2:index-2])

In [19]:
#Test parameters
e=3
N=23584875682328802191106136518968724072456145732480775581334288609587899867767402160085350828722722250114294060364871144115256331527682629035279870373504763179242965176325471310246239370027570968750429900608015793709653353127007701714309468877125096769212654805257606663482343539121460572108497248054678337536669693959505001210943524088963652197589202039757277749467119713382536351281723118957985935802014615991366116676087184039681661211158161359908727521804632847858930716571882125224360404372304055700845702183359142283412919410910422233766200264343210344294403202732601099428583343004029469629762434408535806181813
c1=19686583590390801426906332946819400729379335567451647052008928201531208623567599578228642481479874940960718903983415630308130313718694660626215257836705607401741072984276863143455265665485466592269404782433278092870555751709026824484581155689845050794899225029563049683044832557515523854858078243443442108868258040687432478631721187860624733763293062865687427607984511603703714111156417369093826371846920838999098374765279120229844951990660651179389042482427945046023372170490625532470296487969892985153460164931418475655743558537751613150663600465113969929099358635928040597881847721696063023126046993350197142477270
c2=6987235681298237507950319781447268763646866491177958754331155888315680528916591041132378395549082870864484648600173674212905581581284103790970799671485355198785869407236676398526265914326735248876308897910844593069216461514951573085624208381487947957703060066848230363675275373623743541627383103827606132447867228729591270532668536873329019356318374134554509468773593169780600474379787953735845620534130358607012026516300419321553855915349896708015893358350645632005116375248400704352257458563836925854177246077312489565614683769387185245655962459423872937360508113460713218818652180678262526288408817642275117321435
recoverM(e, N, c1, c2)

A skilled dictator is much more beneficial to humanity than a democracy of ignorant people. (Gene Smilas, Code Geass: Boukoku No Akito)


In [20]:
e=3
N=20755705704733379736327618714441264528240691245030599561796900326984367647494490849921651327838968900807911411785892124692346760484726563037235847558303696070788372167429815431410543373343629866965246226716465691037124452243910447966609324930249364141580234599052609569781877239813113658817125844345910020087467268015819896941096129137538529830360509449125821476651806419876148486658706679278514284656874530799452157082228481649067673214249210101951926517804083275706748545208390424023941400514350059581511556322705156747434046286467538007395907039333186436834003280940498412777387595300297214016743515508457711567381
c1=3147068065420319709305924584030137257313398386095995669952022446428769170541104689540248987028756333743082946791770163166013111105544468277656511809106909519762768189152702418091557358819968303134351762547712626652312451971943045789222791843928220169824566473749847892259413983493364425705483484126657955927519498652481625053032821843116165607534418713753225654479710843559640180979124214726344471235874528059019093850537226307884313792269184623103034026874111180576804753476474139750484220042528780424107779319421760564064715483142445035638655821391216829975510028165273719490808653258320950309797621922521146689160
c2=3133401367164757135929489622671040368339532746311226569598586902030184890493551728587970794745169488101376854458367073571605204682770264934634224304147951741844102259583869396535526585045782256190819589659017171983924296084171874234544286417408584315472114293779874899469749904303586417934984494165433571306458708998788235573244005422394820177176380415542852336944023000791856248965640439665882199888482815024598556654171394197113647418570214127673632584256686134144242678372505982054571065608914448353791830981678726539407672060054017905078558385370982531061317356447749613030468282079962492899804200807052473327780
recoverM(e, N, c1, c2)

All things change in a dynamic environment. Your effort to remain what you are is what limits you. (Puppet Master, Ghost in the Shell)
